# Paths & knobs

In [1]:
# ==== Cell 0: paths & knobs ====
from pathlib import Path
ROOT     = Path("/Users/hsiaopingni/Desktop/octaneX")
DATA_DIR = Path("/Users/hsiaopingni/Desktop/SLM_RAS-main/HW_TELEMETRY_DATA_COLLECTION/TELEMETRY_DATA")

RES_DIR  = ROOT / "Results";            RES_DIR.mkdir(parents=True, exist_ok=True)
PLOTS_DIR= RES_DIR / "ROC_PR";          PLOTS_DIR.mkdir(parents=True, exist_ok=True)

MMI_DIR  = ROOT / "FeatureRankOUT"                 # your existing MMI ranks
HYB_DIR  = ROOT / "FeatureRankOUT_HYBRID";         HYB_DIR.mkdir(parents=True, exist_ok=True)
EXPL_DIR = RES_DIR / "Explainability";             EXPL_DIR.mkdir(parents=True, exist_ok=True)
HYBRID_REPORT_DIR = RES_DIR / "Hybrid_Reports";    HYBRID_REPORT_DIR.mkdir(parents=True, exist_ok=True)

SETUPS        = ["DDR4","DDR5"]
WINDOW_SIZES  = [32, 64, 128, 256, 512, 1024]
KFOLDS_SET    = [3, 5, 10]
SUBSPACES     = ["compute","memory","sensors"]
META          = ["label","setup","run_id"]
SEED          = 42


# Shared helpers

In [2]:
# ==== Cell 1: shared helpers ====
import re, hashlib, numpy as np, pandas as pd

def iter_raw_csvs(root: Path):
    for p in root.rglob("*.csv"): yield p

def read_csv_clean(p: Path) -> pd.DataFrame:
    df = pd.read_csv(p)
    return df.loc[:, ~df.columns.str.startswith("Unnamed")]

def detect_setup_from_path(p: Path):
    s = str(p).lower()
    return "DDR4" if "ddr4" in s else ("DDR5" if "ddr5" in s else None)

def is_benign_path(p: Path) -> bool:
    s = str(p).lower()
    if "benign" in s: return True
    bad = ["attack","anom","fault","inject","trojan","mal","rh","droop","spectre","trrespass"]
    return not any(b in s for b in bad)

def is_anomaly_path(p: Path, anomaly: str) -> bool:
    return (anomaly.lower() in str(p).lower()) and (not is_benign_path(p))

def collect_raw_pairs_by_setup(data_dir: Path, which: str, anomaly: str|None=None):
    out = {"DDR4": [], "DDR5": []}
    for p in iter_raw_csvs(data_dir):
        setup = detect_setup_from_path(p)
        if setup is None: continue
        if which == "benign":
            if not is_benign_path(p): continue
        else:
            if anomaly is None or not is_anomaly_path(p, anomaly): continue
        try:
            out[setup].append((p, read_csv_clean(p)))
        except Exception as e:
            print(f"[WARN] read failed {p}: {e}")
    return out

def mk_run_id(path: Path) -> str:
    return f"run_{hashlib.md5(str(path).encode('utf-8')).hexdigest()[:10]}"

def telemetry_cols(df: pd.DataFrame) -> list[str]:
    return [c for c in df.columns if c not in META and df[c].dtype.kind in "fcbiu"]

def window_collapse_means(df: pd.DataFrame, win: int, setup: str, run_id: str, label: str) -> pd.DataFrame:
    cols = telemetry_cols(df)
    if not cols: return pd.DataFrame()
    rows = []
    for start in range(0, len(df) - win + 1, win):
        means = df.iloc[start:start+win][cols].astype(float).mean(axis=0, numeric_only=True)
        row = means.to_frame().T
        row["setup"] = setup; row["run_id"] = run_id; row["label"] = label
        rows.append(row)
    return pd.concat(rows, ignore_index=True) if rows else pd.DataFrame()

def build_windowed_raw_means(pairs, setup: str, win: int, label: str) -> pd.DataFrame:
    out = []
    for p, df in pairs:
        agg = window_collapse_means(df, win=win, setup=setup, run_id=mk_run_id(p), label=label)
        if not agg.empty: out.append(agg)
    return pd.concat(out, ignore_index=True) if out else pd.DataFrame()

# robust scaling (winsor + z) fit on BENIGN
def robust_scale_train(Xb_np: np.ndarray, winsor=(2.0, 98.0)):
    Q1, Q2 = np.percentile(Xb_np, winsor[0], axis=0), np.percentile(Xb_np, winsor[1], axis=0)
    mu = np.clip(Xb_np, Q1, Q2).mean(axis=0)
    sd = np.clip(Xb_np, Q1, Q2).std(axis=0, ddof=0) + 1e-9
    return mu, sd, Q1, Q2

def apply_robust_scale(X: pd.DataFrame, mu, sd, Q1, Q2):
    Z = (np.clip(X.values, Q1, Q2) - mu) / sd
    return pd.DataFrame(Z, columns=X.columns, index=X.index)

# subspace map
PATS = {
    "memory":  [r"ddr|dram|mem(ory)?\b", r"\bL1(\b|_)|L2(\b|_)|L3(\b|_)", r"(L[123].*(HIT|MISS|MPI))\b",
               r"cache|fill|evict|wb|rd|wr|load|store", r"bandwidth|bw|throughput|qdepth|queue",
               r"lat(ency)?|stall.*mem|tCCD|tRCD|tRP|tCL|page|row|col"],
    "sensors": [r"temp|thermal|hot",
                r"volt|vdd|vcore|vin|vout|cpu[_\- ]?volt|cpu[_\- ]?vdd|vdd[_\- ]?cpu|core[_\- ]?volt|core[_\- ]?voltage",
                r"power|watt|energy|joule", r"fan|throttle|current|amps?"],
    "compute": [r"\bIPC\b|\bPhysIPC\b|\bEXEC\b|\bissue|retire|dispatch\b",
                r"\bINST\b|INSTnom%|branch|mispred|\balu\b|\barith\b|\blogic\b",
                r"C0res%|C1res%|C6res%|C7res%|CFREQ|AFREQ|ACYC|CYC|TIME|clk|cycle|freq|util",
                r"\bcore|cpu|sm|warp|shader\b"],
}
def subspace_of_feature(name: str) -> str:
    b = str(name)
    if any(re.search(p, b, flags=re.I) for p in PATS["memory"]):  return "memory"
    if any(re.search(p, b, flags=re.I) for p in PATS["sensors"]): return "sensors"
    return "compute"

# rank loader (MMI or Hybrid)
def read_rank_list(rank_dir: Path, setup: str, win: int, kfold: int, sub: str) -> list[str]:
    p = rank_dir / f"{setup}_{win}_{kfold}_0_{sub}.csv"
    if not p.exists(): return []
    df = pd.read_csv(p)
    col = "feature" if "feature" in df.columns else df.columns[0]
    return df[col].dropna().astype(str).tolist()


# Feature ranking

## X-OCTANE: RAW windows (WIN, overlap, z-norm, sampling) → KFold → PC1_vs_PC1 (MI) ranking

In [3]:
# Outputs: FeatureRankOUT/DDR{4|5}_{W}_{K}_0_{compute|memory|sensors}.csv  (feature,score)

import re, hashlib, warnings
from pathlib import Path
from typing import List, Tuple, Dict, Optional
import numpy as np
import pandas as pd
from sklearn.model_selection import GroupKFold, KFold
from sklearn.decomposition import PCA

warnings.filterwarnings("ignore", category=RuntimeWarning)

# ------------------ PATHS ------------------
DATA_DIR = Path("/Users/hsiaopingni/Desktop/SLM_RAS-main/HW_TELEMETRY_DATA_COLLECTION/TELEMETRY_DATA")
ROOT     = Path("/Users/hsiaopingni/octaneX_v7_4functions")
OUT_DIR  = ROOT / "FeatureRankOUT"; OUT_DIR.mkdir(parents=True, exist_ok=True)

# ------------------ CONFIG ------------------
WINDOW_SIZES: List[int] = [32, 64, 128, 256, 512, 1024]
OVERLAP: float          = 0.5        # 0.0.. <1.0 ; stride = int(W*(1-OVERLAP))
KFOLDS_SET: List[int]   = [3, 5, 10, 20]
SEED                    = 42
META                    = ["label","setup","run_id"]

# Sampling inside each window -> one scalar per signal per window (no means)
SAMPLING: str           = "center"   # {"center", "first", "last", "offset:<int>"}

# ------------------ File discovery (benign RAW only) ------------------
def is_benign_path(p: Path) -> bool:
    s = str(p).lower()
    if "benign" in s: 
        return True
    bad = ["attack","anom","fault","inject","trojan","mal","rh","droop","spectre","trrespass"]
    return not any(b in s for b in bad)

def detect_setup_from_path(p: Path) -> Optional[str]:
    s = str(p).lower()
    if "ddr4" in s: return "DDR4"
    if "ddr5" in s: return "DDR5"
    return None

def iter_raw_csvs(root: Path):
    for p in root.rglob("*.csv"):
        yield p

def read_csv_clean(p: Path) -> pd.DataFrame:
    df = pd.read_csv(p)
    df = df.loc[:, ~df.columns.str.startswith("Unnamed")]
    return df

def mk_run_id(path: Path) -> str:
    return f"run_{hashlib.md5(str(path).encode('utf-8')).hexdigest()[:10]}"

def unify_benign_by_setup(data_dir: Path) -> Dict[str, List[Tuple[Path, pd.DataFrame]]]:
    buckets = {"DDR4": [], "DDR5": []}
    for p in iter_raw_csvs(data_dir):
        if not is_benign_path(p): 
            continue
        setup = detect_setup_from_path(p)
        if setup is None:
            continue
        try:
            df = read_csv_clean(p)
            if len(df) > 0:
                buckets[setup].append((p, df))
        except Exception as e:
            print(f"[WARN] failed reading {p}: {e}")
    print(f"[DISCOVER] benign RAW files → DDR4: {len(buckets['DDR4'])}, DDR5: {len(buckets['DDR5'])}")
    return buckets

# ------------------ Subspace mapping ------------------
PATS = {
    "memory": [
        r"ddr|dram|mem(ory)?\b", r"\bL1(\b|_)|L2(\b|_)|L3(\b|_)", r"(L[123].*(HIT|MISS|MPI))\b",
        r"cache|fill|evict|wb|rd|wr|load|store", r"bandwidth|bw|throughput|qdepth|queue",
        r"lat(ency)?|stall.*mem|tCCD|tRCD|tRP|tCL|page|row|col",
    ],
    "sensors": [
        r"temp|thermal|hot",
        r"volt|vdd|vcore|vin|vout|cpu[_\- ]?volt|cpu[_\- ]?vdd|vdd[_\- ]?cpu|core[_\- ]?volt|core[_\- ]?voltage",
        r"power|watt|energy|joule",
        r"fan|throttle|current|amps?",
    ],
    "compute": [
        r"\bIPC\b|\bPhysIPC\b|\bEXEC\b|\bissue|retire|dispatch\b",
        r"\bINST\b|INSTnom%|branch|mispred|\balu\b|\barith\b|\blogic\b",
        r"C0res%|C1res%|C6res%|C7res%|CFREQ|AFREQ|ACYC|CYC|TIME|clk|cycle|freq|util",
        r"\bcore|cpu|sm|warp|shader\b",
    ],
}
def subspace_of_feature(base_feat: str) -> str:
    b = str(base_feat)
    if any(re.search(p, b, flags=re.I) for p in PATS["memory"]):  return "memory"
    if any(re.search(p, b, flags=re.I) for p in PATS["sensors"]): return "sensors"
    return "compute"

# ------------------ RAW → overlapped windows → z-norm → sampling ------------------
def telemetry_cols(df: pd.DataFrame) -> list[str]:
    return [c for c in df.columns if c not in META and pd.api.types.is_numeric_dtype(df[c])]

def _stride(win: int, overlap: float) -> int:
    step = max(1, int(round(win * (1.0 - overlap))))
    return max(1, min(step, win))  # never 0; allow step==win for non-overlap

def _pick_index(start: int, win: int) -> int:
    if SAMPLING == "first":
        return start
    if SAMPLING == "last":
        return start + win - 1
    if SAMPLING.startswith("offset:"):
        try:
            off = int(SAMPLING.split(":",1)[1])
        except Exception:
            off = win // 2
        off = np.clip(off, 0, win-1)
        return start + off
    # default = "center"
    return start + (win // 2)

def build_window_samples(df: pd.DataFrame, win: int, overlap: float,
                         setup: str, run_id: str) -> pd.DataFrame:
    """
    Produce one scalar per signal per window by sampling a single time index inside each window
    after global z-normalization per signal (no within-window mean).
    """
    cols = telemetry_cols(df)
    if not cols:
        return pd.DataFrame()

    X = df[cols].astype(float)
    # Global z-norm per signal (over full run)
    X = (X - X.mean(axis=0)) / (X.std(axis=0, ddof=0) + 1e-9)
    X = X.replace([np.inf, -np.inf], 0.0).fillna(0.0)

    n = len(X)
    step = _stride(win, overlap)
    rows = []
    for start in range(0, n - win + 1, step):
        idx = _pick_index(start, win)
        sample = X.iloc[idx:idx+1].copy()
        sample["setup"] = setup
        sample["run_id"] = run_id
        sample["label"] = "BENIGN"
        rows.append(sample)

    return pd.concat(rows, ignore_index=True) if rows else pd.DataFrame()

def build_all_window_samples(pairs: List[Tuple[Path, pd.DataFrame]],
                             setup: str, win: int, overlap: float) -> pd.DataFrame:
    out = []
    for p, df in pairs:
        rid = mk_run_id(p)
        part = build_window_samples(df, win=win, overlap=overlap, setup=setup, run_id=rid)
        if not part.empty:
            out.append(part)
    return pd.concat(out, ignore_index=True) if out else pd.DataFrame()

# ------------------ PC1 vs PC1 (MI) ------------------
def _zscore_df(X: pd.DataFrame) -> pd.DataFrame:
    Z = (X - X.mean(axis=0)) / (X.std(axis=0, ddof=0) + 1e-9)
    return Z.replace([np.inf, -np.inf], 0.0).fillna(0.0)

def _quantile_bin(x: np.ndarray, q: int = 16) -> np.ndarray:
    x = np.asarray(x, float)
    mask = np.isfinite(x)
    if mask.sum() == 0:
        return np.full_like(x, -1, dtype=int)
    edges = np.quantile(x[mask], np.linspace(0,1,q+1))
    edges[0]  -= 1e-9; edges[-1] += 1e-9
    out = np.full_like(x, -1, dtype=int)
    out[mask] = np.digitize(x[mask], edges[1:-1], right=False)
    return out

def _discrete_mi(a: np.ndarray, b: np.ndarray) -> float:
    mask = (a >= 0) & (b >= 0)
    if mask.sum() == 0:
        return 0.0
    A = a[mask].astype(int); B = b[mask].astype(int)
    na, nb = A.max()+1, B.max()+1
    idx = (A * nb + B)
    binc = np.bincount(idx, minlength=na*nb).astype(float).reshape(na, nb)
    joint = binc / binc.sum()
    pa = joint.sum(axis=1, keepdims=True); pb = joint.sum(axis=0, keepdims=True)
    with np.errstate(divide='ignore', invalid='ignore'):
        ratio = joint / (pa @ pb)
        mi = np.nansum(joint * np.log(ratio + 1e-12))
    return float(max(mi, 0.0))

def pc1_vs_pc1_mi_scores(df_windows: pd.DataFrame, qbins: int = 16) -> pd.Series:
    """
    For each feature i:
      - per-signal PC1 = z-scored column Xi (samples = windows)
      - others PC1     = PCA(X_{-i}) first PC scores
      - score_i        = MI(qbin(Xi), qbin(PC1_{others}))
    """
    feat_cols = [c for c in df_windows.columns if c not in META and pd.api.types.is_numeric_dtype(df_windows[c])]
    if not feat_cols:
        return pd.Series(dtype=float)

    X = _zscore_df(df_windows[feat_cols].astype(float))
    names = list(X.columns)
    X_arr = X.values
    n, d = X_arr.shape
    scores = []

    for i in range(d):
        if d == 1:
            scores.append(0.0); continue
        # Per-signal PC1 (1-D) = standardized Xi
        xi = X_arr[:, i]
        # Others PC1
        others = np.delete(X_arr, i, axis=1)
        # If degenerate, fallback to mean of others (still no window mean used anywhere; it's across windows)
        try:
            pca = PCA(n_components=1, svd_solver="auto", random_state=SEED)
            pc1_scores = pca.fit_transform(others).ravel()
        except Exception:
            pc1_scores = others.mean(axis=1)

        ai = _quantile_bin(xi, q=qbins)
        bi = _quantile_bin(pc1_scores, q=qbins)
        scores.append(_discrete_mi(ai, bi))

    s = pd.Series(scores, index=names, name="score").sort_values(ascending=False)
    return s

def _normalize_01(s: pd.Series) -> pd.Series:
    s = s.astype(float)
    lo, hi = float(s.min()), float(s.max())
    if not np.isfinite(lo) or not np.isfinite(hi):
        return s.fillna(0.0)
    if hi == lo:
        return pd.Series(np.zeros(len(s)), index=s.index)
    return (s - lo) / (hi - lo)

def split_and_save(scores: pd.Series, setup: str, win: int, kfold: int):
    s_norm = _normalize_01(scores)
    buckets = {"compute": [], "memory": [], "sensors": []}
    for feat, sc in s_norm.items():
        buckets[subspace_of_feature(feat)].append((feat, sc))
    for sub, pairs in buckets.items():
        if not pairs:
            continue
        df_sub = (
            pd.DataFrame(pairs, columns=["feature","score"])
              .sort_values("score", ascending=False, kind="mergesort")
        )
        out_path = OUT_DIR / f"{setup}_{win}_{kfold}_0_{sub}.csv"
        df_sub.to_csv(out_path, index=False)
        print(f"[WRITE] {out_path}")

# ------------------ MAIN ------------------
if __name__ == "__main__":
    buckets = unify_benign_by_setup(DATA_DIR)

    for setup in ["DDR4","DDR5"]:
        pairs = buckets.get(setup, [])
        if not pairs:
            print(f"[WARN] No benign RAW {setup} files found. Skipping.")
            continue

        for W in WINDOW_SIZES:
            df_win = build_all_window_samples(pairs, setup=setup, win=W, overlap=OVERLAP)
            if df_win.empty:
                print(f"[SKIP] {setup} win={W}: no window samples produced.")
                continue

            # type guard for meta
            for m in META:
                if m in df_win.columns:
                    df_win[m] = df_win[m].astype(str)

            for K in KFOLDS_SET:
                # Prefer grouping by run_id to avoid leakage
                if "run_id" in df_win.columns and df_win["run_id"].nunique() >= K:
                    splitter = GroupKFold(n_splits=K)
                    splits = splitter.split(df_win, groups=df_win["run_id"].values)
                else:
                    splitter = KFold(n_splits=K, shuffle=True, random_state=SEED)
                    splits = splitter.split(df_win)

                fold_scores = []
                for tr_idx, _ in splits:
                    df_tr = df_win.iloc[tr_idx]
                    s = pc1_vs_pc1_mi_scores(df_tr, qbins=16)
                    fold_scores.append(s)

                if not fold_scores:
                    print(f"[SKIP] {setup} win={W} k={K}: no folds produced.")
                    continue

                M = pd.concat(fold_scores, axis=1).fillna(0.0)
                mean_s = M.mean(axis=1).sort_values(ascending=False)
                split_and_save(mean_s, setup=setup, win=W, kfold=K)

    print("\n[DONE] RAW PC1_vs_PC1 (MI) K-fold rankings written.")


[DISCOVER] benign RAW files → DDR4: 13, DDR5: 13
[WRITE] /Users/hsiaopingni/octaneX_v7_4functions/FeatureRankOUT/DDR4_32_3_0_compute.csv
[WRITE] /Users/hsiaopingni/octaneX_v7_4functions/FeatureRankOUT/DDR4_32_3_0_memory.csv
[WRITE] /Users/hsiaopingni/octaneX_v7_4functions/FeatureRankOUT/DDR4_32_3_0_sensors.csv
[WRITE] /Users/hsiaopingni/octaneX_v7_4functions/FeatureRankOUT/DDR4_32_5_0_compute.csv
[WRITE] /Users/hsiaopingni/octaneX_v7_4functions/FeatureRankOUT/DDR4_32_5_0_memory.csv
[WRITE] /Users/hsiaopingni/octaneX_v7_4functions/FeatureRankOUT/DDR4_32_5_0_sensors.csv
[WRITE] /Users/hsiaopingni/octaneX_v7_4functions/FeatureRankOUT/DDR4_32_10_0_compute.csv
[WRITE] /Users/hsiaopingni/octaneX_v7_4functions/FeatureRankOUT/DDR4_32_10_0_memory.csv
[WRITE] /Users/hsiaopingni/octaneX_v7_4functions/FeatureRankOUT/DDR4_32_10_0_sensors.csv
[WRITE] /Users/hsiaopingni/octaneX_v7_4functions/FeatureRankOUT/DDR4_32_20_0_compute.csv
[WRITE] /Users/hsiaopingni/octaneX_v7_4functions/FeatureRankOUT/DDR4_3

# **X-OCTANE paper: FULL MULTI-WIN / MULTI-K PIPELINE (AUTHENTIC)

In [5]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

# ===============================================================================================
# FULL MULTI-WIN / MULTI-K PIPELINE (AUTHENTIC)
# + DSE PLOTS (all cases)
# + TABLE_SUBSPACEWEIGHTS (all cases)
#
# Runs:
#   - WINS   = [32, 64, 128, 512, 1024]
#   - KFOLDS = [3, 5, 10]
#   - SETUPS:
#       DDR4 anomalies: DROOP, RH
#       DDR5 anomalies: DROOP, SPECTRE
#   - PCT sweep: 10..100 step 10
#
# Produces:
#   1) Results/per_run_metrics_all_PIPELINE.csv
#   2) DSE plots for every (setup, anomaly, win, kfold):
#        Results/DesignSpace/PaperStyle_ALLRUNS/<setup>/<anomaly>/WIN{win}_KF{kfold}/...
#   3) Weight tables for every (setup, win, kfold):
#        Results/Table_SubspaceWeights/<setup>/WIN{win}_KF{kfold}/...
#
# Key behaviors:
#   - Workload-matched benign pairing by filename suffix (_dft/_oe/_tr etc.)
#   - DROOP: append CPU Voltage + transient features to sensors selection (does NOT override)
#   - Weight selection: PR-first on holdout split over candidate weight table
#   - Scoring: draft-math dE/dC + aM/aJ (with softplus positivity for aJ)
#
# ===============================================================================================

import gc, re, hashlib, warnings, unicodedata
from pathlib import Path
from typing import List, Dict, Optional, Tuple

import numpy as np
import pandas as pd

import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt

from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.model_selection import StratifiedShuffleSplit

warnings.filterwarnings("ignore", category=RuntimeWarning)

# ===============================================================================================
# 0) PATHS
# ===============================================================================================
DATA_DIR = Path("/Users/hsiaopingni/Desktop/SLM_RAS-main/HW_TELEMETRY_DATA_COLLECTION/TELEMETRY_DATA")
ROOT     = Path("/Users/hsiaopingni/octaneX_v7_4functions")
RES_DIR  = ROOT / "Results"
RES_DIR.mkdir(parents=True, exist_ok=True)

EXT_DRIVE = Path("/Volumes/Untitled")
EXT_RES   = EXT_DRIVE / "octaneX_results"

def _can_write_dir(p: Path) -> bool:
    try:
        p.mkdir(parents=True, exist_ok=True)
        t = p / ".write_test"
        t.write_text("ok")
        t.unlink()
        return True
    except Exception:
        return False

if _can_write_dir(EXT_RES):
    RES_DIR = EXT_RES
    RES_DIR.mkdir(parents=True, exist_ok=True)
    print(f"[OK] Using external results dir: {RES_DIR}")
else:
    print(f"[WARN] Cannot write to {EXT_RES}. Using local: {RES_DIR}")

RANK_DIRS = [
    ROOT / "FeatureRankOUT",
    EXT_DRIVE / "FeatureRankOUT",
    EXT_DRIVE / "octaneX" / "FeatureRankOUT",
]

# ===============================================================================================
# 1) RUN CONFIG
# ===============================================================================================
SETUPS = ["DDR4", "DDR5"]
ANOMALIES_BY_SETUP = {"DDR4": ["DROOP", "RH"], "DDR5": ["DROOP", "SPECTRE"]}

WINS   = [32, 64, 128, 512, 1024]
KFOLDS = [3, 5, 10]
PCT_SWEEP = list(range(10, 101, 10))

OVERLAP_RATIO       = 0.50
OVERLAP_RATIO_DROOP = 0.80

ROBUST_WINSOR = (2.0, 98.0)
SEED = 1337

GC_EVERY_N_PCTS   = 3
GC_EVERY_N_GROUPS = 1

META = ["label", "setup", "run_id"]
DROOP_META_COLS = ["droop_center_found", "droop_best_score_z", "droop_frac_ge_thr", "droop_vcols"]

BALANCE_EVAL_FOR_METRICS = True

DROP_LOW_VARIANCE_COLS = True
LOW_VAR_EPS = 1e-10

# DROOP boost (small, optional)
DROOP_BOOST = True
DROOP_BOOST_ALPHA = 0.20
DROOP_TRANSIENT_PAT = re.compile(r"__(drop|range|slope|min|max|std)", re.I)

METHOD_ORDER = ["dC_aJ", "dC_aM", "dE_aJ", "dE_aM"]

# ===============================================================================================
# 2) PLOT CONFIG (draft-style, no overlaps)
# ===============================================================================================
plt.rcParams.update({
    "font.family": "sans-serif",
    "font.size": 12,
    "axes.titlesize": 13.5,
    "axes.labelsize": 14,
    "xtick.labelsize": 12,
    "ytick.labelsize": 12,
    "legend.fontsize": 12,
    "figure.titlesize": 20,
})

METHOD_TITLE = {
    "dC_aJ": r"Scoring function $d_C(X)$ with Aggregate $a_J$",
    "dC_aM": r"Scoring function $d_C(X)$ with Aggregate $a_M$",
    "dE_aJ": r"Scoring function $d_E(X)$ with Aggregate $a_J$",
    "dE_aM": r"Scoring function $d_E(X)$ with Aggregate $a_M$",
}

PLOT_YMIN = 0.0
PLOT_YMAX = 1.02
ALPHA_MINMAX = 0.28
ALPHA_IQR    = 0.60
GRID_ALPHA   = 0.22
GRID_LS      = "--"
GRID_LW      = 0.6

FIGSIZE = (13.6, 8.4)
WSPACE  = 0.30
HSPACE  = 0.42
TOP     = 0.88
BOTTOM  = 0.20
LEFT    = 0.075
RIGHT   = 0.985
SUPTITLE_Y = 0.975
LEGEND_Y   = 0.055

BAND_EPS = 0.008
def _ensure_visible_band(lo: np.ndarray, hi: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    lo = lo.astype(float).copy()
    hi = hi.astype(float).copy()
    flat = (np.abs(hi - lo) < 1e-12)
    if np.any(flat):
        lo2 = lo - BAND_EPS/2
        hi2 = hi + BAND_EPS/2
        lo[flat] = np.clip(lo2[flat], 0.0, 1.0)
        hi[flat] = np.clip(hi2[flat], 0.0, 1.02)
    return lo, hi

# ===============================================================================================
# 3) WEIGHT CANDIDATES (Table-3 + extras) + LOOKUP
# ===============================================================================================
def build_weight_table() -> pd.DataFrame:
    # Paper Table 3 is (wM,wC,wS) -> store as (wC,wM,wS)
    paper_cases = [
        ("T3_01", "Case 1",  1,    0,    0),
        ("T3_02", "Case 2",  0,    1,    0),
        ("T3_03", "Case 3",  0,    0,    1),
        ("T3_04", "Case 4",  1/3,  1/3,  1/3),
        ("T3_05", "Case 5",  1/4,  1/4,  2/4),
        ("T3_06", "Case 6",  1/5,  1/5,  3/5),
        ("T3_07", "Case 7",  1/6,  1/6,  4/6),
        ("T3_08", "Case 8",  1/8,  2/8,  5/8),
        ("T3_09", "Case 9",  1/8,  1/8,  6/8),
        ("T3_10", "Case 10", 1/10, 1/10, 8/10),
        ("T3_11", "Case 11", 2/3,  1/3,  1/3),
        ("T3_12", "Case 12", 3/4,  1/4,  1/4),
        ("T3_13", "Case 13", 5/8,  2/8,  1/8),
        ("T3_14", "Case 14", 6/8,  1/8,  1/8),
        ("T3_15", "Case 15", 1/20, 1/20, 18/20),
        ("T3_16", "Case 16", 1/40, 1/40, 38/40),
    ]
    rows = []
    for cid, name, wM, wC, wS in paper_cases:
        rows.append({
            "weight_case_id": cid,
            "weight_case_name": name,
            "wC": float(wC),
            "wM": float(wM),
            "wS": float(wS),
            "source": "Table3",
        })

    extras = [
        ("EX_01", "SensorOnly",   0.0, 0.0, 1.0),
        ("EX_02", "Sensor90",     0.05, 0.05, 0.90),
        ("EX_03", "Sensor95",     0.025, 0.025, 0.95),
        ("EX_04", "Sensor98",     0.01, 0.01, 0.98),
    ]
    for cid, name, wC, wM, wS in extras:
        rows.append({
            "weight_case_id": cid,
            "weight_case_name": name,
            "wC": float(wC),
            "wM": float(wM),
            "wS": float(wS),
            "source": "Extra",
        })

    df = pd.DataFrame(rows)
    df["_key"] = df.apply(lambda r: (round(r["wC"], 6), round(r["wM"], 6), round(r["wS"], 6)), axis=1)
    df = df.drop_duplicates("_key").drop(columns=["_key"]).reset_index(drop=True)
    return df

WEIGHT_TABLE = build_weight_table()

def _weight_lookup(w: Tuple[float,float,float]) -> Tuple[str,str,str]:
    wC,wM,wS = (round(float(w[0]),6), round(float(w[1]),6), round(float(w[2]),6))
    m = WEIGHT_TABLE[(WEIGHT_TABLE["wC"].round(6)==wC) &
                     (WEIGHT_TABLE["wM"].round(6)==wM) &
                     (WEIGHT_TABLE["wS"].round(6)==wS)]
    if len(m)>0:
        r = m.iloc[0]
        return str(r["weight_case_id"]), str(r["weight_case_name"]), str(r["source"])
    return ("CUSTOM","Custom","Search")

# ===============================================================================================
# 4) IO + workload parsing
# ===============================================================================================
RUNID2PATH: Dict[str,str] = {}

def detect_setup_from_path(p: Path):
    s = str(p).lower()
    if "ddr4" in s: return "DDR4"
    if "ddr5" in s: return "DDR5"
    return None

def is_benign_path(p: Path) -> bool:
    s = str(p).lower()
    if "benign" in s:
        return True
    bad = ["attack","anom","fault","inject","trojan","mal","rh","droop","spectre","trrespass"]
    return not any(b in s for b in bad)

def is_anomaly_path(p: Path, anomaly: str) -> bool:
    return (anomaly.lower() in str(p).lower()) and (not is_benign_path(p))

def iter_raw_csvs(root: Path):
    for p in root.rglob("*.csv"):
        yield p

def read_csv_clean(p: Path) -> pd.DataFrame:
    df = pd.read_csv(p)
    return df.loc[:, ~df.columns.str.startswith("Unnamed")]

def mk_run_id(path: Path) -> str:
    rid = f"run_{hashlib.md5(str(path).encode('utf-8')).hexdigest()[:10]}"
    RUNID2PATH[rid] = str(path)
    return rid

_WORKLOADS = ["dft","dj","dp","gl","gs","ha","ja","mm","ni","oe","pi","sh","tr"]
def workload_from_path(p: Path) -> str:
    stem = (p.stem or "").lower()
    m = re.search(r"_([a-z]{2,3})$", stem)
    if m and m.group(1) in _WORKLOADS:
        return m.group(1).upper()
    for tok in _WORKLOADS:
        if tok in stem:
            return tok.upper()
    return "UNK"

def collect_raw_pairs_by_setup(data_dir: Path, which: str, anomaly: Optional[str] = None):
    out = {"DDR4": [], "DDR5": []}
    for p in iter_raw_csvs(data_dir):
        setup = detect_setup_from_path(p)
        if setup is None:
            continue
        try:
            if which == "benign":
                if not is_benign_path(p):
                    continue
            else:
                if anomaly is None or not is_anomaly_path(p, anomaly):
                    continue
            df = read_csv_clean(p)
            out[setup].append((p, df))
        except Exception as e:
            print(f"[WARN] read failed {p}: {e}")
    return out

def telemetry_cols(df: pd.DataFrame):
    exclude = set(META) | set(DROOP_META_COLS) | {"workload"}
    return [c for c in df.columns if (c not in exclude) and (df[c].dtype.kind in "fcbiu")]

def drop_low_variance_cols(df: pd.DataFrame, cols: List[str], eps: float = 1e-10) -> List[str]:
    v = df[cols].astype(float).var(axis=0, ddof=0)
    keep = v[v > eps].index.tolist()
    return keep

# ===============================================================================================
# 5) Scaling
# ===============================================================================================
def robust_scale_train(Xb_np: np.ndarray, winsor=(2.0, 98.0)):
    Q1, Q2 = np.percentile(Xb_np, winsor[0], axis=0), np.percentile(Xb_np, winsor[1], axis=0)
    Xb_clip = np.clip(Xb_np, Q1, Q2)
    mu = Xb_clip.mean(axis=0)
    sd = Xb_clip.std(axis=0, ddof=0) + 1e-9
    return mu, sd, Q1, Q2

def apply_robust_scale(X: pd.DataFrame, mu, sd, Q1, Q2):
    Xc = np.clip(X.to_numpy(dtype=float), Q1, Q2)
    Z  = (Xc - mu) / sd
    return pd.DataFrame(Z, columns=X.columns, index=X.index)

# ===============================================================================================
# 6) Windowing (adds sensor transient feats incl CPU Voltage)
# ===============================================================================================
SENSOR_PAT = re.compile(r"cpu\s*voltage|volt|vdd|vcore|vin|vout|power|energy|joule|current|amps?|temp|thermal|hot", re.I)

def window_collapse_means(df: pd.DataFrame, win: int, setup: str, run_id: str, label: str,
                          overlap_ratio: float, src_path: str="") -> pd.DataFrame:
    cols_all = telemetry_cols(df)
    if not cols_all:
        return pd.DataFrame()

    sensor_cols = [c for c in cols_all if SENSOR_PAT.search(c)]
    if "CPU Voltage" in cols_all and "CPU Voltage" not in sensor_cols:
        sensor_cols.append("CPU Voltage")

    n = len(df)
    stride = max(1, int(round(win * (1 - overlap_ratio))))
    starts = list(range(0, n - win + 1, stride))
    if not starts:
        return pd.DataFrame()

    rows = []
    for start in starts:
        chunk = df.iloc[start:start+win]
        X = chunk[cols_all].astype(float)
        means = X.mean(axis=0, numeric_only=True)

        tfeat = {}
        if sensor_cols:
            Xs = X[sensor_cols]
            mins  = Xs.min(axis=0)
            maxs  = Xs.max(axis=0)
            stds  = Xs.std(axis=0, ddof=0)
            means_s = Xs.mean(axis=0)
            drops  = (means_s - mins)
            ranges = (maxs - mins)
            slope  = (Xs.iloc[-1] - Xs.iloc[0])
            for c in sensor_cols:
                tfeat[f"{c}__min"]   = float(mins[c])
                tfeat[f"{c}__max"]   = float(maxs[c])
                tfeat[f"{c}__std"]   = float(stds[c])
                tfeat[f"{c}__drop"]  = float(drops[c])
                tfeat[f"{c}__range"] = float(ranges[c])
                tfeat[f"{c}__slope"] = float(slope[c])

        row = pd.concat([means, pd.Series(tfeat)]).to_frame().T
        row["setup"]    = setup
        row["run_id"]   = run_id
        row["label"]    = label
        row["workload"] = workload_from_path(Path(src_path))
        rows.append(row)

    return pd.concat(rows, ignore_index=True) if rows else pd.DataFrame()

def build_windowed_raw_means(pairs, setup: str, win: int, label: str, overlap_ratio: float) -> pd.DataFrame:
    out = []
    for p, df in pairs:
        rid = mk_run_id(p)
        agg = window_collapse_means(df, win=win, setup=setup, run_id=rid, label=label,
                                    overlap_ratio=overlap_ratio, src_path=str(p))
        if not agg.empty:
            out.append(agg)
    return pd.concat(out, ignore_index=True) if out else pd.DataFrame()

# ===============================================================================================
# 7) Rank lists + robust mapping + CPU Voltage family append for DROOP
# ===============================================================================================
def _read_rank_feats(p: Path):
    try:
        df = pd.read_csv(p)
    except Exception:
        return []
    if df.empty:
        return []
    col = "feature" if "feature" in df.columns else df.columns[0]
    return df[col].dropna().astype(str).tolist()

def load_full_rank_lists(setup: str, win: int, kfold: int):
    def _find_file(setup, win, kfold, sub):
        fname = f"{setup}_{win}_{kfold}_0_{sub}.csv"
        for d in RANK_DIRS:
            pp = d / fname
            if pp.exists():
                return pp
        return None
    out = {}
    for sub in ["compute","memory","sensors"]:
        pp = _find_file(setup, win, kfold, sub)
        out[sub] = _read_rank_feats(pp) if pp else []
    return out

def slice_by_percent(full_lists: Dict[str, List[str]], pct: int):
    sel = {}
    frac = pct / 100.0
    for sub in ("compute","memory","sensors"):
        feats = full_lists.get(sub, []) or []
        k = int(np.ceil(len(feats) * frac))
        if frac > 0 and len(feats) > 0:
            k = max(1, k)
        sel[sub] = feats[:min(len(feats), max(0, k))]
    return sel

ALIAS_MAP = {
    "voltage": ["cpu","voltage","volt","vcore","vdd","vin","vout"],
    "power": ["power","energy","joules","current","amps"],
    "temperature": ["temp","thermal","hot"],
    "frequency": ["freq","afreq","cfreq","clock","clk","mhz","ghz"],
    "ipc": ["ipc","physipc"],
    "cache": ["l1","l2","l3","hit","miss","evict","fill","mpi"],
    "bandwidth": ["read","write","bw","bandwidth"],
}

def _norm(s: str) -> str:
    s = unicodedata.normalize("NFKC", str(s)).lower()
    out = []
    for ch in s:
        out.append(ch if ch.isalnum() else "_")
    s = "".join(out)
    while "__" in s:
        s = s.replace("__","_")
    return s.strip("_")

def _tokens(s: str):
    t = _norm(s).replace("_"," ").split()
    exp = []
    for tok in t:
        exp.append(tok)
        for k, aliases in ALIAS_MAP.items():
            if tok in aliases:
                exp.append(k)
    return set(exp)

def build_column_index(cols: List[str]):
    norm2orig = {}
    tok_index = {}
    for c in cols:
        nc = _norm(c)
        norm2orig.setdefault(nc, c)
        tok_index[c] = _tokens(c)
    return norm2orig, tok_index

def map_ranks_to_existing(ranked_list: List[str], norm2orig, tok_index):
    mapped, seen = [], set()
    for r in ranked_list:
        nr = _norm(r)
        cand = norm2orig.get(nr, None)
        if cand and cand not in seen:
            mapped.append(cand); seen.add(cand); continue
        rtoks = _tokens(r)
        best_c, best_j = None, 0.0
        for c, ctoks in tok_index.items():
            inter = len(rtoks & ctoks)
            union = len(rtoks | ctoks) or 1
            j = inter / union
            if j > best_j:
                best_j, best_c = j, c
        if best_c and best_j >= 0.60 and best_c not in seen:
            mapped.append(best_c); seen.add(best_c)
    return mapped

def intersect_selection_with_columns_robust(sel_raw: Dict[str, List[str]], xb_cols: set):
    norm2orig, tok_index = build_column_index(list(xb_cols))
    out = {}
    for sub in ("compute","memory","sensors"):
        ranked = sel_raw.get(sub, []) or []
        mapped = map_ranks_to_existing(ranked, norm2orig, tok_index)
        out[sub] = [c for c in mapped if c in xb_cols]
    return out

def force_cpu_voltage_family(sel: Dict[str, List[str]], xb_cols: set) -> Dict[str, List[str]]:
    out = dict(sel)
    sensors_now = out.get("sensors", []) or []
    must = []
    if "CPU Voltage" in xb_cols:
        must.append("CPU Voltage")
    for suf in ["__min","__max","__std","__drop","__range","__slope"]:
        c = f"CPU Voltage{suf}"
        if c in xb_cols:
            must.append(c)
    out["sensors"] = list(dict.fromkeys(sensors_now + must))
    return out

# ===============================================================================================
# 8) Balanced evaluation helper  ✅ ADDED (fixes NameError)
# ===============================================================================================
def balanced_concat(df_ben: pd.DataFrame, df_anom: pd.DataFrame, seed: int = 1337):
    """
    Create a balanced evaluation set by sampling the same number of windows from BENIGN and ANOM.
    Returns:
      df_eval : concatenated dataframe (benign first, anomaly second)
      y_true  : numpy array labels (0 benign, 1 anomaly) aligned with df_eval rows
    """
    if df_ben is None or df_ben.empty:
        raise ValueError("balanced_concat(): df_ben is empty")
    if df_anom is None or df_anom.empty:
        raise ValueError("balanced_concat(): df_anom is empty")

    rng = np.random.default_rng(int(seed))

    nb = int(len(df_ben))
    na = int(len(df_anom))
    n  = int(min(nb, na))

    # sample without replacement if possible; otherwise sample with replacement
    replace_b = nb < n
    replace_a = na < n

    idx_b = rng.choice(nb, size=n, replace=replace_b)
    idx_a = rng.choice(na, size=n, replace=replace_a)

    ben_s = df_ben.iloc[idx_b].copy().reset_index(drop=True)
    anm_s = df_anom.iloc[idx_a].copy().reset_index(drop=True)

    df_eval = pd.concat([ben_s, anm_s], ignore_index=True)
    y_true  = np.concatenate([np.zeros(len(ben_s), dtype=int),
                              np.ones(len(anm_s), dtype=int)])
    return df_eval, y_true

# ===============================================================================================
# 9) Weight selection helpers
# ===============================================================================================
def split_holdout(y, test_size=0.3, seed=SEED):
    sss = StratifiedShuffleSplit(n_splits=1, test_size=test_size, random_state=seed)
    idx = np.arange(len(y))
    _, val_idx = next(sss.split(idx, y))
    return val_idx

def pick_best_w_per_method_PRfirst(y_true: np.ndarray, scores_by_w: Dict[Tuple[float,float,float], Dict[str,np.ndarray]], val_idx: np.ndarray):
    yy = y_true[val_idx]
    best = {m: (-1.0, -1.0, -1.0, None) for m in ["dE_aM","dE_aJ","dC_aM","dC_aJ"]}
    for w, md in scores_by_w.items():
        wS = float(w[2])
        for m, s in md.items():
            sv = np.asarray(s)[val_idx]
            pr  = float(average_precision_score(yy, sv))
            roc = float(roc_auc_score(yy, sv))
            key = (pr, roc, wS)
            if key > best[m][:3]:
                best[m] = (pr, roc, wS, w)
    return {m: best[m][3] for m in best}

# ===============================================================================================
# 10) Scoring (draft math)
# ===============================================================================================
def _build_refs(Xb_base: pd.DataFrame, sel: Dict[str, List[str]], eps: float = 1e-12):
    refs = {}
    for sub in ("compute","memory","sensors"):
        cols = [c for c in (sel.get(sub) or []) if c in Xb_base.columns]
        if not cols:
            refs[sub] = None
            continue
        X = Xb_base[cols].astype(float).values
        mu = X.mean(axis=0)
        var = X.var(axis=0, ddof=0)
        var = np.where(var <= eps, eps, var)
        w = 1.0 / var
        denomC = float(np.sum((w * mu) ** 2))
        refs[sub] = {"cols": cols, "mu": mu, "w": w, "denomC": max(denomC, eps)}
    return refs

def _score_dE_parts(Xe_base: pd.DataFrame, refs: dict) -> Dict[str, np.ndarray]:
    n = len(Xe_base)
    out = {}
    for sub in ("compute","memory","sensors"):
        r = refs.get(sub)
        if r is None:
            out[sub] = np.zeros(n)
            continue
        X = Xe_base[r["cols"]].astype(float).values
        D = X - r["mu"]
        s = np.sum((D * D) * r["w"], axis=1)
        out[sub] = np.log1p(np.maximum(s, 0.0))
    return out

def _score_dC_parts(Xe_base: pd.DataFrame, refs: dict, eps: float = 1e-12) -> Dict[str, np.ndarray]:
    n = len(Xe_base)
    out = {}
    for sub in ("compute","memory","sensors"):
        r = refs.get(sub)
        if r is None:
            out[sub] = np.zeros(n)
            continue
        X = Xe_base[r["cols"]].astype(float).values
        w = r["w"]
        mu = r["mu"]
        term1 = np.sum(w * (X * X), axis=1)
        dot   = np.sum(X * (w * mu), axis=1)
        term2 = (dot * dot) / max(r["denomC"], eps)
        s = term1 - term2
        out[sub] = np.log1p(np.maximum(s, 0.0))
    return out

def fit_parts_normalizer_on_benign(Xb_base: pd.DataFrame, refs: dict) -> dict:
    eps = 1e-9
    partsE_b = _score_dE_parts(Xb_base, refs)
    partsC_b = _score_dC_parts(Xb_base, refs)
    norm = {"dE": {}, "dC": {}}
    for sub in ("compute","memory","sensors"):
        vE = np.asarray(partsE_b[sub], float)
        vC = np.asarray(partsC_b[sub], float)
        norm["dE"][sub] = (float(np.nanmean(vE)), float(np.nanstd(vE) + eps))
        norm["dC"][sub] = (float(np.nanmean(vC)), float(np.nanstd(vC) + eps))
    return norm

def apply_parts_normalizer(parts: Dict[str, np.ndarray], norm_sub: dict) -> Dict[str, np.ndarray]:
    out = {}
    for sub, v in parts.items():
        mu, sd = norm_sub.get(sub, (0.0, 1.0))
        v = np.asarray(v, float)
        out[sub] = (v - mu) / (sd if sd > 0 else 1.0)
    return out

def _aggregate_aM(parts: Dict[str, np.ndarray], weights: Tuple[float,float,float]) -> np.ndarray:
    wC, wM, wS = map(float, weights)
    den = (wC + wM + wS) if (wC + wM + wS) > 0 else 1.0
    return (wC*parts["compute"] + wM*parts["memory"] + wS*parts["sensors"]) / den

def _aggregate_aJ(parts: Dict[str, np.ndarray], weights: Tuple[float,float,float], eps: float = 1e-12) -> np.ndarray:
    wC, wM, wS = map(float, weights)
    def softplus(x):
        x = np.asarray(x, float)
        return np.log1p(np.exp(-np.abs(x))) + np.maximum(x, 0.0)
    sC = np.maximum(softplus(parts["compute"]), eps)
    sM = np.maximum(softplus(parts["memory"]),  eps)
    sS = np.maximum(softplus(parts["sensors"]), eps)
    return np.exp(wC*np.log(sC) + wM*np.log(sM) + wS*np.log(sS))

def score_four_methods_benign_norm(Xe_base: pd.DataFrame, refs: dict, weights: Tuple[float,float,float], benign_norm: dict):
    parts_E = apply_parts_normalizer(_score_dE_parts(Xe_base, refs), benign_norm["dE"])
    parts_C = apply_parts_normalizer(_score_dC_parts(Xe_base, refs), benign_norm["dC"])
    return {
        "dE_aM": _aggregate_aM(parts_E, weights),
        "dE_aJ": _aggregate_aJ(parts_E, weights),
        "dC_aM": _aggregate_aM(parts_C, weights),
        "dC_aJ": _aggregate_aJ(parts_C, weights),
    }

def droop_boost_score(Xe_base: pd.DataFrame, sel: Dict[str, List[str]]) -> np.ndarray:
    cols = [c for c in (sel.get("sensors") or []) if ("CPU Voltage" in c and DROOP_TRANSIENT_PAT.search(c))]
    if not cols:
        return np.zeros(len(Xe_base))
    Z = Xe_base[cols].astype(float).to_numpy()
    return np.log1p(np.mean(np.abs(Z), axis=1))

# ===============================================================================================
# 11) DSE plots
# ===============================================================================================
def summarize_for_dse(per_run_df: pd.DataFrame) -> pd.DataFrame:
    keys = ["setup","anomaly","win","kfold","method","pct"]
    def q1(x): return float(np.nanpercentile(x, 25))
    def q3(x): return float(np.nanpercentile(x, 75))
    g = (per_run_df.groupby(keys, as_index=False)
            .agg(
                roc_auc_median=("roc_auc","median"),
                roc_auc_q1=("roc_auc", q1),
                roc_auc_q3=("roc_auc", q3),
                roc_auc_min=("roc_auc","min"),
                roc_auc_max=("roc_auc","max"),
                auc_pr_median=("auc_pr","median"),
                auc_pr_q1=("auc_pr", q1),
                auc_pr_q3=("auc_pr", q3),
                auc_pr_min=("auc_pr","min"),
                auc_pr_max=("auc_pr","max"),
                n_runs=("run_id","nunique"),
            ))
    return g.sort_values(keys).reset_index(drop=True)

def plot_dse_grid(summary_case: pd.DataFrame, metric: str, setup: str, anomaly: str,
                  win: int, kfold: int, out_dir: Path):
    out_dir.mkdir(parents=True, exist_ok=True)
    x_ticks = sorted(summary_case["pct"].unique().tolist())
    if not x_ticks:
        return

    fig, axes = plt.subplots(2, 2, figsize=FIGSIZE, sharex=False, sharey=False)
    axes = axes.flatten()
    fig.subplots_adjust(left=LEFT, right=RIGHT, top=TOP, bottom=BOTTOM, wspace=WSPACE, hspace=HSPACE)

    xlab = "Top-ranked features used (%)"
    ylab = "ROC-AUC" if metric == "roc_auc" else "AUC-PR"

    def _clip01(a): return np.clip(np.asarray(a, float), 0.0, 1.0)

    for i, method in enumerate(METHOD_ORDER):
        ax = axes[i]
        ax.set_title(METHOD_TITLE.get(method, method), pad=7)
        ax.set_xlim(min(x_ticks), max(x_ticks))
        ax.set_ylim(PLOT_YMIN, PLOT_YMAX)
        ax.set_xticks(x_ticks)
        ax.grid(True, alpha=GRID_ALPHA, linestyle=GRID_LS, linewidth=GRID_LW)

        sub = summary_case[summary_case["method"] == method].copy().sort_values("pct")
        if sub.empty:
            ax.text(0.5, 0.5, "No data", ha="center", va="center", transform=ax.transAxes)
        else:
            x = sub["pct"].to_numpy(dtype=float)
            med  = _clip01(sub[f"{metric}_median"].to_numpy(dtype=float))
            q1   = _clip01(sub[f"{metric}_q1"].to_numpy(dtype=float))
            q3   = _clip01(sub[f"{metric}_q3"].to_numpy(dtype=float))
            vmin = _clip01(sub[f"{metric}_min"].to_numpy(dtype=float))
            vmax = _clip01(sub[f"{metric}_max"].to_numpy(dtype=float))

            lo_mm, hi_mm = np.minimum(vmin, vmax), np.maximum(vmin, vmax)
            lo_iq, hi_iq = np.minimum(q1, q3),     np.maximum(q1, q3)
            lo_mm, hi_mm = _ensure_visible_band(lo_mm, hi_mm)
            lo_iq, hi_iq = _ensure_visible_band(lo_iq, hi_iq)

            ax.fill_between(x, lo_mm, hi_mm, alpha=ALPHA_MINMAX, label="min-max", linewidth=0.0)
            ax.fill_between(x, lo_iq, hi_iq, alpha=ALPHA_IQR,   label="IQR",     linewidth=0.0)
            ax.plot(x, med, color="black", marker="o", linewidth=2.0, markersize=5.5, zorder=5, label="Median")

        ax.set_xlabel(xlab, labelpad=6)
        ax.set_ylabel(ylab, labelpad=6)
        ax.tick_params(axis="x", labelbottom=True, pad=3)
        ax.tick_params(axis="y", labelleft=True, pad=3)

    handles, labels = axes[0].get_legend_handles_labels()
    fig.legend(handles, labels, loc="lower center", bbox_to_anchor=(0.5, LEGEND_Y), ncol=3, frameon=False)
    fig.suptitle(f"{setup}: {anomaly} (WIN={win}, K={kfold})", y=SUPTITLE_Y)

    base = f"DSE_{metric.upper()}_{setup}_{anomaly}_WIN{int(win)}_KF{int(kfold)}"
    fig.savefig(out_dir / f"{base}.png", dpi=600, bbox_inches="tight", pad_inches=0.06)
    fig.savefig(out_dir / f"{base}.pdf", bbox_inches="tight", pad_inches=0.06)
    plt.close(fig)

def write_all_dse_plots(per_run_df: pd.DataFrame):
    out_root = RES_DIR / "DesignSpace" / "PaperStyle_ALLRUNS"
    out_root.mkdir(parents=True, exist_ok=True)
    dse = summarize_for_dse(per_run_df)
    for (setup, anomaly, win, kfold), sub in dse.groupby(["setup","anomaly","win","kfold"]):
        out_dir = out_root / str(setup) / str(anomaly) / f"WIN{int(win)}_KF{int(kfold)}"
        plot_dse_grid(sub, "roc_auc", str(setup), str(anomaly), int(win), int(kfold), out_dir)
        plot_dse_grid(sub, "auc_pr",  str(setup), str(anomaly), int(win), int(kfold), out_dir)

# ===============================================================================================
# 12) Subspace-weight tables like your Table 13/14 (per setup,win,kfold)
# ===============================================================================================
def build_workload_weight_tables(per_run_df: pd.DataFrame, setup: str, win: int, kfold: int) -> None:
    base_dir = RES_DIR / "Table_SubspaceWeights" / setup / f"WIN{win}_KF{kfold}"
    base_dir.mkdir(parents=True, exist_ok=True)

    rep_method = "dE_aM"  # for paper-style merged table

    for anomaly in ANOMALIES_BY_SETUP[setup]:
        df = per_run_df[
            (per_run_df["setup"]==setup) &
            (per_run_df["anomaly"]==anomaly) &
            (per_run_df["win"]==win) &
            (per_run_df["kfold"]==kfold)
        ].copy()
        if df.empty:
            continue

        df["_wkey"] = df.apply(lambda r: (round(r["wC"],6), round(r["wM"],6), round(r["wS"],6)), axis=1)

        rows = []
        for (wl, method), g in df.groupby(["workload","method"]):
            mode_key = g["_wkey"].value_counts().idxmax()
            wC, wM, wS = mode_key
            wid, wname, wsrc = _weight_lookup((wC,wM,wS))
            rows.append({
                "workload": wl,
                "method": method,
                "chosen_wC": wC, "chosen_wM": wM, "chosen_wS": wS,
                "weight_case_id": wid,
                "weight_case_name": wname,
                "weight_case_source": wsrc,
                "n_rows": int(len(g)),
                "median_roc": float(np.nanmedian(g["roc_auc"])),
                "median_pr": float(np.nanmedian(g["auc_pr"])),
            })

        df_out = pd.DataFrame(rows).sort_values(["workload","method"]).reset_index(drop=True)
        (base_dir / f"Table_SubspaceWeights_{setup}_WIN{win}_KF{kfold}_{anomaly}.csv").write_text(
            df_out.to_csv(index=False)
        )

    droop = "DROOP"
    other = [a for a in ANOMALIES_BY_SETUP[setup] if a != droop]
    other = other[0] if other else ""

    def _pick_weight_for(wl: str, anom: str) -> Optional[Tuple[float,float,float]]:
        df = per_run_df[
            (per_run_df["setup"]==setup) &
            (per_run_df["anomaly"]==anom) &
            (per_run_df["win"]==win) &
            (per_run_df["kfold"]==kfold) &
            (per_run_df["workload"]==wl) &
            (per_run_df["method"]==rep_method)
        ].copy()
        if df.empty:
            return None
        df["_wkey"] = df.apply(lambda r: (round(r["wC"],6), round(r["wM"],6), round(r["wS"],6)), axis=1)
        return df["_wkey"].value_counts().idxmax()

    all_wl = sorted(per_run_df[
        (per_run_df["setup"]==setup) &
        (per_run_df["win"]==win) &
        (per_run_df["kfold"]==kfold)
    ]["workload"].astype(str).unique().tolist())

    def fmt(w):
        if w is None:
            return ""
        return f"({w[0]:g}, {w[1]:g}, {w[2]:g})"

    rows = []
    for wl in all_wl:
        wd = _pick_weight_for(wl, droop)
        wo = _pick_weight_for(wl, other) if other else None
        rows.append({
            "Workload": wl,
            droop: fmt(wd),
            other if other else "Other": fmt(wo),
        })

    paper_out = pd.DataFrame(rows)
    (base_dir / f"Table_SubspaceWeights_{setup}_WIN{win}_KF{kfold}_PAPERSTYLE.csv").write_text(
        paper_out.to_csv(index=False)
    )

# ===============================================================================================
# 13) Pipeline core
# ===============================================================================================
def run_full_pipeline() -> pd.DataFrame:
    all_rows = []
    group_counter = 0

    benign_pairs = collect_raw_pairs_by_setup(DATA_DIR, which="benign")
    anomaly_pairs_cache: Dict[Tuple[str,str], List[Tuple[Path,pd.DataFrame]]] = {}

    def get_anom_pairs(setup: str, anomaly: str):
        key = (setup, anomaly.upper())
        if key not in anomaly_pairs_cache:
            anomaly_pairs_cache[key] = collect_raw_pairs_by_setup(DATA_DIR, which="anomaly", anomaly=anomaly)[setup]
        return anomaly_pairs_cache[key]

    for setup in SETUPS:
        anomalies = ANOMALIES_BY_SETUP.get(setup, [])
        ben_pairs_all = benign_pairs.get(setup, [])
        if not ben_pairs_all:
            print(f"[SKIP] {setup}: no benign RAW files")
            continue

        for win in WINS:
            df_ben_all = build_windowed_raw_means(ben_pairs_all, setup=setup, win=win, label="BENIGN",
                                                  overlap_ratio=OVERLAP_RATIO)
            if df_ben_all.empty:
                print(f"[SKIP] {setup} WIN={win}: benign windowed empty")
                continue

            Xb_cols = telemetry_cols(df_ben_all)
            if DROP_LOW_VARIANCE_COLS:
                Xb_cols = drop_low_variance_cols(df_ben_all, Xb_cols, eps=LOW_VAR_EPS)
            if not Xb_cols:
                print(f"[SKIP] {setup} WIN={win}: no telemetry cols after low-var drop")
                continue

            Xb = df_ben_all[Xb_cols].astype(float)
            mu_s, sd_s, Q1, Q2 = robust_scale_train(Xb.values, winsor=ROBUST_WINSOR)
            Xb_base = apply_robust_scale(Xb, mu_s, sd_s, Q1, Q2)
            xb_cols_set = set(Xb_base.columns)

            ben_by_wl = {}
            for wl in df_ben_all["workload"].astype(str).unique():
                ben_by_wl[wl] = df_ben_all[df_ben_all["workload"].astype(str)==wl].copy()

            for kfold in KFOLDS:
                # write candidates table once per setup/win/kfold
                wdir = RES_DIR / "Table_SubspaceWeights" / setup / f"WIN{win}_KF{kfold}"
                wdir.mkdir(parents=True, exist_ok=True)
                WEIGHT_TABLE.to_csv(wdir / "Table_SubspaceWeights_CANDIDATES.csv", index=False)

                full_lists = load_full_rank_lists(setup, win, kfold)

                for anomaly in anomalies:
                    group_counter += 1
                    print(f"\n[RUN] {setup} {anomaly} WIN={win} KF={kfold} (group {group_counter})")

                    overlap = OVERLAP_RATIO_DROOP if anomaly.upper() == "DROOP" else OVERLAP_RATIO
                    an_pairs_all = get_anom_pairs(setup, anomaly)
                    if not an_pairs_all:
                        print(f"[SKIP] {setup} {anomaly}: no anomaly RAW files")
                        continue

                    df_anom = build_windowed_raw_means(an_pairs_all, setup=setup, win=win, label=anomaly,
                                                       overlap_ratio=overlap)
                    if df_anom.empty:
                        print(f"[SKIP] {setup} {anomaly} WIN={win}: anomaly windowed empty")
                        continue

                    run_ids = sorted(df_anom["run_id"].astype(str).unique())

                    for rid in run_ids:
                        dfa_run = df_anom[df_anom["run_id"].astype(str) == rid].copy()
                        if dfa_run.empty:
                            continue

                        wl = str(dfa_run["workload"].iloc[0]) if "workload" in dfa_run.columns else "UNK"
                        df_ben = ben_by_wl.get(wl, df_ben_all)
                        if df_ben.empty:
                            df_ben = df_ben_all

                        # balanced evaluation ✅ (now defined)
                        df_eval, y_true = balanced_concat(df_ben, dfa_run, seed=SEED)

                        # Use only columns that exist in eval (prevents KeyError if any mismatch)
                        use_cols = [c for c in Xb_cols if c in df_eval.columns]
                        if not use_cols:
                            continue

                        Xe_all  = df_eval[use_cols].astype(float)
                        Xe_base = apply_robust_scale(Xe_all, mu_s, sd_s, Q1, Q2)

                        for idx_p, pct in enumerate(PCT_SWEEP, start=1):
                            sel_raw = slice_by_percent(full_lists, pct)
                            sel = intersect_selection_with_columns_robust(sel_raw, xb_cols_set)

                            if (not sel.get("compute")) or (not sel.get("memory")) or (not sel.get("sensors")):
                                continue

                            if anomaly.upper() == "DROOP":
                                sel = force_cpu_voltage_family(sel, xb_cols_set)

                            refs = _build_refs(Xb_base, sel, eps=1e-12)
                            benign_norm = fit_parts_normalizer_on_benign(Xb_base, refs)

                            val_idx = split_holdout(y_true, test_size=0.3, seed=SEED)

                            scores_by_w = {}
                            for _, wr in WEIGHT_TABLE.iterrows():
                                w = (float(wr["wC"]), float(wr["wM"]), float(wr["wS"]))
                                scores_by_w[w] = score_four_methods_benign_norm(Xe_base, refs, w, benign_norm)

                            best_w_for = pick_best_w_per_method_PRfirst(y_true, scores_by_w, val_idx)

                            for method in METHOD_ORDER:
                                w_star = best_w_for[method]
                                y_score = score_four_methods_benign_norm(Xe_base, refs, w_star, benign_norm)[method]
                                y_score = np.nan_to_num(y_score, nan=0.0, posinf=0.0, neginf=0.0)

                                if DROOP_BOOST and anomaly.upper() == "DROOP":
                                    boost = droop_boost_score(Xe_base, sel)
                                    y_score = (1.0 - DROOP_BOOST_ALPHA) * y_score + DROOP_BOOST_ALPHA * boost

                                roc = float(np.clip(roc_auc_score(y_true, y_score), 0.0, 1.0))
                                pr  = float(np.clip(average_precision_score(y_true, y_score), 0.0, 1.0))

                                wid, wname, wsrc = _weight_lookup(w_star)

                                all_rows.append({
                                    "setup": setup,
                                    "anomaly": anomaly,
                                    "win": int(win),
                                    "kfold": int(kfold),
                                    "pct": int(pct),
                                    "run_id": str(rid),
                                    "workload": str(wl),
                                    "method": method,
                                    "roc_auc": roc,
                                    "auc_pr": pr,
                                    "weight_case_id": wid,
                                    "weight_case_name": wname,
                                    "weight_case_source": wsrc,
                                    "wC": float(w_star[0]),
                                    "wM": float(w_star[1]),
                                    "wS": float(w_star[2]),
                                })

                            if idx_p % GC_EVERY_N_PCTS == 0:
                                gc.collect()

                    if group_counter % GC_EVERY_N_GROUPS == 0:
                        gc.collect()

    per_run_df = pd.DataFrame(all_rows)

    out_all = RES_DIR / "per_run_metrics_all_PIPELINE.csv"
    per_run_df.to_csv(out_all, index=False)
    print(f"\n[OK] ALLRUNS metrics → {out_all}")

    # Write workload weight tables per setup/win/kfold
    for setup in SETUPS:
        for win in WINS:
            for kfold in KFOLDS:
                build_workload_weight_tables(per_run_df, setup=setup, win=win, kfold=kfold)

    # Write all DSE plots
    write_all_dse_plots(per_run_df)

    return per_run_df

# ===============================================================================================
# 14) MAIN
# ===============================================================================================
def main():
    run_full_pipeline()
    print("[DONE] Pipeline + DSE plots + weight tables per platform")

if __name__ == "__main__":
    main()

[WARN] Cannot write to /Volumes/Untitled/octaneX_results. Using local: /Users/hsiaopingni/octaneX_v7_4functions/Results

[RUN] DDR4 DROOP WIN=32 KF=3 (group 1)

[RUN] DDR4 RH WIN=32 KF=3 (group 2)

[RUN] DDR4 DROOP WIN=32 KF=5 (group 3)

[RUN] DDR4 RH WIN=32 KF=5 (group 4)

[RUN] DDR4 DROOP WIN=32 KF=10 (group 5)

[RUN] DDR4 RH WIN=32 KF=10 (group 6)

[RUN] DDR4 DROOP WIN=64 KF=3 (group 7)

[RUN] DDR4 RH WIN=64 KF=3 (group 8)

[RUN] DDR4 DROOP WIN=64 KF=5 (group 9)

[RUN] DDR4 RH WIN=64 KF=5 (group 10)

[RUN] DDR4 DROOP WIN=64 KF=10 (group 11)

[RUN] DDR4 RH WIN=64 KF=10 (group 12)

[RUN] DDR4 DROOP WIN=128 KF=3 (group 13)

[RUN] DDR4 RH WIN=128 KF=3 (group 14)

[RUN] DDR4 DROOP WIN=128 KF=5 (group 15)

[RUN] DDR4 RH WIN=128 KF=5 (group 16)

[RUN] DDR4 DROOP WIN=128 KF=10 (group 17)

[RUN] DDR4 RH WIN=128 KF=10 (group 18)

[RUN] DDR4 DROOP WIN=512 KF=3 (group 19)

[RUN] DDR4 RH WIN=512 KF=3 (group 20)

[RUN] DDR4 DROOP WIN=512 KF=5 (group 21)

[RUN] DDR4 RH WIN=512 KF=5 (group 22)

[RU

## Find BEST (win, kfold, pct) across full 10–100% sweep

In [7]:
# === Cell: Find BEST (win, kfold, pct) from NEW pipeline outputs ===========
import pathlib as pl
import pandas as pd
import numpy as np

ROOT    = pl.Path("/Users/hsiaopingni/octaneX_v7_4functions")
RES_DIR = ROOT / "Results"

PER_RUN = RES_DIR / "per_run_metrics_all_PIPELINE.csv"
if not PER_RUN.exists():
    raise FileNotFoundError(f"Missing per-run CSV from pipeline: {PER_RUN}")

df = pd.read_csv(PER_RUN).copy()

# Defensive: keep only the grid you actually run
valid_wins   = {32, 64, 128, 512, 1024}
valid_kfolds = {3, 5, 10}
df = df[df["win"].isin(valid_wins) & df["kfold"].isin(valid_kfolds)].copy()

# Fill NaNs for robust sorting
df["auc_pr_filled"]  = df["auc_pr"].fillna(-np.inf)
df["roc_auc_filled"] = df["roc_auc"].fillna(-np.inf)

# ---------------------------------------------------------------------------
# 0) Build a SUMMARY table from per-run (median/IQR/min/max across run_id)
#    This replaces your old per_workload_summary...csv
# ---------------------------------------------------------------------------
def q1(x): return float(np.nanpercentile(x, 25))
def q3(x): return float(np.nanpercentile(x, 75))

# Summary per (setup, anomaly, win, kfold, method, pct)
sum_keys = ["setup","anomaly","win","kfold","method","pct"]
sumdf = (df.groupby(sum_keys, as_index=False)
           .agg(
               auc_pr_median=("auc_pr", "median"),
               auc_pr_q1=("auc_pr", q1),
               auc_pr_q3=("auc_pr", q3),
               auc_pr_min=("auc_pr", "min"),
               auc_pr_max=("auc_pr", "max"),
               roc_auc_median=("roc_auc", "median"),
               roc_auc_q1=("roc_auc", q1),
               roc_auc_q3=("roc_auc", q3),
               roc_auc_min=("roc_auc", "min"),
               roc_auc_max=("roc_auc", "max"),
               n_runs=("run_id","nunique"),
           ))

sumdf["auc_pr_iqr"]  = sumdf["auc_pr_q3"]  - sumdf["auc_pr_q1"]
sumdf["roc_auc_iqr"] = sumdf["roc_auc_q3"] - sumdf["roc_auc_q1"]

# For sorting
sumdf["auc_pr_median_filled"]  = sumdf["auc_pr_median"].fillna(-np.inf)
sumdf["roc_auc_median_filled"] = sumdf["roc_auc_median"].fillna(-np.inf)

# Save summary (optional but useful)
SUMMARY_OUT = RES_DIR / "per_case_summary_from_PIPELINE.csv"
sumdf.to_csv(SUMMARY_OUT, index=False)
print(f"[OK] Wrote summary derived from pipeline → {SUMMARY_OUT}")

ANOMALIES_BY_SETUP = {"DDR4": ["DROOP","RH"], "DDR5": ["DROOP","SPECTRE"]}
METHOD_ORDER = ["dC_aJ", "dC_aM", "dE_aJ", "dE_aM"]

# ---------------------------------------------------------------------------
# A) Best (win,kfold,pct) per TEST CASE per METHOD
#    (setup, anomaly, method)  across whole sweep
#    Sort: AUCPR_median desc, then ROC_median desc, then smaller pct
# ---------------------------------------------------------------------------
winners_rows = []
print("\n[SCAN-A] Best across all WIN×KF×PCT per (setup, anomaly, method)")
for setup, anomalies in ANOMALIES_BY_SETUP.items():
    for anomaly in anomalies:
        for method in METHOD_ORDER:
            sub = sumdf[
                (sumdf["setup"]==setup) &
                (sumdf["anomaly"]==anomaly) &
                (sumdf["method"]==method)
            ].copy()
            if sub.empty:
                print(f"  [WARN] No rows for {setup}–{anomaly}–{method}")
                continue

            sub = sub.sort_values(
                ["auc_pr_median_filled", "roc_auc_median_filled", "pct"],
                ascending=[False, False, True]
            )
            best = sub.iloc[0]

            winners_rows.append({
                "setup": setup,
                "anomaly": anomaly,
                "method": method,
                "win": int(best["win"]),
                "kfold": int(best["kfold"]),
                "best_pct_by_median": int(best["pct"]),
                "auc_pr_median": float(best["auc_pr_median"]),
                "roc_auc_median": float(best["roc_auc_median"]),
                "auc_pr_iqr": float(best["auc_pr_iqr"]),
                "roc_auc_iqr": float(best["roc_auc_iqr"]),
                "min_auc_pr": float(best["auc_pr_min"]),
                "max_auc_pr": float(best["auc_pr_max"]),
                "min_roc_auc": float(best["roc_auc_min"]),
                "max_roc_auc": float(best["roc_auc_max"]),
                "n_runs": int(best["n_runs"]),
            })

            print(f"  [BEST • {setup}–{anomaly}–{method}] "
                  f"WIN={int(best['win'])}  K={int(best['kfold'])}  %={int(best['pct'])}  "
                  f"AUCPR_med={best['auc_pr_median']:.3f}  ROC_med={best['roc_auc_median']:.3f}")

winners = pd.DataFrame(winners_rows).sort_values(["setup","anomaly","method"])
WINNERS_CSV = RES_DIR / "BEST_in_DesignSpace_Post_per_testcase_PER_METHOD.csv"
winners.to_csv(WINNERS_CSV, index=False)
print(f"\n[OK] Saved per-(setup,anomaly,method) winners → {WINNERS_CSV}")

# ---------------------------------------------------------------------------
# B) For every (setup, anomaly, win, kfold, method), find BEST pct
# ---------------------------------------------------------------------------
best_pct_rows = []
gcols = ["setup","anomaly","win","kfold","method"]
print("\n[SCAN-B] Best PCT per (setup, anomaly, win, kfold, method)")
for keys, g in sumdf.groupby(gcols, dropna=False):
    g = g.sort_values(
        ["auc_pr_median_filled", "roc_auc_median_filled", "pct"],
        ascending=[False, False, True]
    )
    top = g.iloc[0]
    best_pct_rows.append({
        "setup": keys[0], "anomaly": keys[1],
        "win": int(keys[2]), "kfold": int(keys[3]),
        "method": keys[4],
        "best_pct_by_median": int(top["pct"]),
        "auc_pr_median": float(top["auc_pr_median"]),
        "roc_auc_median": float(top["roc_auc_median"]),
        "n_runs": int(top["n_runs"]),
    })
    print(f"  {keys[0]}/{keys[1]}  WIN={int(keys[2])}  K={int(keys[3])}  {keys[4]} "
          f"-> best %={int(top['pct'])}  AUCPR_med={top['auc_pr_median']:.3f}, ROC_med={top['roc_auc_median']:.3f}")

best_pct = pd.DataFrame(best_pct_rows).sort_values(["setup","anomaly","win","kfold","method"])
BEST_PCT_CSV = RES_DIR / "BEST_pct_per_win_kfold_PER_METHOD.csv"
best_pct.to_csv(BEST_PCT_CSV, index=False)
print(f"\n[OK] Saved best pct per (win,kfold,method) → {BEST_PCT_CSV}")

# ---------------------------------------------------------------------------
# C) (Optional) One "headline" BEST per (setup, anomaly) by taking BEST METHOD too
#    Sort: AUCPR_median desc, then ROC_median desc, then smaller pct
# ---------------------------------------------------------------------------
headline_rows = []
print("\n[SCAN-C] Headline BEST per (setup, anomaly) across ALL methods too")
for setup, anomalies in ANOMALIES_BY_SETUP.items():
    for anomaly in anomalies:
        sub = sumdf[(sumdf["setup"]==setup) & (sumdf["anomaly"]==anomaly)].copy()
        if sub.empty:
            print(f"  [WARN] No rows for {setup}–{anomaly}")
            continue
        sub = sub.sort_values(
            ["auc_pr_median_filled", "roc_auc_median_filled", "pct"],
            ascending=[False, False, True]
        )
        best = sub.iloc[0]
        headline_rows.append({
            "setup": setup,
            "anomaly": anomaly,
            "method": str(best["method"]),
            "win": int(best["win"]),
            "kfold": int(best["kfold"]),
            "best_pct_by_median": int(best["pct"]),
            "auc_pr_median": float(best["auc_pr_median"]),
            "roc_auc_median": float(best["roc_auc_median"]),
            "auc_pr_iqr": float(best["auc_pr_iqr"]),
            "roc_auc_iqr": float(best["roc_auc_iqr"]),
            "n_runs": int(best["n_runs"]),
        })
        print(f"  [HEADLINE • {setup}–{anomaly}] "
              f"{best['method']}  WIN={int(best['win'])}  K={int(best['kfold'])}  %={int(best['pct'])}  "
              f"AUCPR_med={best['auc_pr_median']:.3f}  ROC_med={best['roc_auc_median']:.3f}")

headline = pd.DataFrame(headline_rows).sort_values(["setup","anomaly"])
HEADLINE_CSV = RES_DIR / "BEST_in_DesignSpace_Post_HEADLINE_per_testcase.csv"
headline.to_csv(HEADLINE_CSV, index=False)
print(f"\n[OK] Saved headline winners → {HEADLINE_CSV}")

[OK] Wrote summary derived from pipeline → /Users/hsiaopingni/octaneX_v7_4functions/Results/per_case_summary_from_PIPELINE.csv

[SCAN-A] Best across all WIN×KF×PCT per (setup, anomaly, method)
  [BEST • DDR4–DROOP–dC_aJ] WIN=512  K=3  %=10  AUCPR_med=1.000  ROC_med=1.000
  [BEST • DDR4–DROOP–dC_aM] WIN=512  K=3  %=10  AUCPR_med=1.000  ROC_med=1.000
  [BEST • DDR4–DROOP–dE_aJ] WIN=512  K=3  %=10  AUCPR_med=1.000  ROC_med=1.000
  [BEST • DDR4–DROOP–dE_aM] WIN=512  K=3  %=10  AUCPR_med=1.000  ROC_med=1.000
  [BEST • DDR4–RH–dC_aJ] WIN=32  K=3  %=10  AUCPR_med=1.000  ROC_med=1.000
  [BEST • DDR4–RH–dC_aM] WIN=32  K=3  %=10  AUCPR_med=1.000  ROC_med=1.000
  [BEST • DDR4–RH–dE_aJ] WIN=32  K=3  %=10  AUCPR_med=1.000  ROC_med=1.000
  [BEST • DDR4–RH–dE_aM] WIN=32  K=3  %=10  AUCPR_med=1.000  ROC_med=1.000
  [BEST • DDR5–DROOP–dC_aJ] WIN=1024  K=5  %=20  AUCPR_med=0.997  ROC_med=0.997
  [BEST • DDR5–DROOP–dC_aM] WIN=1024  K=5  %=20  AUCPR_med=0.990  ROC_med=0.988
  [BEST • DDR5–DROOP–dE_aJ] WIN

In [8]:
# ---------------------------------------------------------------------------
# C) Pick ONE (win, kfold) per platform (DDR4, DDR5)  ✅ UPDATED FOR NEW PIPELINE
#    Uses your NEW pipeline outputs:
#      - per_run_metrics_all_PIPELINE.csv  (per-run rows)
#      - or BEST_pct_per_win_kfold_PER_METHOD.csv  (if you already created it)
#
#    Criterion (default):
#      1) For each (setup, anomaly, win, kfold, method): pick BEST pct (PR-first, ROC, smaller pct)
#      2) For each (setup, win, kfold): aggregate across anomalies by taking the BEST method per anomaly
#         then average AUCPR across anomalies (tie avg ROC, smaller win, smaller k)
#
#    Outputs:
#      - Results/BEST_win_kfold_per_platform.csv
#      - Results/BEST_pct_for_chosen_win_kfold_per_platform.csv
# ---------------------------------------------------------------------------

import numpy as np
import pandas as pd
from pathlib import Path

ROOT    = Path("/Users/hsiaopingni/octaneX_v7_4functions")
RES_DIR = ROOT / "Results"

PER_RUN = RES_DIR / "per_run_metrics_all_PIPELINE.csv"
if not PER_RUN.exists():
    raise FileNotFoundError(f"Missing pipeline per-run CSV: {PER_RUN}")

df = pd.read_csv(PER_RUN).copy()

ANOMALIES_BY_SETUP = {"DDR4": ["DROOP","RH"], "DDR5": ["DROOP","SPECTRE"]}
METHOD_ORDER = ["dC_aJ", "dC_aM", "dE_aJ", "dE_aM"]

# Optional defensive filters (match pipeline grid)
valid_wins   = {32, 64, 128, 512, 1024}
valid_kfolds = {3, 5, 10}
df = df[df["win"].isin(valid_wins) & df["kfold"].isin(valid_kfolds)].copy()
df = df[df["setup"].isin(["DDR4","DDR5"])].copy()
df = df[df["method"].isin(METHOD_ORDER)].copy()

# Fill for robust ranking
df["auc_pr_filled"]  = pd.to_numeric(df["auc_pr"], errors="coerce").fillna(-np.inf)
df["roc_auc_filled"] = pd.to_numeric(df["roc_auc"], errors="coerce").fillna(-np.inf)

# ---------- 0) Summarize per-case at pct level (median across run_id) ----------
def q1(x): return float(np.nanpercentile(x, 25))
def q3(x): return float(np.nanpercentile(x, 75))

sum_keys = ["setup","anomaly","win","kfold","method","pct"]
sumdf = (df.groupby(sum_keys, as_index=False)
           .agg(
               auc_pr_median=("auc_pr", "median"),
               roc_auc_median=("roc_auc", "median"),
               auc_pr_q1=("auc_pr", q1),
               auc_pr_q3=("auc_pr", q3),
               roc_auc_q1=("roc_auc", q1),
               roc_auc_q3=("roc_auc", q3),
               n_runs=("run_id","nunique"),
           ))

sumdf["auc_pr_median"]  = pd.to_numeric(sumdf["auc_pr_median"], errors="coerce").clip(0.0, 1.0)
sumdf["roc_auc_median"] = pd.to_numeric(sumdf["roc_auc_median"], errors="coerce").clip(0.0, 1.0)

sumdf["auc_pr_median_filled"]  = sumdf["auc_pr_median"].fillna(-np.inf)
sumdf["roc_auc_median_filled"] = sumdf["roc_auc_median"].fillna(-np.inf)

# ---------- 1) For each (setup, anomaly, win, kfold, method): choose best pct ----------
# Sort: AUCPR median desc, then ROC median desc, then smaller pct
best_pct = (sumdf.sort_values(
                ["setup","anomaly","win","kfold","method",
                 "auc_pr_median_filled","roc_auc_median_filled","pct"],
                ascending=[True,True,True,True,True, False,False,True]
            )
            .groupby(["setup","anomaly","win","kfold","method"], as_index=False)
            .head(1)
            .rename(columns={"pct":"best_pct_by_median"}))

# Keep only anomalies defined per platform (defensive)
def _keep_defined_anoms(row):
    setup = str(row["setup"]).upper()
    anom  = str(row["anomaly"]).upper()
    return setup in ANOMALIES_BY_SETUP and anom in [a.upper() for a in ANOMALIES_BY_SETUP[setup]]

best_pct = best_pct[best_pct.apply(_keep_defined_anoms, axis=1)].copy()

# Save (optional, helpful for debugging / later steps)
BEST_PCT_PER_METHOD_CSV = RES_DIR / "BEST_pct_per_win_kfold_PER_METHOD_from_PIPELINE.csv"
best_pct.to_csv(BEST_PCT_PER_METHOD_CSV, index=False)
print(f"[OK] Wrote best_pct (per method) derived from pipeline → {BEST_PCT_PER_METHOD_CSV}")

print("\n[SCAN] Best (WIN, K) per platform by mean AUCPR across anomalies")
platform_rows = []
chosen_rows = []

# ---------- 2) Platform selection logic ----------
for setup in ["DDR4", "DDR5"]:
    sub = best_pct[best_pct["setup"].astype(str).str.upper() == setup].copy()
    if sub.empty:
        print(f"  [WARN] No rows for setup={setup} in derived best_pct table")
        continue

    # For each (setup, anomaly, win, kfold), pick BEST METHOD
    # Sort: AUCPR desc, ROC desc, then a stable method preference order
    method_rank = {m:i for i,m in enumerate(METHOD_ORDER)}
    sub["_mrank"] = sub["method"].map(method_rank).fillna(999).astype(int)

    best_method_per_anom = (sub.sort_values(
                                ["setup","anomaly","win","kfold",
                                 "auc_pr_median_filled","roc_auc_median_filled","_mrank",
                                 "best_pct_by_median"],
                                ascending=[True,True,True,True, False,False, True, True]
                            )
                            .groupby(["setup","anomaly","win","kfold"], as_index=False)
                            .head(1))

    # Aggregate per (win, kfold) across anomalies (mean of anomaly-bests)
    agg = (best_method_per_anom.groupby(["setup","win","kfold"], as_index=False)
                           .agg(mean_auc_pr=("auc_pr_median_filled", "mean"),
                                mean_roc_auc=("roc_auc_median_filled", "mean"),
                                n_anoms=("anomaly", "nunique")))

    # Prefer combos that cover more anomalies, then mean AUCPR, mean ROC, smaller win/k
    agg = agg.sort_values(
        ["n_anoms", "mean_auc_pr", "mean_roc_auc", "win", "kfold"],
        ascending=[False, False, False, True, True]
    )
    best_combo = agg.iloc[0]
    win_best = int(best_combo["win"])
    kf_best  = int(best_combo["kfold"])

    platform_rows.append({
        "setup": setup,
        "win": win_best,
        "kfold": kf_best,
        "mean_auc_pr_across_anomalies": float(best_combo["mean_auc_pr"]),
        "mean_roc_auc_across_anomalies": float(best_combo["mean_roc_auc"]),
        "num_anomalies_covered": int(best_combo["n_anoms"]),
        "aggregation_note": "anomaly-wise best method @ best pct, then mean across anomalies",
    })

    # For chosen (win,kfold), list anomaly-wise best pct + best method + metrics
    chosen_detail = best_method_per_anom[
        (best_method_per_anom["win"] == win_best) &
        (best_method_per_anom["kfold"] == kf_best)
    ].copy()

    # Sort for display: higher AUCPR, higher ROC, smaller pct
    chosen_detail = chosen_detail.sort_values(
        ["auc_pr_median_filled", "roc_auc_median_filled", "best_pct_by_median"],
        ascending=[False, False, True]
    )

    print(f"  [PLATFORM BEST • {setup}]  WIN={win_best}  K={kf_best}  "
          f"mean AUCPR={float(best_combo['mean_auc_pr']):.3f}  mean ROC={float(best_combo['mean_roc_auc']):.3f}  "
          f"(across {int(best_combo['n_anoms'])} anomalies)")

    for _, r in chosen_detail.iterrows():
        pct_best = int(r["best_pct_by_median"])
        aucpr = float(r["auc_pr_median"]) if np.isfinite(r["auc_pr_median"]) else float("nan")
        roc   = float(r["roc_auc_median"]) if np.isfinite(r["roc_auc_median"]) else float("nan")
        method = str(r["method"])
        anom = str(r["anomaly"])
        print(f"      {setup}/{anom}: best method={method}  best %={pct_best}  "
              f"AUCPR_med={aucpr:.3f}  ROC_med={roc:.3f}")

        chosen_rows.append({
            "setup": setup,
            "anomaly": anom,
            "win": win_best,
            "kfold": kf_best,
            "best_method": method,
            "best_pct_by_median": pct_best,
            "auc_pr_median": aucpr,
            "roc_auc_median": roc,
            "n_runs": int(r["n_runs"]) if "n_runs" in r else np.nan,
        })

# ---------- 3) Save outputs ----------
platform_best = pd.DataFrame(platform_rows)
PLATFORM_BEST_CSV = RES_DIR / "BEST_win_kfold_per_platform.csv"
platform_best.to_csv(PLATFORM_BEST_CSV, index=False)

chosen_pct = pd.DataFrame(chosen_rows)
CHOSEN_PCT_CSV = RES_DIR / "BEST_pct_for_chosen_win_kfold_per_platform.csv"
chosen_pct.to_csv(CHOSEN_PCT_CSV, index=False)

print(f"\n[OK] Saved platform winners → {PLATFORM_BEST_CSV}")
print(f"[OK] Saved per-anomaly details for chosen (WIN,K) → {CHOSEN_PCT_CSV}")


[OK] Wrote best_pct (per method) derived from pipeline → /Users/hsiaopingni/octaneX_v7_4functions/Results/BEST_pct_per_win_kfold_PER_METHOD_from_PIPELINE.csv

[SCAN] Best (WIN, K) per platform by mean AUCPR across anomalies
  [PLATFORM BEST • DDR4]  WIN=512  K=3  mean AUCPR=1.000  mean ROC=1.000  (across 2 anomalies)
      DDR4/DROOP: best method=dC_aJ  best %=10  AUCPR_med=1.000  ROC_med=1.000
      DDR4/RH: best method=dC_aJ  best %=10  AUCPR_med=1.000  ROC_med=1.000
  [PLATFORM BEST • DDR5]  WIN=1024  K=5  mean AUCPR=0.999  mean ROC=0.998  (across 2 anomalies)
      DDR5/SPECTRE: best method=dC_aJ  best %=10  AUCPR_med=1.000  ROC_med=1.000
      DDR5/DROOP: best method=dC_aJ  best %=20  AUCPR_med=0.997  ROC_med=0.997

[OK] Saved platform winners → /Users/hsiaopingni/octaneX_v7_4functions/Results/BEST_win_kfold_per_platform.csv
[OK] Saved per-anomaly details for chosen (WIN,K) → /Users/hsiaopingni/octaneX_v7_4functions/Results/BEST_pct_for_chosen_win_kfold_per_platform.csv


In [10]:
# === Generate BEST_in_DesignSpace_Post_per_platform.csv (PER PLATFORM, not per testcase) ===
# Input:  Results/per_run_metrics_all_PIPELINE.csv  (from your new pipeline)
# Output: Results/BEST_in_DesignSpace_Post_per_platform.csv
#         Results/BEST_in_DesignSpace_Post_per_platform_details.csv
#
# What "per platform" means here:
#   For each platform (DDR4, DDR5), pick ONE (WIN, K) that is best on average across that
#   platform's anomalies, where each anomaly contributes its BEST (method, pct) under that (WIN,K)
#   using PR-first (median AUCPR), tie ROC, tie smaller pct.
#
# Notes:
# - Uses medians across run_id (so it's stable).
# - Uses anomalies set:
#     DDR4: DROOP, RH
#     DDR5: DROOP, SPECTRE
# - Uses method order as stable tie-break: dC_aJ, dC_aM, dE_aJ, dE_aM
# --------------------------------------------------------------------------------------------

import numpy as np
import pandas as pd
from pathlib import Path

ROOT = Path("/Users/hsiaopingni/octaneX_v7_4functions")
RES_DIR = ROOT / "Results"
PER_RUN = RES_DIR / "per_run_metrics_all_PIPELINE.csv"

if not PER_RUN.exists():
    raise FileNotFoundError(f"Missing pipeline per-run CSV: {PER_RUN}")

df = pd.read_csv(PER_RUN).copy()

ANOMALIES_BY_SETUP = {"DDR4": ["DROOP","RH"], "DDR5": ["DROOP","SPECTRE"]}
METHOD_ORDER = ["dC_aJ", "dC_aM", "dE_aJ", "dE_aM"]

# Defensive grid (match your pipeline)
valid_wins   = {32, 64, 128, 512, 1024}
valid_kfolds = {3, 5, 10}

# Basic cleaning / typing
df = df[df["setup"].isin(["DDR4","DDR5"])].copy()
df = df[df["win"].isin(valid_wins) & df["kfold"].isin(valid_kfolds)].copy()
df = df[df["method"].isin(METHOD_ORDER)].copy()

for c in ["win","kfold","pct","auc_pr","roc_auc"]:
    df[c] = pd.to_numeric(df[c], errors="coerce")

df = df.dropna(subset=["setup","anomaly","win","kfold","pct"]).copy()
df["auc_pr"]  = pd.to_numeric(df["auc_pr"], errors="coerce").clip(0.0, 1.0)
df["roc_auc"] = pd.to_numeric(df["roc_auc"], errors="coerce").clip(0.0, 1.0)

# -------------------------------------------------------------------
# 1) Summarize per (setup, anomaly, win, kfold, method, pct) over run_id
# -------------------------------------------------------------------
def q1(x): return float(np.nanpercentile(x, 25))
def q3(x): return float(np.nanpercentile(x, 75))

sum_keys = ["setup","anomaly","win","kfold","method","pct"]
sumdf = (df.groupby(sum_keys, as_index=False)
           .agg(
               auc_pr_median=("auc_pr","median"),
               roc_auc_median=("roc_auc","median"),
               auc_pr_q1=("auc_pr", q1),
               auc_pr_q3=("auc_pr", q3),
               roc_auc_q1=("roc_auc", q1),
               roc_auc_q3=("roc_auc", q3),
               auc_pr_min=("auc_pr","min"),
               auc_pr_max=("auc_pr","max"),
               roc_auc_min=("roc_auc","min"),
               roc_auc_max=("roc_auc","max"),
               n_runs=("run_id","nunique"),
           ))
sumdf["auc_pr_iqr"]  = sumdf["auc_pr_q3"]  - sumdf["auc_pr_q1"]
sumdf["roc_auc_iqr"] = sumdf["roc_auc_q3"] - sumdf["roc_auc_q1"]

sumdf["auc_pr_median_filled"]  = sumdf["auc_pr_median"].fillna(-np.inf)
sumdf["roc_auc_median_filled"] = sumdf["roc_auc_median"].fillna(-np.inf)

# Keep only defined anomalies
def _keep_defined(row):
    setup = str(row["setup"]).upper()
    anom  = str(row["anomaly"]).upper()
    return setup in ANOMALIES_BY_SETUP and anom in [a.upper() for a in ANOMALIES_BY_SETUP[setup]]

sumdf = sumdf[sumdf.apply(_keep_defined, axis=1)].copy()

# -------------------------------------------------------------------
# 2) For each (setup, anomaly, win, kfold, method): pick BEST pct
#     PR-first, ROC-second, smaller pct
# -------------------------------------------------------------------
best_pct = (sumdf.sort_values(
                ["setup","anomaly","win","kfold","method",
                 "auc_pr_median_filled","roc_auc_median_filled","pct"],
                ascending=[True,True,True,True,True, False,False, True]
            )
            .groupby(["setup","anomaly","win","kfold","method"], as_index=False)
            .head(1)
            .rename(columns={"pct":"best_pct_by_median"}))

# -------------------------------------------------------------------
# 3) For each (setup, anomaly, win, kfold): pick BEST method (using its BEST pct)
#     PR-first, ROC-second, stable method order, then smaller pct
# -------------------------------------------------------------------
method_rank = {m:i for i,m in enumerate(METHOD_ORDER)}
best_pct["_mrank"] = best_pct["method"].map(method_rank).fillna(999).astype(int)

best_method = (best_pct.sort_values(
                    ["setup","anomaly","win","kfold",
                     "auc_pr_median_filled","roc_auc_median_filled","_mrank","best_pct_by_median"],
                    ascending=[True,True,True,True, False,False, True, True]
               )
               .groupby(["setup","anomaly","win","kfold"], as_index=False)
               .head(1))

# -------------------------------------------------------------------
# 4) For each platform, pick ONE (win,kfold) by averaging anomaly-wise best
#     Prefer combos that cover more anomalies, then mean AUCPR, mean ROC,
#     then smaller win, smaller kfold
# -------------------------------------------------------------------
platform_rows = []
detail_rows = []

print("\n[SCAN] Choosing ONE (WIN,K) per platform (mean AUCPR across anomalies)")

for setup in ["DDR4","DDR5"]:
    sub = best_method[best_method["setup"].astype(str).str.upper()==setup].copy()
    if sub.empty:
        print(f"  [WARN] No rows for platform={setup}")
        continue

    agg = (sub.groupby(["setup","win","kfold"], as_index=False)
             .agg(mean_auc_pr=("auc_pr_median_filled","mean"),
                  mean_roc_auc=("roc_auc_median_filled","mean"),
                  n_anoms=("anomaly","nunique")))

    agg = agg.sort_values(
        ["n_anoms","mean_auc_pr","mean_roc_auc","win","kfold"],
        ascending=[False, False, False, True, True]
    )
    top = agg.iloc[0]
    win_best = int(top["win"])
    kf_best  = int(top["kfold"])

    platform_rows.append({
        "setup": setup,
        "win": win_best,
        "kfold": kf_best,
        "best_pct_by_median": np.nan,  # not single pct at platform-level; varies by anomaly
        "method": "BEST_PER_ANOMALY",  # varies by anomaly
        "auc_pr_median": float(top["mean_auc_pr"]),
        "roc_auc_median": float(top["mean_roc_auc"]),
        "auc_pr_iqr": np.nan,
        "roc_auc_iqr": np.nan,
        "auc_pr_min": np.nan,
        "auc_pr_max": np.nan,
        "roc_auc_min": np.nan,
        "roc_auc_max": np.nan,
        "num_anomalies_covered": int(top["n_anoms"]),
        "selection_note": "mean across anomalies; each anomaly uses its best (method,pct) under (win,kfold)",
    })

    chosen = sub[(sub["win"]==win_best) & (sub["kfold"]==kf_best)].copy()
    chosen = chosen.sort_values(
        ["auc_pr_median_filled","roc_auc_median_filled","best_pct_by_median"],
        ascending=[False, False, True]
    )

    print(f"  [PLATFORM BEST • {setup}] WIN={win_best} K={kf_best} "
          f"mean AUCPR={float(top['mean_auc_pr']):.3f} mean ROC={float(top['mean_roc_auc']):.3f} "
          f"(anoms covered={int(top['n_anoms'])})")

    for _, r in chosen.iterrows():
        detail_rows.append({
            "setup": setup,
            "anomaly": str(r["anomaly"]),
            "win": win_best,
            "kfold": kf_best,
            "best_pct_by_median": int(r["best_pct_by_median"]),
            "method": str(r["method"]),
            "auc_pr_median": float(r["auc_pr_median"]),
            "roc_auc_median": float(r["roc_auc_median"]),
            "auc_pr_iqr": float(r["auc_pr_iqr"]),
            "roc_auc_iqr": float(r["roc_auc_iqr"]),
            "auc_pr_min": float(r["auc_pr_min"]),
            "auc_pr_max": float(r["auc_pr_max"]),
            "roc_auc_min": float(r["roc_auc_min"]),
            "roc_auc_max": float(r["roc_auc_max"]),
            "n_runs": int(r["n_runs"]),
        })
        print(f"      {setup}/{r['anomaly']}: method={r['method']}  best %={int(r['best_pct_by_median'])}  "
              f"AUCPR_med={float(r['auc_pr_median']):.3f} ROC_med={float(r['roc_auc_median']):.3f}")

# -------------------------------------------------------------------
# 5) Save outputs
# -------------------------------------------------------------------
platform_best = pd.DataFrame(platform_rows).sort_values(["setup"]).reset_index(drop=True)
details = pd.DataFrame(detail_rows).sort_values(["setup","anomaly"]).reset_index(drop=True)

OUT_PLATFORM = RES_DIR / "BEST_in_DesignSpace_Post_per_platform.csv"
OUT_DETAILS  = RES_DIR / "BEST_in_DesignSpace_Post_per_platform_details.csv"

platform_best.to_csv(OUT_PLATFORM, index=False)
details.to_csv(OUT_DETAILS, index=False)

print(f"\n[OK] Wrote per-platform winners → {OUT_PLATFORM}")
print(f"[OK] Wrote per-platform anomaly details → {OUT_DETAILS}")

print("\n[PLATFORM WINNERS]")
print(platform_best[["setup","win","kfold","auc_pr_median","roc_auc_median","num_anomalies_covered"]].to_string(index=False))



[SCAN] Choosing ONE (WIN,K) per platform (mean AUCPR across anomalies)
  [PLATFORM BEST • DDR4] WIN=512 K=3 mean AUCPR=1.000 mean ROC=1.000 (anoms covered=2)
      DDR4/DROOP: method=dC_aJ  best %=10  AUCPR_med=1.000 ROC_med=1.000
      DDR4/RH: method=dC_aJ  best %=10  AUCPR_med=1.000 ROC_med=1.000
  [PLATFORM BEST • DDR5] WIN=1024 K=5 mean AUCPR=0.999 mean ROC=0.998 (anoms covered=2)
      DDR5/SPECTRE: method=dC_aJ  best %=10  AUCPR_med=1.000 ROC_med=1.000
      DDR5/DROOP: method=dC_aJ  best %=20  AUCPR_med=0.997 ROC_med=0.997

[OK] Wrote per-platform winners → /Users/hsiaopingni/octaneX_v7_4functions/Results/BEST_in_DesignSpace_Post_per_platform.csv
[OK] Wrote per-platform anomaly details → /Users/hsiaopingni/octaneX_v7_4functions/Results/BEST_in_DesignSpace_Post_per_platform_details.csv

[PLATFORM WINNERS]
setup  win  kfold  auc_pr_median  roc_auc_median  num_anomalies_covered
 DDR4  512      3       1.000000        1.000000                      2
 DDR5 1024      5       0.99853

---

# Explainability

### SHAP explainability for BEST cases only (per test case) 

In [55]:
# === SHAP explainability for BEST cases only (PER PLATFORM) ==========================
# Platform = DDR4, DDR5
#
# Reads winners from:
#   Results/BEST_in_DesignSpace_Post_per_platform_details.csv
#   (generated by your per-platform selector; contains one row per platform×anomaly,
#    with chosen WIN/K plus anomaly-specific best METHOD and best PCT)
#
# Uses your pipeline helpers if present; otherwise uses local rank-reader + simple selection.
#
# Outputs:
#   Results/Explainability_SHAP_BestPlatforms/
#       SHAP_BESTPLAT_full_<setup>_<anomaly>_WIN..._KF..._PCT..._METHOD....csv
#       SHAP_BESTPLAT_<subspace>_...csv
#       figs/BAR_BESTPLAT_<subspace>_...png
#       figs/BEE_BESTPLAT_...png
#       SHAP_BESTPLAT_master_all.csv
# =====================================================================================

import os, gc, re, unicodedata
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt

# ---- Paths ----
ROOT        = Path(globals().get("ROOT", "/Users/hsiaopingni/octaneX_v7_4functions"))
RES_DIR     = Path(globals().get("RES_DIR", ROOT / "Results"))
DATA_DIR    = Path(globals().get("DATA_DIR", "/Users/hsiaopingni/Desktop/SLM_RAS-main/HW_TELEMETRY_DATA_COLLECTION/TELEMETRY_DATA"))

DETAILS_CSV = RES_DIR / "BEST_in_DesignSpace_Post_per_platform_details.csv"
OUT_DIR     = RES_DIR / "Explainability_SHAP_BestPlatforms"
FIG_DIR     = OUT_DIR / "figs"
OUT_DIR.mkdir(parents=True, exist_ok=True)
FIG_DIR.mkdir(parents=True, exist_ok=True)

SUBSPACES = ("compute","memory","sensors")
SEED      = int(globals().get("SEED", 42))

# ---- Require helpers from your pipeline (recommended) ----
_required = [
    "collect_raw_pairs_by_setup","build_windowed_raw_means","telemetry_cols",
    "robust_scale_train","apply_robust_scale",
    "slice_by_percent","intersect_selection_with_columns_robust",
]
_missing = [r for r in _required if r not in globals()]
if _missing:
    raise RuntimeError(
        f"Missing required helpers from your pipeline: {_missing}\n"
        f"Run the pipeline script first (or import it) so these functions exist in globals()."
    )

# ---- Rank roots (use your existing RANK_DIRS if present) ----
if "RANK_DIRS" not in globals():
    RANK_DIRS = [
        ROOT / "FeatureRankOUT",
        Path("/Volumes/Untitled") / "FeatureRankOUT",
        Path("/Volumes/Untitled") / "octaneX" / "FeatureRankOUT",
    ]

def _read_rank_list(rank_dirs, setup, win, kfold, sub):
    """Return ordered list of features from MMI FeatureRankOUT (first hit wins)."""
    fname = f"{setup}_{win}_{kfold}_0_{sub}.csv"
    for d in rank_dirs:
        p = Path(d) / fname
        if p.exists():
            try:
                df = pd.read_csv(p)
                col_f = "feature" if "feature" in df.columns else df.columns[0]
                return df[col_f].dropna().astype(str).tolist()
            except Exception as e:
                print(f"[WARN] cannot read {p}: {e}")
    return []

def _train_xgb_simple(X_tr, y_tr, seed=SEED):
    from xgboost import XGBClassifier
    clf = XGBClassifier(
        n_estimators=250, max_depth=4, learning_rate=0.06,
        subsample=0.9, colsample_bytree=0.9,
        random_state=seed, tree_method="hist", n_jobs=os.cpu_count()
    )
    clf.fit(X_tr, y_tr)
    return clf

def _compute_shap(clf, X_ref):
    """Return mean|SHAP| per feature (Series). Fall back to XGB gain if shap not installed."""
    try:
        import shap
        explainer = shap.TreeExplainer(clf, feature_perturbation="interventional")
        sv = explainer.shap_values(X_ref)
        if isinstance(sv, list):  # binary -> [neg,pos]
            vals = np.abs(sv[1]).mean(axis=0)
        else:
            vals = np.abs(sv).mean(axis=0)
        return pd.Series(vals, index=X_ref.columns, name="shap_mean_abs")
    except Exception:
        try:
            booster = clf.get_booster()
            gain = booster.get_score(importance_type="gain")
            s = pd.Series(gain, dtype=float)
            s = s.reindex(X_ref.columns).fillna(0.0)
            s.name = "shap_mean_abs"
            print("[WARN] shap unavailable; fell back to XGB gain.")
            return s
        except Exception:
            return pd.Series(0.0, index=X_ref.columns, name="shap_mean_abs")

def _beeswarm_plot(clf, X, title, out_png, max_display=40):
    try:
        import shap
        explainer = shap.TreeExplainer(clf, feature_perturbation="interventional")
        sv = explainer.shap_values(X)
        if isinstance(sv, list): sv = sv[1]
        plt.figure(figsize=(10,6))
        shap.summary_plot(sv, X, show=False, max_display=max_display)
        plt.title(title); plt.tight_layout()
        plt.savefig(out_png, dpi=220); plt.close()
    except Exception as e:
        print(f"[WARN] SHAP beeswarm skipped: {e}")

def _barplot_subspace(df_sub, title, out_png, topk=30):
    d = df_sub.sort_values("shap_mean_abs", ascending=False)
    if topk is not None and topk > 0:
        d = d.head(topk)
    plt.figure(figsize=(10, max(3.5, 0.25*len(d))))
    plt.barh(d["feature"], d["shap_mean_abs"])
    plt.gca().invert_yaxis()
    plt.xlabel("mean |SHAP| (validation set)")
    plt.title(title)
    plt.tight_layout()
    plt.savefig(out_png, dpi=220)
    plt.close()

# ---- Load per-platform details (winners) ----
if not DETAILS_CSV.exists():
    raise FileNotFoundError(
        f"Missing platform details CSV: {DETAILS_CSV}\n"
        f"Generate it first using your per-platform winners code:\n"
        f"  - BEST_in_DesignSpace_Post_per_platform.csv\n"
        f"  - BEST_in_DesignSpace_Post_per_platform_details.csv"
    )

winners = pd.read_csv(DETAILS_CSV).copy()

# Normalize/validate columns
# expected columns (from your generator):
# setup, anomaly, win, kfold, best_pct_by_median, method, auc_pr_median, roc_auc_median, ...
colmap = {}
if "best_method" in winners.columns and "method" not in winners.columns:
    colmap["best_method"] = "method"
winners = winners.rename(columns=colmap)

required_cols = ["setup","anomaly","win","kfold","best_pct_by_median","method"]
missing = [c for c in required_cols if c not in winners.columns]
if missing:
    raise KeyError(f"Platform details CSV missing required columns: {missing}. Have: {list(winners.columns)}")

# Keep only defined anomalies per platform (defensive)
ANOMALIES_BY_SETUP = {"DDR4": ["DROOP","RH"], "DDR5": ["DROOP","SPECTRE"]}
def _keep(row):
    s = str(row["setup"]).upper()
    a = str(row["anomaly"]).upper()
    return s in ANOMALIES_BY_SETUP and a in [x.upper() for x in ANOMALIES_BY_SETUP[s]]
winners = winners[winners.apply(_keep, axis=1)].copy()
winners = winners.reset_index(drop=True)

print(f"[OK] Loaded platform winners (details) rows: {len(winners)}")
print(winners[["setup","anomaly","win","kfold","best_pct_by_median","method"]].to_string(index=False))

# ---- Run explainability per platform×anomaly best config ----
all_rows = []

for _, row in winners.iterrows():
    setup   = str(row["setup"])
    anomaly = str(row["anomaly"])
    win     = int(row["win"])
    kfold   = int(row["kfold"])
    pct     = int(row["best_pct_by_median"])
    method  = str(row["method"])

    print(f"\n[RUN] SHAP explainability (BEST PLATFORM) → {setup}/{anomaly}  WIN={win}  K={kfold}  PCT={pct}  METHOD={method}")

    # 1) Build BENIGN/ANOM windows
    ben_pairs = collect_raw_pairs_by_setup(DATA_DIR, which="benign").get(setup, [])
    an_pairs  = collect_raw_pairs_by_setup(DATA_DIR, which="anomaly", anomaly=anomaly).get(setup, [])
    if not ben_pairs or not an_pairs:
        print(f"[SKIP] Missing RAW files for {setup}/{anomaly}")
        continue

    # NOTE: build_windowed_raw_means signature includes overlap_ratio in your pipeline.
    # If your helper requires it, it will be in globals() and this call will work.
    # If not, you can add overlap_ratio=... as needed.
    try:
        df_b = build_windowed_raw_means(ben_pairs, setup=setup, win=win, label="BENIGN", overlap_ratio=0.50)
        df_a = build_windowed_raw_means(an_pairs,  setup=setup, win=win, label=anomaly, overlap_ratio=(0.80 if anomaly.upper()=="DROOP" else 0.50))
    except TypeError:
        # fallback if your helper doesn't have overlap_ratio
        df_b = build_windowed_raw_means(ben_pairs, setup=setup, win=win, label="BENIGN")
        df_a = build_windowed_raw_means(an_pairs,  setup=setup, win=win, label=anomaly)

    if df_b.empty or df_a.empty:
        print(f"[SKIP] Empty windows for {setup}/{anomaly} WIN={win}")
        continue

    # 2) Robust scale on BENIGN; apply to all
    xb_cols = telemetry_cols(df_b)
    if not xb_cols:
        print(f"[SKIP] No telemetry cols for {setup}/{anomaly}")
        continue

    Xb = df_b[xb_cols].astype(float)
    mu, sd, q1, q2 = robust_scale_train(Xb.values, winsor=(2.0, 98.0))
    Xb_z = apply_robust_scale(Xb, mu, sd, q1, q2)

    Xa = df_a[[c for c in xb_cols if c in df_a.columns]].astype(float)
    Xa_z = apply_robust_scale(Xa, mu, sd, q1, q2)

    # Align columns (just in case)
    common_cols = [c for c in Xb_z.columns if c in Xa_z.columns]
    if not common_cols:
        print(f"[SKIP] No common scaled columns for {setup}/{anomaly}")
        continue
    Xb_z = Xb_z[common_cols]
    Xa_z = Xa_z[common_cols]

    X_all = pd.concat([Xb_z, Xa_z], ignore_index=True)
    y_all = np.concatenate([np.zeros(len(Xb_z), dtype=int), np.ones(len(Xa_z), dtype=int)])

    # 3) One validation mask per case
    rng = np.random.RandomState(SEED)
    idx = np.arange(len(y_all))
    val_mask = np.zeros_like(y_all, dtype=bool)
    val_mask[rng.choice(idx, size=max(1, int(0.30*len(y_all))), replace=False)] = True

    # 4) MMI selection at best pct (robust mapping against columns)
    full_lists = {sub: _read_rank_list(RANK_DIRS, setup, win, kfold, sub) for sub in SUBSPACES}

    # Your pipeline's slice_by_percent returns sel dict (not tuple). Handle both.
    sel_raw = slice_by_percent(full_lists, pct)
    if isinstance(sel_raw, tuple):
        sel_raw = sel_raw[0]

    sel_map = intersect_selection_with_columns_robust(sel_raw, set(X_all.columns))
    if isinstance(sel_map, tuple):
        sel_map = sel_map[0]

    chosen, feat_sub = [], {}
    for sub in SUBSPACES:
        feats = list(sel_map.get(sub, []) or [])
        for f in feats:
            feat_sub[f] = sub
        chosen.extend(feats)

    chosen = [c for c in dict.fromkeys(chosen) if c in X_all.columns]
    if not chosen:
        print(f"[SKIP] No features after mapping for {setup}/{anomaly} at WIN={win},K={kfold},PCT={pct}")
        continue

    X = X_all[chosen]
    X_tr, y_tr = X.loc[~val_mask], y_all[~val_mask]
    X_va, y_va = X.loc[val_mask],  y_all[val_mask]

    # 5) Train and compute SHAP
    clf     = _train_xgb_simple(X_tr, y_tr, seed=SEED)
    shap_s  = _compute_shap(clf, X_va)

    # z²-lift on validation (anomaly minus benign)
    ben_mask = (y_va == 0)
    an_mask  = (y_va == 1)
    if ben_mask.sum() == 0 or an_mask.sum() == 0:
        z2_lift = np.zeros(X_va.shape[1], dtype=float)
    else:
        z2_lift = ((X_va.values[an_mask]**2).mean(axis=0) - (X_va.values[ben_mask]**2).mean(axis=0))
    z2_s = pd.Series(z2_lift, index=X.columns, name="z2_lift")

    df_all = (
        pd.DataFrame({
            "feature": X.columns,
            "subspace": [feat_sub.get(c, "compute") for c in X.columns],
            "shap_mean_abs": shap_s.reindex(X.columns).fillna(0.0).values,
            "z2_lift": z2_s.reindex(X.columns).fillna(0.0).values,
            "setup": setup, "anomaly": anomaly, "win": win, "kfold": kfold, "pct": pct, "method": method
        })
        .sort_values("shap_mean_abs", ascending=False)
        .reset_index(drop=True)
    )

    # 6) Save outputs (full + per-subspace + plots)
    base = f"{setup}_{anomaly}_WIN{win}_KF{kfold}_PCT{pct}_M{method}"
    out_full = OUT_DIR / f"SHAP_BESTPLAT_full_{base}.csv"
    df_all.to_csv(out_full, index=False)
    print(f"[EXPL] full → {out_full}")

    for sub in SUBSPACES:
        sub_df = df_all[df_all["subspace"]==sub].copy()
        if sub_df.empty:
            continue
        out_sub = OUT_DIR / f"SHAP_BESTPLAT_{sub}_{base}.csv"
        sub_df.to_csv(out_sub, index=False)

        png = FIG_DIR / f"BAR_BESTPLAT_{sub}_{base}.png"
        _barplot_subspace(
            sub_df,
            title=f"{setup}/{anomaly} • {sub} • WIN={win} K={kfold} PCT={pct} • {method}",
            out_png=png,
            topk=30
        )
        print(f"[FIG] {png}")

    bees = FIG_DIR / f"BEE_BESTPLAT_{base}.png"
    _beeswarm_plot(
        clf, X_va,
        title=f"{setup}/{anomaly} • WIN={win} K={kfold} PCT={pct} • {method} (validation)",
        out_png=bees,
        max_display=40
    )

    all_rows.append(df_all)
    gc.collect()

# 7) Master table
if all_rows:
    master = pd.concat(all_rows, ignore_index=True)
    master_csv = OUT_DIR / "SHAP_BESTPLAT_master_all.csv"
    master.to_csv(master_csv, index=False)
    print(f"\n[OK] Master table → {master_csv}")
else:
    print("\n[WARN] No SHAP outputs were produced (check platform winners/ranks/data).")


[OK] Loaded platform winners (details) rows: 4
setup anomaly  win  kfold  best_pct_by_median method
 DDR4   DROOP  512      3                  10  dC_aJ
 DDR4      RH  512      3                  10  dC_aJ
 DDR5   DROOP 1024     10                  10  dC_aJ
 DDR5 SPECTRE 1024     10                  10  dC_aJ

[RUN] SHAP explainability (BEST PLATFORM) → DDR4/DROOP  WIN=512  K=3  PCT=10  METHOD=dC_aJ
[EXPL] full → /Users/hsiaopingni/octaneX_v7_4functions/Results/Explainability_SHAP_BestPlatforms/SHAP_BESTPLAT_full_DDR4_DROOP_WIN512_KF3_PCT10_MdC_aJ.csv
[FIG] /Users/hsiaopingni/octaneX_v7_4functions/Results/Explainability_SHAP_BestPlatforms/figs/BAR_BESTPLAT_compute_DDR4_DROOP_WIN512_KF3_PCT10_MdC_aJ.png
[FIG] /Users/hsiaopingni/octaneX_v7_4functions/Results/Explainability_SHAP_BestPlatforms/figs/BAR_BESTPLAT_memory_DDR4_DROOP_WIN512_KF3_PCT10_MdC_aJ.png
[FIG] /Users/hsiaopingni/octaneX_v7_4functions/Results/Explainability_SHAP_BestPlatforms/figs/BAR_BESTPLAT_sensors_DDR4_DROOP_WIN512_K

## *SHAP explainability for BEST cases only (per platform)

In [56]:
# === PLATFORM-LEVEL SHAP Top-10 plot-only PNGs (ONE per platform) + FULL BLACK BOX + legend ===
# This aggregates SHAP across the platform's anomalies (using the chosen best configs in
# BEST_in_DesignSpace_Post_per_platform_details.csv) and produces ONE Top-10 bar plot per platform.
#
# Inputs:
#   1) Results/BEST_in_DesignSpace_Post_per_platform_details.csv
#   2) Results/Explainability_SHAP_BestPlatforms/
#        SHAP_BESTPLAT_full_<setup>_<anomaly>_WIN<w>_KF<k>_PCT<p>_M<method>.csv
#
# Outputs (3 PNGs total):
#   1) figs/FIG_Top10_SHAP_LEGEND.png
#   2) figs/FIG_Top10_SHAP_PLATFORM_DDR4_PLOT.png
#   3) figs/FIG_Top10_SHAP_PLATFORM_DDR5_PLOT.png
#
# Aggregation rule:
#   - For each platform, load the anomaly-specific SHAP tables (already at each anomaly's best config)
#   - Compute mean SHAP (mean |value|) per feature ACROSS anomalies for that platform:
#       shap_platform(feature) = mean_over_anomalies(shap_mean_abs_anomaly(feature))
#     Missing features in an anomaly contribute 0 for that anomaly.
#   - Subspace label per feature: majority vote across anomalies (tie -> subspace of highest mean SHAP).
#
# Styling:
#   - Full black rectangle frame (all four spines visible & black), like your paper figure
#   - Separate legend PNG
# -------------------------------------------------------------------------------------------

from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt

# ------------------ Paths ------------------
ROOT = Path(globals().get("ROOT", "/Users/hsiaopingni/octaneX_v7_4functions"))
RES_DIR = Path(globals().get("RES_DIR", ROOT / "Results"))

DETAILS = RES_DIR / "BEST_in_DesignSpace_Post_per_platform_details.csv"
if not DETAILS.exists():
    raise FileNotFoundError(f"Missing platform details CSV: {DETAILS}")

PLAT_DIR = RES_DIR / "Explainability_SHAP_BestPlatforms"
FIG_DIR  = PLAT_DIR / "figs"
FIG_DIR.mkdir(parents=True, exist_ok=True)

# ------------------ Colors match paper style ------------------
DOMAIN_COLORS = {"compute": "#1f77b4", "memory": "#ff7f0e", "sensors": "#2ca02c"}  # blue, orange, green
DOMAIN_LABELS = {"compute": "Compute", "memory": "Memory", "sensors": "Sensors"}

plt.rcParams.update({
    "font.family": "sans-serif",
    "font.size": 12,
    "axes.labelsize": 14,
    "xtick.labelsize": 12,
    "ytick.labelsize": 13,
    "axes.grid": True,
    "grid.alpha": 0.25,
    "grid.linestyle": "-",
})

def _apply_full_black_box(ax, lw=1.1):
    """Make a full black rectangle frame like the paper (all 4 spines)."""
    for side in ["left", "right", "top", "bottom"]:
        ax.spines[side].set_visible(True)
        ax.spines[side].set_color("black")
        ax.spines[side].set_linewidth(lw)

def save_legend_only(out_png: Path):
    fig, ax = plt.subplots(figsize=(3.0, 1.4), dpi=220)
    ax.axis("off")

    handles, labels = [], []
    for key in ["compute", "memory", "sensors"]:
        handles.append(plt.Rectangle((0, 0), 1, 1, color=DOMAIN_COLORS[key]))
        labels.append(DOMAIN_LABELS[key])

    ax.legend(handles, labels, loc="center", frameon=True, fontsize=12)
    fig.tight_layout()
    out_png.parent.mkdir(parents=True, exist_ok=True)
    fig.savefig(out_png, dpi=300, bbox_inches="tight", pad_inches=0.05)
    plt.close(fig)
    print("[WROTE]", out_png)

def _load_bestplat_csv(setup: str, anomaly: str, win: int, kfold: int, pct: int, method: str) -> pd.DataFrame:
    p = PLAT_DIR / f"SHAP_BESTPLAT_full_{setup}_{anomaly}_WIN{win}_KF{kfold}_PCT{pct}_M{method}.csv"
    if not p.exists():
        raise FileNotFoundError(f"Missing best-platform SHAP CSV: {p}")
    df = pd.read_csv(p)
    df.columns = [c.lower() for c in df.columns]
    need = {"feature", "subspace", "shap_mean_abs"}
    if not need.issubset(df.columns):
        raise KeyError(f"{p} missing columns {need - set(df.columns)}. Have: {list(df.columns)}")
    df["feature"] = df["feature"].astype(str)
    df["subspace"] = df["subspace"].astype(str).str.lower()
    df["shap_mean_abs"] = pd.to_numeric(df["shap_mean_abs"], errors="coerce").fillna(0.0)
    return df[["feature","subspace","shap_mean_abs"]].copy()

def _aggregate_platform_shap(cfg_rows: pd.DataFrame) -> pd.DataFrame:
    """
    cfg_rows: rows for one setup (platform), each row includes anomaly/win/kfold/pct/method.
    Returns: aggregated per-feature table with columns:
      feature, subspace, shap_mean_abs_platform
    """
    if cfg_rows.empty:
        return pd.DataFrame(columns=["feature","subspace","shap_mean_abs_platform"])

    dfs = []
    for _, r in cfg_rows.iterrows():
        setup  = str(r["setup"])
        anomaly= str(r["anomaly"])
        win    = int(r["win"])
        kfold  = int(r["kfold"])
        pct    = int(r["best_pct_by_median"])
        method = str(r["method"])
        d = _load_bestplat_csv(setup, anomaly, win, kfold, pct, method)
        d = d.rename(columns={"shap_mean_abs": f"shap_{anomaly}"})
        dfs.append(d)

    # Outer-merge across anomalies on feature; keep subspace columns for voting
    # We merge in a way that preserves all features ever seen across anomalies.
    merged = None
    for d in dfs:
        if merged is None:
            merged = d.copy()
        else:
            # when merging, keep both subspace columns (we'll vote later)
            merged = merged.merge(d, on="feature", how="outer", suffixes=("", "_r"))

            # consolidate/rename subspace columns into a list-friendly set
            # after merge, we may have 'subspace' and 'subspace_r'
            if "subspace_r" in merged.columns:
                # keep both; rename to unique col name
                # we’ll just leave them and handle later
                pass

    if merged is None or merged.empty:
        return pd.DataFrame(columns=["feature","subspace","shap_mean_abs_platform"])

    # Identify all shap columns (one per anomaly)
    shap_cols = [c for c in merged.columns if c.startswith("shap_")]
    if not shap_cols:
        return pd.DataFrame(columns=["feature","subspace","shap_mean_abs_platform"])

    # Fill missing SHAP with 0 (means feature absent in that anomaly)
    merged[shap_cols] = merged[shap_cols].fillna(0.0)

    # Platform aggregate = mean over anomalies (equal weight)
    merged["shap_mean_abs_platform"] = merged[shap_cols].mean(axis=1)

    # Determine subspace via majority vote across available subspace columns
    sub_cols = [c for c in merged.columns if c.startswith("subspace")]
    def vote_subspace(row):
        vals = [str(row[c]).lower() for c in sub_cols if pd.notna(row[c]) and str(row[c]).strip() != ""]
        vals = [v for v in vals if v in ("compute","memory","sensors")]
        if not vals:
            return "compute"
        # majority vote
        counts = {k: vals.count(k) for k in ("compute","memory","sensors")}
        best = max(counts, key=lambda k: counts[k])
        # tie-break: choose the subspace corresponding to the anomaly where this feature has max SHAP
        top_cnt = counts[best]
        ties = [k for k,v in counts.items() if v == top_cnt]
        if len(ties) == 1:
            return best
        # tie: pick subspace of the anomaly with largest shap value among tie subspaces
        # (use first non-empty subspace aligned with max shap col)
        max_col = shap_cols[int(np.argmax([row[c] for c in shap_cols]))]
        # find a subspace column that likely came from same anomaly merge stage:
        # simplest tie-break: pick first subspace that is in ties
        for v in vals:
            if v in ties:
                return v
        return ties[0]

    merged["subspace"] = merged.apply(vote_subspace, axis=1)

    out = merged[["feature","subspace","shap_mean_abs_platform"]].copy()
    out["shap_mean_abs_platform"] = pd.to_numeric(out["shap_mean_abs_platform"], errors="coerce").fillna(0.0)
    return out

def save_platform_plot_only(df_platform: pd.DataFrame, setup: str, out_png: Path, topk: int = 10):
    d = df_platform.sort_values("shap_mean_abs_platform", ascending=False).head(topk).copy()
    if d.empty:
        print("[SKIP] empty platform df for", setup)
        return

    colors = [DOMAIN_COLORS.get(s, "#777777") for s in d["subspace"]]

    fig, ax = plt.subplots(figsize=(7.2, 4.4), dpi=220)
    ax.barh(d["feature"], d["shap_mean_abs_platform"], color=colors, edgecolor="none")
    ax.invert_yaxis()

    ax.set_xlabel("SHAP (mean |value|) • platform-avg", fontweight="bold")
    ax.grid(True, axis="both", alpha=0.25)

    _apply_full_black_box(ax, lw=1.1)

    fig.tight_layout()
    out_png.parent.mkdir(parents=True, exist_ok=True)
    fig.savefig(out_png, dpi=300, bbox_inches="tight", pad_inches=0.05)
    plt.close(fig)
    print("[WROTE]", out_png)

# ------------------ Load per-platform details and build configs ------------------
w = pd.read_csv(DETAILS).copy()
if "best_method" in w.columns and "method" not in w.columns:
    w = w.rename(columns={"best_method":"method"})
if "best_pct_by_median" not in w.columns and "pct" in w.columns:
    w = w.rename(columns={"pct":"best_pct_by_median"})

need = {"setup","anomaly","win","kfold","best_pct_by_median","method"}
missing = need - set(w.columns)
if missing:
    raise KeyError(f"{DETAILS} missing columns {sorted(missing)}. Have: {list(w.columns)}")

# ------------------ 1) Legend PNG (shared) ------------------
legend_png = FIG_DIR / "FIG_Top10_SHAP_LEGEND.png"
save_legend_only(legend_png)

# ------------------ 2) One plot per platform ------------------
for setup in ["DDR4","DDR5"]:
    cfg = w[w["setup"].astype(str).str.upper() == setup].copy()
    if cfg.empty:
        print("[WARN] No config rows for", setup)
        continue

    # Build platform-level aggregated SHAP
    plat = _aggregate_platform_shap(cfg)

    # Save the aggregated table too (useful for checking)
    agg_csv = PLAT_DIR / f"SHAP_PLATFORM_AGG_{setup}.csv"
    plat.sort_values("shap_mean_abs_platform", ascending=False).to_csv(agg_csv, index=False)
    print("[WROTE]", agg_csv)

    out_plot = FIG_DIR / f"FIG_Top10_SHAP_PLATFORM_{setup}_PLOT.png"
    save_platform_plot_only(plat, setup=setup, out_png=out_plot, topk=10)

[WROTE] /Users/hsiaopingni/octaneX_v7_4functions/Results/Explainability_SHAP_BestPlatforms/figs/FIG_Top10_SHAP_LEGEND.png
[WROTE] /Users/hsiaopingni/octaneX_v7_4functions/Results/Explainability_SHAP_BestPlatforms/SHAP_PLATFORM_AGG_DDR4.csv
[WROTE] /Users/hsiaopingni/octaneX_v7_4functions/Results/Explainability_SHAP_BestPlatforms/figs/FIG_Top10_SHAP_PLATFORM_DDR4_PLOT.png
[WROTE] /Users/hsiaopingni/octaneX_v7_4functions/Results/Explainability_SHAP_BestPlatforms/SHAP_PLATFORM_AGG_DDR5.csv
[WROTE] /Users/hsiaopingni/octaneX_v7_4functions/Results/Explainability_SHAP_BestPlatforms/figs/FIG_Top10_SHAP_PLATFORM_DDR5_PLOT.png


# **Compare Explainability: SHAP vs CP-MI (BEST cases)

In [14]:
# === SHAP vs CP-MI (PER PLATFORM) — stability + concordance plots ============================
# This is a PER-PLATFORM revision of your original "BestCases" script.
#
# What changes:
#   - Uses per-platform winners from:
#       Results/BEST_in_DesignSpace_Post_per_platform_details.csv
#   - Uses SHAP tables generated by your per-platform SHAP run:
#       Results/Explainability_SHAP_BestPlatforms/SHAP_BESTPLAT_full_<setup>_<anomaly>_WIN..._KF..._PCT..._M<method>.csv
#   - For each PLATFORM (DDR4, DDR5) and each SUBSPACE (compute/memory/sensors), it:
#       1) Loads CP-MI ranks (from FeatureRankOUT) for that platform's chosen WIN/K
#       2) Aggregates SHAP across the platform's anomalies (equal-weight mean, missing=0)
#       3) Computes Spearman (rank/percentile/score) and top-% Jaccard/overlap
#       4) Produces platform-level plots (scatter + jaccard curve + overlap bars)
#
# Outputs:
#   Results/Explainability_SHAP_BestPlatforms/
#     - SHAP_vs_CPMI_summary_PLATFORM.csv
#     - comparison_plots_platform/
#         DDR4_compute_scatter_cpmi_vs_shap.png
#         DDR4_compute_scatter_percentile_ranks.png
#         DDR4_compute_jaccard_vs_pct.png
#         DDR4_compute_overlap_bar_pct.png
#         ... (memory/sensors, DDR5)
# ============================================================================================

import math
from pathlib import Path
from typing import Optional, Dict, List

import numpy as np
import pandas as pd
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
from scipy.stats import spearmanr

# ---------- Paths (ROBUST) ----------
ROOT = Path(globals().get("ROOT", "/Users/hsiaopingni/octaneX_v7_4functions"))
BASE_RES = ROOT / "Results"

# Use the PER-PLATFORM SHAP directory
OUT_DIR = BASE_RES / "Explainability_SHAP_BestPlatforms"
FIG_DIR = OUT_DIR / "comparison_plots_platform"
FIG_DIR.mkdir(parents=True, exist_ok=True)

DETAILS_CSV = BASE_RES / "BEST_in_DesignSpace_Post_per_platform_details.csv"
if not DETAILS_CSV.exists():
    raise FileNotFoundError(f"Missing per-platform details CSV: {DETAILS_CSV}")

SUBSPACES = ("compute", "memory", "sensors")
TOP_PCTS  = list(range(10, 101, 10))   # for overlap/jaccard sweep
TAKE_PCTS = list(range(10, 101, 10))   # which points to record

NORMALIZE_BY_PERCENTILE = True

# Where CP-MI ranks might be (first hit wins)
if "RANK_DIRS" not in globals():
    RANK_DIRS = [
        ROOT / "FeatureRankOUT",
        Path("/Volumes/Untitled") / "FeatureRankOUT",
        Path("/Volumes/Untitled") / "octaneX" / "FeatureRankOUT",
        Path.home() / "Desktop" / "octaneX" / "FeatureRankOUT",
    ]

def _read_csv_safe(p: Path) -> pd.DataFrame:
    try:
        return pd.read_csv(p)
    except Exception:
        return pd.DataFrame()

# ------------------------ CP-MI loader ------------------------
def _cpmi_rank_path(setup: str, win: int, kfold: int, sub: str) -> Optional[Path]:
    fname = f"{setup}_{win}_{kfold}_0_{sub}.csv"
    for d in RANK_DIRS:
        p = Path(d) / fname
        if p.exists():
            return p
    # fallback search (rare)
    for d in RANK_DIRS:
        d = Path(d)
        if not d.exists():
            continue
        hits = list(d.rglob(f"*{setup}*{sub}*CPMI*.csv")) + list(d.rglob(f"*{setup}*{sub}*CP-MI*.csv"))
        if hits:
            return hits[0]
    return None

def _load_cpmi(setup: str, win: int, kfold: int) -> Dict[str, pd.DataFrame]:
    """
    Returns dict[subspace] -> DataFrame(feature, cpmi_score, cpmi_rank), sorted by score desc.
    """
    out: Dict[str, pd.DataFrame] = {}
    for sub in SUBSPACES:
        rp = _cpmi_rank_path(setup, win, kfold, sub)
        if not rp:
            out[sub] = pd.DataFrame()
            continue
        df = _read_csv_safe(rp)
        if df.empty:
            out[sub] = df
            continue

        feat_col = None
        for c in df.columns:
            if c.lower() in ("feature", "features", "name", "signal", "column"):
                feat_col = c
                break
        if feat_col is None:
            nonnum = df.select_dtypes(exclude=[np.number]).columns
            feat_col = nonnum[0] if len(nonnum) else df.columns[0]

        score_col = None
        for c in df.columns:
            if c.lower() in ("cpmi", "cp-mi", "score", "mi", "mi_score", "rank_score"):
                score_col = c
                break
        if score_col is None:
            nums = df.select_dtypes(include=[np.number]).columns
            score_col = nums[0] if len(nums) else df.columns[-1]

        d = (
            df[[feat_col, score_col]]
            .rename(columns={feat_col: "feature", score_col: "cpmi_score"})
            .assign(feature=lambda x: x["feature"].astype(str).str.strip())
            .dropna(subset=["feature"])
            .drop_duplicates(subset=["feature"])
            .sort_values("cpmi_score", ascending=False)
            .reset_index(drop=True)
        )
        d["cpmi_rank"] = np.arange(1, len(d) + 1)
        out[sub] = d
    return out

# ------------------------ SHAP loader (per-platform best) ------------------------
def _shap_bestplat_full_path(setup: str, anomaly: str, win: int, kfold: int, pct: int, method: str) -> Path:
    return OUT_DIR / f"SHAP_BESTPLAT_full_{setup}_{anomaly}_WIN{win}_KF{kfold}_PCT{pct}_M{method}.csv"

def _load_shap_bestplat_full(setup: str, anomaly: str, win: int, kfold: int, pct: int, method: str) -> pd.DataFrame:
    p = _shap_bestplat_full_path(setup, anomaly, win, kfold, pct, method)
    df = _read_csv_safe(p)
    if df.empty:
        return df
    df.columns = [c.lower() for c in df.columns]
    # allow alt naming
    if "shap_mean_abs" not in df.columns and "importance" in df.columns:
        df = df.rename(columns={"importance": "shap_mean_abs"})
    # keep minimum set
    need = {"feature", "subspace", "shap_mean_abs"}
    if not need.issubset(df.columns):
        return pd.DataFrame()
    df["feature"] = df["feature"].astype(str).str.strip()
    df["subspace"] = df["subspace"].astype(str).str.lower()
    df["shap_mean_abs"] = pd.to_numeric(df["shap_mean_abs"], errors="coerce").fillna(0.0)
    return df[["feature", "subspace", "shap_mean_abs"]].copy()

# ------------------------ Aggregation helpers ------------------------
def _percentile_rank(series: pd.Series) -> pd.Series:
    n = len(series)
    if n <= 1:
        return pd.Series(np.zeros(n), index=series.index, dtype=float)
    r = series.rank(method="average", ascending=True)
    return (r - 1) / (n - 1)

def _aggregate_shap_platform(details_rows: pd.DataFrame, setup: str, subspace: str) -> pd.DataFrame:
    """
    details_rows: rows from BEST_in_DesignSpace_Post_per_platform_details.csv for this setup
    Returns: DataFrame(feature, shap_mean_abs_platform) for one subspace
    """
    parts: List[pd.DataFrame] = []
    for _, r in details_rows.iterrows():
        anomaly = str(r["anomaly"])
        win     = int(r["win"])
        kfold   = int(r["kfold"])
        pct     = int(r["best_pct_by_median"])
        method  = str(r["method"])

        df_shap = _load_shap_bestplat_full(setup, anomaly, win, kfold, pct, method)
        if df_shap.empty:
            continue

        sub_df = df_shap[df_shap["subspace"] == subspace][["feature", "shap_mean_abs"]].copy()
        sub_df = sub_df.rename(columns={"shap_mean_abs": f"shap_{anomaly}"})
        parts.append(sub_df)

    if not parts:
        return pd.DataFrame(columns=["feature", "shap_mean_abs_platform"])

    merged = None
    for d in parts:
        if merged is None:
            merged = d.copy()
        else:
            merged = merged.merge(d, on="feature", how="outer")

    shap_cols = [c for c in merged.columns if c.startswith("shap_")]
    merged[shap_cols] = merged[shap_cols].fillna(0.0)

    # equal-weight mean across anomalies
    merged["shap_mean_abs_platform"] = merged[shap_cols].mean(axis=1)

    out = merged[["feature", "shap_mean_abs_platform"]].copy()
    out["feature"] = out["feature"].astype(str)
    out["shap_mean_abs_platform"] = pd.to_numeric(out["shap_mean_abs_platform"], errors="coerce").fillna(0.0)
    return out

# ------------------------ Comparison ------------------------
def _compare_subspace(cpmi_df: pd.DataFrame, shap_df: pd.DataFrame) -> Optional[dict]:
    """
    cpmi_df: feature, cpmi_score, cpmi_rank
    shap_df: feature, shap_mean_abs_platform
    """
    if cpmi_df.empty or shap_df.empty:
        return None

    m = pd.merge(
        cpmi_df[["feature", "cpmi_score", "cpmi_rank"]],
        shap_df[["feature", "shap_mean_abs_platform"]],
        on="feature", how="inner"
    )
    if m.empty:
        return None

    m = m.copy()
    m["shap_rank"] = m["shap_mean_abs_platform"].rank(ascending=False, method="average").astype(float)

    if NORMALIZE_BY_PERCENTILE:
        m["cpmi_prank"] = _percentile_rank(-m["cpmi_rank"])
        m["shap_prank"] = _percentile_rank(-m["shap_rank"])
    else:
        m["cpmi_prank"] = np.nan
        m["shap_prank"] = np.nan

    rho_rank,  p_rank  = spearmanr(-m["cpmi_rank"], -m["shap_rank"])
    rho_prank, p_prank = (np.nan, np.nan)
    if NORMALIZE_BY_PERCENTILE:
        rho_prank, p_prank = spearmanr(m["cpmi_prank"], m["shap_prank"])
    rho_score, p_score = spearmanr(m["cpmi_score"], m["shap_mean_abs_platform"])

    rows = []
    n = len(m)
    for pct in TOP_PCTS:
        k = max(1, math.floor(pct * n / 100))
        top_c = set(m.sort_values("cpmi_rank").head(k)["feature"])
        top_s = set(m.sort_values("shap_rank").head(k)["feature"])
        inter = len(top_c & top_s)
        union = len(top_c | top_s) if (top_c or top_s) else 1
        rows.append({"k": f"top{pct}%", "overlap": inter, "jaccard": inter / union, "n_aligned": n})

    return {
        "aligned": m,
        "rho_rank": rho_rank, "p_rank": p_rank,
        "rho_prank": rho_prank, "p_prank": p_prank,
        "rho_score": rho_score, "p_score": p_score,
        "summary": pd.DataFrame(rows)
    }

# ------------------------ Plotting ------------------------
def _plot_platform(setup: str, subspace: str, comp: dict):
    aligned = comp["aligned"]
    case_tag = f"{setup}_{subspace}"

    plt.figure(figsize=(6, 5))
    plt.scatter(aligned["cpmi_score"], aligned["shap_mean_abs_platform"], s=12)
    plt.xlabel("CP-MI score")
    plt.ylabel("SHAP importance (mean |SHAP|) • platform-avg")
    plt.title(f"{setup} • {subspace} • CP-MI vs SHAP (ρ={comp['rho_score']:.2f})")
    plt.tight_layout()
    plt.savefig(FIG_DIR / f"{case_tag}_scatter_cpmi_vs_shap.png", dpi=220)
    plt.close()

    if "cpmi_prank" in aligned.columns and "shap_prank" in aligned.columns:
        plt.figure(figsize=(6, 5))
        plt.scatter(aligned["cpmi_prank"], aligned["shap_prank"], s=12)
        plt.xlabel("CP-MI percentile rank (higher = more important)")
        plt.ylabel("SHAP percentile rank (higher = more important)")
        plt.title(f"{setup} • {subspace} • Percentile Concordance (ρ={comp['rho_prank']:.2f})")
        plt.tight_layout()
        plt.savefig(FIG_DIR / f"{case_tag}_scatter_percentile_ranks.png", dpi=220)
        plt.close()

    s_pct = comp["summary"].copy()
    s_pct["pct"] = s_pct["k"].str.extract(r"top(\d+)%").astype(int)
    s_pct = s_pct.sort_values("pct")

    plt.figure(figsize=(6, 4))
    plt.plot(s_pct["pct"], s_pct["jaccard"], marker="o")
    plt.xlabel("Top-% of features")
    plt.ylabel("Jaccard (CP-MI vs SHAP)")
    plt.title(f"{setup} • {subspace} • Agreement vs Percentage")
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(FIG_DIR / f"{case_tag}_jaccard_vs_pct.png", dpi=220)
    plt.close()

    plt.figure(figsize=(6, 4))
    plt.bar(s_pct["pct"].astype(str), s_pct["overlap"])
    plt.xlabel("Top-% of features")
    plt.ylabel("Overlap count")
    plt.title(f"{setup} • {subspace} • Overlap (CP-MI ∩ SHAP) vs %")
    plt.tight_layout()
    plt.savefig(FIG_DIR / f"{case_tag}_overlap_bar_pct.png", dpi=220)
    plt.close()

# ====================== Main ======================
details = pd.read_csv(DETAILS_CSV).copy()

# Normalize column naming
if "best_method" in details.columns and "method" not in details.columns:
    details = details.rename(columns={"best_method":"method"})
if "best_pct_by_median" not in details.columns and "pct" in details.columns:
    details = details.rename(columns={"pct":"best_pct_by_median"})

need_det = {"setup","anomaly","win","kfold","best_pct_by_median","method"}
missing = need_det - set(details.columns)
if missing:
    raise KeyError(f"{DETAILS_CSV} missing columns {sorted(missing)}. Have: {list(details.columns)}")

rows = []

for setup in ["DDR4","DDR5"]:
    dsetup = details[details["setup"].astype(str).str.upper() == setup].copy()
    if dsetup.empty:
        print(f"[WARN] No details rows for {setup}")
        continue

    # Use platform's WIN/K from the first row (they should be identical across anomalies for that setup)
    win_best = int(dsetup["win"].iloc[0])
    kf_best  = int(dsetup["kfold"].iloc[0])

    print(f"\n[PLATFORM] {setup} using WIN={win_best} K={kf_best}")

    cpmi = _load_cpmi(setup, win_best, kf_best)

    for sub in SUBSPACES:
        shap_plat = _aggregate_shap_platform(dsetup, setup=setup, subspace=sub)
        comp = _compare_subspace(cpmi.get(sub, pd.DataFrame()), shap_plat)

        if comp is None:
            rows.append({
                "setup": setup, "win": win_best, "kfold": kf_best,
                "subspace": sub, "k": "NA",
                "jaccard": np.nan, "overlap": np.nan,
                "rho_rank": np.nan, "rho_prank": np.nan, "rho_score": np.nan,
                "aligned_features": 0
            })
            continue

        _plot_platform(setup, sub, comp)

        s = comp["summary"].copy()
        s["pct"] = s["k"].str.extract(r"top(\d+)%").astype(int)
        s["k_eff"] = (np.floor(s["pct"] * s["n_aligned"] / 100)).astype(int).clip(lower=1)
        s = s.sort_values("pct").drop_duplicates(subset=["k_eff"], keep="first")
        take = s[s["pct"].isin(TAKE_PCTS)].copy()

        for _, rr in take.iterrows():
            rows.append({
                "setup": setup, "win": win_best, "kfold": kf_best,
                "subspace": sub, "k": rr["k"],
                "jaccard": rr["jaccard"], "overlap": rr["overlap"],
                "rho_rank": comp["rho_rank"], "rho_prank": comp["rho_prank"], "rho_score": comp["rho_score"],
                "aligned_features": int(comp["aligned"].shape[0])
            })

out_csv = OUT_DIR / "SHAP_vs_CPMI_summary_PLATFORM.csv"
pd.DataFrame(rows).to_csv(out_csv, index=False)
print(f"\n[OK] Summary → {out_csv}")
print(f"[OK] Plots   → {FIG_DIR}")



[PLATFORM] DDR4 using WIN=512 K=3

[PLATFORM] DDR5 using WIN=1024 K=5

[OK] Summary → /Users/hsiaopingni/octaneX_v7_4functions/Results/Explainability_SHAP_BestPlatforms/SHAP_vs_CPMI_summary_PLATFORM.csv
[OK] Plots   → /Users/hsiaopingni/octaneX_v7_4functions/Results/Explainability_SHAP_BestPlatforms/comparison_plots_platform


In [15]:
# === Aggregates: subspace-level & PLATFORM×subspace (percentages-only run) ===
# UPDATED for PER-PLATFORM workflow.
#
# Reads:
#   Results/Explainability_SHAP_BestPlatforms/SHAP_vs_CPMI_summary_PLATFORM.csv
#
# Writes:
#   Results/Explainability_SHAP_BestPlatforms/
#     - SHAP_vs_CPMI_aggregate_corr_by_subspace_PLATFORM.csv
#     - SHAP_vs_CPMI_aggregate_corr_by_platform_subspace.csv
#     - SHAP_vs_CPMI_aggregate_jaccard_by_subspace_k_PLATFORM.csv
#     - SHAP_vs_CPMI_aggregate_jaccard_by_subspace_pivot_PLATFORM.csv
#     - SHAP_vs_CPMI_aggregate_jaccard_by_platform_subspace_k.csv
#     - SHAP_vs_CPMI_aggregate_jaccard_by_platform_subspace_pivot.csv
# -------------------------------------------------------------------------------------------

from pathlib import Path
import pandas as pd

ROOT = Path(globals().get("ROOT", "/Users/hsiaopingni/octaneX_v7_4functions"))

BASE_RES = ROOT / "Results"
OUT_DIR  = BASE_RES / "Explainability_SHAP_BestPlatforms"
OUT_DIR.mkdir(parents=True, exist_ok=True)

def _find_summary_csv() -> Path:
    fname = "SHAP_vs_CPMI_summary_PLATFORM.csv"
    candidates = [
        OUT_DIR / fname,
        BASE_RES / fname,
        # if someone nested folders accidentally
        OUT_DIR / "Explainability_SHAP_BestPlatforms" / fname,
        BASE_RES / "Explainability_SHAP_BestPlatforms" / fname,
    ]
    for p in candidates:
        if p.exists():
            return p
    hits = list(BASE_RES.rglob(fname))
    if hits:
        return hits[0]
    raise FileNotFoundError(
        f"Could not find {fname}. Tried:\n  - " +
        "\n  - ".join(str(x) for x in candidates) +
        f"\nAlso searched under: {BASE_RES}"
    )

summary_path = _find_summary_csv()
print("[OK] Using summary:", summary_path)

df = pd.read_csv(summary_path)

# Expected per-platform summary schema:
#   setup, win, kfold, subspace, k, jaccard, overlap, rho_rank, rho_prank, rho_score, aligned_features
required = {"setup","win","kfold","subspace","k","jaccard","overlap","rho_rank","rho_prank","rho_score","aligned_features"}
missing = required - set(df.columns)
if missing:
    raise KeyError(f"Summary missing required columns: {sorted(missing)}. Have: {list(df.columns)}")

# Keys for a unique "platform case"
case_keys = ["setup","win","kfold","subspace"]

# --- Correlations aggregated per subspace (one row per platform/subspace) ---
corr_cols = ["rho_rank","rho_prank","rho_score","aligned_features"]

corr_per_case = df[case_keys + corr_cols].drop_duplicates(case_keys).reset_index(drop=True)

# n_cases per subspace (across platforms; typically 2 platforms => n_cases=2)
n_cases = (
    corr_per_case.groupby("subspace")
    .size()
    .rename("n_cases")
    .reset_index()
)

agg_corr_sub = (
    corr_per_case.groupby("subspace")[corr_cols]
    .median(numeric_only=True)
    .reset_index()
)
agg_corr_sub = agg_corr_sub.merge(n_cases, on="subspace", how="left")
agg_corr_sub.to_csv(OUT_DIR / "SHAP_vs_CPMI_aggregate_corr_by_subspace_PLATFORM.csv", index=False)

# Correlations per PLATFORM × subspace
agg_corr_plat_sub = (
    corr_per_case.groupby(["setup","subspace"])[corr_cols]
    .median(numeric_only=True)
    .reset_index()
)
agg_corr_plat_sub.to_csv(OUT_DIR / "SHAP_vs_CPMI_aggregate_corr_by_platform_subspace.csv", index=False)

# --- Jaccard aggregated per subspace at each percentage threshold ---
jacc_sub_k = (
    df.groupby(["subspace","k"])["jaccard"]
      .median()
      .reset_index()
      .sort_values(["subspace","k"])
)
jacc_sub_k.to_csv(OUT_DIR / "SHAP_vs_CPMI_aggregate_jaccard_by_subspace_k_PLATFORM.csv", index=False)

# Pivot for a compact table: subspace × selected thresholds
take_pcts = ["top10%","top25%","top50%","top100%"]
available = [c for c in take_pcts if c in set(jacc_sub_k["k"].astype(str).unique())]

jacc_pivot = (
    jacc_sub_k.pivot(index="subspace", columns="k", values="jaccard")
      .reindex(columns=available)
      .reset_index()
)
jacc_pivot.to_csv(OUT_DIR / "SHAP_vs_CPMI_aggregate_jaccard_by_subspace_pivot_PLATFORM.csv", index=False)

# --- Jaccard per PLATFORM × subspace at each percentage threshold ---
jacc_plat_sub_k = (
    df.groupby(["setup","subspace","k"])["jaccard"]
      .median()
      .reset_index()
      .sort_values(["setup","subspace","k"])
)
jacc_plat_sub_k.to_csv(OUT_DIR / "SHAP_vs_CPMI_aggregate_jaccard_by_platform_subspace_k.csv", index=False)

available2 = [c for c in take_pcts if c in set(jacc_plat_sub_k["k"].astype(str).unique())]
jacc_plat_sub_pivot = (
    jacc_plat_sub_k.pivot(index=["setup","subspace"], columns="k", values="jaccard")
      .reindex(columns=available2)
      .reset_index()
)
jacc_plat_sub_pivot.to_csv(OUT_DIR / "SHAP_vs_CPMI_aggregate_jaccard_by_platform_subspace_pivot.csv", index=False)

print("[OK] Aggregates →",
      OUT_DIR / "SHAP_vs_CPMI_aggregate_corr_by_subspace_PLATFORM.csv", ",",
      OUT_DIR / "SHAP_vs_CPMI_aggregate_corr_by_platform_subspace.csv", ",",
      OUT_DIR / "SHAP_vs_CPMI_aggregate_jaccard_by_subspace_k_PLATFORM.csv", ",",
      OUT_DIR / "SHAP_vs_CPMI_aggregate_jaccard_by_subspace_pivot_PLATFORM.csv", ",",
      OUT_DIR / "SHAP_vs_CPMI_aggregate_jaccard_by_platform_subspace_k.csv", ",",
      OUT_DIR / "SHAP_vs_CPMI_aggregate_jaccard_by_platform_subspace_pivot.csv")


[OK] Using summary: /Users/hsiaopingni/octaneX_v7_4functions/Results/Explainability_SHAP_BestPlatforms/SHAP_vs_CPMI_summary_PLATFORM.csv
[OK] Aggregates → /Users/hsiaopingni/octaneX_v7_4functions/Results/Explainability_SHAP_BestPlatforms/SHAP_vs_CPMI_aggregate_corr_by_subspace_PLATFORM.csv , /Users/hsiaopingni/octaneX_v7_4functions/Results/Explainability_SHAP_BestPlatforms/SHAP_vs_CPMI_aggregate_corr_by_platform_subspace.csv , /Users/hsiaopingni/octaneX_v7_4functions/Results/Explainability_SHAP_BestPlatforms/SHAP_vs_CPMI_aggregate_jaccard_by_subspace_k_PLATFORM.csv , /Users/hsiaopingni/octaneX_v7_4functions/Results/Explainability_SHAP_BestPlatforms/SHAP_vs_CPMI_aggregate_jaccard_by_subspace_pivot_PLATFORM.csv , /Users/hsiaopingni/octaneX_v7_4functions/Results/Explainability_SHAP_BestPlatforms/SHAP_vs_CPMI_aggregate_jaccard_by_platform_subspace_k.csv , /Users/hsiaopingni/octaneX_v7_4functions/Results/Explainability_SHAP_BestPlatforms/SHAP_vs_CPMI_aggregate_jaccard_by_platform_subspace_p

## Consistency bars (CP-MI selection supported by SHAP, BEST cases)

In [17]:
# ==== Cell: Consistency bars (CP-MI selection supported by SHAP) — PER PLATFORM ====
# PER-PLATFORM revision of your "BEST cases" script.
#
# Reads platform winners from:
#   Results/BEST_in_DesignSpace_Post_per_platform_details.csv
#
# Uses SHAP outputs from:
#   Results/Explainability_SHAP_BestPlatforms/SHAP_BESTPLAT_full_<setup>_<anomaly>_WIN<w>_KF<k>_PCT<p>_M<method>.csv
#
# CP-MI rank files under FeatureRankOUT:
#   <setup>_<win>_<kfold>_0_<subspace>.csv  (preferred)
#
# Produces:
#   - ONE plot per PLATFORM (DDR4, DDR5):
#       Results/Explainability_SHAP_BestPlatforms/consistency_v2_platform/
#         CONSISTENCY_BARS_PLATFORM_<setup>_WIN<w>_KF<k>_PLOT.png
#   - Also writes platform-level counts CSV:
#       CONSISTENCY_BARS_PLATFORM_counts.csv
#
# How it works (platform-level):
#   1) For a platform, choose (WIN,KF) from the details file (consistent across anomalies).
#   2) CP-MI selection:
#        - For each anomaly, select top-<pct>% CP-MI features per subspace (from CP-MI rank lists)
#        - Union these anomaly selections (so CP-MI selection reflects the platform's anomaly mix)
#   3) SHAP support:
#        - Load SHAP tables for each anomaly at its best config (method+pct+WIN+KF)
#        - For each anomaly, split features into SHAP-low vs SHAP-high by median SHAP
#        - Count how many CP-MI-selected features fall into SHAP-high vs SHAP-low, per subspace
#        - Sum counts across anomalies (platform totals)
#
# Note:
#   - This is a COUNT-based view (like your original). It does NOT weight by SHAP magnitude.
# -------------------------------------------------------------------------------

import os, re, glob
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt

# --- Paths (reuse if already defined in your notebook) -----------------------
ROOT     = Path(globals().get("ROOT", "/Users/hsiaopingni/octaneX_v7_4functions"))
RES_DIR  = Path(globals().get("RES_DIR", ROOT / "Results"))

DETAILS_CSV = RES_DIR / "BEST_in_DesignSpace_Post_per_platform_details.csv"
EXPL_DIR    = RES_DIR / "Explainability_SHAP_BestPlatforms"  # SHAP_BESTPLAT_full_*.csv live here
CONS_DIR    = EXPL_DIR / "consistency_v2_platform"
CONS_DIR.mkdir(parents=True, exist_ok=True)

# CP-MI rank roots (use global RANK_DIRS if already set)
if "RANK_DIRS" in globals():
    RANK_DIRS = list(globals()["RANK_DIRS"])
else:
    RANK_DIRS = [
        ROOT / "FeatureRankOUT",
        Path("/Volumes/Untitled") / "FeatureRankOUT",
        Path("/Volumes/Untitled") / "octaneX" / "FeatureRankOUT",
        Path.home() / "Desktop" / "octaneX" / "FeatureRankOUT",
    ]

SUBSPACES = ("compute","memory","sensors")

# --- Helpers -----------------------------------------------------------------
def _norm(s: str) -> str:
    """Normalize feature names to increase intersection chances."""
    s = re.sub(r"\s+", "_", str(s)).lower()
    s = s.replace("%", "pct")
    s = re.sub(r"[^a-z0-9_]+", "_", s)
    s = re.sub(r"_+", "_", s).strip("_")
    return s

def _read_csv_safe(p: Path) -> pd.DataFrame:
    try:
        return pd.read_csv(p)
    except Exception:
        return pd.DataFrame()

def _read_cpmi_list(setup: str, win: int, kfold: int, sub: str) -> list[str]:
    """
    Load CP-MI FeatureRankOUT list for one subspace. Returns ordered original names.
    Expected filename: <setup>_<win>_<kfold>_0_<sub>.csv
    Falls back to *CPMI*/*CP-MI* patterns.
    """
    fname = f"{setup}_{win}_{kfold}_0_{sub}.csv"
    # 1) direct hits
    for rd in RANK_DIRS:
        p = Path(rd) / fname
        if p.exists():
            df = _read_csv_safe(p)
            if df.empty:
                continue
            col = "feature" if "feature" in df.columns else df.columns[0]
            return df[col].dropna().astype(str).tolist()
    # 2) fallback search
    for rd in RANK_DIRS:
        rd = Path(rd)
        if not rd.exists():
            continue
        hits = list(rd.rglob(f"*{setup}*{sub}*CPMI*.csv")) + list(rd.rglob(f"*{setup}*{sub}*CP-MI*.csv"))
        if hits:
            df = _read_csv_safe(hits[0])
            if df.empty:
                continue
            col = "feature" if "feature" in df.columns else df.columns[0]
            return df[col].dropna().astype(str).tolist()
    return []

def _cpmi_selection_at_pct(setup: str, win: int, kfold: int, pct: int) -> dict:
    """
    Return dict{subspace: [features]}: top-<pct>% per subspace (ceil, min 1 if pct>0).
    """
    out = {}
    for sub in SUBSPACES:
        feats = _read_cpmi_list(setup, win, kfold, sub)
        if feats:
            k = int(np.ceil(len(feats) * (pct / 100.0)))
            if pct > 0 and k == 0:
                k = 1
            out[sub] = feats[:k]
        else:
            out[sub] = []
    return out

def _find_shap_bestplat_full(setup: str, anomaly: str, win: int, kfold: int, pct: int, method: str) -> Path | None:
    """
    Prefer exact BEST file:
      SHAP_BESTPLAT_full_<setup>_<anomaly>_WIN<w>_KF<k>_PCT<p>_M<method>.csv
    Fallback: any PCT for the same (setup, anomaly, win, kfold, method).
    """
    exact = EXPL_DIR / f"SHAP_BESTPLAT_full_{setup}_{anomaly}_WIN{win}_KF{kfold}_PCT{pct}_M{method}.csv"
    if exact.exists():
        return exact
    pat = str(EXPL_DIR / f"SHAP_BESTPLAT_full_{setup}_{anomaly}_WIN{win}_KF{kfold}_PCT*_M{method}.csv")
    alts = sorted(glob.glob(pat))
    return Path(alts[0]) if alts else None

def _split_high_low(shap_df: pd.DataFrame):
    """
    Binary split by median SHAP so both sides are non-empty when >1 feature.
    Returns (low_features, high_features) as pd.Series of feature strings.
    """
    if shap_df.empty:
        return pd.Series([], dtype=str), pd.Series([], dtype=str)
    med = shap_df["shap_mean_abs"].median()
    high_idx = shap_df[shap_df["shap_mean_abs"] >= med]["feature"].astype(str)
    low_idx  = shap_df[shap_df["shap_mean_abs"]  < med]["feature"].astype(str)
    return low_idx, high_idx

def _debug_platform(setup, win, kfold, detail_rows, by_sub_counts):
    print(f"\n[PLATFORM CHECK] {setup}  WIN={win}  K={kfold}")
    print("  anomalies:", ", ".join(sorted(detail_rows["anomaly"].astype(str).unique().tolist())))
    for sub in SUBSPACES:
        lo = by_sub_counts[sub]["low"]
        hi = by_sub_counts[sub]["high"]
        tot = lo + hi
        print(f"  {sub}: SHAP-low={lo}  SHAP-high={hi}  total={tot}")

# --- Main --------------------------------------------------------------------
def plot_cpmi_supported_by_shap_per_platform():
    if not DETAILS_CSV.exists():
        raise FileNotFoundError(f"Missing platform details CSV: {DETAILS_CSV}")

    details = pd.read_csv(DETAILS_CSV).copy()
    details.columns = [c.lower() for c in details.columns]

    # normalize column names
    if "best_method" in details.columns and "method" not in details.columns:
        details = details.rename(columns={"best_method":"method"})
    if "best_pct_by_median" not in details.columns and "pct" in details.columns:
        details = details.rename(columns={"pct":"best_pct_by_median"})

    need = {"setup","anomaly","win","kfold","best_pct_by_median","method"}
    missing = need - set(details.columns)
    if missing:
        raise KeyError(f"Platform details CSV missing columns: {sorted(missing)}. Have: {list(details.columns)}")

    out_rows = []

    for setup in ["DDR4","DDR5"]:
        dplat = details[details["setup"].astype(str).str.upper() == setup].copy()
        if dplat.empty:
            print(f"[WARN] No detail rows for platform={setup}")
            continue

        # WIN/K should be consistent across anomalies for the platform
        win = int(pd.to_numeric(dplat["win"], errors="coerce").dropna().iloc[0])
        kfold = int(pd.to_numeric(dplat["kfold"], errors="coerce").dropna().iloc[0])

        # Platform aggregate counts
        by_sub = {sub: {"low": 0, "high": 0} for sub in SUBSPACES}

        # --- Loop anomalies in this platform ---
        for _, r in dplat.iterrows():
            anomaly = str(r["anomaly"]).strip()
            pct     = int(pd.to_numeric(r["best_pct_by_median"], errors="coerce"))
            method  = str(r["method"]).strip()

            # 1) CP-MI selection for THIS anomaly at its best pct (based on CP-MI lists at platform WIN/K)
            sel = _cpmi_selection_at_pct(setup, win, kfold, pct)

            # Build per-subspace normalized selection sets (for quick membership)
            sel_norm = {sub: set(_norm(f) for f in sel[sub]) for sub in SUBSPACES}

            # 2) Load SHAP best full for THIS anomaly
            shap_csv = _find_shap_bestplat_full(setup, anomaly, win, kfold, pct, method)
            shap_df = pd.DataFrame()
            if shap_csv is not None and shap_csv.exists():
                shap_df = _read_csv_safe(shap_csv)
                if not shap_df.empty:
                    shap_df.columns = [c.lower() for c in shap_df.columns]
                    if "feature" not in shap_df.columns:
                        shap_df = shap_df.rename(columns={shap_df.columns[0]: "feature"})
                    if "shap_mean_abs" not in shap_df.columns and "importance" in shap_df.columns:
                        shap_df = shap_df.rename(columns={"importance": "shap_mean_abs"})
                    if "subspace" not in shap_df.columns:
                        # if missing, we can't do subspace-wise counts reliably
                        shap_df["subspace"] = "compute"
                    shap_df["feature"] = shap_df["feature"].astype(str)
                    shap_df["subspace"] = shap_df["subspace"].astype(str).str.lower()
                    shap_df["shap_mean_abs"] = pd.to_numeric(shap_df["shap_mean_abs"], errors="coerce").fillna(0.0)

            # If missing SHAP, skip this anomaly gracefully
            if shap_df.empty:
                print(f"[WARN] Missing/empty SHAP for {setup}/{anomaly} (WIN={win} K={kfold} PCT={pct} M={method})")
                continue

            # 3) SHAP split (low/high by median)
            low_f, high_f = _split_high_low(shap_df)
            low_norm  = set(low_f.map(_norm))
            high_norm = set(high_f.map(_norm))

            # 4) Count CP-MI-selected features that fall into SHAP-low vs SHAP-high, per subspace
            #    We do membership by normalized name (robust)
            for sub in SUBSPACES:
                if not sel_norm[sub]:
                    continue
                # A feature is "supported by SHAP-high" if it appears in high_norm
                hi = len(sel_norm[sub] & high_norm)
                # Count "low" as those selected but not in high; we can intersect with low_norm explicitly
                lo = len(sel_norm[sub] & low_norm)
                # If some selected features are in neither (e.g., not present in SHAP file), ignore them
                by_sub[sub]["high"] += hi
                by_sub[sub]["low"]  += lo

        # --- Plot ONE grouped-bar figure per platform ---
        cats  = list(SUBSPACES)
        lows  = [by_sub[s]["low"]  for s in cats]
        highs = [by_sub[s]["high"] for s in cats]

        fig = plt.figure(figsize=(12, 6))
        x = np.arange(len(cats)); w = 0.35
        plt.bar(x - w/2, lows,  width=w, label="SHAP-low")
        plt.bar(x + w/2, highs, width=w, label="SHAP-high")
        plt.xticks(x, cats)
        plt.ylabel("Count of CP-MI-selected features")
        plt.title(f"{setup} • WIN={win} K={kfold}\nCP-MI selection supported by SHAP (summed across anomalies)")
        plt.legend()
        plt.tight_layout()

        outp = CONS_DIR / f"CONSISTENCY_BARS_PLATFORM_{setup}_WIN{win}_KF{kfold}_PLOT.png"
        fig.savefig(outp, dpi=220)
        plt.close(fig)
        print(f"[OK] Plot saved: {outp}")

        # record counts for CSV
        for sub in SUBSPACES:
            out_rows.append({
                "setup": setup, "win": win, "kfold": kfold,
                "subspace": sub,
                "count_shap_low": int(by_sub[sub]["low"]),
                "count_shap_high": int(by_sub[sub]["high"]),
                "count_total_counted": int(by_sub[sub]["low"] + by_sub[sub]["high"]),
                "note": "counts are summed across platform anomalies; only features present in SHAP file contribute",
            })

        _debug_platform(setup, win, kfold, dplat, by_sub)

    # Save platform-level counts
    if out_rows:
        out_df = pd.DataFrame(out_rows)
        out_csv = CONS_DIR / "CONSISTENCY_BARS_PLATFORM_counts.csv"
        out_df.to_csv(out_csv, index=False)
        print(f"\n[OK] Counts CSV → {out_csv}")
    else:
        print("\n[WARN] No platform consistency counts produced (check SHAP + details CSV).")

# Run
plot_cpmi_supported_by_shap_per_platform()


[OK] Plot saved: /Users/hsiaopingni/octaneX_v7_4functions/Results/Explainability_SHAP_BestPlatforms/consistency_v2_platform/CONSISTENCY_BARS_PLATFORM_DDR4_WIN512_KF3_PLOT.png

[PLATFORM CHECK] DDR4  WIN=512  K=3
  anomalies: DROOP, RH
  compute: SHAP-low=16  SHAP-high=10  total=26
  memory: SHAP-low=2  SHAP-high=6  total=8
  sensors: SHAP-low=0  SHAP-high=4  total=4
[OK] Plot saved: /Users/hsiaopingni/octaneX_v7_4functions/Results/Explainability_SHAP_BestPlatforms/consistency_v2_platform/CONSISTENCY_BARS_PLATFORM_DDR5_WIN1024_KF5_PLOT.png

[PLATFORM CHECK] DDR5  WIN=1024  K=5
  anomalies: DROOP, SPECTRE
  compute: SHAP-low=52  SHAP-high=44  total=96
  memory: SHAP-low=12  SHAP-high=21  total=33
  sensors: SHAP-low=5  SHAP-high=6  total=11

[OK] Counts CSV → /Users/hsiaopingni/octaneX_v7_4functions/Results/Explainability_SHAP_BestPlatforms/consistency_v2_platform/CONSISTENCY_BARS_PLATFORM_counts.csv


# OPTIONAL hybrid promoter (CP-MI baseline + SHAP boost)

In [20]:
# === JUPYTER: Build FeatureRankOUT_HYBRID PER PLATFORM — ACROSS ALL (WIN,KF) GRID ===
# UPDATED per your request:
#   ✅ "building hybrid ranking across WIN and F like CP-MI ranking"
#
# What this does:
#   - Instead of hard-coding (WIN,KF) per platform, it scans ALL (WINS × KFOLDS) available,
#     exactly like your CP-MI rank grid exists.
#   - For each platform (DDR4, DDR5) and for each (win,kfold):
#       1) Load CP-MI ranks from FeatureRankOUT/<setup>_<win>_<kfold>_0_{sub}.csv
#       2) Pool SHAP across that platform's anomalies at that SAME (win,kfold),
#          using the best pct + method per anomaly from:
#             Results/BEST_in_DesignSpace_Post_per_platform_details.csv
#       3) Join CP-MI and pooled SHAP by normalized feature name
#       4) Compute hybrid_score per subspace and write:
#             FeatureRankOUT_HYBRID/<setup>_<win>_<kfold>_0_{compute|memory|sensors}.csv
#       5) Save the joined table for audit:
#             Results/Explainability_SHAP_BestPlatforms/cmp_cpmi_vs_shap/PLATFORM_<setup>_WIN<w>_KF<k>__SHAP_vs_CPMI_joined.csv
#
# Inputs:
#   - Results/BEST_in_DesignSpace_Post_per_platform_details.csv
#   - FeatureRankOUT/<setup>_<win>_<kfold>_0_{compute|memory|sensors}.csv  (CP-MI ranks)
#   - Results/Explainability_SHAP_BestPlatforms/SHAP_BESTPLAT_full_<setup>_<anomaly>_WIN<w>_KF<k>_PCT<p>_M<method>.csv (SHAP)
#
# Output:
#   - FeatureRankOUT_HYBRID/<setup>_<win>_<kfold>_0_{compute|memory|sensors}.csv
#
# Notes:
#   - If SHAP for a given (win,kfold) isn't available (no SHAP files), it will SKIP that combo.
#   - If CP-MI ranks missing for a given (win,kfold), it will SKIP that combo.
# --------------------------------------------------------------------------------------------

import glob, re
from pathlib import Path
from typing import Optional, Dict, Tuple, List

import numpy as np
import pandas as pd

SUBSPACES = ("compute", "memory", "sensors")

# ------------------ Normalization ------------------
def _norm_name(s: str) -> str:
    s = re.sub(r"\s+", "_", str(s)).lower()
    s = s.replace("%", "pct")
    s = re.sub(r"[^a-z0-9_]+", "_", s)
    s = re.sub(r"_+", "_", s).strip("_")
    return s

def _read_csv(p: Path) -> pd.DataFrame:
    try:
        return pd.read_csv(p)
    except Exception:
        return pd.DataFrame()

# ------------------ Locate CP-MI rank CSVs ------------------
def _find_cpmi_csvs(root: Path, setup: str, win: int, kfold: int, rank_dirs=None) -> Dict[str, Optional[Path]]:
    out = {s: None for s in SUBSPACES}
    search_roots = rank_dirs or [
        root / "FeatureRankOUT",
        Path("/Volumes/Untitled") / "FeatureRankOUT",
        Path("/Volumes/Untitled") / "octaneX" / "FeatureRankOUT",
        Path.home() / "Desktop" / "octaneX" / "FeatureRankOUT",
    ]
    for sub in SUBSPACES:
        fname = f"{setup}_{win}_{kfold}_0_{sub}.csv"
        for rd in search_roots:
            p = Path(rd) / fname
            if p.exists():
                out[sub] = p
                break
        if out[sub] is None:
            for rd in search_roots:
                rd = Path(rd)
                if not rd.exists():
                    continue
                hits = list(rd.rglob(f"*{setup}*{sub}*CPMI*.csv")) + list(rd.rglob(f"*{setup}*{sub}*CP-MI*.csv"))
                if hits:
                    out[sub] = Path(hits[0])
                    break
    return out

def _read_cpmi_rank(root: Path, setup: str, win: int, kfold: int, rank_dirs=None) -> pd.DataFrame:
    """
    Return a long table:
      feature_cpmi, subspace_cpmi, cpmi_score, cpmi_rank, feature_norm
    """
    paths = _find_cpmi_csvs(root, setup, win, kfold, rank_dirs=rank_dirs)
    rows = []
    for sub, p in paths.items():
        if p is None:
            continue
        df = _read_csv(p)
        if df.empty:
            continue

        cols_lower = {c.lower(): c for c in df.columns}
        fcol = cols_lower.get("feature") or list(df.columns)[0]

        # Prefer a numeric score column if present
        num_cols = [c for c in df.columns if c != fcol and pd.api.types.is_numeric_dtype(df[c])]
        if num_cols:
            s_col = num_cols[0]
            scores = pd.to_numeric(df[s_col], errors="coerce").fillna(0.0).values
        else:
            # fallback: descending pseudo-scores by position
            scores = np.linspace(1.0, 0.0, num=len(df), endpoint=False)

        tmp = pd.DataFrame({
            "feature_cpmi": df[fcol].astype(str).values,
            "subspace_cpmi": sub,
            "cpmi_score": scores
        })
        tmp["cpmi_rank"] = pd.Series(scores).rank(ascending=False, method="dense").astype(int)
        tmp["feature_norm"] = tmp["feature_cpmi"].map(_norm_name)
        rows.append(tmp)

    if not rows:
        return pd.DataFrame(columns=["feature_cpmi","subspace_cpmi","cpmi_score","cpmi_rank","feature_norm"])
    return pd.concat(rows, ignore_index=True)

# ------------------ SHAP BESTPLAT full reader ------------------
def _find_shap_bestplat(expl_dir: Path, setup: str, anomaly: str, win: int, kfold: int, pct: int, method: str) -> Optional[Path]:
    exact = expl_dir / f"SHAP_BESTPLAT_full_{setup}_{anomaly}_WIN{win}_KF{kfold}_PCT{pct}_M{method}.csv"
    if exact.exists():
        return exact
    # fallback: any pct for same (setup,anomaly,win,kfold,method)
    pat = str(expl_dir / f"SHAP_BESTPLAT_full_{setup}_{anomaly}_WIN{win}_KF{kfold}_PCT*_M{method}.csv")
    hits = sorted(glob.glob(pat))
    return Path(hits[0]) if hits else None

def _read_shap_bestplat(expl_dir: Path, setup: str, anomaly: str, win: int, kfold: int, pct: int, method: str) -> pd.DataFrame:
    """
    Returns long table:
      feature_shap, subspace_shap, shap_mean_abs, shap_rank, feature_norm
    """
    p = _find_shap_bestplat(expl_dir, setup, anomaly, win, kfold, pct, method)
    if p is None:
        return pd.DataFrame(columns=["feature_shap","subspace_shap","shap_mean_abs","shap_rank","feature_norm"])
    df = _read_csv(p)
    if df.empty:
        return pd.DataFrame(columns=["feature_shap","subspace_shap","shap_mean_abs","shap_rank","feature_norm"])

    cols_lower = {c.lower(): c for c in df.columns}
    fcol = cols_lower.get("feature") or list(df.columns)[0]
    scol = cols_lower.get("shap_mean_abs") or cols_lower.get("importance")
    subc = cols_lower.get("subspace")

    out = pd.DataFrame({"feature_shap": df[fcol].astype(str)})
    out["subspace_shap"] = df[subc].astype(str).str.lower() if subc else "compute"
    out["shap_mean_abs"] = pd.to_numeric(df[scol], errors="coerce").fillna(0.0) if scol else 0.0
    out["feature_norm"]  = out["feature_shap"].map(_norm_name)

    # If multiple rows map to same normalized feature/subspace, keep MAX SHAP
    out = (out.groupby(["feature_norm","subspace_shap"], as_index=False)["shap_mean_abs"]
              .max()
              .merge(out.drop_duplicates(["feature_norm","subspace_shap"])[["feature_norm","subspace_shap","feature_shap"]],
                     on=["feature_norm","subspace_shap"], how="left"))

    out["shap_rank"] = out.groupby("subspace_shap")["shap_mean_abs"].rank(ascending=False, method="dense").astype(int)
    return out[["feature_shap","subspace_shap","shap_mean_abs","shap_rank","feature_norm"]]

# ------------------ Pool SHAP across platform anomalies for a given (win,kfold) ------------------
def _pool_shap_for_platform(details_df: pd.DataFrame, expl_dir: Path, setup: str, win: int, kfold: int) -> pd.DataFrame:
    """
    Pool SHAP across all anomalies for this (setup,win,kfold):
      - Use best_pct_by_median + method per anomaly from BEST_in_DesignSpace_Post_per_platform_details.csv
      - Load SHAP_BESTPLAT_full_* for each anomaly (if exists)
      - Pool by (feature_norm, subspace) using MAX shap_mean_abs across anomalies
    """
    mask = (
        (details_df["setup"].astype(str).str.upper() == setup.upper()) &
        (pd.to_numeric(details_df["win"], errors="coerce") == int(win)) &
        (pd.to_numeric(details_df["kfold"], errors="coerce") == int(kfold))
    )
    sub = details_df.loc[mask].copy()
    if sub.empty:
        return pd.DataFrame(columns=["feature_shap","subspace_shap","shap_mean_abs","shap_rank","feature_norm"])

    shap_parts = []
    for _, r in sub.iterrows():
        anomaly = str(r["anomaly"]).strip().upper()
        pct = int(pd.to_numeric(r["best_pct_by_median"], errors="coerce"))
        method = str(r["method"]).strip()
        s = _read_shap_bestplat(expl_dir, setup, anomaly, win, kfold, pct, method)
        if not s.empty:
            s = s.copy()
            s["anomaly"] = anomaly
            shap_parts.append(s)

    if not shap_parts:
        return pd.DataFrame(columns=["feature_shap","subspace_shap","shap_mean_abs","shap_rank","feature_norm"])

    shap_all = pd.concat(shap_parts, ignore_index=True)

    pooled = (shap_all.groupby(["feature_norm","subspace_shap"], as_index=False)["shap_mean_abs"]
                    .max()
                    .merge(shap_all.drop_duplicates(["feature_norm","subspace_shap"])[["feature_norm","subspace_shap","feature_shap"]],
                           on=["feature_norm","subspace_shap"], how="left"))

    pooled["shap_rank"] = pooled.groupby("subspace_shap")["shap_mean_abs"].rank(ascending=False, method="dense").astype(int)
    return pooled[["feature_shap","subspace_shap","shap_mean_abs","shap_rank","feature_norm"]]

# ------------------ Hybrid scoring + writer ------------------
def _percentile(series: pd.Series) -> pd.Series:
    x = pd.Series(series).astype(float).fillna(0.0)
    if len(x) <= 1:
        return pd.Series(np.zeros(len(x)), index=x.index, dtype=float)
    r = x.rank(ascending=True, method="average")
    return (r - 1) / (len(x) - 1)

def _write_hybrid_from_joined(root: Path, joined_df: pd.DataFrame, setup: str, win: int, kfold: int,
                              alpha=0.60, beta=0.40, lam=0.05, q=0.80):
    """
    Writes one hybrid CSV per subspace:
      FeatureRankOUT_HYBRID/<setup>_<win>_<kfold>_0_<sub>.csv
    """
    hyb_dir = root / "FeatureRankOUT_HYBRID"
    hyb_dir.mkdir(parents=True, exist_ok=True)

    for sub in ("compute", "memory", "sensors"):
        sub_df = joined_df[joined_df["subspace"] == sub].copy()
        if sub_df.empty:
            continue

        sub_df["cpmi_score"] = pd.to_numeric(sub_df.get("cpmi_score", 0.0), errors="coerce").fillna(0.0)
        sub_df["shap_mean_abs"] = pd.to_numeric(sub_df.get("shap_mean_abs", 0.0), errors="coerce").fillna(0.0)

        sub_df["r_cpmi"] = _percentile(sub_df["cpmi_score"])
        sub_df["r_shap"] = _percentile(sub_df["shap_mean_abs"])

        thr = np.nanpercentile(sub_df["r_shap"], q * 100.0) if len(sub_df) else 1.0
        sub_df["bonus"] = (sub_df["r_shap"] >= thr).astype(float) * lam

        sub_df["hybrid_score"] = alpha * sub_df["r_cpmi"] + beta * sub_df["r_shap"] + sub_df["bonus"]
        sub_df["hybrid_rank"]  = sub_df["hybrid_score"].rank(ascending=False, method="dense").astype(int)

        out = hyb_dir / f"{setup}_{win}_{kfold}_0_{sub}.csv"
        sub_df[["feature_display","hybrid_score","hybrid_rank","cpmi_score","shap_mean_abs"]].to_csv(out, index=False)
        print(f"[OK] Hybrid written → {out}")

# ------------------ Discover available (win,kfold) combos ------------------
def _discover_grid_from_cpmi(root: Path, setup: str, rank_dirs=None) -> List[Tuple[int,int]]:
    """
    Scan FeatureRankOUT files for this setup to discover all (win,kfold) combos present.
    Looks for: <setup>_<win>_<kfold>_0_<sub>.csv
    """
    search_roots = rank_dirs or [
        root / "FeatureRankOUT",
        Path("/Volumes/Untitled") / "FeatureRankOUT",
        Path("/Volumes/Untitled") / "octaneX" / "FeatureRankOUT",
        Path.home() / "Desktop" / "octaneX" / "FeatureRankOUT",
    ]
    combos = set()
    pat = re.compile(rf"^{re.escape(setup)}_(\d+)_(\d+)_0_(compute|memory|sensors)\.csv$", re.I)

    for rd in search_roots:
        rd = Path(rd)
        if not rd.exists():
            continue
        for p in rd.glob(f"{setup}_*_*_0_*.csv"):
            m = pat.match(p.name)
            if not m:
                continue
            win = int(m.group(1))
            kf  = int(m.group(2))
            combos.add((win, kf))

    return sorted(combos, key=lambda x: (x[0], x[1]))

# ------------------ Main driver ------------------
def build_hybrid_rankouts_per_platform_across_grid(
    root_path: str,
    alpha=0.60, beta=0.40, lam=0.05, q=0.80,
    rank_dirs=None,
):
    root = Path(root_path).expanduser().resolve()
    res_dir  = root / "Results"
    expl_dir = res_dir / "Explainability_SHAP_BestPlatforms"
    details_path = res_dir / "BEST_in_DesignSpace_Post_per_platform_details.csv"

    if not details_path.exists():
        raise FileNotFoundError(f"Missing per-platform details CSV: {details_path}")

    details = pd.read_csv(details_path)
    details.columns = [c.lower() for c in details.columns]
    if "best_method" in details.columns and "method" not in details.columns:
        details = details.rename(columns={"best_method":"method"})
    if "best_pct_by_median" not in details.columns and "pct" in details.columns:
        details = details.rename(columns={"pct":"best_pct_by_median"})

    need = {"setup","anomaly","win","kfold","best_pct_by_median","method"}
    miss = need - set(details.columns)
    if miss:
        raise KeyError(f"Details CSV missing required columns: {sorted(miss)}")

    out_join_dir = expl_dir / "cmp_cpmi_vs_shap"
    out_join_dir.mkdir(parents=True, exist_ok=True)

    processed = 0
    skipped = 0

    for setup in ["DDR4", "DDR5"]:
        print(f"\n[PLATFORM] {setup}: discovering CP-MI grid ...")
        combos = _discover_grid_from_cpmi(root, setup, rank_dirs=rank_dirs)
        if not combos:
            print(f"[WARN] No CP-MI rank files found for {setup}. Skipping.")
            continue

        print(f"[INFO] {setup}: found {len(combos)} (WIN,KF) combos from CP-MI ranks.")

        for win, kfold in combos:
            # 1) load CP-MI ranks for this (win,kfold)
            cpmi = _read_cpmi_rank(root, setup, win, kfold, rank_dirs=rank_dirs)
            if cpmi.empty:
                skipped += 1
                continue

            # 2) pool SHAP for this (win,kfold) using per-platform details (best pct/method per anomaly)
            shap = _pool_shap_for_platform(details, expl_dir, setup, win, kfold)
            if shap.empty:
                # no SHAP available for this combo -> skip
                skipped += 1
                continue

            # 3) join
            joined = pd.merge(
                cpmi.drop_duplicates(["feature_norm","subspace_cpmi"]),
                shap.drop_duplicates(["feature_norm","subspace_shap"]),
                on="feature_norm",
                how="outer",
                suffixes=("_cpmi", "_shap")
            )

            joined["feature_display"] = joined["feature_cpmi"].fillna(joined["feature_shap"])
            joined["subspace"] = joined["subspace_cpmi"].fillna(joined["subspace_shap"])

            joined["cpmi_rank"] = pd.to_numeric(joined.get("cpmi_rank"), errors="coerce")
            joined["shap_rank"] = pd.to_numeric(joined.get("shap_rank"), errors="coerce")
            joined["cpmi_score"] = pd.to_numeric(joined.get("cpmi_score"), errors="coerce").fillna(0.0)
            joined["shap_mean_abs"] = pd.to_numeric(joined.get("shap_mean_abs"), errors="coerce").fillna(0.0)

            joined["delta_rank"]  = joined["cpmi_rank"].fillna(np.inf) - joined["shap_rank"].fillna(np.inf)
            joined["delta_score"] = joined["shap_mean_abs"] - joined["cpmi_score"]

            joined = joined.sort_values(["subspace", "shap_rank", "cpmi_rank"], na_position="last")

            out_csv = out_join_dir / f"PLATFORM_{setup}_WIN{win}_KF{kfold}__SHAP_vs_CPMI_joined.csv"
            joined.to_csv(out_csv, index=False)
            print(f"[OK] Joined saved → {out_csv.name}")

            # 4) write hybrid rankouts per subspace
            _write_hybrid_from_joined(
                root, joined, setup, win, kfold,
                alpha=alpha, beta=beta, lam=lam, q=q
            )

            processed += 1

    print(f"\n[DONE] Hybrid rankouts built for {processed} (platform,WIN,KF) combos. Skipped={skipped}.")
    print(f"       Output folder: {root/'FeatureRankOUT_HYBRID'}")

# -------------------- RUN HERE --------------------
build_hybrid_rankouts_per_platform_across_grid(
    root_path="/Users/hsiaopingni/octaneX_v7_4functions",
    alpha=0.60, beta=0.40, lam=0.05, q=0.80,
    rank_dirs=None,   # optionally pass your own RANK_DIRS list
)


[PLATFORM] DDR4: discovering CP-MI grid ...
[INFO] DDR4: found 24 (WIN,KF) combos from CP-MI ranks.
[OK] Joined saved → PLATFORM_DDR4_WIN512_KF3__SHAP_vs_CPMI_joined.csv
[OK] Hybrid written → /Users/hsiaopingni/octaneX_v7_4functions/FeatureRankOUT_HYBRID/DDR4_512_3_0_compute.csv
[OK] Hybrid written → /Users/hsiaopingni/octaneX_v7_4functions/FeatureRankOUT_HYBRID/DDR4_512_3_0_memory.csv
[OK] Hybrid written → /Users/hsiaopingni/octaneX_v7_4functions/FeatureRankOUT_HYBRID/DDR4_512_3_0_sensors.csv

[PLATFORM] DDR5: discovering CP-MI grid ...
[INFO] DDR5: found 24 (WIN,KF) combos from CP-MI ranks.
[OK] Joined saved → PLATFORM_DDR5_WIN1024_KF5__SHAP_vs_CPMI_joined.csv
[OK] Hybrid written → /Users/hsiaopingni/octaneX_v7_4functions/FeatureRankOUT_HYBRID/DDR5_1024_5_0_compute.csv
[OK] Hybrid written → /Users/hsiaopingni/octaneX_v7_4functions/FeatureRankOUT_HYBRID/DDR5_1024_5_0_memory.csv
[OK] Hybrid written → /Users/hsiaopingni/octaneX_v7_4functions/FeatureRankOUT_HYBRID/DDR5_1024_5_0_sensors.

In [21]:
# === PER-PLATFORM editor package export (AUTO from per-platform winners) ===
# UPDATED per your current per-platform pipeline.
#
# What this does (per platform):
#   - Uses per-platform winners (auto; no hard-coded WIN/K/PCT):
#       Results/BEST_in_DesignSpace_Post_per_platform.csv
#       Results/BEST_in_DesignSpace_Post_per_platform_details.csv
#   - Exports:
#       1) _editor_package/workloads_performance_platform.csv
#          (platform + anomaly rows with chosen WIN/K, best_pct, best_method and metrics if present)
#       2) _editor_package/sweep_counts_by_subspace_platform.csv
#          (counts vs top-% thresholds for CP-MI ranks at the chosen (WIN,K) per platform)
#          OPTIONAL: also include HYBRID ranks if FeatureRankOUT_HYBRID exists (toggle below)
#       3) Plots:
#          - _editor_package/sweep_feature_counts_per_subspace_<setup>.png
#          - _editor_package/sweep_feature_counts_per_subspace_PLATFORM.png
#
# Inputs:
#   - Results/BEST_in_DesignSpace_Post_per_platform.csv
#   - Results/BEST_in_DesignSpace_Post_per_platform_details.csv
#   - FeatureRankOUT/<setup>_<win>_<kfold>_0_{compute|memory|sensors}.csv
#   - (optional) FeatureRankOUT_HYBRID/<setup>_<win>_<kfold>_0_{compute|memory|sensors}.csv
# ---------------------------------------------------------------------------------------------

from pathlib import Path
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt

# ----------- CONFIG ----------
ROOT = Path(globals().get("ROOT", "/Users/hsiaopingni/octaneX_v7_4functions"))
RES  = Path(globals().get("RES_DIR", ROOT / "Results"))
# normalize if notebook points too deep
if RES.name in ("Explainability_SHAP_BestCases", "Explainability_SHAP_BestPlatforms"):
    RES = RES.parent

BEST_PLATFORM        = RES / "BEST_in_DesignSpace_Post_per_platform.csv"
BEST_PLATFORM_DETAIL = RES / "BEST_in_DesignSpace_Post_per_platform_details.csv"

RANK_DIR = ROOT / "FeatureRankOUT"            # CP-MI baseline ranks
HYB_DIR  = ROOT / "FeatureRankOUT_HYBRID"     # optional hybrid ranks
OUT_DIR  = RES / "_editor_package"
OUT_DIR.mkdir(parents=True, exist_ok=True)

SUBSPACES = ("compute","memory","sensors")
TOP_PCTS  = list(range(10, 101, 10))

INCLUDE_HYBRID_COUNTS = True   # set False if you only want CP-MI counts

# ---------- Helpers ----------
def read_platform_best(best_csv: Path) -> pd.DataFrame:
    df = pd.read_csv(best_csv)
    df.columns = [c.lower() for c in df.columns]
    # minimal expected columns: setup, win, kfold
    need = {"setup","win","kfold"}
    miss = need - set(df.columns)
    if miss:
        raise KeyError(f"{best_csv} missing required columns: {sorted(miss)}. Have: {list(df.columns)}")
    return df

def read_platform_details(details_csv: Path) -> pd.DataFrame:
    df = pd.read_csv(details_csv)
    df.columns = [c.lower() for c in df.columns]
    if "best_method" in df.columns and "method" not in df.columns:
        df = df.rename(columns={"best_method":"method"})
    if "best_pct_by_median" not in df.columns and "pct" in df.columns:
        df = df.rename(columns={"pct":"best_pct_by_median"})
    need = {"setup","anomaly","win","kfold","best_pct_by_median","method"}
    miss = need - set(df.columns)
    if miss:
        raise KeyError(f"{details_csv} missing required columns: {sorted(miss)}. Have: {list(df.columns)}")
    return df

def pick_score_col(df: pd.DataFrame) -> str | None:
    for c in ("cpmi_score","hybrid_score","shap_mean_abs"):
        if c in df.columns:
            return c
    # fallback: first numeric col after feature if any
    nums = df.select_dtypes(include=[np.number]).columns.tolist()
    return nums[0] if nums else (df.columns[1] if df.shape[1] >= 2 else None)

def load_rank_table(rank_dir: Path, setup: str, win: int, kfold: int, subspace: str) -> pd.DataFrame:
    p = rank_dir / f"{setup}_{win}_{kfold}_0_{subspace}.csv"
    if not p.exists():
        return pd.DataFrame()
    df = pd.read_csv(p)
    # feature column may differ by source
    feat_col = (
        "feature" if "feature" in df.columns else
        ("feature_display" if "feature_display" in df.columns else df.columns[0])
    )
    df = df.rename(columns={feat_col: "feature"})
    return df

def top_k_count(n_total: int, pct: int) -> int:
    k = int(math.ceil(n_total * pct / 100.0))
    return max(1, k) if pct > 0 else 0

def enumerate_mean_rank(values: pd.Series) -> float:
    """
    Your prior 'enumerated mean rank 1..5' scheme.
    """
    if values is None or len(values) == 0:
        return np.nan
    v = pd.to_numeric(values, errors="coerce").fillna(0.0)
    r = v.rank(ascending=True, method="average")
    p = (r - 1) / (len(v) - 1) if len(v) > 1 else pd.Series([0.0]*len(v))
    buckets = 5 - (p*5).astype(int).clip(0,4)
    return float(buckets.mean())

# ---------- Load platform winners ----------
if not BEST_PLATFORM.exists():
    raise FileNotFoundError(f"Missing platform BEST CSV: {BEST_PLATFORM}")
if not BEST_PLATFORM_DETAIL.exists():
    raise FileNotFoundError(f"Missing platform BEST details CSV: {BEST_PLATFORM_DETAIL}")

best_plat = read_platform_best(BEST_PLATFORM)
detail    = read_platform_details(BEST_PLATFORM_DETAIL)

# Build PLATFORM_CFG automatically
PLATFORM_CFG = {}
for _, r in best_plat.iterrows():
    setup = str(r["setup"]).strip()
    PLATFORM_CFG[setup] = dict(
        win=int(pd.to_numeric(r["win"], errors="coerce")),
        kfold=int(pd.to_numeric(r["kfold"], errors="coerce")),
    )

if not PLATFORM_CFG:
    raise RuntimeError("No platform configs found in BEST_in_DesignSpace_Post_per_platform.csv")

print("[OK] PLATFORM_CFG (auto):", PLATFORM_CFG)

# ---------- A) workloads_performance_platform.csv ----------
# Use the per-platform details (anomaly-specific best pct & method).
# Include any metric columns that exist (roc/auc/pr/iqr/min/max etc.).
metric_cols = [c for c in detail.columns if any(tok in c for tok in ["auc", "roc", "iqr", "min", "max", "median", "n_runs"])]

perf_rows = []
for setup, cfg in PLATFORM_CFG.items():
    win, kfold = int(cfg["win"]), int(cfg["kfold"])
    sub = detail[
        (detail["setup"].astype(str).str.upper() == setup.upper()) &
        (pd.to_numeric(detail["win"], errors="coerce") == win) &
        (pd.to_numeric(detail["kfold"], errors="coerce") == kfold)
    ].copy()
    if sub.empty:
        continue
    sub["platform_win"] = win
    sub["platform_kfold"] = kfold
    cols_keep = ["setup","anomaly","platform_win","platform_kfold","win","kfold","best_pct_by_median","method"] + metric_cols
    cols_keep = [c for c in cols_keep if c in sub.columns]
    perf_rows.append(sub[cols_keep])

perf_platform = pd.concat(perf_rows, ignore_index=True) if perf_rows else pd.DataFrame()
perf_csv = OUT_DIR / "workloads_performance_platform.csv"
perf_platform.to_csv(perf_csv, index=False)

# ---------- B) sweep_counts_by_subspace_platform.csv ----------
rows = []
for setup, cfg in PLATFORM_CFG.items():
    win, kfold = int(cfg["win"]), int(cfg["kfold"])

    for sub in SUBSPACES:
        # ---- CP-MI counts ----
        df_rank = load_rank_table(RANK_DIR, setup, win, kfold, sub)
        if not df_rank.empty:
            score_col = pick_score_col(df_rank)
            n_total = len(df_rank)
            enum_mean = enumerate_mean_rank(df_rank[score_col]) if score_col and score_col in df_rank.columns else np.nan

            for pct in TOP_PCTS:
                k = top_k_count(n_total, pct)
                rows.append({
                    "setup": setup, "win": win, "kfold": kfold,
                    "rank_source": "CPMI",
                    "subspace": sub, "top_pct": pct,
                    "n_total": n_total, "n_selected": k,
                    "enumerated_mean_rank_1to5": enum_mean
                })
        else:
            print(f"[WARN] Missing CP-MI rank file for {setup} WIN={win} K={kfold} sub={sub}")

        # ---- HYBRID counts (optional) ----
        if INCLUDE_HYBRID_COUNTS:
            df_hyb = load_rank_table(HYB_DIR, setup, win, kfold, sub)
            if not df_hyb.empty:
                score_col_h = pick_score_col(df_hyb)
                n_total_h = len(df_hyb)
                enum_mean_h = enumerate_mean_rank(df_hyb[score_col_h]) if score_col_h and score_col_h in df_hyb.columns else np.nan

                for pct in TOP_PCTS:
                    k = top_k_count(n_total_h, pct)
                    rows.append({
                        "setup": setup, "win": win, "kfold": kfold,
                        "rank_source": "HYBRID",
                        "subspace": sub, "top_pct": pct,
                        "n_total": n_total_h, "n_selected": k,
                        "enumerated_mean_rank_1to5": enum_mean_h
                    })

sweep_platform = pd.DataFrame(rows)
sweep_csv = OUT_DIR / "sweep_counts_by_subspace_platform.csv"
sweep_platform.to_csv(sweep_csv, index=False)

print("Saved:")
print(" -", perf_csv)
print(" -", sweep_csv)

# ---------- C) Plot: top-% vs number of features per subspace (per platform) ----------
plt.rcParams.update({
    "figure.dpi": 200,
    "savefig.dpi": 300,
    "font.size": 12,
    "axes.labelsize": 10,
    "axes.titlesize": 14,
    "legend.fontsize": 11,
    "lines.linewidth": 2.2,
})

colors = {"compute":"#1f77b4", "memory":"#66b3ff", "sensors":"#ff7f0e"}

# One plot per platform, CPMI only (clean)
for setup, cfg in PLATFORM_CFG.items():
    fig, ax = plt.subplots(figsize=(7.5, 4.5))
    d0 = sweep_platform[(sweep_platform["rank_source"]=="CPMI") & (sweep_platform["setup"]==setup)]
    if d0.empty:
        plt.close(fig)
        continue

    for sub in SUBSPACES:
        d = d0[d0["subspace"]==sub]
        if d.empty:
            continue
        g = d.groupby("top_pct")["n_selected"].median().reset_index()
        ax.plot(g["top_pct"], g["n_selected"], marker="o", label=sub.capitalize(), color=colors[sub])

    ax.set_xlabel("Top-% threshold")
    ax.set_ylabel("# features selected")
    ax.set_title(f"Feature count vs top-% threshold (CP-MI) — {setup} (WIN={cfg['win']}, K={cfg['kfold']})")
    ax.legend(loc="center left", bbox_to_anchor=(1.02, 0.5), frameon=False)
    fig.tight_layout(rect=[0,0,0.82,1])
    outp = OUT_DIR / f"sweep_feature_counts_per_subspace_{setup}.png"
    fig.savefig(outp, bbox_inches="tight")
    plt.close(fig)
    print(" -", outp)

# Combined plot (two platforms shown by linestyle)
fig, ax = plt.subplots(figsize=(7.8, 4.7))
linestyles = {"DDR4":"-", "DDR5":"--"}

for setup, cfg in PLATFORM_CFG.items():
    d0 = sweep_platform[(sweep_platform["rank_source"]=="CPMI") & (sweep_platform["setup"]==setup)]
    if d0.empty:
        continue
    for sub in SUBSPACES:
        d = d0[d0["subspace"]==sub]
        if d.empty:
            continue
        g = d.groupby("top_pct")["n_selected"].median().reset_index()
        ax.plot(g["top_pct"], g["n_selected"], marker="o",
                label=f"{setup} {sub.capitalize()}",
                color=colors[sub], linestyle=linestyles.get(setup, "-"))

ax.set_xlabel("Top-% threshold")
ax.set_ylabel("# features selected")
ax.set_title("Feature count vs top-% threshold (per subspace, CP-MI) — per platform")
ax.legend(loc="center left", bbox_to_anchor=(1.02, 0.5), frameon=False)
fig.tight_layout(rect=[0,0,0.78,1])

out_comb = OUT_DIR / "sweep_feature_counts_per_subspace_PLATFORM.png"
fig.savefig(out_comb, bbox_inches="tight")
plt.close(fig)
print(" -", out_comb)

[OK] PLATFORM_CFG (auto): {'DDR4': {'win': 512, 'kfold': 3}, 'DDR5': {'win': 1024, 'kfold': 5}}
Saved:
 - /Users/hsiaopingni/octaneX_v7_4functions/Results/_editor_package/workloads_performance_platform.csv
 - /Users/hsiaopingni/octaneX_v7_4functions/Results/_editor_package/sweep_counts_by_subspace_platform.csv
 - /Users/hsiaopingni/octaneX_v7_4functions/Results/_editor_package/sweep_feature_counts_per_subspace_DDR4.png
 - /Users/hsiaopingni/octaneX_v7_4functions/Results/_editor_package/sweep_feature_counts_per_subspace_DDR5.png
 - /Users/hsiaopingni/octaneX_v7_4functions/Results/_editor_package/sweep_feature_counts_per_subspace_PLATFORM.png


In [22]:
# === PER-PLATFORM enumrank tables (AUTO WIN/K from per-platform winners) ===
# UPDATED so you do NOT hard-code DDR4/DDR5 WIN/K.
#
# Builds ONE enumrank CSV per platform × subspace using the chosen (WIN,K) per platform from:
#   Results/BEST_in_DesignSpace_Post_per_platform.csv
#
# Output (6 files total):
#   OUT_DIR/enumrank__DDR4_PLATFORM_WIN<win>_KF<kfold>__compute.csv
#   OUT_DIR/enumrank__DDR4_PLATFORM_WIN<win>_KF<kfold>__memory.csv
#   OUT_DIR/enumrank__DDR4_PLATFORM_WIN<win>_KF<kfold>__sensors.csv
#   OUT_DIR/enumrank__DDR5_PLATFORM_WIN<win>_KF<kfold>__compute.csv
#   OUT_DIR/enumrank__DDR5_PLATFORM_WIN<win>_KF<kfold>__memory.csv
#   OUT_DIR/enumrank__DDR5_PLATFORM_WIN<win>_KF<kfold>__sensors.csv
#
# Notes:
# - Uses CP-MI rank tables from FeatureRankOUT by (setup,win,kfold,subspace).
# - No anomaly dimension; “PLATFORM” indicates platform-level export.
# --------------------------------------------------------------------------------------------

from pathlib import Path
import pandas as pd
import numpy as np

# ---------- paths ----------
ROOT = Path(globals().get("ROOT", "/Users/hsiaopingni/octaneX_v7_4functions"))
RES  = Path(globals().get("RES_DIR", ROOT / "Results"))
# normalize if notebook points too deep
if RES.name in ("Explainability_SHAP_BestCases", "Explainability_SHAP_BestPlatforms"):
    RES = RES.parent

BEST_PLATFORM = RES / "BEST_in_DesignSpace_Post_per_platform.csv"
if not BEST_PLATFORM.exists():
    raise FileNotFoundError(f"Missing per-platform winners CSV: {BEST_PLATFORM}")

RANK_DIR = Path(globals().get("RANK_DIR", ROOT / "FeatureRankOUT"))
OUT_DIR  = Path(globals().get("OUT_DIR", RES / "_editor_package"))
OUT_DIR.mkdir(parents=True, exist_ok=True)

SUBSPACES = ("compute","memory","sensors")

# ---------- helpers ----------
def pick_score_col(df: pd.DataFrame) -> str | None:
    for c in ("cpmi_score","hybrid_score","shap_mean_abs","score"):
        if c in df.columns:
            return c
    feat_col = (
        "feature" if "feature" in df.columns else
        ("feature_display" if "feature_display" in df.columns else df.columns[0])
    )
    nums = [c for c in df.columns if c != feat_col and pd.api.types.is_numeric_dtype(df[c])]
    return nums[0] if nums else None

def load_rank_table(rank_dir: Path, setup: str, win: int, kfold: int, subspace: str) -> pd.DataFrame:
    p = rank_dir / f"{setup}_{win}_{kfold}_0_{subspace}.csv"
    if not p.exists():
        return pd.DataFrame()
    df = pd.read_csv(p)
    feat_col = (
        "feature" if "feature" in df.columns else
        ("feature_display" if "feature_display" in df.columns else df.columns[0])
    )
    df = df.rename(columns={feat_col: "feature"})
    return df

def enum_bucket_per_feature(scores: pd.Series) -> pd.Series:
    """Return a 1..5 bucket per feature (5=top 20%)."""
    s = pd.to_numeric(scores, errors="coerce").fillna(0.0)
    if len(s) <= 1:
        return pd.Series([3]*len(s), index=s.index, dtype=int)  # neutral if trivial
    # percentile rank 0..1 (higher = better)
    r = s.rank(ascending=True, method="average")
    p = (r - 1) / (len(s) - 1)
    # map to 1..5 (5 best)
    return (5 - (p*5).astype(int).clip(0,4)).astype(int)

def write_enumrank_platform_table(setup: str, win: int, kfold: int, subspace: str) -> Path | None:
    df = load_rank_table(RANK_DIR, setup, win, kfold, subspace)
    if df.empty:
        print(f"[SKIP] Missing rank table: {setup} WIN={win} KF={kfold} sub={subspace}")
        return None

    score_col = pick_score_col(df)
    if score_col is None:
        print(f"[SKIP] No score col found in rank table for {setup} WIN={win} KF={kfold} sub={subspace}")
        return None

    tmp = df[["feature", score_col]].copy()
    tmp = tmp.rename(columns={score_col: "score"})
    tmp["score"] = pd.to_numeric(tmp["score"], errors="coerce").fillna(0.0)

    tmp["EnumRank_1to5"] = enum_bucket_per_feature(tmp["score"])

    # Percentile for transparency (0..1)
    if len(tmp) > 1:
        r = tmp["score"].rank(ascending=True, method="average")
        tmp["Percentile_0to1"] = (r - 1) / (len(tmp) - 1)
    else:
        tmp["Percentile_0to1"] = 0.5

    tmp = tmp.sort_values(["EnumRank_1to5","score"], ascending=[False, False]).reset_index(drop=True)

    out_csv = OUT_DIR / f"enumrank__{setup}_PLATFORM_WIN{win}_KF{kfold}__{subspace}.csv"
    tmp.to_csv(out_csv, index=False)
    print("[WROTE]", out_csv)
    return out_csv

# ---------- Load per-platform winners (AUTO WIN/K) ----------
bp = pd.read_csv(BEST_PLATFORM).copy()
bp.columns = [c.lower() for c in bp.columns]
need = {"setup","win","kfold"}
missing = need - set(bp.columns)
if missing:
    raise KeyError(f"{BEST_PLATFORM} missing required columns {sorted(missing)}. Have: {list(bp.columns)}")

# Build PLATFORM_CFG automatically
PLATFORM_CFG = {}
for _, r in bp.iterrows():
    setup = str(r["setup"]).strip()
    win = int(pd.to_numeric(r["win"], errors="coerce"))
    kf  = int(pd.to_numeric(r["kfold"], errors="coerce"))
    PLATFORM_CFG[setup] = dict(win=win, kfold=kf)

print("[OK] PLATFORM_CFG (auto):", PLATFORM_CFG)

# ---------- Run per platform × subspace ----------
for setup, cfg in PLATFORM_CFG.items():
    win, kfold = int(cfg["win"]), int(cfg["kfold"])
    for sub in SUBSPACES:
        write_enumrank_platform_table(setup, win, kfold, sub)


[OK] PLATFORM_CFG (auto): {'DDR4': {'win': 512, 'kfold': 3}, 'DDR5': {'win': 1024, 'kfold': 5}}
[WROTE] /Users/hsiaopingni/octaneX_v7_4functions/Results/_editor_package/enumrank__DDR4_PLATFORM_WIN512_KF3__compute.csv
[WROTE] /Users/hsiaopingni/octaneX_v7_4functions/Results/_editor_package/enumrank__DDR4_PLATFORM_WIN512_KF3__memory.csv
[WROTE] /Users/hsiaopingni/octaneX_v7_4functions/Results/_editor_package/enumrank__DDR4_PLATFORM_WIN512_KF3__sensors.csv
[WROTE] /Users/hsiaopingni/octaneX_v7_4functions/Results/_editor_package/enumrank__DDR5_PLATFORM_WIN1024_KF5__compute.csv
[WROTE] /Users/hsiaopingni/octaneX_v7_4functions/Results/_editor_package/enumrank__DDR5_PLATFORM_WIN1024_KF5__memory.csv
[WROTE] /Users/hsiaopingni/octaneX_v7_4functions/Results/_editor_package/enumrank__DDR5_PLATFORM_WIN1024_KF5__sensors.csv


# **HYBRID OCTANE — Best-Case Sweep

# *FULL MULTI-WIN / MULTI-K PIPELINE (AUTHENTIC) - hybrid feature ranking

In [49]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

# ===============================================================================================
# FULL MULTI-WIN / MULTI-K PIPELINE (AUTHENTIC)
# + DSE PLOTS (all cases)
# + TABLE_SUBSPACEWEIGHTS (all cases)
#
# UPDATE (your request):
#   ✅ Auto-adjust y-axis for EVERY DSE subplot (all WIN/K) to remove empty white space below.
#   ✅ Do NOT show any values/regions above 1.00 (metrics are clipped at 1.00).
#   ✅ BUT still leave a small visual headroom so markers/lines at 1.00 are NOT cut off.
#
# How:
#   - All data are clipped to [0, 1.00] (never > 1.00).
#   - Y-axis upper limit is allowed up to 1.02 for DISPLAY ONLY (headroom),
#     but data are never plotted above 1.00.
#   - Each subplot computes tight y-limits from its own bands/median, adds padding,
#     then clamps to [0, 1.02] with a forced minimum headroom above 1.00 if needed.
# ===============================================================================================

import gc, re, hashlib, warnings, unicodedata
from pathlib import Path
from typing import List, Dict, Optional, Tuple

import numpy as np
import pandas as pd

import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt

from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.model_selection import StratifiedShuffleSplit

warnings.filterwarnings("ignore", category=RuntimeWarning)

# ===============================================================================================
# 0) PATHS
# ===============================================================================================
DATA_DIR = Path("/Users/hsiaopingni/Desktop/SLM_RAS-main/HW_TELEMETRY_DATA_COLLECTION/TELEMETRY_DATA")
ROOT     = Path("/Users/hsiaopingni/octaneX_v7_4functions")
RES_DIR  = ROOT / "Results"
RES_DIR.mkdir(parents=True, exist_ok=True)

EXT_DRIVE = Path("/Volumes/Untitled")
EXT_RES   = EXT_DRIVE / "octaneX_results"

def _can_write_dir(p: Path) -> bool:
    try:
        p.mkdir(parents=True, exist_ok=True)
        t = p / ".write_test"
        t.write_text("ok")
        t.unlink()
        return True
    except Exception:
        return False

if _can_write_dir(EXT_RES):
    RES_DIR = EXT_RES
    RES_DIR.mkdir(parents=True, exist_ok=True)
    print(f"[OK] Using external results dir: {RES_DIR}")
else:
    print(f"[WARN] Cannot write to {EXT_RES}. Using local: {RES_DIR}")

RANK_DIRS = [
    ROOT / "FeatureRankOUT_HYBRID",
    EXT_DRIVE / "FeatureRankOUT_HYBRID",
    EXT_DRIVE / "octaneX" / "FeatureRankOUT_HYBRID",
]

# ===============================================================================================
# 1) RUN CONFIG
# ===============================================================================================
SETUPS = ["DDR4", "DDR5"]
ANOMALIES_BY_SETUP = {"DDR4": ["DROOP", "RH"], "DDR5": ["DROOP", "SPECTRE"]}

WINS   = [32, 64, 128, 512, 1024]
KFOLDS = [3, 5, 10]
PCT_SWEEP = list(range(10, 101, 10))

OVERLAP_RATIO       = 0.50
OVERLAP_RATIO_DROOP = 0.80

ROBUST_WINSOR = (2.0, 98.0)
SEED = 1337

GC_EVERY_N_PCTS   = 3
GC_EVERY_N_GROUPS = 1

META = ["label", "setup", "run_id"]
DROOP_META_COLS = ["droop_center_found", "droop_best_score_z", "droop_frac_ge_thr", "droop_vcols"]

DROP_LOW_VARIANCE_COLS = True
LOW_VAR_EPS = 1e-10

# DROOP boost (small, optional)
DROOP_BOOST = True
DROOP_BOOST_ALPHA = 0.20
DROOP_TRANSIENT_PAT = re.compile(r"__(drop|range|slope|min|max|std)", re.I)

METHOD_ORDER = ["dC_aJ", "dC_aM", "dE_aJ", "dE_aM"]

# ===============================================================================================
# 2) PLOT CONFIG
# ===============================================================================================
plt.rcParams.update({
    "font.family": "sans-serif",
    "font.size": 12,
    "axes.titlesize": 13.5,
    "axes.labelsize": 14,
    "xtick.labelsize": 12,
    "ytick.labelsize": 12,
    "legend.fontsize": 12,
    "figure.titlesize": 20,
})

METHOD_TITLE = {
    "dC_aJ": r"Scoring function $d_C(X)$ with Aggregate $a_J$",
    "dC_aM": r"Scoring function $d_C(X)$ with Aggregate $a_M$",
    "dE_aJ": r"Scoring function $d_E(X)$ with Aggregate $a_J$",
    "dE_aM": r"Scoring function $d_E(X)$ with Aggregate $a_M$",
}

# ---- Data never exceeds 1.00 ----
Y_DATA_MAX = 1.00
Y_DATA_MIN = 0.00

# ---- Display headroom so points at 1.00 aren't cut off ----
Y_DISPLAY_MAX = 1.02      # axis can go slightly above 1.00
Y_HEADROOM = 0.015        # if ymax is near 1.00, ensure at least this much headroom

# y-axis auto-range per subplot
Y_AUTO     = True
Y_PAD_FRAC = 0.06
Y_MIN_SPAN = 0.04

ALPHA_MINMAX = 0.28
ALPHA_IQR    = 0.60
GRID_ALPHA   = 0.22
GRID_LS      = "--"
GRID_LW      = 0.6

FIGSIZE = (13.6, 8.4)
WSPACE  = 0.30
HSPACE  = 0.42
TOP     = 0.88
BOTTOM  = 0.20
LEFT    = 0.075
RIGHT   = 0.985
SUPTITLE_Y = 0.975
LEGEND_Y   = 0.055

BAND_EPS = 0.008
def _ensure_visible_band(lo: np.ndarray, hi: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    lo = lo.astype(float).copy()
    hi = hi.astype(float).copy()
    flat = (np.abs(hi - lo) < 1e-12)
    if np.any(flat):
        lo2 = lo - BAND_EPS/2
        hi2 = hi + BAND_EPS/2
        lo[flat] = np.clip(lo2[flat], Y_DATA_MIN, Y_DATA_MAX)
        hi[flat] = np.clip(hi2[flat], Y_DATA_MIN, Y_DATA_MAX)
    return lo, hi

def _clip_metric(a):
    """Hard clip metrics (data) to [0,1]."""
    return np.clip(np.asarray(a, float), Y_DATA_MIN, Y_DATA_MAX)

def _auto_ylim_from_arrays(arrs: List[np.ndarray]) -> Tuple[float, float]:
    """
    Tight ylim based on plotted data bands/median.
    Axis can extend above 1.00 up to 1.02 for visual headroom, but data are clipped at 1.00.
    """
    vals = []
    for a in arrs:
        if a is None:
            continue
        a = np.asarray(a, float).ravel()
        a = a[np.isfinite(a)]
        if a.size:
            vals.append(a)
    if not vals:
        return (Y_DATA_MIN, Y_DISPLAY_MAX)

    v = np.concatenate(vals)
    ymin = float(np.min(v))
    ymax = float(np.max(v))
    if not np.isfinite(ymin) or not np.isfinite(ymax):
        return (Y_DATA_MIN, Y_DISPLAY_MAX)

    # base padding
    span = max(ymax - ymin, Y_MIN_SPAN)
    pad = Y_PAD_FRAC * span
    ymin2 = float(np.clip(ymin - pad, Y_DATA_MIN, Y_DATA_MAX))
    ymax2 = float(np.clip(ymax + pad, Y_DATA_MIN, Y_DATA_MAX))

    # enforce minimal span
    if ymax2 - ymin2 < Y_MIN_SPAN:
        mid = 0.5 * (ymin2 + ymax2)
        ymin2 = float(np.clip(mid - Y_MIN_SPAN/2, Y_DATA_MIN, Y_DATA_MAX))
        ymax2 = float(np.clip(mid + Y_MIN_SPAN/2, Y_DATA_MIN, Y_DATA_MAX))

    # ---- headroom: if we are close to 1.00, allow y-axis to go slightly above 1.00 (up to 1.02)
    # This prevents markers/lines at 1.00 from being clipped by the axis boundary.
    if ymax2 >= (Y_DATA_MAX - 1e-6):
        ymax_display = min(Y_DISPLAY_MAX, Y_DATA_MAX + Y_HEADROOM)
        ymax2 = max(ymax2, ymax_display)
    else:
        # still allow a tiny pad beyond ymax (but not beyond 1.02)
        ymax2 = min(Y_DISPLAY_MAX, ymax2)

    # lower bound should never go below 0
    ymin2 = max(Y_DATA_MIN, ymin2)

    return ymin2, ymax2

# ===============================================================================================
# 3) WEIGHT CANDIDATES (Table-3 + extras) + LOOKUP
# ===============================================================================================
def build_weight_table() -> pd.DataFrame:
    paper_cases = [
        ("T3_01", "Case 1",  1,    0,    0),
        ("T3_02", "Case 2",  0,    1,    0),
        ("T3_03", "Case 3",  0,    0,    1),
        ("T3_04", "Case 4",  1/3,  1/3,  1/3),
        ("T3_05", "Case 5",  1/4,  1/4,  2/4),
        ("T3_06", "Case 6",  1/5,  1/5,  3/5),
        ("T3_07", "Case 7",  1/6,  1/6,  4/6),
        ("T3_08", "Case 8",  1/8,  2/8,  5/8),
        ("T3_09", "Case 9",  1/8,  1/8,  6/8),
        ("T3_10", "Case 10", 1/10, 1/10, 8/10),
        ("T3_11", "Case 11", 2/3,  1/3,  1/3),
        ("T3_12", "Case 12", 3/4,  1/4,  1/4),
        ("T3_13", "Case 13", 5/8,  2/8,  1/8),
        ("T3_14", "Case 14", 6/8,  1/8,  1/8),
        ("T3_15", "Case 15", 1/20, 1/20, 18/20),
        ("T3_16", "Case 16", 1/40, 1/40, 38/40),
    ]
    rows = []
    for cid, name, wM, wC, wS in paper_cases:
        rows.append({"weight_case_id": cid, "weight_case_name": name, "wC": float(wC), "wM": float(wM), "wS": float(wS), "source": "Table3"})
    extras = [
        ("EX_01", "SensorOnly", 0.0, 0.0, 1.0),
        ("EX_02", "Sensor90",   0.05, 0.05, 0.90),
        ("EX_03", "Sensor95",   0.025, 0.025, 0.95),
        ("EX_04", "Sensor98",   0.01, 0.01, 0.98),
    ]
    for cid, name, wC, wM, wS in extras:
        rows.append({"weight_case_id": cid, "weight_case_name": name, "wC": float(wC), "wM": float(wM), "wS": float(wS), "source": "Extra"})
    df = pd.DataFrame(rows)
    df["_key"] = df.apply(lambda r: (round(r["wC"], 6), round(r["wM"], 6), round(r["wS"], 6)), axis=1)
    df = df.drop_duplicates("_key").drop(columns=["_key"]).reset_index(drop=True)
    return df

WEIGHT_TABLE = build_weight_table()

def _weight_lookup(w: Tuple[float,float,float]) -> Tuple[str,str,str]:
    wC,wM,wS = (round(float(w[0]),6), round(float(w[1]),6), round(float(w[2]),6))
    m = WEIGHT_TABLE[(WEIGHT_TABLE["wC"].round(6)==wC) & (WEIGHT_TABLE["wM"].round(6)==wM) & (WEIGHT_TABLE["wS"].round(6)==wS)]
    if len(m)>0:
        r = m.iloc[0]
        return str(r["weight_case_id"]), str(r["weight_case_name"]), str(r["source"])
    return ("CUSTOM","Custom","Search")

# ===============================================================================================
# 4) IO + workload parsing
# ===============================================================================================
RUNID2PATH: Dict[str,str] = {}

def detect_setup_from_path(p: Path):
    s = str(p).lower()
    if "ddr4" in s: return "DDR4"
    if "ddr5" in s: return "DDR5"
    return None

def is_benign_path(p: Path) -> bool:
    s = str(p).lower()
    if "benign" in s:
        return True
    bad = ["attack","anom","fault","inject","trojan","mal","rh","droop","spectre","trrespass"]
    return not any(b in s for b in bad)

def is_anomaly_path(p: Path, anomaly: str) -> bool:
    return (anomaly.lower() in str(p).lower()) and (not is_benign_path(p))

def iter_raw_csvs(root: Path):
    for p in root.rglob("*.csv"):
        yield p

def read_csv_clean(p: Path) -> pd.DataFrame:
    df = pd.read_csv(p)
    return df.loc[:, ~df.columns.str.startswith("Unnamed")]

def mk_run_id(path: Path) -> str:
    rid = f"run_{hashlib.md5(str(path).encode('utf-8')).hexdigest()[:10]}"
    RUNID2PATH[rid] = str(path)
    return rid

_WORKLOADS = ["dft","dj","dp","gl","gs","ha","ja","mm","ni","oe","pi","sh","tr"]
def workload_from_path(p: Path) -> str:
    stem = (p.stem or "").lower()
    m = re.search(r"_([a-z]{2,3})$", stem)
    if m and m.group(1) in _WORKLOADS:
        return m.group(1).upper()
    for tok in _WORKLOADS:
        if tok in stem:
            return tok.upper()
    return "UNK"

def collect_raw_pairs_by_setup(data_dir: Path, which: str, anomaly: Optional[str] = None):
    out = {"DDR4": [], "DDR5": []}
    for p in iter_raw_csvs(data_dir):
        setup = detect_setup_from_path(p)
        if setup is None:
            continue
        try:
            if which == "benign":
                if not is_benign_path(p):
                    continue
            else:
                if anomaly is None or not is_anomaly_path(p, anomaly):
                    continue
            df = read_csv_clean(p)
            out[setup].append((p, df))
        except Exception as e:
            print(f"[WARN] read failed {p}: {e}")
    return out

def telemetry_cols(df: pd.DataFrame):
    exclude = set(META) | set(DROOP_META_COLS) | {"workload"}
    return [c for c in df.columns if (c not in exclude) and (df[c].dtype.kind in "fcbiu")]

def drop_low_variance_cols(df: pd.DataFrame, cols: List[str], eps: float = 1e-10) -> List[str]:
    v = df[cols].astype(float).var(axis=0, ddof=0)
    keep = v[v > eps].index.tolist()
    return keep

# ===============================================================================================
# 5) Scaling
# ===============================================================================================
def robust_scale_train(Xb_np: np.ndarray, winsor=(2.0, 98.0)):
    Q1, Q2 = np.percentile(Xb_np, winsor[0], axis=0), np.percentile(Xb_np, winsor[1], axis=0)
    Xb_clip = np.clip(Xb_np, Q1, Q2)
    mu = Xb_clip.mean(axis=0)
    sd = Xb_clip.std(axis=0, ddof=0) + 1e-9
    return mu, sd, Q1, Q2

def apply_robust_scale(X: pd.DataFrame, mu, sd, Q1, Q2):
    Xc = np.clip(X.to_numpy(dtype=float), Q1, Q2)
    Z  = (Xc - mu) / sd
    return pd.DataFrame(Z, columns=X.columns, index=X.index)

# ===============================================================================================
# 6) Windowing (adds sensor transient feats incl CPU Voltage)
# ===============================================================================================
SENSOR_PAT = re.compile(r"cpu\s*voltage|volt|vdd|vcore|vin|vout|power|energy|joule|current|amps?|temp|thermal|hot", re.I)

def window_collapse_means(df: pd.DataFrame, win: int, setup: str, run_id: str, label: str,
                          overlap_ratio: float, src_path: str="") -> pd.DataFrame:
    cols_all = telemetry_cols(df)
    if not cols_all:
        return pd.DataFrame()

    sensor_cols = [c for c in cols_all if SENSOR_PAT.search(c)]
    if "CPU Voltage" in cols_all and "CPU Voltage" not in sensor_cols:
        sensor_cols.append("CPU Voltage")

    n = len(df)
    stride = max(1, int(round(win * (1 - overlap_ratio))))
    starts = list(range(0, n - win + 1, stride))
    if not starts:
        return pd.DataFrame()

    rows = []
    for start in starts:
        chunk = df.iloc[start:start+win]
        X = chunk[cols_all].astype(float)
        means = X.mean(axis=0, numeric_only=True)

        tfeat = {}
        if sensor_cols:
            Xs = X[sensor_cols]
            mins  = Xs.min(axis=0)
            maxs  = Xs.max(axis=0)
            stds  = Xs.std(axis=0, ddof=0)
            means_s = Xs.mean(axis=0)
            drops  = (means_s - mins)
            ranges = (maxs - mins)
            slope  = (Xs.iloc[-1] - Xs.iloc[0])
            for c in sensor_cols:
                tfeat[f"{c}__min"]   = float(mins[c])
                tfeat[f"{c}__max"]   = float(maxs[c])
                tfeat[f"{c}__std"]   = float(stds[c])
                tfeat[f"{c}__drop"]  = float(drops[c])
                tfeat[f"{c}__range"] = float(ranges[c])
                tfeat[f"{c}__slope"] = float(slope[c])

        row = pd.concat([means, pd.Series(tfeat)]).to_frame().T
        row["setup"]    = setup
        row["run_id"]   = run_id
        row["label"]    = label
        row["workload"] = workload_from_path(Path(src_path))
        rows.append(row)

    return pd.concat(rows, ignore_index=True) if rows else pd.DataFrame()

def build_windowed_raw_means(pairs, setup: str, win: int, label: str, overlap_ratio: float) -> pd.DataFrame:
    out = []
    for p, df in pairs:
        rid = mk_run_id(p)
        agg = window_collapse_means(df, win=win, setup=setup, run_id=rid, label=label,
                                    overlap_ratio=overlap_ratio, src_path=str(p))
        if not agg.empty:
            out.append(agg)
    return pd.concat(out, ignore_index=True) if out else pd.DataFrame()

# ===============================================================================================
# 7) Rank lists + robust mapping + CPU Voltage family append for DROOP
# ===============================================================================================
def _read_rank_feats(p: Path):
    try:
        df = pd.read_csv(p)
    except Exception:
        return []
    if df.empty:
        return []
    col = "feature" if "feature" in df.columns else df.columns[0]
    return df[col].dropna().astype(str).tolist()

def load_full_rank_lists(setup: str, win: int, kfold: int):
    def _find_file(setup, win, kfold, sub):
        fname = f"{setup}_{win}_{kfold}_0_{sub}.csv"
        for d in RANK_DIRS:
            pp = d / fname
            if pp.exists():
                return pp
        return None
    out = {}
    for sub in ["compute","memory","sensors"]:
        pp = _find_file(setup, win, kfold, sub)
        out[sub] = _read_rank_feats(pp) if pp else []
    return out

def slice_by_percent(full_lists: Dict[str, List[str]], pct: int):
    sel = {}
    frac = pct / 100.0
    for sub in ("compute","memory","sensors"):
        feats = full_lists.get(sub, []) or []
        k = int(np.ceil(len(feats) * frac))
        if frac > 0 and len(feats) > 0:
            k = max(1, k)
        sel[sub] = feats[:min(len(feats), max(0, k))]
    return sel

ALIAS_MAP = {
    "voltage": ["cpu","voltage","volt","vcore","vdd","vin","vout"],
    "power": ["power","energy","joules","current","amps"],
    "temperature": ["temp","thermal","hot"],
    "frequency": ["freq","afreq","cfreq","clock","clk","mhz","ghz"],
    "ipc": ["ipc","physipc"],
    "cache": ["l1","l2","l3","hit","miss","evict","fill","mpi"],
    "bandwidth": ["read","write","bw","bandwidth"],
}

def _norm(s: str) -> str:
    s = unicodedata.normalize("NFKC", str(s)).lower()
    out = []
    for ch in s:
        out.append(ch if ch.isalnum() else "_")
    s = "".join(out)
    while "__" in s:
        s = s.replace("__","_")
    return s.strip("_")

def _tokens(s: str):
    t = _norm(s).replace("_"," ").split()
    exp = []
    for tok in t:
        exp.append(tok)
        for k, aliases in ALIAS_MAP.items():
            if tok in aliases:
                exp.append(k)
    return set(exp)

def build_column_index(cols: List[str]):
    norm2orig = {}
    tok_index = {}
    for c in cols:
        nc = _norm(c)
        norm2orig.setdefault(nc, c)
        tok_index[c] = _tokens(c)
    return norm2orig, tok_index

def map_ranks_to_existing(ranked_list: List[str], norm2orig, tok_index):
    mapped, seen = [], set()
    for r in ranked_list:
        nr = _norm(r)
        cand = norm2orig.get(nr, None)
        if cand and cand not in seen:
            mapped.append(cand); seen.add(cand); continue
        rtoks = _tokens(r)
        best_c, best_j = None, 0.0
        for c, ctoks in tok_index.items():
            inter = len(rtoks & ctoks)
            union = len(rtoks | ctoks) or 1
            j = inter / union
            if j > best_j:
                best_j, best_c = j, c
        if best_c and best_j >= 0.60 and best_c not in seen:
            mapped.append(best_c); seen.add(best_c)
    return mapped

def intersect_selection_with_columns_robust(sel_raw: Dict[str, List[str]], xb_cols: set):
    norm2orig, tok_index = build_column_index(list(xb_cols))
    out = {}
    for sub in ("compute","memory","sensors"):
        ranked = sel_raw.get(sub, []) or []
        mapped = map_ranks_to_existing(ranked, norm2orig, tok_index)
        out[sub] = [c for c in mapped if c in xb_cols]
    return out

def force_cpu_voltage_family(sel: Dict[str, List[str]], xb_cols: set) -> Dict[str, List[str]]:
    out = dict(sel)
    sensors_now = out.get("sensors", []) or []
    must = []
    if "CPU Voltage" in xb_cols:
        must.append("CPU Voltage")
    for suf in ["__min","__max","__std","__drop","__range","__slope"]:
        c = f"CPU Voltage{suf}"
        if c in xb_cols:
            must.append(c)
    out["sensors"] = list(dict.fromkeys(sensors_now + must))
    return out

# ===============================================================================================
# 8) Balanced evaluation helper
# ===============================================================================================
def balanced_concat(df_ben: pd.DataFrame, df_anom: pd.DataFrame, seed: int = 1337):
    if df_ben is None or df_ben.empty:
        raise ValueError("balanced_concat(): df_ben is empty")
    if df_anom is None or df_anom.empty:
        raise ValueError("balanced_concat(): df_anom is empty")

    rng = np.random.default_rng(int(seed))
    nb = int(len(df_ben))
    na = int(len(df_anom))
    n  = int(min(nb, na))

    idx_b = rng.choice(nb, size=n, replace=(nb < n))
    idx_a = rng.choice(na, size=n, replace=(na < n))

    ben_s = df_ben.iloc[idx_b].copy().reset_index(drop=True)
    anm_s = df_anom.iloc[idx_a].copy().reset_index(drop=True)

    df_eval = pd.concat([ben_s, anm_s], ignore_index=True)
    y_true  = np.concatenate([np.zeros(len(ben_s), dtype=int),
                              np.ones(len(anm_s), dtype=int)])
    return df_eval, y_true

# ===============================================================================================
# 9) Weight selection helpers
# ===============================================================================================
def split_holdout(y, test_size=0.3, seed=SEED):
    sss = StratifiedShuffleSplit(n_splits=1, test_size=test_size, random_state=seed)
    idx = np.arange(len(y))
    _, val_idx = next(sss.split(idx, y))
    return val_idx

def pick_best_w_per_method_PRfirst(y_true: np.ndarray, scores_by_w: Dict[Tuple[float,float,float], Dict[str,np.ndarray]], val_idx: np.ndarray):
    yy = y_true[val_idx]
    best = {m: (-1.0, -1.0, -1.0, None) for m in ["dE_aM","dE_aJ","dC_aM","dC_aJ"]}
    for w, md in scores_by_w.items():
        wS = float(w[2])
        for m, s in md.items():
            sv = np.asarray(s)[val_idx]
            pr  = float(average_precision_score(yy, sv))
            roc = float(roc_auc_score(yy, sv))
            key = (pr, roc, wS)
            if key > best[m][:3]:
                best[m] = (pr, roc, wS, w)
    return {m: best[m][3] for m in best}

# ===============================================================================================
# 10) DSE plots
# ===============================================================================================
def summarize_for_dse(per_run_df: pd.DataFrame) -> pd.DataFrame:
    keys = ["setup","anomaly","win","kfold","method","pct"]
    def q1(x): return float(np.nanpercentile(x, 25))
    def q3(x): return float(np.nanpercentile(x, 75))
    g = (per_run_df.groupby(keys, as_index=False)
            .agg(
                roc_auc_median=("roc_auc","median"),
                roc_auc_q1=("roc_auc", q1),
                roc_auc_q3=("roc_auc", q3),
                roc_auc_min=("roc_auc","min"),
                roc_auc_max=("roc_auc","max"),
                auc_pr_median=("auc_pr","median"),
                auc_pr_q1=("auc_pr", q1),
                auc_pr_q3=("auc_pr", q3),
                auc_pr_min=("auc_pr","min"),
                auc_pr_max=("auc_pr","max"),
                n_runs=("run_id","nunique"),
            ))
    return g.sort_values(keys).reset_index(drop=True)

def plot_dse_grid(summary_case: pd.DataFrame, metric: str, setup: str, anomaly: str,
                  win: int, kfold: int, out_dir: Path):
    out_dir.mkdir(parents=True, exist_ok=True)
    x_ticks = sorted(summary_case["pct"].unique().tolist())
    if not x_ticks:
        return

    fig, axes = plt.subplots(2, 2, figsize=FIGSIZE, sharex=False, sharey=False)
    axes = axes.flatten()
    fig.subplots_adjust(left=LEFT, right=RIGHT, top=TOP, bottom=BOTTOM, wspace=WSPACE, hspace=HSPACE)

    xlab = "Top-ranked features used (%)"
    ylab = "ROC-AUC" if metric == "roc_auc" else "AUC-PR"

    for i, method in enumerate(METHOD_ORDER):
        ax = axes[i]
        ax.set_title(METHOD_TITLE.get(method, method), pad=7)
        ax.set_xlim(min(x_ticks), max(x_ticks))
        ax.set_xticks(x_ticks)
        ax.grid(True, alpha=GRID_ALPHA, linestyle=GRID_LS, linewidth=GRID_LW)

        sub = summary_case[summary_case["method"] == method].copy().sort_values("pct")
        if sub.empty:
            ax.text(0.5, 0.5, "No data", ha="center", va="center", transform=ax.transAxes)
            ax.set_ylim(Y_DATA_MIN, Y_DISPLAY_MAX)
        else:
            x    = sub["pct"].to_numpy(dtype=float)
            med  = _clip_metric(sub[f"{metric}_median"].to_numpy(dtype=float))
            q1   = _clip_metric(sub[f"{metric}_q1"].to_numpy(dtype=float))
            q3   = _clip_metric(sub[f"{metric}_q3"].to_numpy(dtype=float))
            vmin = _clip_metric(sub[f"{metric}_min"].to_numpy(dtype=float))
            vmax = _clip_metric(sub[f"{metric}_max"].to_numpy(dtype=float))

            lo_mm, hi_mm = np.minimum(vmin, vmax), np.maximum(vmin, vmax)
            lo_iq, hi_iq = np.minimum(q1, q3),     np.maximum(q1, q3)
            lo_mm, hi_mm = _ensure_visible_band(lo_mm, hi_mm)
            lo_iq, hi_iq = _ensure_visible_band(lo_iq, hi_iq)

            ax.fill_between(x, lo_mm, hi_mm, alpha=ALPHA_MINMAX, label="min-max", linewidth=0.0)
            ax.fill_between(x, lo_iq, hi_iq, alpha=ALPHA_IQR,   label="IQR",     linewidth=0.0)
            ax.plot(x, med, color="black", marker="o", linewidth=2.0, markersize=5.5, zorder=5, label="Median")

            if Y_AUTO:
                ylo, yhi = _auto_ylim_from_arrays([lo_mm, hi_mm, lo_iq, hi_iq, med])
                ax.set_ylim(ylo, yhi)
            else:
                ax.set_ylim(Y_DATA_MIN, Y_DISPLAY_MAX)

        ax.set_xlabel(xlab, labelpad=6)
        ax.set_ylabel(ylab, labelpad=6)
        ax.tick_params(axis="x", labelbottom=True, pad=3)
        ax.tick_params(axis="y", labelleft=True, pad=3)

    handles, labels = axes[0].get_legend_handles_labels()
    fig.legend(handles, labels, loc="lower center", bbox_to_anchor=(0.5, LEGEND_Y), ncol=3, frameon=False)
    fig.suptitle(f"{setup}: {anomaly} (WIN={win}, K={kfold})", y=SUPTITLE_Y)

    base = f"DSE_{metric.upper()}_{setup}_{anomaly}_WIN{int(win)}_KF{int(kfold)}"
    fig.savefig(out_dir / f"{base}.png", dpi=600, bbox_inches="tight", pad_inches=0.06)
    fig.savefig(out_dir / f"{base}.pdf", bbox_inches="tight", pad_inches=0.06)
    plt.close(fig)

def write_all_dse_plots(per_run_df: pd.DataFrame):
    out_root = RES_DIR / "DesignSpace" / "PaperStyle_ALLRUNS"
    out_root.mkdir(parents=True, exist_ok=True)
    dse = summarize_for_dse(per_run_df)
    for (setup, anomaly, win, kfold), sub in dse.groupby(["setup","anomaly","win","kfold"]):
        out_dir = out_root / str(setup) / str(anomaly) / f"WIN{int(win)}_KF{int(kfold)}"
        plot_dse_grid(sub, "roc_auc", str(setup), str(anomaly), int(win), int(kfold), out_dir)
        plot_dse_grid(sub, "auc_pr",  str(setup), str(anomaly), int(win), int(kfold), out_dir)

# ===============================================================================================
# 11) Subspace-weight tables (same as before; omitted here for brevity)
# ===============================================================================================
def build_workload_weight_tables(per_run_df: pd.DataFrame, setup: str, win: int, kfold: int) -> None:
    # (same implementation you already use; keep unchanged)
    base_dir = RES_DIR / "Table_SubspaceWeights" / setup / f"WIN{win}_KF{kfold}"
    base_dir.mkdir(parents=True, exist_ok=True)

    rep_method = "dE_aM"
    for anomaly in ANOMALIES_BY_SETUP[setup]:
        df = per_run_df[
            (per_run_df["setup"]==setup) &
            (per_run_df["anomaly"]==anomaly) &
            (per_run_df["win"]==win) &
            (per_run_df["kfold"]==kfold)
        ].copy()
        if df.empty:
            continue

        df["_wkey"] = df.apply(lambda r: (round(r["wC"],6), round(r["wM"],6), round(r["wS"],6)), axis=1)
        rows = []
        for (wl, method), g in df.groupby(["workload","method"]):
            mode_key = g["_wkey"].value_counts().idxmax()
            wC, wM, wS = mode_key
            wid, wname, wsrc = _weight_lookup((wC,wM,wS))
            rows.append({
                "workload": wl,
                "method": method,
                "chosen_wC": wC, "chosen_wM": wM, "chosen_wS": wS,
                "weight_case_id": wid,
                "weight_case_name": wname,
                "weight_case_source": wsrc,
                "n_rows": int(len(g)),
                "median_roc": float(np.nanmedian(g["roc_auc"])),
                "median_pr": float(np.nanmedian(g["auc_pr"])),
            })
        df_out = pd.DataFrame(rows).sort_values(["workload","method"]).reset_index(drop=True)
        (base_dir / f"Table_SubspaceWeights_{setup}_WIN{win}_KF{kfold}_{anomaly}.csv").write_text(df_out.to_csv(index=False))

# ===============================================================================================
# 12) Pipeline core (same logic as your current version; unchanged except DSE y-axis)
# ===============================================================================================
def run_full_pipeline() -> pd.DataFrame:
    all_rows = []
    group_counter = 0

    benign_pairs = collect_raw_pairs_by_setup(DATA_DIR, which="benign")
    anomaly_pairs_cache: Dict[Tuple[str,str], List[Tuple[Path,pd.DataFrame]]] = {}

    def get_anom_pairs(setup: str, anomaly: str):
        key = (setup, anomaly.upper())
        if key not in anomaly_pairs_cache:
            anomaly_pairs_cache[key] = collect_raw_pairs_by_setup(DATA_DIR, which="anomaly", anomaly=anomaly)[setup]
        return anomaly_pairs_cache[key]

    for setup in SETUPS:
        anomalies = ANOMALIES_BY_SETUP.get(setup, [])
        ben_pairs_all = benign_pairs.get(setup, [])
        if not ben_pairs_all:
            print(f"[SKIP] {setup}: no benign RAW files")
            continue

        for win in WINS:
            df_ben_all = build_windowed_raw_means(ben_pairs_all, setup=setup, win=win, label="BENIGN",
                                                  overlap_ratio=OVERLAP_RATIO)
            if df_ben_all.empty:
                print(f"[SKIP] {setup} WIN={win}: benign windowed empty")
                continue

            Xb_cols = telemetry_cols(df_ben_all)
            if DROP_LOW_VARIANCE_COLS:
                Xb_cols = drop_low_variance_cols(df_ben_all, Xb_cols, eps=LOW_VAR_EPS)
            if not Xb_cols:
                print(f"[SKIP] {setup} WIN={win}: no telemetry cols after low-var drop")
                continue

            Xb = df_ben_all[Xb_cols].astype(float)
            mu_s, sd_s, Q1, Q2 = robust_scale_train(Xb.values, winsor=ROBUST_WINSOR)
            Xb_base = apply_robust_scale(Xb, mu_s, sd_s, Q1, Q2)
            xb_cols_set = set(Xb_base.columns)

            ben_by_wl = {}
            for wl in df_ben_all["workload"].astype(str).unique():
                ben_by_wl[wl] = df_ben_all[df_ben_all["workload"].astype(str)==wl].copy()

            for kfold in KFOLDS:
                wdir = RES_DIR / "Table_SubspaceWeights" / setup / f"WIN{win}_KF{kfold}"
                wdir.mkdir(parents=True, exist_ok=True)
                WEIGHT_TABLE.to_csv(wdir / "Table_SubspaceWeights_CANDIDATES.csv", index=False)

                full_lists = load_full_rank_lists(setup, win, kfold)

                for anomaly in anomalies:
                    group_counter += 1
                    print(f"\n[RUN] {setup} {anomaly} WIN={win} KF={kfold} (group {group_counter})")

                    overlap = OVERLAP_RATIO_DROOP if anomaly.upper() == "DROOP" else OVERLAP_RATIO
                    an_pairs_all = get_anom_pairs(setup, anomaly)
                    if not an_pairs_all:
                        print(f"[SKIP] {setup} {anomaly}: no anomaly RAW files")
                        continue

                    df_anom = build_windowed_raw_means(an_pairs_all, setup=setup, win=win, label=anomaly,
                                                       overlap_ratio=overlap)
                    if df_anom.empty:
                        print(f"[SKIP] {setup} {anomaly} WIN={win}: anomaly windowed empty")
                        continue

                    run_ids = sorted(df_anom["run_id"].astype(str).unique())

                    for rid in run_ids:
                        dfa_run = df_anom[df_anom["run_id"].astype(str) == rid].copy()
                        if dfa_run.empty:
                            continue

                        wl = str(dfa_run["workload"].iloc[0]) if "workload" in dfa_run.columns else "UNK"
                        df_ben = ben_by_wl.get(wl, df_ben_all)
                        if df_ben.empty:
                            df_ben = df_ben_all

                        df_eval, y_true = balanced_concat(df_ben, dfa_run, seed=SEED)

                        Xe_all  = df_eval[Xb_cols].astype(float)
                        Xe_base = apply_robust_scale(Xe_all, mu_s, sd_s, Q1, Q2)

                        for idx_p, pct in enumerate(PCT_SWEEP, start=1):
                            sel_raw = slice_by_percent(full_lists, pct)
                            sel = intersect_selection_with_columns_robust(sel_raw, xb_cols_set)
                            if (not sel.get("compute")) or (not sel.get("memory")) or (not sel.get("sensors")):
                                continue
                            if anomaly.upper() == "DROOP":
                                sel = force_cpu_voltage_family(sel, xb_cols_set)

                            refs = _build_refs(Xb_base, sel, eps=1e-12)
                            benign_norm = fit_parts_normalizer_on_benign(Xb_base, refs)

                            val_idx = split_holdout(y_true, test_size=0.3, seed=SEED)

                            scores_by_w = {}
                            for _, wr in WEIGHT_TABLE.iterrows():
                                w = (float(wr["wC"]), float(wr["wM"]), float(wr["wS"]))
                                scores_by_w[w] = score_four_methods_benign_norm(Xe_base, refs, w, benign_norm)

                            best_w_for = pick_best_w_per_method_PRfirst(y_true, scores_by_w, val_idx)

                            for method in METHOD_ORDER:
                                w_star = best_w_for[method]
                                y_score = score_four_methods_benign_norm(Xe_base, refs, w_star, benign_norm)[method]
                                y_score = np.nan_to_num(y_score, nan=0.0, posinf=0.0, neginf=0.0)

                                if DROOP_BOOST and anomaly.upper() == "DROOP":
                                    boost = droop_boost_score(Xe_base, sel)
                                    y_score = (1.0 - DROOP_BOOST_ALPHA) * y_score + DROOP_BOOST_ALPHA * boost

                                roc = float(np.clip(roc_auc_score(y_true, y_score), Y_DATA_MIN, Y_DATA_MAX))
                                pr  = float(np.clip(average_precision_score(y_true, y_score), Y_DATA_MIN, Y_DATA_MAX))

                                wid, wname, wsrc = _weight_lookup(w_star)

                                all_rows.append({
                                    "setup": setup,
                                    "anomaly": anomaly,
                                    "win": int(win),
                                    "kfold": int(kfold),
                                    "pct": int(pct),
                                    "run_id": str(rid),
                                    "workload": str(wl),
                                    "method": method,
                                    "roc_auc": roc,
                                    "auc_pr": pr,
                                    "weight_case_id": wid,
                                    "weight_case_name": wname,
                                    "weight_case_source": wsrc,
                                    "wC": float(w_star[0]),
                                    "wM": float(w_star[1]),
                                    "wS": float(w_star[2]),
                                })

                            if idx_p % GC_EVERY_N_PCTS == 0:
                                gc.collect()

                    if group_counter % GC_EVERY_N_GROUPS == 0:
                        gc.collect()

    per_run_df = pd.DataFrame(all_rows)
    out_all = RES_DIR / "per_run_metrics_all_PIPELINE.csv"
    per_run_df.to_csv(out_all, index=False)
    print(f"\n[OK] ALLRUNS metrics → {out_all}")

    for setup in SETUPS:
        for win in WINS:
            for kfold in KFOLDS:
                build_workload_weight_tables(per_run_df, setup=setup, win=win, kfold=kfold)

    write_all_dse_plots(per_run_df)
    return per_run_df

# ===============================================================================================
# 13) MAIN
# ===============================================================================================
def main():
    run_full_pipeline()
    print("[DONE] Pipeline + DSE plots + weight tables per platform")

if __name__ == "__main__":
    main()

[WARN] Cannot write to /Volumes/Untitled/octaneX_results. Using local: /Users/hsiaopingni/octaneX_v7_4functions/Results

[RUN] DDR4 DROOP WIN=32 KF=3 (group 1)

[RUN] DDR4 RH WIN=32 KF=3 (group 2)

[RUN] DDR4 DROOP WIN=32 KF=5 (group 3)

[RUN] DDR4 RH WIN=32 KF=5 (group 4)

[RUN] DDR4 DROOP WIN=32 KF=10 (group 5)

[RUN] DDR4 RH WIN=32 KF=10 (group 6)

[RUN] DDR4 DROOP WIN=64 KF=3 (group 7)

[RUN] DDR4 RH WIN=64 KF=3 (group 8)

[RUN] DDR4 DROOP WIN=64 KF=5 (group 9)

[RUN] DDR4 RH WIN=64 KF=5 (group 10)

[RUN] DDR4 DROOP WIN=64 KF=10 (group 11)

[RUN] DDR4 RH WIN=64 KF=10 (group 12)

[RUN] DDR4 DROOP WIN=128 KF=3 (group 13)

[RUN] DDR4 RH WIN=128 KF=3 (group 14)

[RUN] DDR4 DROOP WIN=128 KF=5 (group 15)

[RUN] DDR4 RH WIN=128 KF=5 (group 16)

[RUN] DDR4 DROOP WIN=128 KF=10 (group 17)

[RUN] DDR4 RH WIN=128 KF=10 (group 18)

[RUN] DDR4 DROOP WIN=512 KF=3 (group 19)

[RUN] DDR4 RH WIN=512 KF=3 (group 20)

[RUN] DDR4 DROOP WIN=512 KF=5 (group 21)

[RUN] DDR4 RH WIN=512 KF=5 (group 22)

[RU

In [64]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

# ===============================================================================================
# FULL MULTI-WIN / MULTI-K PIPELINE (AUTHENTIC)
# + DSE PLOTS (all cases)
# + TABLE_SUBSPACEWEIGHTS (all cases)
#
# UPDATE (your request):
#   ✅ Auto-adjust y-axis for EVERY DSE subplot (all WIN/K) to remove empty white space below.
#   ✅ Do NOT show any values/regions above 1.00 (metrics are clipped at 1.00).
#   ✅ BUT still leave a small visual headroom so markers/lines at 1.00 are NOT cut off.
#
# FIX (your request, v2):
#   ✅ RH plots: further reduce white space above y=1.00 by tightening the DISPLAY headroom:
#        - reduce global Y_DISPLAY_MAX from 1.02 → 1.006
#        - reduce adaptive headroom floor/cap so near-saturated plots only get ~0.2%–0.6% headroom
#      This matches the DROOP-style “tight top” while still preventing marker clipping at y=1.00.
#
# NOTE:
#   - DATA are still clipped to <= 1.00 (never plotted above 1.00).
#   - Only the y-axis DISPLAY max can go slightly above 1.00.
# ===============================================================================================

import gc, re, hashlib, warnings, unicodedata
from pathlib import Path
from typing import List, Dict, Optional, Tuple

import numpy as np
import pandas as pd

import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt

from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.model_selection import StratifiedShuffleSplit

warnings.filterwarnings("ignore", category=RuntimeWarning)

# ===============================================================================================
# 0) PATHS
# ===============================================================================================
DATA_DIR = Path("/Users/hsiaopingni/Desktop/SLM_RAS-main/HW_TELEMETRY_DATA_COLLECTION/TELEMETRY_DATA")
ROOT     = Path("/Users/hsiaopingni/octaneX_v7_4functions")
RES_DIR  = ROOT / "Results"
RES_DIR.mkdir(parents=True, exist_ok=True)

EXT_DRIVE = Path("/Volumes/Untitled")
EXT_RES   = EXT_DRIVE / "octaneX_results"

def _can_write_dir(p: Path) -> bool:
    try:
        p.mkdir(parents=True, exist_ok=True)
        t = p / ".write_test"
        t.write_text("ok")
        t.unlink()
        return True
    except Exception:
        return False

if _can_write_dir(EXT_RES):
    RES_DIR = EXT_RES
    RES_DIR.mkdir(parents=True, exist_ok=True)
    print(f"[OK] Using external results dir: {RES_DIR}")
else:
    print(f"[WARN] Cannot write to {EXT_RES}. Using local: {RES_DIR}")

RANK_DIRS = [
    ROOT / "FeatureRankOUT_HYBRID",
    EXT_DRIVE / "FeatureRankOUT_HYBRID",
    EXT_DRIVE / "octaneX" / "FeatureRankOUT_HYBRID",
]

# ===============================================================================================
# 1) RUN CONFIG
# ===============================================================================================
SETUPS = ["DDR4", "DDR5"]
ANOMALIES_BY_SETUP = {"DDR4": ["DROOP", "RH"], "DDR5": ["DROOP", "SPECTRE"]}

WINS   = [32, 64, 128, 512, 1024]
KFOLDS = [3, 5, 10]
PCT_SWEEP = list(range(10, 101, 10))

OVERLAP_RATIO       = 0.50
OVERLAP_RATIO_DROOP = 0.80

ROBUST_WINSOR = (2.0, 98.0)
SEED = 1337

GC_EVERY_N_PCTS   = 3
GC_EVERY_N_GROUPS = 1

META = ["label", "setup", "run_id"]
DROOP_META_COLS = ["droop_center_found", "droop_best_score_z", "droop_frac_ge_thr", "droop_vcols"]

DROP_LOW_VARIANCE_COLS = True
LOW_VAR_EPS = 1e-10

# DROOP boost (small, optional)
DROOP_BOOST = True
DROOP_BOOST_ALPHA = 0.20
DROOP_TRANSIENT_PAT = re.compile(r"__(drop|range|slope|min|max|std)", re.I)

METHOD_ORDER = ["dC_aJ", "dC_aM", "dE_aJ", "dE_aM"]

# ===============================================================================================
# 2) PLOT CONFIG
# ===============================================================================================
plt.rcParams.update({
    "font.family": "sans-serif",
    "font.size": 12,
    "axes.titlesize": 13.5,
    "axes.labelsize": 14,
    "xtick.labelsize": 12,
    "ytick.labelsize": 12,
    "legend.fontsize": 12,
    "figure.titlesize": 20,
})

METHOD_TITLE = {
    "dC_aJ": r"Scoring function $d_C(X)$ with Aggregate $a_J$",
    "dC_aM": r"Scoring function $d_C(X)$ with Aggregate $a_M$",
    "dE_aJ": r"Scoring function $d_E(X)$ with Aggregate $a_J$",
    "dE_aM": r"Scoring function $d_E(X)$ with Aggregate $a_M$",
}

# ---- Data never exceeds 1.00 ----
Y_DATA_MAX = 1.00
Y_DATA_MIN = 0.00

# ---- Display headroom so points at 1.00 aren't cut off ----
# (TIGHTENED for RH plots)
Y_DISPLAY_MAX = 1.006     # was 1.02 → reduce top whitespace while still preventing clipping
Y_HEADROOM    = 0.006     # cap for adaptive headroom (display-only), data remain <= 1.00

# y-axis auto-range per subplot
Y_AUTO     = True
Y_PAD_FRAC = 0.06
Y_MIN_SPAN = 0.04

ALPHA_MINMAX = 0.28
ALPHA_IQR    = 0.60
GRID_ALPHA   = 0.22
GRID_LS      = "--"
GRID_LW      = 0.6

FIGSIZE = (13.6, 8.4)
WSPACE  = 0.30
HSPACE  = 0.42
TOP     = 0.88
BOTTOM  = 0.20
LEFT    = 0.075
RIGHT   = 0.985
SUPTITLE_Y = 0.975
LEGEND_Y   = 0.055

BAND_EPS = 0.008
def _ensure_visible_band(lo: np.ndarray, hi: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    lo = lo.astype(float).copy()
    hi = hi.astype(float).copy()
    flat = (np.abs(hi - lo) < 1e-12)
    if np.any(flat):
        lo2 = lo - BAND_EPS/2
        hi2 = hi + BAND_EPS/2
        lo[flat] = np.clip(lo2[flat], Y_DATA_MIN, Y_DATA_MAX)
        hi[flat] = np.clip(hi2[flat], Y_DATA_MIN, Y_DATA_MAX)
    return lo, hi

def _clip_metric(a):
    """Hard clip metrics (data) to [0,1]."""
    return np.clip(np.asarray(a, float), Y_DATA_MIN, Y_DATA_MAX)

def _auto_ylim_from_arrays(arrs: List[np.ndarray]) -> Tuple[float, float]:
    """
    Tight ylim based on plotted data bands/median.
    Axis can extend slightly above 1.00 up to Y_DISPLAY_MAX for visual headroom,
    but data are clipped at 1.00.

    v2: Even tighter headroom for near-saturated plots (RH).
    """
    vals = []
    for a in arrs:
        if a is None:
            continue
        a = np.asarray(a, float).ravel()
        a = a[np.isfinite(a)]
        if a.size:
            vals.append(a)
    if not vals:
        return (Y_DATA_MIN, Y_DISPLAY_MAX)

    v = np.concatenate(vals)
    ymin = float(np.min(v))
    ymax = float(np.max(v))
    if not np.isfinite(ymin) or not np.isfinite(ymax):
        return (Y_DATA_MIN, Y_DISPLAY_MAX)

    # base padding
    raw_span = max(ymax - ymin, 1e-12)
    span = max(ymax - ymin, Y_MIN_SPAN)
    pad = Y_PAD_FRAC * span
    ymin2 = float(np.clip(ymin - pad, Y_DATA_MIN, Y_DATA_MAX))
    ymax2 = float(np.clip(ymax + pad, Y_DATA_MIN, Y_DATA_MAX))

    # enforce minimal span inside [0,1]
    if ymax2 - ymin2 < Y_MIN_SPAN:
        mid = 0.5 * (ymin2 + ymax2)
        ymin2 = float(np.clip(mid - Y_MIN_SPAN/2, Y_DATA_MIN, Y_DATA_MAX))
        ymax2 = float(np.clip(mid + Y_MIN_SPAN/2, Y_DATA_MIN, Y_DATA_MAX))

    # ---- ADAPTIVE headroom near 1.00 (TIGHTER) ----
    if ymax2 >= (Y_DATA_MAX - 1e-6):
        # headroom ~0.2%–0.6% for near-saturated cases:
        # - floor prevents marker clipping
        # - cap keeps RH from showing large white space
        head = min(Y_HEADROOM, max(0.002, 0.10 * max(raw_span, 0.02)))  # floor 0.002, typically 0.002–0.006
        if ymin2 >= 0.85:
            head = min(head, 0.004)  # extra-tight for RH-like (high lower bound)
        ymax_display = min(Y_DISPLAY_MAX, Y_DATA_MAX + head)
        ymax2 = max(ymax2, ymax_display)
    else:
        ymax2 = min(Y_DISPLAY_MAX, ymax2)

    ymin2 = max(Y_DATA_MIN, ymin2)
    return ymin2, ymax2

# ===============================================================================================
# 3) WEIGHT CANDIDATES (Table-3 + extras) + LOOKUP
# ===============================================================================================
def build_weight_table() -> pd.DataFrame:
    paper_cases = [
        ("T3_01", "Case 1",  1,    0,    0),
        ("T3_02", "Case 2",  0,    1,    0),
        ("T3_03", "Case 3",  0,    0,    1),
        ("T3_04", "Case 4",  1/3,  1/3,  1/3),
        ("T3_05", "Case 5",  1/4,  1/4,  2/4),
        ("T3_06", "Case 6",  1/5,  1/5,  3/5),
        ("T3_07", "Case 7",  1/6,  1/6,  4/6),
        ("T3_08", "Case 8",  1/8,  2/8,  5/8),
        ("T3_09", "Case 9",  1/8,  1/8,  6/8),
        ("T3_10", "Case 10", 1/10, 1/10, 8/10),
        ("T3_11", "Case 11", 2/3,  1/3,  1/3),
        ("T3_12", "Case 12", 3/4,  1/4,  1/4),
        ("T3_13", "Case 13", 5/8,  2/8,  1/8),
        ("T3_14", "Case 14", 6/8,  1/8,  1/8),
        ("T3_15", "Case 15", 1/20, 1/20, 18/20),
        ("T3_16", "Case 16", 1/40, 1/40, 38/40),
    ]
    rows = []
    for cid, name, wM, wC, wS in paper_cases:
        rows.append({"weight_case_id": cid, "weight_case_name": name, "wC": float(wC), "wM": float(wM), "wS": float(wS), "source": "Table3"})
    extras = [
        ("EX_01", "SensorOnly", 0.0, 0.0, 1.0),
        ("EX_02", "Sensor90",   0.05, 0.05, 0.90),
        ("EX_03", "Sensor95",   0.025, 0.025, 0.95),
        ("EX_04", "Sensor98",   0.01, 0.01, 0.98),
    ]
    for cid, name, wC, wM, wS in extras:
        rows.append({"weight_case_id": cid, "weight_case_name": name, "wC": float(wC), "wM": float(wM), "wS": float(wS), "source": "Extra"})
    df = pd.DataFrame(rows)
    df["_key"] = df.apply(lambda r: (round(r["wC"], 6), round(r["wM"], 6), round(r["wS"], 6)), axis=1)
    df = df.drop_duplicates("_key").drop(columns=["_key"]).reset_index(drop=True)
    return df

WEIGHT_TABLE = build_weight_table()

def _weight_lookup(w: Tuple[float,float,float]) -> Tuple[str,str,str]:
    wC,wM,wS = (round(float(w[0]),6), round(float(w[1]),6), round(float(w[2]),6))
    m = WEIGHT_TABLE[(WEIGHT_TABLE["wC"].round(6)==wC) & (WEIGHT_TABLE["wM"].round(6)==wM) & (WEIGHT_TABLE["wS"].round(6)==wS)]
    if len(m)>0:
        r = m.iloc[0]
        return str(r["weight_case_id"]), str(r["weight_case_name"]), str(r["source"])
    return ("CUSTOM","Custom","Search")

# ===============================================================================================
# 4) IO + workload parsing
# ===============================================================================================
RUNID2PATH: Dict[str,str] = {}

def detect_setup_from_path(p: Path):
    s = str(p).lower()
    if "ddr4" in s: return "DDR4"
    if "ddr5" in s: return "DDR5"
    return None

def is_benign_path(p: Path) -> bool:
    s = str(p).lower()
    if "benign" in s:
        return True
    bad = ["attack","anom","fault","inject","trojan","mal","rh","droop","spectre","trrespass"]
    return not any(b in s for b in bad)

def is_anomaly_path(p: Path, anomaly: str) -> bool:
    return (anomaly.lower() in str(p).lower()) and (not is_benign_path(p))

def iter_raw_csvs(root: Path):
    for p in root.rglob("*.csv"):
        yield p

def read_csv_clean(p: Path) -> pd.DataFrame:
    df = pd.read_csv(p)
    return df.loc[:, ~df.columns.str.startswith("Unnamed")]

def mk_run_id(path: Path) -> str:
    rid = f"run_{hashlib.md5(str(path).encode('utf-8')).hexdigest()[:10]}"
    RUNID2PATH[rid] = str(path)
    return rid

_WORKLOADS = ["dft","dj","dp","gl","gs","ha","ja","mm","ni","oe","pi","sh","tr"]
def workload_from_path(p: Path) -> str:
    stem = (p.stem or "").lower()
    m = re.search(r"_([a-z]{2,3})$", stem)
    if m and m.group(1) in _WORKLOADS:
        return m.group(1).upper()
    for tok in _WORKLOADS:
        if tok in stem:
            return tok.upper()
    return "UNK"

def collect_raw_pairs_by_setup(data_dir: Path, which: str, anomaly: Optional[str] = None):
    out = {"DDR4": [], "DDR5": []}
    for p in iter_raw_csvs(data_dir):
        setup = detect_setup_from_path(p)
        if setup is None:
            continue
        try:
            if which == "benign":
                if not is_benign_path(p):
                    continue
            else:
                if anomaly is None or not is_anomaly_path(p, anomaly):
                    continue
            df = read_csv_clean(p)
            out[setup].append((p, df))
        except Exception as e:
            print(f"[WARN] read failed {p}: {e}")
    return out

def telemetry_cols(df: pd.DataFrame):
    exclude = set(META) | set(DROOP_META_COLS) | {"workload"}
    return [c for c in df.columns if (c not in exclude) and (df[c].dtype.kind in "fcbiu")]

def drop_low_variance_cols(df: pd.DataFrame, cols: List[str], eps: float = 1e-10) -> List[str]:
    v = df[cols].astype(float).var(axis=0, ddof=0)
    keep = v[v > eps].index.tolist()
    return keep

# ===============================================================================================
# 5) Scaling
# ===============================================================================================
def robust_scale_train(Xb_np: np.ndarray, winsor=(2.0, 98.0)):
    Q1, Q2 = np.percentile(Xb_np, winsor[0], axis=0), np.percentile(Xb_np, winsor[1], axis=0)
    Xb_clip = np.clip(Xb_np, Q1, Q2)
    mu = Xb_clip.mean(axis=0)
    sd = Xb_clip.std(axis=0, ddof=0) + 1e-9
    return mu, sd, Q1, Q2

def apply_robust_scale(X: pd.DataFrame, mu, sd, Q1, Q2):
    Xc = np.clip(X.to_numpy(dtype=float), Q1, Q2)
    Z  = (Xc - mu) / sd
    return pd.DataFrame(Z, columns=X.columns, index=X.index)

# ===============================================================================================
# 6) Windowing (adds sensor transient feats incl CPU Voltage)
# ===============================================================================================
SENSOR_PAT = re.compile(r"cpu\s*voltage|volt|vdd|vcore|vin|vout|power|energy|joule|current|amps?|temp|thermal|hot", re.I)

def window_collapse_means(df: pd.DataFrame, win: int, setup: str, run_id: str, label: str,
                          overlap_ratio: float, src_path: str="") -> pd.DataFrame:
    cols_all = telemetry_cols(df)
    if not cols_all:
        return pd.DataFrame()

    sensor_cols = [c for c in cols_all if SENSOR_PAT.search(c)]
    if "CPU Voltage" in cols_all and "CPU Voltage" not in sensor_cols:
        sensor_cols.append("CPU Voltage")

    n = len(df)
    stride = max(1, int(round(win * (1 - overlap_ratio))))
    starts = list(range(0, n - win + 1, stride))
    if not starts:
        return pd.DataFrame()

    rows = []
    for start in starts:
        chunk = df.iloc[start:start+win]
        X = chunk[cols_all].astype(float)
        means = X.mean(axis=0, numeric_only=True)

        tfeat = {}
        if sensor_cols:
            Xs = X[sensor_cols]
            mins  = Xs.min(axis=0)
            maxs  = Xs.max(axis=0)
            stds  = Xs.std(axis=0, ddof=0)
            means_s = Xs.mean(axis=0)
            drops  = (means_s - mins)
            ranges = (maxs - mins)
            slope  = (Xs.iloc[-1] - Xs.iloc[0])
            for c in sensor_cols:
                tfeat[f"{c}__min"]   = float(mins[c])
                tfeat[f"{c}__max"]   = float(maxs[c])
                tfeat[f"{c}__std"]   = float(stds[c])
                tfeat[f"{c}__drop"]  = float(drops[c])
                tfeat[f"{c}__range"] = float(ranges[c])
                tfeat[f"{c}__slope"] = float(slope[c])

        row = pd.concat([means, pd.Series(tfeat)]).to_frame().T
        row["setup"]    = setup
        row["run_id"]   = run_id
        row["label"]    = label
        row["workload"] = workload_from_path(Path(src_path))
        rows.append(row)

    return pd.concat(rows, ignore_index=True) if rows else pd.DataFrame()

def build_windowed_raw_means(pairs, setup: str, win: int, label: str, overlap_ratio: float) -> pd.DataFrame:
    out = []
    for p, df in pairs:
        rid = mk_run_id(p)
        agg = window_collapse_means(df, win=win, setup=setup, run_id=rid, label=label,
                                    overlap_ratio=overlap_ratio, src_path=str(p))
        if not agg.empty:
            out.append(agg)
    return pd.concat(out, ignore_index=True) if out else pd.DataFrame()

# ===============================================================================================
# 7) Rank lists + robust mapping + CPU Voltage family append for DROOP
# ===============================================================================================
def _read_rank_feats(p: Path):
    try:
        df = pd.read_csv(p)
    except Exception:
        return []
    if df.empty:
        return []
    col = "feature" if "feature" in df.columns else df.columns[0]
    return df[col].dropna().astype(str).tolist()

def load_full_rank_lists(setup: str, win: int, kfold: int):
    def _find_file(setup, win, kfold, sub):
        fname = f"{setup}_{win}_{kfold}_0_{sub}.csv"
        for d in RANK_DIRS:
            pp = d / fname
            if pp.exists():
                return pp
        return None
    out = {}
    for sub in ["compute","memory","sensors"]:
        pp = _find_file(setup, win, kfold, sub)
        out[sub] = _read_rank_feats(pp) if pp else []
    return out

def slice_by_percent(full_lists: Dict[str, List[str]], pct: int):
    sel = {}
    frac = pct / 100.0
    for sub in ("compute","memory","sensors"):
        feats = full_lists.get(sub, []) or []
        k = int(np.ceil(len(feats) * frac))
        if frac > 0 and len(feats) > 0:
            k = max(1, k)
        sel[sub] = feats[:min(len(feats), max(0, k))]
    return sel

ALIAS_MAP = {
    "voltage": ["cpu","voltage","volt","vcore","vdd","vin","vout"],
    "power": ["power","energy","joules","current","amps"],
    "temperature": ["temp","thermal","hot"],
    "frequency": ["freq","afreq","cfreq","clock","clk","mhz","ghz"],
    "ipc": ["ipc","physipc"],
    "cache": ["l1","l2","l3","hit","miss","evict","fill","mpi"],
    "bandwidth": ["read","write","bw","bandwidth"],
}

def _norm(s: str) -> str:
    s = unicodedata.normalize("NFKC", str(s)).lower()
    out = []
    for ch in s:
        out.append(ch if ch.isalnum() else "_")
    s = "".join(out)
    while "__" in s:
        s = s.replace("__","_")
    return s.strip("_")

def _tokens(s: str):
    t = _norm(s).replace("_"," ").split()
    exp = []
    for tok in t:
        exp.append(tok)
        for k, aliases in ALIAS_MAP.items():
            if tok in aliases:
                exp.append(k)
    return set(exp)

def build_column_index(cols: List[str]):
    norm2orig = {}
    tok_index = {}
    for c in cols:
        nc = _norm(c)
        norm2orig.setdefault(nc, c)
        tok_index[c] = _tokens(c)
    return norm2orig, tok_index

def map_ranks_to_existing(ranked_list: List[str], norm2orig, tok_index):
    mapped, seen = [], set()
    for r in ranked_list:
        nr = _norm(r)
        cand = norm2orig.get(nr, None)
        if cand and cand not in seen:
            mapped.append(cand); seen.add(cand); continue
        rtoks = _tokens(r)
        best_c, best_j = None, 0.0
        for c, ctoks in tok_index.items():
            inter = len(rtoks & ctoks)
            union = len(rtoks | ctoks) or 1
            j = inter / union
            if j > best_j:
                best_j, best_c = j, c
        if best_c and best_j >= 0.60 and best_c not in seen:
            mapped.append(best_c); seen.add(best_c)
    return mapped

def intersect_selection_with_columns_robust(sel_raw: Dict[str, List[str]], xb_cols: set):
    norm2orig, tok_index = build_column_index(list(xb_cols))
    out = {}
    for sub in ("compute","memory","sensors"):
        ranked = sel_raw.get(sub, []) or []
        mapped = map_ranks_to_existing(ranked, norm2orig, tok_index)
        out[sub] = [c for c in mapped if c in xb_cols]
    return out

def force_cpu_voltage_family(sel: Dict[str, List[str]], xb_cols: set) -> Dict[str, List[str]]:
    out = dict(sel)
    sensors_now = out.get("sensors", []) or []
    must = []
    if "CPU Voltage" in xb_cols:
        must.append("CPU Voltage")
    for suf in ["__min","__max","__std","__drop","__range","__slope"]:
        c = f"CPU Voltage{suf}"
        if c in xb_cols:
            must.append(c)
    out["sensors"] = list(dict.fromkeys(sensors_now + must))
    return out

# ===============================================================================================
# 8) Balanced evaluation helper
# ===============================================================================================
def balanced_concat(df_ben: pd.DataFrame, df_anom: pd.DataFrame, seed: int = 1337):
    if df_ben is None or df_ben.empty:
        raise ValueError("balanced_concat(): df_ben is empty")
    if df_anom is None or df_anom.empty:
        raise ValueError("balanced_concat(): df_anom is empty")

    rng = np.random.default_rng(int(seed))
    nb = int(len(df_ben))
    na = int(len(df_anom))
    n  = int(min(nb, na))

    idx_b = rng.choice(nb, size=n, replace=(nb < n))
    idx_a = rng.choice(na, size=n, replace=(na < n))

    ben_s = df_ben.iloc[idx_b].copy().reset_index(drop=True)
    anm_s = df_anom.iloc[idx_a].copy().reset_index(drop=True)

    df_eval = pd.concat([ben_s, anm_s], ignore_index=True)
    y_true  = np.concatenate([np.zeros(len(ben_s), dtype=int),
                              np.ones(len(anm_s), dtype=int)])
    return df_eval, y_true

# ===============================================================================================
# 9) Weight selection helpers
# ===============================================================================================
def split_holdout(y, test_size=0.3, seed=SEED):
    sss = StratifiedShuffleSplit(n_splits=1, test_size=test_size, random_state=seed)
    idx = np.arange(len(y))
    _, val_idx = next(sss.split(idx, y))
    return val_idx

def pick_best_w_per_method_PRfirst(y_true: np.ndarray, scores_by_w: Dict[Tuple[float,float,float], Dict[str,np.ndarray]], val_idx: np.ndarray):
    yy = y_true[val_idx]
    best = {m: (-1.0, -1.0, -1.0, None) for m in ["dE_aM","dE_aJ","dC_aM","dC_aJ"]}
    for w, md in scores_by_w.items():
        wS = float(w[2])
        for m, s in md.items():
            sv = np.asarray(s)[val_idx]
            pr  = float(average_precision_score(yy, sv))
            roc = float(roc_auc_score(yy, sv))
            key = (pr, roc, wS)
            if key > best[m][:3]:
                best[m] = (pr, roc, wS, w)
    return {m: best[m][3] for m in best}

# ===============================================================================================
# 10) DSE plots
# ===============================================================================================
def summarize_for_dse(per_run_df: pd.DataFrame) -> pd.DataFrame:
    keys = ["setup","anomaly","win","kfold","method","pct"]
    def q1(x): return float(np.nanpercentile(x, 25))
    def q3(x): return float(np.nanpercentile(x, 75))
    g = (per_run_df.groupby(keys, as_index=False)
            .agg(
                roc_auc_median=("roc_auc","median"),
                roc_auc_q1=("roc_auc", q1),
                roc_auc_q3=("roc_auc", q3),
                roc_auc_min=("roc_auc","min"),
                roc_auc_max=("roc_auc","max"),
                auc_pr_median=("auc_pr","median"),
                auc_pr_q1=("auc_pr", q1),
                auc_pr_q3=("auc_pr", q3),
                auc_pr_min=("auc_pr","min"),
                auc_pr_max=("auc_pr","max"),
                n_runs=("run_id","nunique"),
            ))
    return g.sort_values(keys).reset_index(drop=True)

def plot_dse_grid(summary_case: pd.DataFrame, metric: str, setup: str, anomaly: str,
                  win: int, kfold: int, out_dir: Path):
    out_dir.mkdir(parents=True, exist_ok=True)
    x_ticks = sorted(summary_case["pct"].unique().tolist())
    if not x_ticks:
        return

    fig, axes = plt.subplots(2, 2, figsize=FIGSIZE, sharex=False, sharey=False)
    axes = axes.flatten()
    fig.subplots_adjust(left=LEFT, right=RIGHT, top=TOP, bottom=BOTTOM, wspace=WSPACE, hspace=HSPACE)

    xlab = "Top-ranked features used (%)"
    ylab = "ROC-AUC" if metric == "roc_auc" else "AUC-PR"

    for i, method in enumerate(METHOD_ORDER):
        ax = axes[i]
        ax.set_title(METHOD_TITLE.get(method, method), pad=7)
        ax.set_xlim(min(x_ticks), max(x_ticks))
        ax.set_xticks(x_ticks)
        ax.grid(True, alpha=GRID_ALPHA, linestyle=GRID_LS, linewidth=GRID_LW)

        sub = summary_case[summary_case["method"] == method].copy().sort_values("pct")
        if sub.empty:
            ax.text(0.5, 0.5, "No data", ha="center", va="center", transform=ax.transAxes)
            ax.set_ylim(Y_DATA_MIN, Y_DISPLAY_MAX)
        else:
            x    = sub["pct"].to_numpy(dtype=float)
            med  = _clip_metric(sub[f"{metric}_median"].to_numpy(dtype=float))
            q1   = _clip_metric(sub[f"{metric}_q1"].to_numpy(dtype=float))
            q3   = _clip_metric(sub[f"{metric}_q3"].to_numpy(dtype=float))
            vmin = _clip_metric(sub[f"{metric}_min"].to_numpy(dtype=float))
            vmax = _clip_metric(sub[f"{metric}_max"].to_numpy(dtype=float))

            lo_mm, hi_mm = np.minimum(vmin, vmax), np.maximum(vmin, vmax)
            lo_iq, hi_iq = np.minimum(q1, q3),     np.maximum(q1, q3)
            lo_mm, hi_mm = _ensure_visible_band(lo_mm, hi_mm)
            lo_iq, hi_iq = _ensure_visible_band(lo_iq, hi_iq)

            ax.fill_between(x, lo_mm, hi_mm, alpha=ALPHA_MINMAX, label="min-max", linewidth=0.0)
            ax.fill_between(x, lo_iq, hi_iq, alpha=ALPHA_IQR,   label="IQR",     linewidth=0.0)
            ax.plot(x, med, color="black", marker="o", linewidth=2.0, markersize=5.5, zorder=5, label="Median")

            if Y_AUTO:
                ylo, yhi = _auto_ylim_from_arrays([lo_mm, hi_mm, lo_iq, hi_iq, med])
                ax.set_ylim(ylo, yhi)
            else:
                ax.set_ylim(Y_DATA_MIN, Y_DISPLAY_MAX)

        ax.set_xlabel(xlab, labelpad=6)
        ax.set_ylabel(ylab, labelpad=6)
        ax.tick_params(axis="x", labelbottom=True, pad=3)
        ax.tick_params(axis="y", labelleft=True, pad=3)

    handles, labels = axes[0].get_legend_handles_labels()
    fig.legend(handles, labels, loc="lower center", bbox_to_anchor=(0.5, LEGEND_Y), ncol=3, frameon=False)
    fig.suptitle(f"{setup}: {anomaly} (WIN={win}, K={kfold})", y=SUPTITLE_Y)

    base = f"DSE_{metric.upper()}_{setup}_{anomaly}_WIN{int(win)}_KF{int(kfold)}"
    fig.savefig(out_dir / f"{base}.png", dpi=600, bbox_inches="tight", pad_inches=0.06)
    fig.savefig(out_dir / f"{base}.pdf", bbox_inches="tight", pad_inches=0.06)
    plt.close(fig)

def write_all_dse_plots(per_run_df: pd.DataFrame):
    out_root = RES_DIR / "DesignSpace" / "PaperStyle_ALLRUNS"
    out_root.mkdir(parents=True, exist_ok=True)
    dse = summarize_for_dse(per_run_df)
    for (setup, anomaly, win, kfold), sub in dse.groupby(["setup","anomaly","win","kfold"]):
        out_dir = out_root / str(setup) / str(anomaly) / f"WIN{int(win)}_KF{int(kfold)}"
        plot_dse_grid(sub, "roc_auc", str(setup), str(anomaly), int(win), int(kfold), out_dir)
        plot_dse_grid(sub, "auc_pr",  str(setup), str(anomaly), int(win), int(kfold), out_dir)

# ===============================================================================================
# 11) Subspace-weight tables (same as before; omitted here for brevity)
# ===============================================================================================
def build_workload_weight_tables(per_run_df: pd.DataFrame, setup: str, win: int, kfold: int) -> None:
    # (same implementation you already use; keep unchanged)
    base_dir = RES_DIR / "Table_SubspaceWeights" / setup / f"WIN{win}_KF{kfold}"
    base_dir.mkdir(parents=True, exist_ok=True)

    rep_method = "dE_aM"
    for anomaly in ANOMALIES_BY_SETUP[setup]:
        df = per_run_df[
            (per_run_df["setup"]==setup) &
            (per_run_df["anomaly"]==anomaly) &
            (per_run_df["win"]==win) &
            (per_run_df["kfold"]==kfold)
        ].copy()
        if df.empty:
            continue

        df["_wkey"] = df.apply(lambda r: (round(r["wC"],6), round(r["wM"],6), round(r["wS"],6)), axis=1)
        rows = []
        for (wl, method), g in df.groupby(["workload","method"]):
            mode_key = g["_wkey"].value_counts().idxmax()
            wC, wM, wS = mode_key
            wid, wname, wsrc = _weight_lookup((wC,wM,wS))
            rows.append({
                "workload": wl,
                "method": method,
                "chosen_wC": wC, "chosen_wM": wM, "chosen_wS": wS,
                "weight_case_id": wid,
                "weight_case_name": wname,
                "weight_case_source": wsrc,
                "n_rows": int(len(g)),
                "median_roc": float(np.nanmedian(g["roc_auc"])),
                "median_pr": float(np.nanmedian(g["auc_pr"])),
            })
        df_out = pd.DataFrame(rows).sort_values(["workload","method"]).reset_index(drop=True)
        (base_dir / f"Table_SubspaceWeights_{setup}_WIN{win}_KF{kfold}_{anomaly}.csv").write_text(df_out.to_csv(index=False))

# ===============================================================================================
# 12) Pipeline core (same logic as your current version; unchanged except DSE y-axis)
# ===============================================================================================
def run_full_pipeline() -> pd.DataFrame:
    all_rows = []
    group_counter = 0

    benign_pairs = collect_raw_pairs_by_setup(DATA_DIR, which="benign")
    anomaly_pairs_cache: Dict[Tuple[str,str], List[Tuple[Path,pd.DataFrame]]] = {}

    def get_anom_pairs(setup: str, anomaly: str):
        key = (setup, anomaly.upper())
        if key not in anomaly_pairs_cache:
            anomaly_pairs_cache[key] = collect_raw_pairs_by_setup(DATA_DIR, which="anomaly", anomaly=anomaly)[setup]
        return anomaly_pairs_cache[key]

    for setup in SETUPS:
        anomalies = ANOMALIES_BY_SETUP.get(setup, [])
        ben_pairs_all = benign_pairs.get(setup, [])
        if not ben_pairs_all:
            print(f"[SKIP] {setup}: no benign RAW files")
            continue

        for win in WINS:
            df_ben_all = build_windowed_raw_means(ben_pairs_all, setup=setup, win=win, label="BENIGN",
                                                  overlap_ratio=OVERLAP_RATIO)
            if df_ben_all.empty:
                print(f"[SKIP] {setup} WIN={win}: benign windowed empty")
                continue

            Xb_cols = telemetry_cols(df_ben_all)
            if DROP_LOW_VARIANCE_COLS:
                Xb_cols = drop_low_variance_cols(df_ben_all, Xb_cols, eps=LOW_VAR_EPS)
            if not Xb_cols:
                print(f"[SKIP] {setup} WIN={win}: no telemetry cols after low-var drop")
                continue

            Xb = df_ben_all[Xb_cols].astype(float)
            mu_s, sd_s, Q1, Q2 = robust_scale_train(Xb.values, winsor=ROBUST_WINSOR)
            Xb_base = apply_robust_scale(Xb, mu_s, sd_s, Q1, Q2)
            xb_cols_set = set(Xb_base.columns)

            ben_by_wl = {}
            for wl in df_ben_all["workload"].astype(str).unique():
                ben_by_wl[wl] = df_ben_all[df_ben_all["workload"].astype(str)==wl].copy()

            for kfold in KFOLDS:
                wdir = RES_DIR / "Table_SubspaceWeights" / setup / f"WIN{win}_KF{kfold}"
                wdir.mkdir(parents=True, exist_ok=True)
                WEIGHT_TABLE.to_csv(wdir / "Table_SubspaceWeights_CANDIDATES.csv", index=False)

                full_lists = load_full_rank_lists(setup, win, kfold)

                for anomaly in anomalies:
                    group_counter += 1
                    print(f"\n[RUN] {setup} {anomaly} WIN={win} KF={kfold} (group {group_counter})")

                    overlap = OVERLAP_RATIO_DROOP if anomaly.upper() == "DROOP" else OVERLAP_RATIO
                    an_pairs_all = get_anom_pairs(setup, anomaly)
                    if not an_pairs_all:
                        print(f"[SKIP] {setup} {anomaly}: no anomaly RAW files")
                        continue

                    df_anom = build_windowed_raw_means(an_pairs_all, setup=setup, win=win, label=anomaly,
                                                       overlap_ratio=overlap)
                    if df_anom.empty:
                        print(f"[SKIP] {setup} {anomaly} WIN={win}: anomaly windowed empty")
                        continue

                    run_ids = sorted(df_anom["run_id"].astype(str).unique())

                    for rid in run_ids:
                        dfa_run = df_anom[df_anom["run_id"].astype(str) == rid].copy()
                        if dfa_run.empty:
                            continue

                        wl = str(dfa_run["workload"].iloc[0]) if "workload" in dfa_run.columns else "UNK"
                        df_ben = ben_by_wl.get(wl, df_ben_all)
                        if df_ben.empty:
                            df_ben = df_ben_all

                        df_eval, y_true = balanced_concat(df_ben, dfa_run, seed=SEED)

                        Xe_all  = df_eval[Xb_cols].astype(float)
                        Xe_base = apply_robust_scale(Xe_all, mu_s, sd_s, Q1, Q2)

                        for idx_p, pct in enumerate(PCT_SWEEP, start=1):
                            sel_raw = slice_by_percent(full_lists, pct)
                            sel = intersect_selection_with_columns_robust(sel_raw, xb_cols_set)
                            if (not sel.get("compute")) or (not sel.get("memory")) or (not sel.get("sensors")):
                                continue
                            if anomaly.upper() == "DROOP":
                                sel = force_cpu_voltage_family(sel, xb_cols_set)

                            refs = _build_refs(Xb_base, sel, eps=1e-12)
                            benign_norm = fit_parts_normalizer_on_benign(Xb_base, refs)

                            val_idx = split_holdout(y_true, test_size=0.3, seed=SEED)

                            scores_by_w = {}
                            for _, wr in WEIGHT_TABLE.iterrows():
                                w = (float(wr["wC"]), float(wr["wM"]), float(wr["wS"]))
                                scores_by_w[w] = score_four_methods_benign_norm(Xe_base, refs, w, benign_norm)

                            best_w_for = pick_best_w_per_method_PRfirst(y_true, scores_by_w, val_idx)

                            for method in METHOD_ORDER:
                                w_star = best_w_for[method]
                                y_score = score_four_methods_benign_norm(Xe_base, refs, w_star, benign_norm)[method]
                                y_score = np.nan_to_num(y_score, nan=0.0, posinf=0.0, neginf=0.0)

                                if DROOP_BOOST and anomaly.upper() == "DROOP":
                                    boost = droop_boost_score(Xe_base, sel)
                                    y_score = (1.0 - DROOP_BOOST_ALPHA) * y_score + DROOP_BOOST_ALPHA * boost

                                roc = float(np.clip(roc_auc_score(y_true, y_score), Y_DATA_MIN, Y_DATA_MAX))
                                pr  = float(np.clip(average_precision_score(y_true, y_score), Y_DATA_MIN, Y_DATA_MAX))

                                wid, wname, wsrc = _weight_lookup(w_star)

                                all_rows.append({
                                    "setup": setup,
                                    "anomaly": anomaly,
                                    "win": int(win),
                                    "kfold": int(kfold),
                                    "pct": int(pct),
                                    "run_id": str(rid),
                                    "workload": str(wl),
                                    "method": method,
                                    "roc_auc": roc,
                                    "auc_pr": pr,
                                    "weight_case_id": wid,
                                    "weight_case_name": wname,
                                    "weight_case_source": wsrc,
                                    "wC": float(w_star[0]),
                                    "wM": float(w_star[1]),
                                    "wS": float(w_star[2]),
                                })

                            if idx_p % GC_EVERY_N_PCTS == 0:
                                gc.collect()

                    if group_counter % GC_EVERY_N_GROUPS == 0:
                        gc.collect()

    per_run_df = pd.DataFrame(all_rows)
    out_all = RES_DIR / "per_run_metrics_all_PIPELINE.csv"
    per_run_df.to_csv(out_all, index=False)
    print(f"\n[OK] ALLRUNS metrics → {out_all}")

    for setup in SETUPS:
        for win in WINS:
            for kfold in KFOLDS:
                build_workload_weight_tables(per_run_df, setup=setup, win=win, kfold=kfold)

    write_all_dse_plots(per_run_df)
    return per_run_df

# ===============================================================================================
# 13) MAIN
# ===============================================================================================
def main():
    run_full_pipeline()
    print("[DONE] Pipeline + DSE plots + weight tables per platform")

if __name__ == "__main__":
    main()

[WARN] Cannot write to /Volumes/Untitled/octaneX_results. Using local: /Users/hsiaopingni/octaneX_v7_4functions/Results

[RUN] DDR4 DROOP WIN=32 KF=3 (group 1)

[RUN] DDR4 RH WIN=32 KF=3 (group 2)

[RUN] DDR4 DROOP WIN=32 KF=5 (group 3)

[RUN] DDR4 RH WIN=32 KF=5 (group 4)

[RUN] DDR4 DROOP WIN=32 KF=10 (group 5)

[RUN] DDR4 RH WIN=32 KF=10 (group 6)

[RUN] DDR4 DROOP WIN=64 KF=3 (group 7)

[RUN] DDR4 RH WIN=64 KF=3 (group 8)

[RUN] DDR4 DROOP WIN=64 KF=5 (group 9)

[RUN] DDR4 RH WIN=64 KF=5 (group 10)

[RUN] DDR4 DROOP WIN=64 KF=10 (group 11)

[RUN] DDR4 RH WIN=64 KF=10 (group 12)

[RUN] DDR4 DROOP WIN=128 KF=3 (group 13)

[RUN] DDR4 RH WIN=128 KF=3 (group 14)

[RUN] DDR4 DROOP WIN=128 KF=5 (group 15)

[RUN] DDR4 RH WIN=128 KF=5 (group 16)

[RUN] DDR4 DROOP WIN=128 KF=10 (group 17)

[RUN] DDR4 RH WIN=128 KF=10 (group 18)

[RUN] DDR4 DROOP WIN=512 KF=3 (group 19)

[RUN] DDR4 RH WIN=512 KF=3 (group 20)

[RUN] DDR4 DROOP WIN=512 KF=5 (group 21)

[RUN] DDR4 RH WIN=512 KF=5 (group 22)

[RU

### Print DSE stats (mean, IQR, min, max) for selected cases

In [24]:
# === Print DSE stats (mean, IQR, min, max) for selected cases — PER PLATFORM ===
# This version uses your NEW pipeline outputs:
#   - Results/per_run_metrics_all_PIPELINE.csv  (always produced)
# and reads platform winners from:
#   - Results/BEST_in_DesignSpace_Post_per_platform_details.csv
#
# For each platform (DDR4, DDR5), it prints DSE summary for the chosen (WIN,KF)
# for each anomaly on that platform, across ALL PCT values, for each method.
#
# Summary per (setup, anomaly, win, kfold, method):
#   - mean_of_medians over pct (median across run_id at each pct, then mean over pct)
#   - mean IQR over pct
#   - min_over_pct (global min of per-pct min across run_id)
#   - max_over_pct (global max of per-pct max across run_id)
#
# Output:
#   - prints a table (and writes CSV Results/DSE_summary_per_platform.csv)
# ------------------------------------------------------------------------------

from pathlib import Path
import pandas as pd
import numpy as np

ROOT     = Path(globals().get("ROOT", "/Users/hsiaopingni/octaneX_v7_4functions"))
RES_DIR  = Path(globals().get("RES_DIR", ROOT / "Results"))
if RES_DIR.name in ("Explainability_SHAP_BestCases", "Explainability_SHAP_BestPlatforms"):
    RES_DIR = RES_DIR.parent

PER_RUN = RES_DIR / "per_run_metrics_all_PIPELINE.csv"
DETAILS = RES_DIR / "BEST_in_DesignSpace_Post_per_platform_details.csv"

if not PER_RUN.exists():
    raise FileNotFoundError(f"Missing per-run pipeline CSV: {PER_RUN}")
if not DETAILS.exists():
    raise FileNotFoundError(f"Missing per-platform details CSV: {DETAILS}")

df = pd.read_csv(PER_RUN).copy()
det = pd.read_csv(DETAILS).copy()

# Normalize columns
df.columns = [c.lower() for c in df.columns]
det.columns = [c.lower() for c in det.columns]
if "best_method" in det.columns and "method" not in det.columns:
    det = det.rename(columns={"best_method":"method"})
if "best_pct_by_median" not in det.columns and "pct" in det.columns:
    det = det.rename(columns={"pct":"best_pct_by_median"})

need_df = {"setup","anomaly","win","kfold","pct","run_id","method","roc_auc","auc_pr"}
miss = need_df - set(df.columns)
if miss:
    raise KeyError(f"{PER_RUN} missing columns: {sorted(miss)}. Have: {list(df.columns)}")

need_det = {"setup","anomaly","win","kfold"}
miss2 = need_det - set(det.columns)
if miss2:
    raise KeyError(f"{DETAILS} missing columns: {sorted(miss2)}. Have: {list(det.columns)}")

# Coerce numeric
for c in ["win","kfold","pct","roc_auc","auc_pr"]:
    df[c] = pd.to_numeric(df[c], errors="coerce")

df = df.dropna(subset=["setup","anomaly","win","kfold","pct","method"]).copy()
df["roc_auc"] = df["roc_auc"].clip(0.0, 1.0)
df["auc_pr"]  = df["auc_pr"].clip(0.0, 1.0)

# ------------------ Build DSE stats at each pct ------------------
def q1(x): return float(np.nanpercentile(x, 25))
def q3(x): return float(np.nanpercentile(x, 75))

keys = ["setup","anomaly","win","kfold","method","pct"]
dse = (df.groupby(keys, as_index=False)
         .agg(
             roc_auc_median=("roc_auc","median"),
             roc_auc_q1=("roc_auc", q1),
             roc_auc_q3=("roc_auc", q3),
             roc_auc_min=("roc_auc","min"),
             roc_auc_max=("roc_auc","max"),
             auc_pr_median=("auc_pr","median"),
             auc_pr_q1=("auc_pr", q1),
             auc_pr_q3=("auc_pr", q3),
             auc_pr_min=("auc_pr","min"),
             auc_pr_max=("auc_pr","max"),
             n_runs=("run_id","nunique"),
         ))
dse["roc_auc_iqr"] = dse["roc_auc_q3"] - dse["roc_auc_q1"]
dse["auc_pr_iqr"]  = dse["auc_pr_q3"]  - dse["auc_pr_q1"]

# ------------------ Helper: summarize over ALL PCT for a single case ------------------
def _summarize_case_over_pct(df_case: pd.DataFrame) -> dict | None:
    if df_case.empty:
        return None

    def agg(prefix):
        arr_med = df_case[f"{prefix}_median"].to_numpy(dtype=float)
        arr_min = df_case[f"{prefix}_min"].to_numpy(dtype=float)
        arr_max = df_case[f"{prefix}_max"].to_numpy(dtype=float)
        arr_iqr = df_case[f"{prefix}_iqr"].to_numpy(dtype=float)
        return {
            f"{prefix}_mean_of_medians": float(np.nanmean(arr_med)),
            f"{prefix}_iqr_mean": float(np.nanmean(arr_iqr)),
            f"{prefix}_min_over_pct": float(np.nanmin(arr_min)),
            f"{prefix}_max_over_pct": float(np.nanmax(arr_max)),
        }

    out = {
        "pct_values": sorted(df_case["pct"].astype(int).unique().tolist()),
        "n_pcts": int(df_case["pct"].nunique()),
        "n_runs_median": float(np.nanmedian(df_case["n_runs"].astype(float).values)),
    }
    out.update(agg("auc_pr"))
    out.update(agg("roc_auc"))
    return out

# ------------------ Per-platform selection from details CSV ------------------
# Use (setup, win, kfold) from per-platform details (should be consistent per platform)
platform_rows = []
missing = []

METHOD_ORDER = ["dC_aJ", "dC_aM", "dE_aJ", "dE_aM"]

for setup in ["DDR4","DDR5"]:
    dplat = det[det["setup"].astype(str).str.upper() == setup].copy()
    if dplat.empty:
        missing.append((setup, "ALL", "NO_DETAILS"))
        continue

    win_best = int(pd.to_numeric(dplat["win"], errors="coerce").dropna().iloc[0])
    kf_best  = int(pd.to_numeric(dplat["kfold"], errors="coerce").dropna().iloc[0])

    for anomaly in sorted(dplat["anomaly"].astype(str).unique().tolist()):
        for method in METHOD_ORDER:
            sub = dse[
                (dse["setup"].astype(str).str.upper() == setup.upper()) &
                (dse["anomaly"].astype(str).str.upper() == str(anomaly).upper()) &
                (pd.to_numeric(dse["win"], errors="coerce") == win_best) &
                (pd.to_numeric(dse["kfold"], errors="coerce") == kf_best) &
                (dse["method"].astype(str) == method)
            ].copy()

            res = _summarize_case_over_pct(sub)
            if res is None:
                missing.append((setup, anomaly, f"WIN{win_best}_KF{kf_best}_{method}"))
                continue

            platform_rows.append({
                "setup": setup,
                "anomaly": anomaly,
                "win": win_best,
                "kfold": kf_best,
                "method": method,
                **res
            })

# ------------------ Present / save ------------------
if platform_rows:
    out = pd.DataFrame(platform_rows)
    out_csv = RES_DIR / "DSE_summary_per_platform.csv"
    out.to_csv(out_csv, index=False)

    # Pretty print
    disp_cols = [
        "setup","anomaly","win","kfold","method","n_pcts",
        "auc_pr_mean_of_medians","auc_pr_iqr_mean","auc_pr_min_over_pct","auc_pr_max_over_pct",
        "roc_auc_mean_of_medians","roc_auc_iqr_mean","roc_auc_min_over_pct","roc_auc_max_over_pct",
    ]
    show = out[disp_cols].copy()

    # Format numeric columns
    num_cols = [c for c in show.columns if c not in ("setup","anomaly","win","kfold","method","n_pcts")]
    for c in num_cols:
        show[c] = show[c].map(lambda x: f"{float(x):.4f}")

    print("=== DSE summary (mean/IQR/min/max across PCT) — per platform’s chosen (WIN,KF) ===")
    try:
        display(show)
    except Exception:
        print(show.to_string(index=False))

    print(f"\n[OK] Wrote CSV → {out_csv}")
else:
    print("No matching rows found for per-platform selected cases.")

if missing:
    print("\n[WARN] Missing combinations (no rows in per-run DSE aggregation):")
    for m in missing[:30]:
        print("  - setup={}, anomaly={}, tag={}".format(*m))
    if len(missing) > 30:
        print(f"  ... and {len(missing)-30} more")
    print("Tip: verify that the per-run CSV includes these (setup, anomaly, win, kfold, method) rows.")


=== DSE summary (mean/IQR/min/max across PCT) — per platform’s chosen (WIN,KF) ===


,setup,anomaly,win,kfold,method,n_pcts,auc_pr_mean_of_medians,auc_pr_iqr_mean,auc_pr_min_over_pct,auc_pr_max_over_pct,roc_auc_mean_of_medians,roc_auc_iqr_mean,roc_auc_min_over_pct,roc_auc_max_over_pct
0,DDR4,DROOP,512,3,dC_aJ,10,0.9996,0.0899,0.6027,1.0000,0.9995,0.0482,0.7445,1.0000
1,DDR4,DROOP,512,3,dC_aM,10,0.9989,0.0854,0.6027,1.0000,0.9988,0.0546,0.7445,1.0000
2,DDR4,DROOP,512,3,dE_aJ,10,0.9996,0.0899,0.6027,1.0000,0.9995,0.0482,0.7445,1.0000
3,DDR4,DROOP,512,3,dE_aM,10,0.9989,0.0854,0.6027,1.0000,0.9988,0.0546,0.7445,1.0000
4,DDR4,RH,512,3,dC_aJ,10,1.0000,0.0000,0.9467,1.0000,1.0000,0.0000,0.9294,1.0000
5,DDR4,RH,512,3,dC_aM,10,1.0000,0.0000,0.9360,1.0000,1.0000,0.0000,0.9155,1.0000
6,DDR4,RH,512,3,dE_aJ,10,1.0000,0.0000,0.9467,1.0000,1.0000,0.0000,0.9294,1.0000
7,DDR4,RH,512,3,dE_aM,10,1.0000,0.0000,0.9360,1.0000,1.0000,0.0000,0.9155,1.0000
8,DDR5,DROOP,1024,5,dC_aJ,10,0.9718,0.1413,0.4225,1.0000,0.9719,0.1188,0.3889,1.0000
9,DDR5,DROOP,1024,5,dC_aM,10,0.9701,0.1194,0.4159,1.0000,0.9667,0.1105,0.3704,1.0000



[OK] Wrote CSV → /Users/hsiaopingni/octaneX_v7_4functions/Results/DSE_summary_per_platform.csv


###  Find best Top-% (PCT) per (setup, anomaly, win, kfold)

In [25]:
# === Find best Top-% (PCT) per PLATFORM (setup) for the chosen (WIN,KF) ===
# PER-PLATFORM version:
#   - Reads per-run metrics from your NEW pipeline:
#       Results/per_run_metrics_all_PIPELINE.csv
#   - Uses per-platform chosen (WIN,KF) from:
#       Results/BEST_in_DesignSpace_Post_per_platform.csv
#   - For each platform and each anomaly on that platform, finds BEST PCT
#     at that platform's (WIN,KF), per METHOD, using:
#       Sort key: AUCPR_median desc, then ROC_median desc, then smaller pct
#
# Outputs:
#   - Prints a table
#   - Writes Results/BEST_pct_per_platform_WIN_KF.csv
# ---------------------------------------------------------------------------

from pathlib import Path
import pandas as pd
import numpy as np

ROOT    = Path(globals().get("ROOT", "/Users/hsiaopingni/octaneX_v7_4functions"))
RES_DIR = Path(globals().get("RES_DIR", ROOT / "Results"))
if RES_DIR.name in ("Explainability_SHAP_BestCases", "Explainability_SHAP_BestPlatforms"):
    RES_DIR = RES_DIR.parent

PER_RUN   = RES_DIR / "per_run_metrics_all_PIPELINE.csv"
PLAT_BEST = RES_DIR / "BEST_in_DesignSpace_Post_per_platform.csv"

if not PER_RUN.exists():
    raise FileNotFoundError(f"Missing pipeline per-run CSV: {PER_RUN}")
if not PLAT_BEST.exists():
    raise FileNotFoundError(f"Missing per-platform winners CSV: {PLAT_BEST}")

df = pd.read_csv(PER_RUN).copy()
pb = pd.read_csv(PLAT_BEST).copy()

# Normalize
df.columns = [c.lower() for c in df.columns]
pb.columns = [c.lower() for c in pb.columns]

need_df = {"setup","anomaly","win","kfold","pct","run_id","method","roc_auc","auc_pr"}
miss = need_df - set(df.columns)
if miss:
    raise KeyError(f"{PER_RUN} missing columns: {sorted(miss)}. Have: {list(df.columns)}")

need_pb = {"setup","win","kfold"}
miss2 = need_pb - set(pb.columns)
if miss2:
    raise KeyError(f"{PLAT_BEST} missing columns: {sorted(miss2)}. Have: {list(pb.columns)}")

# Ensure numeric
for c in ["win","kfold","pct","roc_auc","auc_pr"]:
    df[c] = pd.to_numeric(df[c], errors="coerce")

df = df.dropna(subset=["setup","anomaly","win","kfold","pct","method"]).copy()
df["roc_auc"] = df["roc_auc"].clip(0.0, 1.0)
df["auc_pr"]  = df["auc_pr"].clip(0.0, 1.0)

METHOD_ORDER = ["dC_aJ", "dC_aM", "dE_aJ", "dE_aM"]
df = df[df["method"].isin(METHOD_ORDER)].copy()

# ---- Build pct-level medians per (setup, anomaly, win, kfold, method, pct) ----
def q1(x): return float(np.nanpercentile(x, 25))
def q3(x): return float(np.nanpercentile(x, 75))

keys = ["setup","anomaly","win","kfold","method","pct"]
sumdf = (df.groupby(keys, as_index=False)
           .agg(
               auc_pr_median=("auc_pr","median"),
               roc_auc_median=("roc_auc","median"),
               auc_pr_q1=("auc_pr", q1),
               auc_pr_q3=("auc_pr", q3),
               roc_auc_q1=("roc_auc", q1),
               roc_auc_q3=("roc_auc", q3),
               n_runs=("run_id","nunique"),
           ))
sumdf["auc_pr_iqr"]  = sumdf["auc_pr_q3"]  - sumdf["auc_pr_q1"]
sumdf["roc_auc_iqr"] = sumdf["roc_auc_q3"] - sumdf["roc_auc_q1"]
sumdf["auc_pr_median_filled"]  = sumdf["auc_pr_median"].fillna(-np.inf)
sumdf["roc_auc_median_filled"] = sumdf["roc_auc_median"].fillna(-np.inf)

# ---- Per platform: choose (WIN,KF) and then best pct per anomaly×method ----
rows = []
for _, r in pb.iterrows():
    setup = str(r["setup"]).strip()
    win_best = int(pd.to_numeric(r["win"], errors="coerce"))
    kf_best  = int(pd.to_numeric(r["kfold"], errors="coerce"))

    sub_plat = sumdf[
        (sumdf["setup"].astype(str).str.upper() == setup.upper()) &
        (pd.to_numeric(sumdf["win"], errors="coerce") == win_best) &
        (pd.to_numeric(sumdf["kfold"], errors="coerce") == kf_best)
    ].copy()
    if sub_plat.empty:
        continue

    for anomaly in sorted(sub_plat["anomaly"].astype(str).unique().tolist()):
        for method in METHOD_ORDER:
            sub = sub_plat[
                (sub_plat["anomaly"].astype(str).str.upper() == str(anomaly).upper()) &
                (sub_plat["method"] == method)
            ].copy()
            if sub.empty:
                continue

            sub = sub.sort_values(
                ["auc_pr_median_filled", "roc_auc_median_filled", "pct"],
                ascending=[False, False, True]
            )
            best = sub.iloc[0]
            rows.append({
                "setup": setup,
                "win": win_best,
                "kfold": kf_best,
                "anomaly": anomaly,
                "method": method,
                "best_pct": int(best["pct"]),
                "auc_pr_median": float(best["auc_pr_median"]),
                "roc_auc_median": float(best["roc_auc_median"]),
                "auc_pr_iqr": float(best["auc_pr_iqr"]),
                "roc_auc_iqr": float(best["roc_auc_iqr"]),
                "n_runs": int(best["n_runs"]),
            })

out = pd.DataFrame(rows).sort_values(["setup","anomaly","method"]).reset_index(drop=True)
OUT_CSV = RES_DIR / "BEST_pct_per_platform_WIN_KF.csv"
out.to_csv(OUT_CSV, index=False)

print(f"[OK] Wrote → {OUT_CSV}")

# Pretty print
if out.empty:
    print("[WARN] No rows produced (check that per_run_metrics_all_PIPELINE.csv contains those WIN/KF combos).")
else:
    show = out.copy()
    for c in ["auc_pr_median","roc_auc_median","auc_pr_iqr","roc_auc_iqr"]:
        show[c] = show[c].map(lambda x: f"{x:.4f}")
    try:
        display(show)
    except Exception:
        print(show.to_string(index=False))

[OK] Wrote → /Users/hsiaopingni/octaneX_v7_4functions/Results/BEST_pct_per_platform_WIN_KF.csv


,setup,win,kfold,anomaly,method,best_pct,auc_pr_median,roc_auc_median,auc_pr_iqr,roc_auc_iqr,n_runs
0,DDR4,512,3,DROOP,dC_aJ,10,1.0000,1.0000,0.0387,0.0353,13
1,DDR4,512,3,DROOP,dC_aM,10,1.0000,1.0000,0.0021,0.0021,13
2,DDR4,512,3,DROOP,dE_aJ,10,1.0000,1.0000,0.0387,0.0353,13
3,DDR4,512,3,DROOP,dE_aM,10,1.0000,1.0000,0.0021,0.0021,13
4,DDR4,512,3,RH,dC_aJ,10,1.0000,1.0000,0.0000,0.0000,13
5,DDR4,512,3,RH,dC_aM,10,1.0000,1.0000,0.0000,0.0000,13
6,DDR4,512,3,RH,dE_aJ,10,1.0000,1.0000,0.0000,0.0000,13
7,DDR4,512,3,RH,dE_aM,10,1.0000,1.0000,0.0000,0.0000,13
8,DDR5,1024,5,DROOP,dC_aJ,20,0.9971,0.9969,0.2914,0.2222,13
9,DDR5,1024,5,DROOP,dC_aM,20,0.9899,0.9877,0.2570,0.2160,13


## Explainability results in Overleaf

In [26]:
# === PER-PLATFORM SHAP–CPMI agreement (top-% Jaccard) + plot for Overleaf ===
# UPDATED to be PER PLATFORM (DDR4, DDR5), not per test case.
#
# What it does:
#   - Uses per-platform winners/configs from:
#       Results/BEST_in_DesignSpace_Post_per_platform_details.csv
#     (gives anomaly-specific best pct + method at the platform-chosen WIN/KF)
#   - Loads SHAP per anomaly from:
#       Results/Explainability_SHAP_BestPlatforms/SHAP_BESTPLAT_full_<setup>_<anomaly>_WIN<w>_KF<k>_PCT<p>_M<method>.csv
#   - Loads CP-MI rank lists from:
#       FeatureRankOUT/<setup>_<win>_<kfold>_0_<subspace>.csv
#   - Computes Jaccard overlap between:
#       top-% CP-MI features  vs  top-% SHAP features
#     for each subspace, then averages across subspaces to get a platform curve.
#
# Outputs:
#   - CSV: Results/Explainability_SHAP_BestPlatforms/SHAP_vs_CPMI_agreement_PLATFORM.csv
#   - Figure: <ROOT>/figs/Explainability/SHAP_vs_CPMI_agreement_PLATFORM.png
#   - (optional) per-platform tables: <ROOT>/tables/SHAP_vs_CPMI_agreement_PLATFORM_summary.csv
#
# Notes:
#   - Agreement metric is Jaccard(top-k CP-MI, top-k SHAP) where k is computed separately for each list.
#   - Platform aggregation:
#       For each (setup, subspace, pct): average Jaccard across that platform's anomalies
#       Then for plotting: average across subspaces (compute/memory/sensors)
# ------------------------------------------------------------------------------

from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import glob, re

# -------------------- Paths --------------------
ROOT   = Path(globals().get("ROOT", "/Users/hsiaopingni/octaneX_v7_4functions"))
RES    = ROOT / "Results"
EXPL   = RES / "Explainability_SHAP_BestPlatforms"
DETAIL = RES / "BEST_in_DesignSpace_Post_per_platform_details.csv"

FIGDIR = ROOT / "figs" / "Explainability"    # copy to Overleaf/figs/Explainability
TABDIR = ROOT / "tables"                     # copy to Overleaf/tables
FIGDIR.mkdir(parents=True, exist_ok=True)
TABDIR.mkdir(parents=True, exist_ok=True)

if not DETAIL.exists():
    raise FileNotFoundError(f"Missing platform details CSV: {DETAIL}")
if not EXPL.exists():
    raise FileNotFoundError(f"Missing SHAP platform explainability dir: {EXPL}")

# -------------------- Params --------------------
SUBSPACES = ("compute","memory","sensors")
PCTS = list(range(10, 101, 10))

def _norm(s):
    s = re.sub(r"\s+","_",str(s)).lower().replace("%","pct")
    return re.sub(r"[^a-z0-9_]+","_", s).strip("_")

def _cpmi_path(setup, win, kf, sub):
    return ROOT / "FeatureRankOUT" / f"{setup}_{win}_{kf}_0_{sub}.csv"

def _read_ranklist(p: Path):
    if not p.exists():
        return []
    df = pd.read_csv(p)
    # first column is usually feature; handle feature_display too
    col0 = df.columns[0]
    feats = df[col0].astype(str).tolist()
    return [_norm(x) for x in feats]

def _find_shap_bestplat(setup, anomaly, win, kf, pct, method):
    # exact name from BestPlatforms script
    p = EXPL / f"SHAP_BESTPLAT_full_{setup}_{anomaly}_WIN{win}_KF{kf}_PCT{pct}_M{method}.csv"
    if p.exists():
        return p
    # fallback: any pct for same (setup, anomaly, win, kf, method)
    hits = sorted(glob.glob(str(EXPL / f"SHAP_BESTPLAT_full_{setup}_{anomaly}_WIN{win}_KF{kf}_PCT*_M{method}.csv")))
    return Path(hits[0]) if hits else None

def _read_shap_norm(p: Path):
    if (p is None) or (not p.exists()):
        return pd.DataFrame()
    df = pd.read_csv(p)
    cols = {c.lower(): c for c in df.columns}
    fcol = cols.get("feature", df.columns[0])
    subc = cols.get("subspace", None)
    # choose shap column robustly
    if "shap_mean_abs" in cols:
        scol = cols["shap_mean_abs"]
    else:
        shap_like = [c for c in df.columns if "shap" in c.lower()]
        scol = shap_like[0] if shap_like else df.columns[-1]

    out = pd.DataFrame({
        "feature_norm": df[fcol].astype(str).map(_norm),
        "shap": pd.to_numeric(df[scol], errors="coerce").fillna(0.0),
        "subspace": df[subc].astype(str).str.lower() if subc else "compute",
    })
    out["rank"] = out["shap"].rank(ascending=False, method="dense").astype(int)
    return out

def _jaccard(top1, top2):
    inter = len(top1 & top2)
    denom = max(1, len(top1 | top2))
    return inter / denom

def agreement_for_platform(setup: str, dplat: pd.DataFrame):
    """
    Returns a long DataFrame with columns:
      setup, win, kfold, anomaly, subspace, pct, jaccard
    """
    setup = str(setup).strip()
    if dplat.empty:
        return pd.DataFrame()

    # Platform WIN/KF should be consistent across anomalies
    win = int(pd.to_numeric(dplat["win"], errors="coerce").dropna().iloc[0])
    kf  = int(pd.to_numeric(dplat["kfold"], errors="coerce").dropna().iloc[0])

    rows = []

    for _, r in dplat.iterrows():
        anomaly = str(r["anomaly"]).strip()
        pct     = int(pd.to_numeric(r["best_pct_by_median"], errors="coerce"))
        method  = str(r["method"]).strip()

        shap_csv = _find_shap_bestplat(setup, anomaly, win, kf, pct, method)
        shap = _read_shap_norm(shap_csv)
        if shap.empty:
            continue

        for sub in SUBSPACES:
            cpmi = _read_ranklist(_cpmi_path(setup, win, kf, sub))
            if not cpmi:
                continue

            shap_sub = shap[shap["subspace"].astype(str).str.lower() == sub].copy()
            shap_ranked = shap_sub.sort_values("rank")["feature_norm"].tolist()
            if not shap_ranked:
                continue

            for pct2 in PCTS:
                k1 = max(1, int(np.ceil(len(cpmi) * (pct2/100))))
                k2 = max(1, int(np.ceil(len(shap_ranked) * (pct2/100))))
                top1 = set(cpmi[:k1])
                top2 = set(shap_ranked[:k2])
                rows.append({
                    "setup": setup,
                    "win": win,
                    "kfold": kf,
                    "anomaly": anomaly,
                    "subspace": sub,
                    "pct": int(pct2),
                    "jaccard": float(_jaccard(top1, top2)),
                })

    return pd.DataFrame(rows)

# -------------------- Load per-platform detail rows --------------------
det = pd.read_csv(DETAIL).copy()
det.columns = [c.lower() for c in det.columns]
if "best_method" in det.columns and "method" not in det.columns:
    det = det.rename(columns={"best_method":"method"})
if "best_pct_by_median" not in det.columns and "pct" in det.columns:
    det = det.rename(columns={"pct":"best_pct_by_median"})

need = {"setup","anomaly","win","kfold","best_pct_by_median","method"}
missing = need - set(det.columns)
if missing:
    raise KeyError(f"{DETAIL} missing columns: {sorted(missing)}. Have: {list(det.columns)}")

# -------------------- Run for each platform --------------------
agreeds = []
for setup in ["DDR4","DDR5"]:
    dplat = det[det["setup"].astype(str).str.upper() == setup].copy()
    if dplat.empty:
        print("[WARN] No detail rows for", setup)
        continue
    dfp = agreement_for_platform(setup, dplat)
    if not dfp.empty:
        agreeds.append(dfp)

agree = pd.concat(agreeds, ignore_index=True) if agreeds else pd.DataFrame()

out_csv = EXPL / "SHAP_vs_CPMI_agreement_PLATFORM.csv"
agree.to_csv(out_csv, index=False)
print("Wrote", out_csv)

# -------------------- Plot: PLATFORM curves (averaged across anomalies + subspaces) --------------------
if not agree.empty:
    plt.figure(figsize=(6.6,4.3), dpi=160)

    # platform avg:
    #   for each setup,pct: mean over anomalies and subspaces
    for setup, g in agree.groupby("setup"):
        gg = (g.groupby("pct", as_index=False)["jaccard"].mean())
        win = int(g["win"].iloc[0])
        kf  = int(g["kfold"].iloc[0])
        label = f"{setup} (W{win},K{kf})"
        plt.plot(gg["pct"], gg["jaccard"], lw=2.4, marker="o", label=label)

    plt.xlabel("Top-% threshold")
    plt.ylabel("SHAP–CPMI agreement (Jaccard)")
    plt.ylim(0, 1)
    plt.grid(True, alpha=0.25)
    plt.legend(loc="lower right", fontsize=9)
    plt.tight_layout()
    fig_out = FIGDIR / "SHAP_vs_CPMI_agreement_PLATFORM.png"
    plt.savefig(fig_out, dpi=300)
    plt.close()
    print("Wrote", fig_out)

    # Optional summary table for Overleaf
    summary = (agree.groupby(["setup","pct"], as_index=False)["jaccard"].mean())
    summ_out = TABDIR / "SHAP_vs_CPMI_agreement_PLATFORM_summary.csv"
    summary.to_csv(summ_out, index=False)
    print("Wrote", summ_out)
else:
    print("[WARN] No agreement data produced (missing SHAP or CP-MI ranks).")


Wrote /Users/hsiaopingni/octaneX_v7_4functions/Results/Explainability_SHAP_BestPlatforms/SHAP_vs_CPMI_agreement_PLATFORM.csv
Wrote /Users/hsiaopingni/octaneX_v7_4functions/figs/Explainability/SHAP_vs_CPMI_agreement_PLATFORM.png
Wrote /Users/hsiaopingni/octaneX_v7_4functions/tables/SHAP_vs_CPMI_agreement_PLATFORM_summary.csv


In [27]:
# === PER-PLATFORM horizontal bar chart of top-k SHAP features (ONE per platform) ===
# This aggregates SHAP across the platform's anomalies (equal-weight mean; missing=0)
# using the chosen best configs in:
#   Results/BEST_in_DesignSpace_Post_per_platform_details.csv
#
# Inputs:
#   - Results/BEST_in_DesignSpace_Post_per_platform_details.csv
#   - Results/Explainability_SHAP_BestPlatforms/SHAP_BESTPLAT_full_<setup>_<anomaly>_WIN<w>_KF<k>_PCT<p>_M<method>.csv
#
# Outputs:
#   - <ROOT>/figs/Explainability/SHAP_topK_PLATFORM_<setup>.png
#   - (optional) Results/Explainability_SHAP_BestPlatforms/SHAP_PLATFORM_topK_<setup>.csv
# -----------------------------------------------------------------------------------

from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import glob, re

# -------------------- Paths --------------------
ROOT   = Path(globals().get("ROOT", "/Users/hsiaopingni/octaneX_v7_4functions"))
RES    = ROOT / "Results"
EXPL   = RES / "Explainability_SHAP_BestPlatforms"
DETAIL = RES / "BEST_in_DesignSpace_Post_per_platform_details.csv"

FIGDIR = ROOT / "figs" / "Explainability"
FIGDIR.mkdir(parents=True, exist_ok=True)

if not DETAIL.exists():
    raise FileNotFoundError(f"Missing platform details CSV: {DETAIL}")
if not EXPL.exists():
    raise FileNotFoundError(f"Missing SHAP platform explainability dir: {EXPL}")

TOPK = 15
SUBSPACES = ("compute","memory","sensors")

# Let matplotlib handle default colors; we only segment bars by subspace
COLOR_MAP = {"compute": None, "memory": None, "sensors": None}

def _norm(s):
    s = re.sub(r"\s+","_",str(s)).lower().replace("%","pct")
    return re.sub(r"[^a-z0-9_]+","_", s).strip("_")

def _find_shap_bestplat(setup, anomaly, win, kf, pct, method):
    p = EXPL / f"SHAP_BESTPLAT_full_{setup}_{anomaly}_WIN{win}_KF{kf}_PCT{pct}_M{method}.csv"
    if p.exists():
        return p
    hits = sorted(glob.glob(str(EXPL / f"SHAP_BESTPLAT_full_{setup}_{anomaly}_WIN{win}_KF{kf}_PCT*_M{method}.csv")))
    return Path(hits[0]) if hits else None

def _read_shap_norm(p: Path):
    if (p is None) or (not p.exists()):
        return pd.DataFrame()
    df = pd.read_csv(p)
    cols = {c.lower(): c for c in df.columns}
    fcol = cols.get("feature", df.columns[0])
    subc = cols.get("subspace", None)
    if "shap_mean_abs" in cols:
        scol = cols["shap_mean_abs"]
    else:
        shap_like = [c for c in df.columns if "shap" in c.lower()]
        scol = shap_like[0] if shap_like else df.columns[-1]

    out = pd.DataFrame({
        "feature": df[fcol].astype(str),
        "feature_norm": df[fcol].astype(str).map(_norm),
        "shap": pd.to_numeric(df[scol], errors="coerce").fillna(0.0),
        "subspace": df[subc].astype(str).str.lower() if subc else "compute",
    })
    # If duplicates, keep max SHAP per (feature_norm, subspace)
    out = (out.groupby(["feature_norm","subspace"], as_index=False)["shap"].max()
             .merge(out.drop_duplicates(["feature_norm","subspace"])[["feature_norm","subspace","feature"]],
                    on=["feature_norm","subspace"], how="left"))
    return out

def _aggregate_platform_shap(details_rows: pd.DataFrame, setup: str) -> pd.DataFrame:
    """
    Returns: DataFrame(feature_norm, feature_display, subspace, shap_platform)
    where shap_platform is mean across anomalies (missing=0).
    """
    parts = []
    # platform win/kfold should be consistent across anomalies
    win = int(pd.to_numeric(details_rows["win"], errors="coerce").dropna().iloc[0])
    kf  = int(pd.to_numeric(details_rows["kfold"], errors="coerce").dropna().iloc[0])

    for _, r in details_rows.iterrows():
        anomaly = str(r["anomaly"]).strip()
        pct     = int(pd.to_numeric(r["best_pct_by_median"], errors="coerce"))
        method  = str(r["method"]).strip()

        p = _find_shap_bestplat(setup, anomaly, win, kf, pct, method)
        d = _read_shap_norm(p)
        if d.empty:
            continue
        d = d.rename(columns={"shap": f"shap_{anomaly}"})
        parts.append(d)

    if not parts:
        return pd.DataFrame(columns=["feature_norm","feature_display","subspace","shap_platform"])

    merged = None
    for d in parts:
        if merged is None:
            merged = d.copy()
        else:
            merged = merged.merge(d, on=["feature_norm","subspace"], how="outer", suffixes=("", "_r"))

            # Keep a display feature column if present
            if "feature_r" in merged.columns:
                merged["feature"] = merged["feature"].fillna(merged["feature_r"])
                merged = merged.drop(columns=["feature_r"])

    shap_cols = [c for c in merged.columns if c.startswith("shap_")]
    merged[shap_cols] = merged[shap_cols].fillna(0.0)

    merged["shap_platform"] = merged[shap_cols].mean(axis=1)  # equal-weight across anomalies
    merged = merged.rename(columns={"feature": "feature_display"})
    return merged[["feature_norm","feature_display","subspace","shap_platform"]].copy()

def plot_platform_shap_bar(setup: str, details_rows: pd.DataFrame):
    plat = _aggregate_platform_shap(details_rows, setup)
    if plat.empty:
        print("[SKIP] No platform SHAP aggregated for", setup)
        return

    # Save topk CSV for inspection
    top_csv = EXPL / f"SHAP_PLATFORM_topK_{setup}.csv"
    plat.sort_values("shap_platform", ascending=False).head(200).to_csv(top_csv, index=False)
    print("Wrote", top_csv)

    g = plat.sort_values("shap_platform", ascending=False).head(TOPK).copy()
    g["subspace"] = g["subspace"].astype(str).str.lower()
    y = np.arange(len(g))[::-1]

    plt.figure(figsize=(7.2, 5.2), dpi=160)

    # Draw bars grouped by subspace (default colors)
    for sub, gi in g.groupby("subspace", sort=False):
        idx = g.index.isin(gi.index)
        plt.barh(y[idx], g.loc[idx, "shap_platform"], label=sub.capitalize())

    plt.yticks(y, g["feature_norm"].iloc[::-1])
    plt.xlabel("SHAP (mean |value|) • platform-avg")
    plt.ylabel("Feature (normalized)")
    plt.legend(loc="lower right")
    plt.tight_layout()

    out = FIGDIR / f"SHAP_top{TOPK}_PLATFORM_{setup}.png"
    plt.savefig(out, dpi=300)
    plt.close()
    print("Wrote", out)

# -------------------- Load details and run for DDR4/DDR5 --------------------
det = pd.read_csv(DETAIL).copy()
det.columns = [c.lower() for c in det.columns]
if "best_method" in det.columns and "method" not in det.columns:
    det = det.rename(columns={"best_method":"method"})
if "best_pct_by_median" not in det.columns and "pct" in det.columns:
    det = det.rename(columns={"pct":"best_pct_by_median"})

need = {"setup","anomaly","win","kfold","best_pct_by_median","method"}
missing = need - set(det.columns)
if missing:
    raise KeyError(f"{DETAIL} missing columns: {sorted(missing)}. Have: {list(det.columns)}")

for setup in ["DDR4", "DDR5"]:
    dplat = det[det["setup"].astype(str).str.upper() == setup].copy()
    if dplat.empty:
        print("[WARN] No detail rows for", setup)
        continue
    plot_platform_shap_bar(setup, dplat)

Wrote /Users/hsiaopingni/octaneX_v7_4functions/Results/Explainability_SHAP_BestPlatforms/SHAP_PLATFORM_topK_DDR4.csv
Wrote /Users/hsiaopingni/octaneX_v7_4functions/figs/Explainability/SHAP_top15_PLATFORM_DDR4.png
Wrote /Users/hsiaopingni/octaneX_v7_4functions/Results/Explainability_SHAP_BestPlatforms/SHAP_PLATFORM_topK_DDR5.csv
Wrote /Users/hsiaopingni/octaneX_v7_4functions/figs/Explainability/SHAP_top15_PLATFORM_DDR5.png


In [29]:
# === PER-PLATFORM LaTeX table export: "best PCT" DSE summary (AUC-PR / ROC-AUC) ===
# This replaces your old "cases = [...]" test-case list with PER-PLATFORM selection.
#
# Uses NEW pipeline source-of-truth:
#   Results/per_run_metrics_all_PIPELINE.csv
# And per-platform chosen (WIN,KF):
#   Results/BEST_in_DesignSpace_Post_per_platform.csv
#
# For each platform (DDR4, DDR5) and each anomaly on that platform:
#   - Restrict to the platform's chosen (WIN,KF)
#   - Choose BEST PCT by:
#       AUCPR_median desc, then ROC_median desc, then smaller pct
#   - Report:
#       Best_AUC_PR = median ± IQR
#       Best_ROC_AUC = median ± IQR
#
# Also adds a TOTAL row (mean ± std of the medians across platform-anomaly rows)
#
# Outputs:
#   tables/explainability_hybrid_octane_summary_PLATFORM.tex
# ----------------------------------------------------------------------------------

from pathlib import Path
import pandas as pd
import numpy as np

ROOT   = Path(globals().get("ROOT", "/Users/hsiaopingni/octaneX_v7_4functions"))
RES    = Path(globals().get("RES_DIR", ROOT / "Results"))
if RES.name in ("Explainability_SHAP_BestCases", "Explainability_SHAP_BestPlatforms"):
    RES = RES.parent

TABDIR = Path(globals().get("TABDIR", ROOT / "tables"))
TABDIR.mkdir(parents=True, exist_ok=True)

PER_RUN   = RES / "per_run_metrics_all_PIPELINE.csv"
PLAT_BEST = RES / "BEST_in_DesignSpace_Post_per_platform.csv"

assert PER_RUN.exists(), f"Missing per-run CSV: {PER_RUN}"
assert PLAT_BEST.exists(), f"Missing per-platform winners CSV: {PLAT_BEST}"

pr = pd.read_csv(PER_RUN).copy()
pb = pd.read_csv(PLAT_BEST).copy()

# normalize cols
pr.columns = [c.lower() for c in pr.columns]
pb.columns = [c.lower() for c in pb.columns]

need_pr = {"setup","anomaly","win","kfold","pct","method","run_id","auc_pr","roc_auc"}
miss = need_pr - set(pr.columns)
if miss:
    raise KeyError(f"{PER_RUN} missing columns: {sorted(miss)}. Have: {list(pr.columns)}")

need_pb = {"setup","win","kfold"}
miss2 = need_pb - set(pb.columns)
if miss2:
    raise KeyError(f"{PLAT_BEST} missing columns: {sorted(miss2)}. Have: {list(pb.columns)}")

# numeric
for c in ["win","kfold","pct","auc_pr","roc_auc"]:
    pr[c] = pd.to_numeric(pr[c], errors="coerce")

pr = pr.dropna(subset=["setup","anomaly","win","kfold","pct","method"]).copy()
pr["auc_pr"]  = pr["auc_pr"].clip(0.0, 1.0)
pr["roc_auc"] = pr["roc_auc"].clip(0.0, 1.0)

# summarize per pct (median/IQR across run_id)
def q1(x): return float(np.nanpercentile(x, 25))
def q3(x): return float(np.nanpercentile(x, 75))

keys = ["setup","anomaly","win","kfold","method","pct"]
dse = (pr.groupby(keys, as_index=False)
         .agg(
             auc_pr_median=("auc_pr","median"),
             auc_pr_q1=("auc_pr", q1),
             auc_pr_q3=("auc_pr", q3),
             roc_auc_median=("roc_auc","median"),
             roc_auc_q1=("roc_auc", q1),
             roc_auc_q3=("roc_auc", q3),
             n_runs=("run_id","nunique"),
         ))
dse["auc_pr_iqr"]  = dse["auc_pr_q3"]  - dse["auc_pr_q1"]
dse["roc_auc_iqr"] = dse["roc_auc_q3"] - dse["roc_auc_q1"]

dse["auc_pr_median_filled"]  = dse["auc_pr_median"].fillna(-np.inf)
dse["roc_auc_median_filled"] = dse["roc_auc_median"].fillna(-np.inf)

METHOD_ORDER = ["dC_aJ","dC_aM","dE_aJ","dE_aM"]

# build platform config map
plat_cfg = {str(r["setup"]).strip(): (int(r["win"]), int(r["kfold"])) for _, r in pb.iterrows()}

ANOMALIES_BY_SETUP = {"DDR4": ["DROOP","RH"], "DDR5": ["DROOP","SPECTRE"]}

def summarize_best_pct_for_row(df_case: pd.DataFrame) -> dict | None:
    """
    df_case: rows for fixed (setup, anomaly, win, kfold), across methods and pct
    Strategy:
      - pick BEST method+pct: AUCPR_median desc, ROC desc, smaller pct
      - report median±IQR
    """
    if df_case.empty:
        return None
    df_case = df_case[df_case["method"].isin(METHOD_ORDER)].copy()
    if df_case.empty:
        return None
    df_case["mrank"] = df_case["method"].map({m:i for i,m in enumerate(METHOD_ORDER)}).fillna(999).astype(int)

    df_case = df_case.sort_values(
        ["auc_pr_median_filled","roc_auc_median_filled","pct","mrank"],
        ascending=[False, False, True, True]
    )
    best = df_case.iloc[0]
    return dict(
        Method=str(best["method"]),
        PCT=f"{int(best['pct'])} \\%",
        Best_AUC_PR=f"{best['auc_pr_median']:.2f} $\\pm$ {best['auc_pr_iqr']:.2f}",
        Best_ROC_AUC=f"{best['roc_auc_median']:.2f} $\\pm$ {best['roc_auc_iqr']:.2f}",
        auc_pr_median=float(best["auc_pr_median"]),
        roc_auc_median=float(best["roc_auc_median"]),
    )

rows = []
for setup in ["DDR4","DDR5"]:
    if setup not in plat_cfg:
        continue
    win, kf = plat_cfg[setup]
    for anomaly in ANOMALIES_BY_SETUP.get(setup, []):
        sub = dse[
            (dse["setup"].astype(str).str.upper() == setup.upper()) &
            (dse["anomaly"].astype(str).str.upper() == anomaly.upper()) &
            (pd.to_numeric(dse["win"], errors="coerce") == win) &
            (pd.to_numeric(dse["kfold"], errors="coerce") == kf)
        ].copy()

        s = summarize_best_pct_for_row(sub)
        if s is None:
            continue

        rows.append({
            "Platform": setup,
            "Anomaly": anomaly,
            "WIN": int(win),
            "KF": int(kf),
            "Method": s["Method"],
            "PCT": s["PCT"],
            "Best_AUC_PR": s["Best_AUC_PR"],
            "Best_ROC_AUC": s["Best_ROC_AUC"],
            "_auc_m": s["auc_pr_median"],
            "_roc_m": s["roc_auc_median"],
        })

df_out = pd.DataFrame(rows)

if df_out.empty:
    raise RuntimeError("No rows found for per-platform selection. Check per_run_metrics_all_PIPELINE.csv and platform WIN/KF.")

# TOTAL line (mean ± std across rows, using medians)
auc_m = df_out["_auc_m"].astype(float).tolist()
roc_m = df_out["_roc_m"].astype(float).tolist()

df_out.loc[len(df_out)] = {
    "Platform":"TOTAL","Anomaly":"—","WIN":"—","KF":"—","Method":"—","PCT":"—",
    "Best_AUC_PR":f"{np.mean(auc_m):.2f} $\\pm$ {np.std(auc_m):.2f}",
    "Best_ROC_AUC":f"{np.mean(roc_m):.2f} $\\pm$ {np.std(roc_m):.2f}",
    "_auc_m": np.nan,
    "_roc_m": np.nan,
}

# Export to LaTeX
latex = df_out[["Platform","Anomaly","WIN","KF","Method","PCT","Best_AUC_PR","Best_ROC_AUC"]].to_latex(
    index=False, escape=False
)

out_tex = TABDIR / "explainability_hybrid_octane_summary_PLATFORM.tex"
out_tex.write_text(latex)
print("Wrote", out_tex)

Wrote /Users/hsiaopingni/octaneX_v7_4functions/tables/explainability_hybrid_octane_summary_PLATFORM.tex


In [30]:
# === PER-PLATFORM rank mean/var LaTeX export (ROC-AUC) ===
# This is the PER-PLATFORM rewrite of your rank-summary logic.
#
# Instead of ranking "models" across many datasets/workloads, we rank METHODS (dC_aJ,dC_aM,dE_aJ,dE_aM)
# for each PLATFORM×ANOMALY dataset at the platform-chosen (WIN,KF), using the BEST pct per method.
#
# Inputs:
#   - Results/per_run_metrics_all_PIPELINE.csv
#   - Results/BEST_in_DesignSpace_Post_per_platform.csv
#
# Outputs:
#   - tables/rank_meanvar_rocauc_PLATFORM.tex
#
# Ranking rule per dataset_id (setup|anomaly|win|kfold):
#   1) For each METHOD, pick best pct by ROC-AUC median (desc), tie smaller pct.
#   2) Rank METHODS by roc_auc_median (desc) within that dataset_id (rank 1 = best).
#   3) Aggregate mean/var rank per anomaly across platforms, plus Overall.
# --------------------------------------------------------------------------------------------

from pathlib import Path
import pandas as pd
import numpy as np

ROOT   = Path(globals().get("ROOT", "/Users/hsiaopingni/octaneX_v7_4functions"))
RES    = Path(globals().get("RES_DIR", ROOT / "Results"))
if RES.name in ("Explainability_SHAP_BestCases", "Explainability_SHAP_BestPlatforms"):
    RES = RES.parent

TABDIR = Path(globals().get("TABDIR", ROOT / "tables"))
TABDIR.mkdir(parents=True, exist_ok=True)

PER_RUN   = RES / "per_run_metrics_all_PIPELINE.csv"
PLAT_BEST = RES / "BEST_in_DesignSpace_Post_per_platform.csv"

assert PER_RUN.exists(), f"Missing per-run CSV: {PER_RUN}"
assert PLAT_BEST.exists(), f"Missing per-platform winners CSV: {PLAT_BEST}"

df = pd.read_csv(PER_RUN).copy()
pb = pd.read_csv(PLAT_BEST).copy()

df.columns = [c.lower() for c in df.columns]
pb.columns = [c.lower() for c in pb.columns]

need = {"setup","anomaly","win","kfold","pct","method","run_id","roc_auc"}
miss = need - set(df.columns)
if miss:
    raise KeyError(f"{PER_RUN} missing columns: {sorted(miss)}. Have: {list(df.columns)}")

METHODS = ["dC_aJ","dC_aM","dE_aJ","dE_aM"]

# numeric & clean
for c in ["win","kfold","pct","roc_auc"]:
    df[c] = pd.to_numeric(df[c], errors="coerce")
df = df.dropna(subset=["setup","anomaly","win","kfold","pct","method","roc_auc"]).copy()
df = df[df["method"].isin(METHODS)].copy()
df["roc_auc"] = df["roc_auc"].clip(0.0, 1.0)

# platform (WIN,KF) map
plat_cfg = {str(r["setup"]).strip(): (int(r["win"]), int(r["kfold"])) for _, r in pb.iterrows()}

# restrict to platform-chosen (WIN,KF) only
keep_parts = []
for setup, (win, kf) in plat_cfg.items():
    sub = df[
        (df["setup"].astype(str).str.upper() == setup.upper()) &
        (pd.to_numeric(df["win"], errors="coerce") == win) &
        (pd.to_numeric(df["kfold"], errors="coerce") == kf)
    ].copy()
    if not sub.empty:
        keep_parts.append(sub)
dfp = pd.concat(keep_parts, ignore_index=True) if keep_parts else pd.DataFrame()
if dfp.empty:
    raise RuntimeError("No rows after filtering to per-platform (WIN,KF). Check PLAT_BEST and PER_RUN.")

# summarize ROC-AUC median per pct
def q1(x): return float(np.nanpercentile(x, 25))
def q3(x): return float(np.nanpercentile(x, 75))

keys = ["setup","anomaly","win","kfold","method","pct"]
dse = (dfp.groupby(keys, as_index=False)
         .agg(
             roc_auc_median=("roc_auc","median"),
             n_runs=("run_id","nunique"),
         ))
dse["roc_auc_median"] = pd.to_numeric(dse["roc_auc_median"], errors="coerce")
dse = dse.dropna(subset=["roc_auc_median"]).copy()

# dataset_id = platform-level dataset (no workload dimension)
dse["dataset_id"] = (
    dse["setup"].astype(str) + "|" +
    dse["anomaly"].astype(str) + "|" +
    dse["win"].astype(str) + "|" +
    dse["kfold"].astype(str)
)

# For each dataset_id × method: pick best pct by ROC median (desc), tie smaller pct
best = (dse.sort_values(["dataset_id","method","roc_auc_median","pct"],
                        ascending=[True, True, False, True])
          .groupby(["dataset_id","method"], as_index=False)
          .head(1))

# Rank METHODS within each dataset_id: higher ROC -> lower (better) rank number
def _rank_within(df_):
    df_ = df_.copy()
    df_["rank"] = df_["roc_auc_median"].rank(ascending=False, method="average")
    return df_

ranked = best.groupby("dataset_id", group_keys=False).apply(_rank_within)

# Aggregate mean/var rank by anomaly × method
agg = (ranked.groupby(["anomaly","method"], as_index=False)
             .agg(mean_rank=("rank","mean"),
                  var_rank=("rank","var"),
                  n=("dataset_id","nunique"))
             .sort_values(["anomaly","mean_rank","method"]))

agg["Mean (Var)"] = agg.apply(
    lambda r: f"{r['mean_rank']:.2f} ({0.0 if pd.isna(r['var_rank']) else r['var_rank']:.2f})",
    axis=1
)

# Overall across anomalies
overall = (ranked.groupby("method", as_index=False)
                 .agg(mean_rank=("rank","mean"),
                      var_rank=("rank","var"),
                      n=("dataset_id","nunique")))
overall["anomaly"] = "Overall"
overall["Mean (Var)"] = overall.apply(
    lambda r: f"{r['mean_rank']:.2f} ({0.0 if pd.isna(r['var_rank']) else r['var_rank']:.2f})",
    axis=1
)

rank_table = pd.concat([agg, overall], ignore_index=True)

# Display + LaTeX
disp = (rank_table[["anomaly","method","n","Mean (Var)"]]
        .rename(columns={
            "anomaly":"Anomaly",
            "method":"Method",
            "n":"Datasets",
            "Mean (Var)":"Mean (Var) Rank (ROC-AUC) ↓"
        }))

latex = disp.to_latex(index=False, escape=False)
out_tex = TABDIR / "rank_meanvar_rocauc_PLATFORM.tex"
out_tex.write_text(latex)
print("Wrote", out_tex)

Wrote /Users/hsiaopingni/octaneX_v7_4functions/tables/rank_meanvar_rocauc_PLATFORM.tex


/var/folders/2z/b1lzf17n4z90pz80khfc42fw0000gp/T/ipykernel_14285/3044125755.py:108: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ranked = best.groupby("dataset_id", group_keys=False).apply(_rank_within)


In [31]:
# === Explainability comparison (PER PLATFORM): HYBRID vs CP-MI (Jaccard + Spearman) ==========
# PER-PLATFORM revision:
#   - Only evaluates the platform-chosen (WIN,KF) for DDR4 and DDR5
#     (loaded from Results/BEST_in_DesignSpace_Post_per_platform.csv).
#   - Computes:
#       1) Jaccard@Top-% (10..100) per platform×subspace
#       2) Spearman rho per platform×subspace (full rankings over union; missing -> worst+1)
#   - Saves CSVs + a simple plot (optional).
#
# Inputs:
#   - FeatureRankOUT_HYBRID/<setup>_<win>_<kfold>_0_<sub>.csv
#   - FeatureRankOUT/<setup>_<win>_<kfold>_0_<sub>.csv
#   - Results/BEST_in_DesignSpace_Post_per_platform.csv
#
# Outputs:
#   - Results/Explainability_Comparisons_PLATFORM/hybrid_vs_cpmi_jaccard_topk_PLATFORM.csv
#   - Results/Explainability_Comparisons_PLATFORM/hybrid_vs_cpmi_spearman_PLATFORM.csv
#   - Results/Explainability_Comparisons_PLATFORM/hybrid_vs_cpmi_jaccard_PLATFORM.png
# ---------------------------------------------------------------------------------------------

from pathlib import Path
import pandas as pd, numpy as np, re
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt

# ---- Paths (EDIT root if needed) ----
ROOT = Path(globals().get("ROOT", "/Users/hsiaopingni/octaneX_v7_4functions")).expanduser().resolve()
RES  = ROOT / "Results"

R_HYB = ROOT / "FeatureRankOUT_HYBRID"   # <setup>_<win>_<kfold>_0_<sub>.csv
R_CPM = ROOT / "FeatureRankOUT"          # <setup>_<win>_<kfold>_0_<sub>.csv
PLAT  = RES / "BEST_in_DesignSpace_Post_per_platform.csv"

OUT   = RES / "Explainability_Comparisons_PLATFORM"
OUT.mkdir(parents=True, exist_ok=True)

SUBSPACES = ("compute","memory","sensors")
PCTS = list(range(10, 101, 10))  # 10%, 20%, ..., 100%

if not PLAT.exists():
    raise FileNotFoundError(f"Missing per-platform winners CSV: {PLAT}")
if not R_HYB.exists():
    raise FileNotFoundError(f"Missing HYBRID rank dir: {R_HYB}")
if not R_CPM.exists():
    raise FileNotFoundError(f"Missing CP-MI rank dir: {R_CPM}")

def _norm(s: str) -> str:
    s = re.sub(r"\s+","_",str(s)).lower().replace("%","pct")
    s = re.sub(r"[^a-z0-9_]+","_", s)
    return re.sub(r"_+","_", s).strip("_")

def _read_rank_csv(p: Path, prefer_col=None):
    """
    Returns list of normalized feature names (ranking order top->bottom).
    prefer_col is used for HYBRID files that may have a 'feature_display' column.
    """
    if (not p) or (not p.exists()):
        return []
    df = pd.read_csv(p)
    col = prefer_col if (prefer_col and prefer_col in df.columns) else df.columns[0]
    feats = df[col].astype(str).tolist()
    return [_norm(x) for x in feats]

def _path_hybrid(setup, win, kfold, sub):
    return R_HYB / f"{setup}_{win}_{kfold}_0_{sub}.csv"

def _path_cpmi(setup, win, kfold, sub):
    return R_CPM / f"{setup}_{win}_{kfold}_0_{sub}.csv"

# ---------------- Platform cases (ONLY) ----------------
pb = pd.read_csv(PLAT).copy()
pb.columns = [c.lower() for c in pb.columns]
need = {"setup","win","kfold"}
missing = need - set(pb.columns)
if missing:
    raise KeyError(f"{PLAT} missing required columns: {sorted(missing)}. Have: {list(pb.columns)}")

platform_cases = []
for _, r in pb.iterrows():
    setup = str(r["setup"]).strip()
    win   = int(pd.to_numeric(r["win"], errors="coerce"))
    kf    = int(pd.to_numeric(r["kfold"], errors="coerce"))
    platform_cases.append((setup, win, kf))

# Defensive: keep only DDR4/DDR5 if present
platform_cases = [c for c in platform_cases if str(c[0]).upper() in ("DDR4","DDR5")]
platform_cases = sorted(platform_cases, key=lambda x: str(x[0]))

print("[OK] Platform cases:", platform_cases)

# --------------- Metrics ---------------
def jaccard_topk(A_list, B_list, pct):
    if not A_list or not B_list:
        return np.nan
    kA = max(1, int(np.ceil(len(A_list) * (pct/100.0))))
    kB = max(1, int(np.ceil(len(B_list) * (pct/100.0))))
    A = set(A_list[:kA]); B = set(B_list[:kB])
    denom = len(A | B)
    return (len(A & B) / denom) if denom else np.nan

def spearman_between_lists(A_list, B_list):
    """
    Spearman rank corr between two full rankings defined over the union of features.
    Missing items get a default 'worst+1' rank.
    """
    if not A_list or not B_list:
        return np.nan
    union = list(dict.fromkeys(A_list + B_list))
    rA_index = {f:i for i,f in enumerate(A_list)}
    rB_index = {f:i for i,f in enumerate(B_list)}
    worstA = len(A_list) + 1
    worstB = len(B_list) + 1
    a = np.array([rA_index.get(f, worstA) for f in union], dtype=float)
    b = np.array([rB_index.get(f, worstB) for f in union], dtype=float)
    if np.all(a == a[0]) or np.all(b == b[0]):
        return np.nan
    # Pearson on positions (equivalent monotonic rank corr here)
    a_mean, b_mean = a.mean(), b.mean()
    num = np.sum((a-a_mean)*(b-b_mean))
    den = np.sqrt(np.sum((a-a_mean)**2) * np.sum((b-b_mean)**2))
    return float(num/den) if den > 0 else np.nan

def compute_overlap_and_corr_platform(cases):
    rows_j = []
    rows_s = []
    for setup, win, kf in cases:
        for sub in SUBSPACES:
            pH = _path_hybrid(setup, win, kf, sub)
            pC = _path_cpmi(setup, win, kf, sub)
            H = _read_rank_csv(pH, prefer_col="feature_display")
            C = _read_rank_csv(pC)
            if (not H) or (not C):
                continue

            # Jaccard across PCTs
            for pct in PCTS:
                rows_j.append({
                    "setup": setup, "win": win, "kfold": kf, "subspace": sub, "pct": pct,
                    "jaccard": jaccard_topk(H, C, pct)
                })

            # Spearman (single value per platform×subspace)
            rows_s.append({
                "setup": setup, "win": win, "kfold": kf, "subspace": sub,
                "spearman": spearman_between_lists(H, C)
            })
    return pd.DataFrame(rows_j), pd.DataFrame(rows_s)

JACCARD, SPEARMAN = compute_overlap_and_corr_platform(platform_cases)

j_csv = OUT / "hybrid_vs_cpmi_jaccard_topk_PLATFORM.csv"
s_csv = OUT / "hybrid_vs_cpmi_spearman_PLATFORM.csv"
JACCARD.to_csv(j_csv, index=False)
SPEARMAN.to_csv(s_csv, index=False)
print(f"[WROTE] {j_csv}\n[WROTE] {s_csv}")

# Optional: plot mean Jaccard across subspaces per platform
if not JACCARD.empty:
    plt.figure(figsize=(6.6,4.3), dpi=160)
    for (setup, win, kf), g in JACCARD.groupby(["setup","win","kfold"]):
        gg = g.groupby("pct", as_index=False)["jaccard"].mean()
        label = f"{setup} (W{win},K{kf})"
        plt.plot(gg["pct"], gg["jaccard"], lw=2.4, marker="o", label=label)
    plt.xlabel("Top-% threshold")
    plt.ylabel("Jaccard(HYBRID, CP-MI) • avg over subspaces")
    plt.ylim(0, 1)
    plt.grid(True, alpha=0.25)
    plt.legend(loc="lower right", fontsize=9)
    plt.tight_layout()
    out_png = OUT / "hybrid_vs_cpmi_jaccard_PLATFORM.png"
    plt.savefig(out_png, dpi=300)
    plt.close()
    print(f"[WROTE] {out_png}")

# Quick sanity view (optional)
try:
    from IPython.display import display
    display(JACCARD.head())
    display(SPEARMAN.head())
except Exception:
    pass


[OK] Platform cases: [('DDR4', 512, 3), ('DDR5', 1024, 5)]
[WROTE] /Users/hsiaopingni/octaneX_v7_4functions/Results/Explainability_Comparisons_PLATFORM/hybrid_vs_cpmi_jaccard_topk_PLATFORM.csv
[WROTE] /Users/hsiaopingni/octaneX_v7_4functions/Results/Explainability_Comparisons_PLATFORM/hybrid_vs_cpmi_spearman_PLATFORM.csv
[WROTE] /Users/hsiaopingni/octaneX_v7_4functions/Results/Explainability_Comparisons_PLATFORM/hybrid_vs_cpmi_jaccard_PLATFORM.png


,setup,win,kfold,subspace,pct,jaccard
0,DDR4,512,3,compute,10,1.0
1,DDR4,512,3,compute,20,1.0
2,DDR4,512,3,compute,30,1.0
3,DDR4,512,3,compute,40,1.0
4,DDR4,512,3,compute,50,1.0


,setup,win,kfold,subspace,spearman
0,DDR4,512,3,compute,0.993706
1,DDR4,512,3,memory,0.999343
2,DDR4,512,3,sensors,0.983459
3,DDR5,1024,5,compute,0.993905
4,DDR5,1024,5,memory,0.994734


In [32]:
# === PER-PLATFORM Plot: Jaccard@Top-% curves (avg over subspaces) ===
# Uses JACCARD DataFrame produced by the per-platform HYBRID-vs-CPMI script:
#   columns: setup, win, kfold, subspace, pct, jaccard
#
# Outputs one PNG per platform:
#   OUT/jaccard_curve_PLATFORM_<setup>_WIN<w>_KF<k>.png

import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt

def plot_jaccard_curves_platform(J, out_dir, setups=("DDR4","DDR5")):
    if J is None or J.empty:
        print("[SKIP] Jaccard plot (no data).")
        return

    # Defensive typing
    JJ = J.copy()
    JJ["pct"] = pd.to_numeric(JJ["pct"], errors="coerce")
    JJ["jaccard"] = pd.to_numeric(JJ["jaccard"], errors="coerce")

    for setup in setups:
        g0 = JJ[JJ["setup"].astype(str).str.upper() == setup.upper()].copy()
        if g0.empty:
            continue

        # If multiple (win,kfold) exist, make one plot per (win,kfold) anyway,
        # but naming still says PLATFORM.
        for (win, kf), g in g0.groupby(["win","kfold"]):
            gg = g.groupby("pct", as_index=False)["jaccard"].mean()
            gg = gg.sort_values("pct")

            plt.figure(figsize=(6.0, 4.0), dpi=160)
            plt.plot(gg["pct"], gg["jaccard"], lw=2.4, marker="o")
            plt.ylim(0, 1)
            plt.xlabel("Top-% threshold")
            plt.ylabel("Jaccard overlap (Hybrid vs CP-MI)")
            plt.title(f"{setup} (WIN={int(win)}, KF={int(kf)}) — Avg over subspaces")
            plt.grid(True, alpha=0.25)
            plt.tight_layout()

            out = Path(out_dir) / f"jaccard_curve_PLATFORM_{setup}_WIN{int(win)}_KF{int(kf)}.png"
            plt.savefig(out, dpi=300)
            plt.close()
            print("Wrote", out)

# Run (expects OUT and JACCARD already exist in your notebook)
plot_jaccard_curves_platform(JACCARD, OUT)


Wrote /Users/hsiaopingni/octaneX_v7_4functions/Results/Explainability_Comparisons_PLATFORM/jaccard_curve_PLATFORM_DDR4_WIN512_KF3.png
Wrote /Users/hsiaopingni/octaneX_v7_4functions/Results/Explainability_Comparisons_PLATFORM/jaccard_curve_PLATFORM_DDR5_WIN1024_KF5.png


In [34]:
# === PER-PLATFORM: (1) HYBRID vs CP-MI explainability (Jaccard + Spearman) + (2) Jaccard plots
#                  + (3) Build per_workload_system_auc.csv (52 rows) ============
#
# This is a SINGLE, combined, per-platform script that:
#   A) Uses platform-chosen (WIN,KF) from Results/BEST_in_DesignSpace_Post_per_platform.csv
#   B) Computes HYBRID vs CP-MI similarity (Jaccard@Top-% and Spearman) at those (WIN,KF)
#   C) Saves CSVs + one Jaccard curve PNG per platform
#   D) Builds Results/paper_exports/per_workload_system_auc.csv (52 rows)
#      using ONE (WIN,KF,top%) per platform selected by probe workloads mean AUCROC.
#
# IMPORTANT differences vs your old mixed scripts:
#   - All “platform-level” configs come from:
#       Results/BEST_in_DesignSpace_Post_per_platform.csv  (for (WIN,KF))
#   - SHAP for HYBRID feature blending is loaded from PER-PLATFORM SHAP outputs:
#       Results/Explainability_SHAP_BestPlatforms/SHAP_BESTPLAT_full_<setup>_<scenario>_WIN<w>_KF<k>_PCT<p>_M<method>.csv
#     via Results/BEST_in_DesignSpace_Post_per_platform_details.csv (best pct+method per scenario).
#
# Outputs:
#   1) Results/Explainability_Comparisons_PLATFORM/
#        hybrid_vs_cpmi_jaccard_topk_PLATFORM.csv
#        hybrid_vs_cpmi_spearman_PLATFORM.csv
#        jaccard_curve_PLATFORM_<setup>_WIN<w>_KF<k>.png
#   2) Results/paper_exports/
#        per_workload_system_auc.csv
#        build_per_workload_system_auc.log
# --------------------------------------------------------------------------------------------

import os, re, glob, math, gc, warnings, logging
from pathlib import Path
from typing import Optional, Dict, Tuple, List

import numpy as np
import pandas as pd
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
from xgboost import XGBClassifier
import xgboost as xgb

warnings.filterwarnings("ignore")
os.environ.setdefault("PYTHONWARNINGS", "ignore")

# ---------------- You can edit these paths if needed -------------------
ROOT = Path(globals().get("ROOT", "/Users/hsiaopingni/octaneX_v7_4functions")).expanduser().resolve()
RES_DIR_RAW = Path(globals().get("RES_DIR", ROOT / "Results"))
DATA_DIR = Path(globals().get("DATA_DIR", "/Users/hsiaopingni/Desktop/SLM_RAS-main/HW_TELEMETRY_DATA_COLLECTION/TELEMETRY_DATA"))

# Normalize RES_DIR to ".../Results"
RES_DIR = RES_DIR_RAW.parent if RES_DIR_RAW.name in ("Explainability_SHAP_BestCases", "Explainability_SHAP_BestPlatforms") else RES_DIR_RAW

# Rank dirs for CP-MI files (searched in order)
RANK_DIRS = list(globals().get("RANK_DIRS", [
    ROOT / "FeatureRankOUT",
    Path("/Volumes/Untitled") / "FeatureRankOUT",
    Path("/Volumes/Untitled") / "octaneX" / "FeatureRankOUT",
    Path.home() / "Desktop" / "octaneX" / "FeatureRankOUT",
]))

# Per-platform configs
PLAT_BEST = RES_DIR / "BEST_in_DesignSpace_Post_per_platform.csv"
PLAT_DETAILS = RES_DIR / "BEST_in_DesignSpace_Post_per_platform_details.csv"

# SHAP dir (per platform)
SHAP_DIR = RES_DIR / "Explainability_SHAP_BestPlatforms"

# Output dirs
OUT_COMP = RES_DIR / "Explainability_Comparisons_PLATFORM"
OUT_COMP.mkdir(parents=True, exist_ok=True)

PAPER_OUT = RES_DIR / "paper_exports"
PAPER_OUT.mkdir(parents=True, exist_ok=True)

def _ensure_writable_dir(p: Path, hint_name: str):
    p.mkdir(parents=True, exist_ok=True)
    testfile = p / ".write_test.tmp"
    try:
        with open(testfile, "w") as f:
            f.write("ok")
        try:
            testfile.unlink()
        except Exception:
            pass
    except PermissionError as e:
        raise RuntimeError(
            f"Cannot write to {p} for {hint_name}.\n"
            "On macOS, give your terminal/Jupyter app Full Disk Access:\n"
            "  System Settings → Privacy & Security → Full Disk Access → enable Terminal/VS Code/Jupyter\n"
            "Then restart it and rerun this cell."
        ) from e

_ensure_writable_dir(OUT_COMP, "explainability comparisons")
_ensure_writable_dir(PAPER_OUT, "paper exports")

# Log file for the AUC CSV build
LOG_FILE = PAPER_OUT / "build_per_workload_system_auc.log"
logging.basicConfig(
    filename=str(LOG_FILE),
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s",
)
log = logging.getLogger("papercsv")
log.info("==== START combined PER-PLATFORM run ====")
log.info(f"ROOT={ROOT} RES_DIR={RES_DIR} DATA_DIR={DATA_DIR}")
log.info(f"RANK_DIRS={RANK_DIRS}")

# ---------------- Paper configuration ----------------------------------------
SUBSPACES = ("compute","memory","sensors")
PLATFORMS = ("DDR4","DDR5")
SCENARIOS = {"DDR4": ("DROOP","RH"), "DDR5": ("DROOP","SPECTRE")}
WORKLOADS = ["dft","dj","dp","gl","gs","ha","ja","mm","ni","oe","pi","sh","tr"]

# Hybrid blend for selection inside per_workload_system_auc.csv
ALPHA    = float(globals().get("ALPHA", 0.40))
SEED     = int(globals().get("SEED", 42))
TOP_PCTS = [10, 25, 50, 75, 100]  # candidates to choose per platform

# Selection probe set (keeps selection fast; then final pass writes all 52 rows)
PROBE_WORKLOADS = list(globals().get("PROBE_WORKLOADS", ["dft","gs","mm"]))

# XGBoost runtime knobs
XGB_TREES   = int(globals().get("XGB_TREES", 150))
XGB_ESR     = int(globals().get("XGB_ESR", 20))
XGB_THREADS = max(1, min(8, (os.cpu_count() or 4)))
os.environ.setdefault("OMP_NUM_THREADS", str(XGB_THREADS))

# ---------------- Self-contained helpers -------------------------------------
def _norm(s: str) -> str:
    s = re.sub(r"\s+","_",str(s)).lower().replace("%","pct")
    s = re.sub(r"[^a-z0-9_]+","_", s)
    return re.sub(r"_+","_", s).strip("_")

def telemetry_cols(df: pd.DataFrame):
    drop_like = {"label","setup","anomaly","run_id","timestamp","time","idx","index"}
    num_cols = df.select_dtypes(include=[np.number]).columns
    return [c for c in num_cols if c.lower() not in drop_like]

def robust_scale_train(X: np.ndarray, winsor=(2.0, 98.0)):
    X = np.asarray(X, dtype=float)
    q1 = np.nanpercentile(X, winsor[0], axis=0)
    q2 = np.nanpercentile(X, winsor[1], axis=0)
    Xw = np.clip(X, q1, q2)
    mu = np.nanmedian(Xw, axis=0)
    mad = np.nanmedian(np.abs(Xw - mu), axis=0)
    sd  = mad * 1.4826
    iqr = np.nanpercentile(Xw, 75, axis=0) - np.nanpercentile(Xw, 25, axis=0)
    sd  = np.where(sd < 1e-9, iqr/1.349, sd)
    sd  = np.where(sd < 1e-9, 1.0, sd)
    return mu, sd, q1, q2

def apply_robust_scale(df: pd.DataFrame, mu, sd, q1, q2) -> pd.DataFrame:
    X = df.values.astype(float)
    X = np.clip(X, q1, q2)
    Z = (X - mu) / sd
    return pd.DataFrame(Z, columns=df.columns, index=df.index)

def _read_csv_safe(p: Path) -> pd.DataFrame:
    try:
        return pd.read_csv(p)
    except Exception as e:
        log.warning(f"read_csv failed for {p}: {e}")
        return pd.DataFrame()

def build_windowed_raw_means(pairs, win: int, label: str, overlap_ratio=None) -> pd.DataFrame:
    rows = []
    step = win if overlap_ratio is None else max(1, int(win * (1 - float(overlap_ratio))))
    for _, df in pairs:
        feats = telemetry_cols(df)
        if not feats:
            continue
        D = df[feats].astype(float)

        # If already aggregated: pass-through
        if len(D) <= max(2*win, 32) and D.shape[1] >= 16:
            out_chunk = D.copy()
            out_chunk["label"] = label
            rows.append(out_chunk)
            continue

        n = len(D)
        if n < win:
            continue
        for start in range(0, n - win + 1, step):
            sl = D.iloc[start:start+win]
            rows.append(pd.Series({**sl.mean(axis=0, skipna=True).to_dict(), "label": label}))

    if not rows:
        return pd.DataFrame()
    out = pd.DataFrame(rows).reset_index(drop=True)
    feat_cols = telemetry_cols(out)
    return out[feat_cols + ["label"]]

def _percentile_rank(series: pd.Series, higher_is_better=True) -> pd.Series:
    x = series.copy()
    if not higher_is_better:
        x = -x
    n = len(x)
    if n <= 1:
        return pd.Series(np.zeros(n), index=series.index, dtype=float)
    r = x.rank(method="average", ascending=False)  # largest→1
    return (n - r) / (n - 1)

def _top_k_from_pct(n: int, pct: int) -> int:
    k = int(math.ceil(n * pct / 100.0))
    return max(1, k) if pct > 0 else 0

# ---------------- CP-MI loader (cached) --------------------------------------
def _cpmi_rank_path(setup: str, win: int, kfold: int, sub: str) -> Optional[Path]:
    fname = f"{setup}_{win}_{kfold}_0_{sub}.csv"
    for d in RANK_DIRS:
        p = Path(d) / fname
        if p.exists():
            return p
    for d in RANK_DIRS:
        d = Path(d)
        if not d.exists():
            continue
        hits = list(d.rglob(f"*{setup}*{sub}*CPMI*.csv")) + list(d.rglob(f"*{setup}*{sub}*CP-MI*.csv"))
        if hits:
            return hits[0]
    return None

_CPMI_CACHE: Dict[Tuple[str,int,int,str], pd.DataFrame] = {}

def _load_cpmi_one(setup: str, win: int, kfold: int, sub: str) -> pd.DataFrame:
    key = (setup, int(win), int(kfold), sub)
    if key in _CPMI_CACHE:
        return _CPMI_CACHE[key]

    rp = _cpmi_rank_path(setup, win, kfold, sub)
    if not rp:
        _CPMI_CACHE[key] = pd.DataFrame()
        return _CPMI_CACHE[key]

    df = _read_csv_safe(rp)
    if df.empty:
        _CPMI_CACHE[key] = df
        return df

    feat_col = next((c for c in df.columns if c.lower() in ("feature","features","name","signal","column")), df.columns[0])
    score_col= next((c for c in df.columns if c.lower() in ("cpmi","cp-mi","score","mi","mi_score","rank_score")),
                    (df.select_dtypes(include=[np.number]).columns.tolist() or [df.columns[-1]])[0])

    d = (
        df[[feat_col, score_col]].rename(columns={feat_col:"feature", score_col:"cpmi_score"})
          .assign(feature=lambda x: x["feature"].astype(str).str.strip())
          .dropna(subset=["feature"]).drop_duplicates(subset=["feature"])
    )
    d["cpmi_score"] = pd.to_numeric(d["cpmi_score"], errors="coerce").fillna(0.0)
    d = d.sort_values("cpmi_score", ascending=False).reset_index(drop=True)
    d["cpmi_rank"]  = np.arange(1, len(d)+1)
    d["cpmi_prank"] = _percentile_rank(d["cpmi_score"], True)
    d["subspace"]   = sub

    _CPMI_CACHE[key] = d
    return d

# ---------------- SHAP loader (per platform, exact via details) --------------
_SHAP_CACHE: Dict[Tuple[str,str,int,int,int,str], pd.DataFrame] = {}

def _find_shap_best_exact(setup: str, scenario: str, win: int, kfold: int, pct: int, method: str) -> Optional[Path]:
    p = SHAP_DIR / f"SHAP_BESTPLAT_full_{setup}_{scenario}_WIN{win}_KF{kfold}_PCT{pct}_M{method}.csv"
    if p.exists():
        return p
    patt = str(SHAP_DIR / f"SHAP_BESTPLAT_full_{setup}_{scenario}_WIN{win}_KF{kfold}_PCT*_M{method}.csv")
    hits = sorted(glob.glob(patt))
    return Path(hits[0]) if hits else None

def _load_shap_best_full_exact(setup: str, scenario: str, win: int, kfold: int, pct: int, method: str) -> pd.DataFrame:
    key = (setup, scenario, int(win), int(kfold), int(pct), str(method))
    if key in _SHAP_CACHE:
        return _SHAP_CACHE[key]

    p = _find_shap_best_exact(setup, scenario, win, kfold, pct, method)
    if not p:
        _SHAP_CACHE[key] = pd.DataFrame()
        return _SHAP_CACHE[key]

    df = _read_csv_safe(p)
    if df.empty:
        _SHAP_CACHE[key] = df
        return df

    cols = {c.lower(): c for c in df.columns}
    if "feature" not in cols:
        df = df.rename(columns={list(df.columns)[0]: "feature"})
    else:
        df = df.rename(columns={cols["feature"]: "feature"})
    if "subspace" in cols:
        df = df.rename(columns={cols["subspace"]: "subspace"})
    if "shap_mean_abs" in cols:
        df = df.rename(columns={cols["shap_mean_abs"]: "shap_mean_abs"})
    elif "importance" in cols:
        df = df.rename(columns={cols["importance"]: "shap_mean_abs"})

    keep = [c for c in ["feature","subspace","shap_mean_abs"] if c in df.columns]
    df = df[keep].dropna()
    if "subspace" not in df.columns:
        df["subspace"] = "compute"
    df["subspace"] = df["subspace"].astype(str)
    df["shap_mean_abs"] = pd.to_numeric(df["shap_mean_abs"], errors="coerce").fillna(0.0)
    df["feature"] = df["feature"].astype(str).str.strip()
    df = df.sort_values("shap_mean_abs", ascending=False).reset_index(drop=True)
    df["shap_rank"]  = np.arange(1, len(df)+1)
    df["shap_prank"] = _percentile_rank(df["shap_mean_abs"], True)

    _SHAP_CACHE[key] = df
    return df

# ---------------- Hybrid feature list for scenario (CPMI + SHAP blend) --------
def _hybrid_rank_table(cpmi_df: pd.DataFrame, shap_df: pd.DataFrame) -> pd.DataFrame:
    if cpmi_df.empty:
        return pd.DataFrame()
    left = cpmi_df[["feature","cpmi_score","cpmi_rank","cpmi_prank","subspace"]].copy()
    if shap_df is None or shap_df.empty:
        m = left.copy()
        m["shap_prank"] = 0.0
    else:
        right = shap_df[["feature","shap_mean_abs","shap_rank","shap_prank"]].copy()
        m = left.merge(right, on="feature", how="left")
        m["shap_prank"] = pd.to_numeric(m["shap_prank"], errors="coerce").fillna(0.0)
    m["hybrid_score"] = (1 - ALPHA) * m["cpmi_prank"] + ALPHA * m["shap_prank"]
    m = m.sort_values("hybrid_score", ascending=False).reset_index(drop=True)
    m["hybrid_rank"] = np.arange(1, len(m)+1)
    return m

# ---------------- Data indexing ----------------------------------------------
def _index_files(data_dir: Path):
    files = {}
    for p in Path(data_dir).glob("*.csv"):
        m = re.match(r"(?i)^(DDR4|DDR5)_([A-Za-z]+)_([A-Za-z0-9]+)\.csv$", p.name)
        if not m:
            continue
        plat, scen, wl = m.group(1).upper(), m.group(2).upper(), m.group(3).lower()
        files[(plat, scen, wl)] = p
    return files

FILES = _index_files(DATA_DIR)
if len(FILES) < 52:
    log.error(f"Expected ~52 workload files. Found {len(FILES)} in {DATA_DIR}")
    raise RuntimeError(f"Missing workload CSVs in {DATA_DIR}. See log: {LOG_FILE}")
log.info(f"Data files indexed: {len(FILES)}")

# ---------------- XGBoost fit wrapper (API compatible) -----------------------
def fit_xgb(clf, X_tr, y_tr, X_va, y_va, esr: int):
    try:
        clf.fit(X_tr, y_tr, eval_set=[(X_va, y_va)], verbose=False, early_stopping_rounds=esr)
        return
    except TypeError:
        pass
    try:
        cb = [xgb.callback.EarlyStopping(rounds=esr, save_best=True)]
        clf.fit(X_tr, y_tr, eval_set=[(X_va, y_va)], verbose=False, callbacks=cb)
        return
    except TypeError:
        pass
    clf.fit(X_tr, y_tr)

# ---------------- Load per-platform winners (WIN,KF) + details (pct/method) ---
if not PLAT_BEST.exists():
    raise FileNotFoundError(f"Missing per-platform winners CSV: {PLAT_BEST}")
if not PLAT_DETAILS.exists():
    raise FileNotFoundError(f"Missing per-platform details CSV: {PLAT_DETAILS}")

pb = pd.read_csv(PLAT_BEST).copy()
pb.columns = [c.lower() for c in pb.columns]
need_pb = {"setup","win","kfold"}
miss_pb = need_pb - set(pb.columns)
if miss_pb:
    raise KeyError(f"{PLAT_BEST} missing columns: {sorted(miss_pb)}. Have: {list(pb.columns)}")

platform_cases = []
for _, r in pb.iterrows():
    setup = str(r["setup"]).strip().upper()
    win   = int(pd.to_numeric(r["win"], errors="coerce"))
    kf    = int(pd.to_numeric(r["kfold"], errors="coerce"))
    if setup in PLATFORMS:
        platform_cases.append((setup, win, kf))
platform_cases = sorted(platform_cases, key=lambda x: x[0])
print("[OK] Platform cases:", platform_cases)

det = pd.read_csv(PLAT_DETAILS).copy()
det.columns = [c.lower() for c in det.columns]
if "best_method" in det.columns and "method" not in det.columns:
    det = det.rename(columns={"best_method":"method"})
if "best_pct_by_median" not in det.columns and "pct" in det.columns:
    det = det.rename(columns={"pct":"best_pct_by_median"})
need_det = {"setup","anomaly","win","kfold","best_pct_by_median","method"}
miss_det = need_det - set(det.columns)
if miss_det:
    raise KeyError(f"{PLAT_DETAILS} missing columns: {sorted(miss_det)}. Have: {list(det.columns)}")

# Map (setup, scenario) -> (pct, method) from details
detail_map: Dict[Tuple[str,str], Tuple[int,str]] = {}
for _, r in det.iterrows():
    setup = str(r["setup"]).strip().upper()
    scen  = str(r["anomaly"]).strip().upper()
    pct   = int(pd.to_numeric(r["best_pct_by_median"], errors="coerce"))
    method= str(r["method"]).strip()
    detail_map[(setup, scen)] = (pct, method)

# ---------------- PART A: HYBRID vs CP-MI (Jaccard + Spearman) ----------------
def _read_rank_csv(p: Path, prefer_col=None):
    if (not p) or (not p.exists()):
        return []
    df = pd.read_csv(p)
    col = prefer_col if (prefer_col and prefer_col in df.columns) else df.columns[0]
    feats = df[col].astype(str).tolist()
    return [_norm(x) for x in feats]

def _path_hybrid(setup, win, kfold, sub):
    return ROOT / "FeatureRankOUT_HYBRID" / f"{setup}_{win}_{kfold}_0_{sub}.csv"

def _path_cpmi(setup, win, kfold, sub):
    return ROOT / "FeatureRankOUT" / f"{setup}_{win}_{kfold}_0_{sub}.csv"

def jaccard_topk(A_list, B_list, pct):
    if not A_list or not B_list:
        return np.nan
    kA = max(1, int(np.ceil(len(A_list) * (pct/100.0))))
    kB = max(1, int(np.ceil(len(B_list) * (pct/100.0))))
    A = set(A_list[:kA]); B = set(B_list[:kB])
    denom = len(A | B)
    return (len(A & B) / denom) if denom else np.nan

def spearman_between_lists(A_list, B_list):
    if not A_list or not B_list:
        return np.nan
    union = list(dict.fromkeys(A_list + B_list))
    rA_index = {f:i for i,f in enumerate(A_list)}
    rB_index = {f:i for i,f in enumerate(B_list)}
    worstA = len(A_list) + 1
    worstB = len(B_list) + 1
    a = np.array([rA_index.get(f, worstA) for f in union], dtype=float)
    b = np.array([rB_index.get(f, worstB) for f in union], dtype=float)
    if np.all(a == a[0]) or np.all(b == b[0]):
        return np.nan
    a_mean, b_mean = a.mean(), b.mean()
    num = np.sum((a-a_mean)*(b-b_mean))
    den = np.sqrt(np.sum((a-a_mean)**2) * np.sum((b-b_mean)**2))
    return float(num/den) if den > 0 else np.nan

rows_j, rows_s = [], []
for setup, win, kf in platform_cases:
    for sub in SUBSPACES:
        H = _read_rank_csv(_path_hybrid(setup, win, kf, sub), prefer_col="feature_display")
        C = _read_rank_csv(_path_cpmi(setup, win, kf, sub))
        if (not H) or (not C):
            continue
        for pct in list(range(10,101,10)):
            rows_j.append({"setup":setup,"win":win,"kfold":kf,"subspace":sub,"pct":pct,
                           "jaccard": jaccard_topk(H,C,pct)})
        rows_s.append({"setup":setup,"win":win,"kfold":kf,"subspace":sub,
                       "spearman": spearman_between_lists(H,C)})

JACCARD = pd.DataFrame(rows_j)
SPEARMAN = pd.DataFrame(rows_s)

j_csv = OUT_COMP / "hybrid_vs_cpmi_jaccard_topk_PLATFORM.csv"
s_csv = OUT_COMP / "hybrid_vs_cpmi_spearman_PLATFORM.csv"
JACCARD.to_csv(j_csv, index=False)
SPEARMAN.to_csv(s_csv, index=False)
print(f"[WROTE] {j_csv}\n[WROTE] {s_csv}")

# ---------------- PART B: Plot Jaccard curves per platform -------------------
def plot_jaccard_curves_platform(J, out_dir):
    if J is None or J.empty:
        print("[SKIP] Jaccard plot (no data).")
        return
    JJ = J.copy()
    JJ["pct"] = pd.to_numeric(JJ["pct"], errors="coerce")
    JJ["jaccard"] = pd.to_numeric(JJ["jaccard"], errors="coerce")
    for (setup, win, kf), g in JJ.groupby(["setup","win","kfold"]):
        gg = g.groupby("pct", as_index=False)["jaccard"].mean().sort_values("pct")
        plt.figure(figsize=(6.0,4.0), dpi=160)
        plt.plot(gg["pct"], gg["jaccard"], lw=2.4, marker="o")
        plt.ylim(0,1)
        plt.xlabel("Top-% threshold")
        plt.ylabel("Jaccard overlap (Hybrid vs CP-MI)")
        plt.title(f"{setup} (WIN={int(win)}, KF={int(kf)}) — Avg over subspaces")
        plt.grid(True, alpha=0.25)
        plt.tight_layout()
        out = Path(out_dir) / f"jaccard_curve_PLATFORM_{setup}_WIN{int(win)}_KF{int(kf)}.png"
        plt.savefig(out, dpi=300)
        plt.close()
        print("Wrote", out)

plot_jaccard_curves_platform(JACCARD, OUT_COMP)

# ---------------- PART C: Build per_workload_system_auc.csv (52 rows) --------
def _available_win_kfold(setup: str):
    pairs = set()
    patt_list = []
    for d in RANK_DIRS:
        d = Path(d)
        patt_list += [
            str(d / f"{setup}_*_0_compute.csv"),
            str(d / f"{setup}_*_0_memory.csv"),
            str(d / f"{setup}_*_0_sensors.csv"),
            str(d / f"{setup}_*_0_*.csv"),
        ]
    for patt in patt_list:
        for path in glob.glob(patt):
            m = re.search(rf"{setup}_(\d+)_([0-9]+)_0_", str(path))
            if m:
                pairs.add((int(m.group(1)), int(m.group(2))))
    return sorted(pairs)

def _evaluate_mean_auc(setup: str, win: int, kfold: int, pct: int, workloads_probe=None) -> float:
    workloads_probe = workloads_probe or WORKLOADS
    aucs = []
    count = 0

    cpmi_tables = {sub: _load_cpmi_one(setup, win, kfold, sub) for sub in SUBSPACES}
    if all(v.empty for v in cpmi_tables.values()):
        return float("nan")

    for scen in SCENARIOS[setup]:
        # use exact pct/method from per-platform details if available; else fallback to first match
        pct_s, method_s = detail_map.get((setup, scen.upper()), (None, None))
        shap = pd.DataFrame()
        if pct_s is not None and method_s is not None:
            shap = _load_shap_best_full_exact(setup, scen.upper(), win, kfold, int(pct_s), str(method_s))

        sel = []
        for sub in SUBSPACES:
            base = cpmi_tables[sub]
            if base.empty:
                continue
            shap_sub = shap[shap["subspace"].astype(str).str.lower() == sub] if (not shap.empty and "subspace" in shap.columns) else shap
            H = _hybrid_rank_table(base, shap_sub)
            if H.empty:
                continue
            k_sel = _top_k_from_pct(len(H), pct)
            sel.extend(H.head(k_sel)["feature"].tolist())
        sel = sorted(set(sel))
        if not sel:
            continue

        for wl in workloads_probe:
            if (setup, "BENIGN", wl) not in FILES or (setup, scen.upper(), wl) not in FILES:
                continue

            def _pairs(key):
                p = FILES[key]
                return [(p, _read_csv_safe(p))]

            df_b = build_windowed_raw_means(_pairs((setup,"BENIGN",wl)), win=win, label="BENIGN")
            df_a = build_windowed_raw_means(_pairs((setup,scen.upper(),wl)), win=win, label=scen.upper())
            if df_b.empty or df_a.empty:
                continue

            Xb = df_b[telemetry_cols(df_b)].astype(float)
            mu, sd, q1, q2 = robust_scale_train(Xb.values, winsor=(2.0, 98.0))
            Xb_z = apply_robust_scale(Xb, mu, sd, q1, q2); Xb_z.columns = Xb.columns
            Xa   = df_a[telemetry_cols(df_a)].astype(float)
            Xa_z = apply_robust_scale(Xa, mu, sd, q1, q2); Xa_z.columns = Xa.columns

            X_all = pd.concat([Xb_z, Xa_z], ignore_index=True)
            y_all = np.concatenate([np.zeros(len(Xb_z), dtype=int), np.ones(len(Xa_z), dtype=int)])

            present = [f for f in sel if f in X_all.columns]
            if not present:
                continue

            rng = np.random.RandomState(SEED)
            idx = np.arange(len(y_all))
            val_mask = np.zeros_like(y_all, dtype=bool)
            val_mask[rng.choice(idx, size=max(1, int(0.30*len(y_all))), replace=False)] = True

            X = X_all[present]
            X_tr, y_tr = X.loc[~val_mask], y_all[~val_mask]
            X_va, y_va = X.loc[val_mask],  y_all[val_mask]
            if len(np.unique(y_va)) < 2 or len(y_tr) < 2:
                continue

            clf = XGBClassifier(
                n_estimators=XGB_TREES, max_depth=4, learning_rate=0.08,
                subsample=0.9, colsample_bytree=0.9,
                random_state=SEED, tree_method="hist",
                n_jobs=XGB_THREADS, verbosity=0
            )
            fit_xgb(clf, X_tr, y_tr, X_va, y_va, esr=XGB_ESR)
            pred = clf.predict_proba(X_va)[:, 1]
            aucs.append(roc_auc_score(y_va, pred))

            count += 1
            if count % 8 == 0:
                gc.collect()

    return float(np.nanmean(aucs)) if aucs else float("nan")

best_combo = {}
for setup in PLATFORMS:
    pairs = _available_win_kfold(setup)
    if not pairs:
        log.error(f"No (win,kfold) pairs found for {setup} under {RANK_DIRS}")
        raise RuntimeError(f"No CP-MI rank files for {setup}. See log: {LOG_FILE}")

    best_auc, best_t = -1.0, None
    for (win, kfold) in pairs:
        for pct in TOP_PCTS:
            mean_auc = _evaluate_mean_auc(setup, int(win), int(kfold), int(pct), workloads_probe=PROBE_WORKLOADS)
            log.info(f"[SELECT_TEST] {setup} win={win} kfold={kfold} top%={pct} probeMeanAUC={mean_auc:.4f}")
            if not np.isnan(mean_auc) and mean_auc > best_auc:
                best_auc, best_t = mean_auc, (int(win), int(kfold), int(pct))

    if best_t is None:
        best_t = (pairs[0][0], pairs[0][1], 50)
    best_combo[setup] = best_t
    log.info(f"[SELECT] {setup}: win={best_t[0]} kfold={best_t[1]} top%={best_t[2]} (probeMeanAUC≈{best_auc:.3f})")

def _select_features_for_scenario(setup: str, scenario: str, win: int, kfold: int, pct: int):
    cpmi = {sub: _load_cpmi_one(setup, win, kfold, sub) for sub in SUBSPACES}
    pct_s, method_s = detail_map.get((setup, scenario.upper()), (None, None))
    shap = pd.DataFrame()
    if pct_s is not None and method_s is not None:
        shap = _load_shap_best_full_exact(setup, scenario.upper(), win, kfold, int(pct_s), str(method_s))

    sel_by_sub, all_sel = {}, []
    for sub in SUBSPACES:
        base = cpmi[sub]
        if base.empty:
            sel_by_sub[sub] = []
            continue
        shap_sub = shap[shap["subspace"].astype(str).str.lower() == sub] if (not shap.empty and "subspace" in shap.columns) else shap
        H = _hybrid_rank_table(base, shap_sub)
        if H.empty:
            sel_by_sub[sub] = []
            continue
        k_sel = _top_k_from_pct(len(H), pct)
        feats = H.head(k_sel)["feature"].tolist()
        sel_by_sub[sub] = feats
        all_sel.extend(feats)
    return sel_by_sub, sorted(set(all_sel))

out_csv = PAPER_OUT / "per_workload_system_auc.csv"
cols = ["Workload name","platform","test scenario","win","k-fold","top %",
        "num features total","num features compute","num features memory","num features sensors","AUCROC"]
pd.DataFrame(columns=cols).to_csv(out_csv, index=False)

for setup in PLATFORMS:
    win, kfold, pct = best_combo[setup]
    for scen in SCENARIOS[setup]:
        sel_by_sub, all_sel = _select_features_for_scenario(setup, scen, win, kfold, pct)

        n_comp = len(sel_by_sub.get("compute", []))
        n_mem  = len(sel_by_sub.get("memory", []))
        n_sens = len(sel_by_sub.get("sensors", []))
        n_tot  = n_comp + n_mem + n_sens

        for wl in WORKLOADS:
            key_b = (setup,"BENIGN",wl)
            key_a = (setup,scen.upper(),wl)

            auc = float("nan")
            if key_b in FILES and key_a in FILES:
                def _pairs(key):
                    p = FILES[key]
                    return [(p, _read_csv_safe(p))]

                df_b = build_windowed_raw_means(_pairs(key_b), win=win, label="BENIGN")
                df_a = build_windowed_raw_means(_pairs(key_a), win=win, label=scen.upper())

                if not df_b.empty and not df_a.empty:
                    Xb = df_b[telemetry_cols(df_b)].astype(float)
                    mu, sd, q1, q2 = robust_scale_train(Xb.values, winsor=(2.0, 98.0))
                    Xb_z = apply_robust_scale(Xb, mu, sd, q1, q2); Xb_z.columns = Xb.columns
                    Xa   = df_a[telemetry_cols(df_a)].astype(float)
                    Xa_z = apply_robust_scale(Xa, mu, sd, q1, q2); Xa_z.columns = Xa.columns

                    X_all = pd.concat([Xb_z, Xa_z], ignore_index=True)
                    y_all = np.concatenate([np.zeros(len(Xb_z), dtype=int), np.ones(len(Xa_z), dtype=int)])

                    present = [f for f in all_sel if f in X_all.columns]
                    if present:
                        rng = np.random.RandomState(SEED)
                        idx = np.arange(len(y_all))
                        val_mask = np.zeros_like(y_all, dtype=bool)
                        val_mask[rng.choice(idx, size=max(1, int(0.30*len(y_all))), replace=False)] = True

                        X = X_all[present]
                        X_tr, y_tr = X.loc[~val_mask], y_all[~val_mask]
                        X_va, y_va = X.loc[val_mask],  y_all[val_mask]

                        if len(np.unique(y_va)) > 1 and len(y_tr) > 1:
                            clf = XGBClassifier(
                                n_estimators=XGB_TREES, max_depth=4, learning_rate=0.08,
                                subsample=0.9, colsample_bytree=0.9,
                                random_state=SEED, tree_method="hist",
                                n_jobs=XGB_THREADS, verbosity=0
                            )
                            fit_xgb(clf, X_tr, y_tr, X_va, y_va, esr=XGB_ESR)
                            pred = clf.predict_proba(X_va)[:, 1]
                            auc = float(roc_auc_score(y_va, pred))

            row = {
                "Workload name": wl,
                "platform": setup,
                "test scenario": scen,
                "win": int(win),
                "k-fold": int(kfold),
                "top %": int(pct),
                "num features total": int(n_tot),
                "num features compute": int(n_comp),
                "num features memory": int(n_mem),
                "num features sensors": int(n_sens),
                "AUCROC": auc
            }
            pd.DataFrame([row]).to_csv(out_csv, mode="a", header=False, index=False)

        gc.collect()

log.info(f"[OK] CSV → {out_csv}")
log.info("==== END combined run ====")

print("Done.")
print("Explainability OUT:", OUT_COMP)
print("CSV:", out_csv)
print("Log:", LOG_FILE)

[OK] Platform cases: [('DDR4', 512, 3), ('DDR5', 1024, 5)]
[WROTE] /Users/hsiaopingni/octaneX_v7_4functions/Results/Explainability_Comparisons_PLATFORM/hybrid_vs_cpmi_jaccard_topk_PLATFORM.csv
[WROTE] /Users/hsiaopingni/octaneX_v7_4functions/Results/Explainability_Comparisons_PLATFORM/hybrid_vs_cpmi_spearman_PLATFORM.csv
Wrote /Users/hsiaopingni/octaneX_v7_4functions/Results/Explainability_Comparisons_PLATFORM/jaccard_curve_PLATFORM_DDR4_WIN512_KF3.png
Wrote /Users/hsiaopingni/octaneX_v7_4functions/Results/Explainability_Comparisons_PLATFORM/jaccard_curve_PLATFORM_DDR5_WIN1024_KF5.png
Done.
Explainability OUT: /Users/hsiaopingni/octaneX_v7_4functions/Results/Explainability_Comparisons_PLATFORM
CSV: /Users/hsiaopingni/octaneX_v7_4functions/Results/paper_exports/per_workload_system_auc.csv
Log: /Users/hsiaopingni/octaneX_v7_4functions/Results/paper_exports/build_per_workload_system_auc.log


---

In [38]:
# === PER-PLATFORM: Find BEST (win, kfold) and BEST pct per anomaly across full 10–100% sweep ===
# UPDATED for your NEW pipeline outputs (no dependency on old per_workload_summary_all_raw...csv).
#
# Inputs:
#   - Results/per_run_metrics_all_PIPELINE.csv   (from your full pipeline)
#
# Outputs:
#   1) Results/BEST_in_DesignSpace_Post_per_platform.csv
#        One row per platform (DDR4, DDR5) with chosen WIN/KF.
#        Criterion: mean AUCPR across that platform's anomalies,
#                   where each anomaly uses its own BEST (method,pct) under that (WIN,KF).
#
#   2) Results/BEST_in_DesignSpace_Post_per_platform_details.csv
#        One row per platform×anomaly with that platform's WIN/KF and the anomaly-specific:
#          best method, best pct, median/IQR/min/max for AUCPR and ROC (at the best pct)
#
#   3) Results/BEST_pct_per_win_kfold_PER_METHOD.csv   (optional but useful)
#        Best pct for each (setup, anomaly, win, kfold, method)
#
# Notes:
#   - Sort rule for “best”: AUCPR_median desc, ROC_median desc, smaller pct, then method order.
#   - Methods considered: dC_aJ, dC_aM, dE_aJ, dE_aM
# ---------------------------------------------------------------------------

import pathlib as pl
import pandas as pd
import numpy as np

ROOT    = pl.Path(globals().get("ROOT", "/Users/hsiaopingni/octaneX_v7_4functions"))
RES_DIR = pl.Path(globals().get("RES_DIR", ROOT / "Results"))
if RES_DIR.name in ("Explainability_SHAP_BestCases", "Explainability_SHAP_BestPlatforms"):
    RES_DIR = RES_DIR.parent

PER_RUN = RES_DIR / "per_run_metrics_all_PIPELINE.csv"
if not PER_RUN.exists():
    raise FileNotFoundError(f"Missing per-run pipeline CSV: {PER_RUN}")

# Load per-run once
df = pd.read_csv(PER_RUN).copy()
df.columns = [c.lower() for c in df.columns]

# Defensive grid you run
valid_wins   = {32, 64, 128, 512, 1024}
valid_kfolds = {3, 5, 10}
df = df[df["win"].isin(valid_wins) & df["kfold"].isin(valid_kfolds)].copy()

# Required columns
need = {"setup","anomaly","win","kfold","pct","run_id","method","roc_auc","auc_pr"}
miss = need - set(df.columns)
if miss:
    raise KeyError(f"{PER_RUN} missing columns: {sorted(miss)}. Have: {list(df.columns)}")

# Clean numeric
for c in ["win","kfold","pct","roc_auc","auc_pr"]:
    df[c] = pd.to_numeric(df[c], errors="coerce")
df = df.dropna(subset=["setup","anomaly","win","kfold","pct","method"]).copy()
df["auc_pr"]  = df["auc_pr"].clip(0.0, 1.0)
df["roc_auc"] = df["roc_auc"].clip(0.0, 1.0)

ANOMALIES_BY_SETUP = {"DDR4": ["DROOP","RH"], "DDR5": ["DROOP","SPECTRE"]}
METHOD_ORDER = ["dC_aJ", "dC_aM", "dE_aJ", "dE_aM"]
method_rank = {m:i for i,m in enumerate(METHOD_ORDER)}

df = df[df["setup"].isin(["DDR4","DDR5"])].copy()
df = df[df["method"].isin(METHOD_ORDER)].copy()

# ---------------------------------------------------------------------------
# 0) Summarize per (setup, anomaly, win, kfold, method, pct) across run_id
# ---------------------------------------------------------------------------
def q1(x): return float(np.nanpercentile(x, 25))
def q3(x): return float(np.nanpercentile(x, 75))

keys = ["setup","anomaly","win","kfold","method","pct"]
sumdf = (df.groupby(keys, as_index=False)
           .agg(
               auc_pr_median=("auc_pr","median"),
               auc_pr_q1=("auc_pr", q1),
               auc_pr_q3=("auc_pr", q3),
               auc_pr_min=("auc_pr","min"),
               auc_pr_max=("auc_pr","max"),
               roc_auc_median=("roc_auc","median"),
               roc_auc_q1=("roc_auc", q1),
               roc_auc_q3=("roc_auc", q3),
               roc_auc_min=("roc_auc","min"),
               roc_auc_max=("roc_auc","max"),
               n_runs=("run_id","nunique"),
           ))

sumdf["auc_pr_iqr"]  = sumdf["auc_pr_q3"]  - sumdf["auc_pr_q1"]
sumdf["roc_auc_iqr"] = sumdf["roc_auc_q3"] - sumdf["roc_auc_q1"]

# Fill NaNs for robust sorting
sumdf["auc_pr_median_filled"]  = sumdf["auc_pr_median"].fillna(-np.inf)
sumdf["roc_auc_median_filled"] = sumdf["roc_auc_median"].fillna(-np.inf)
sumdf["_mrank"] = sumdf["method"].map(method_rank).fillna(999).astype(int)

# Keep only anomalies defined per platform
def _keep_defined(row):
    setup = str(row["setup"]).upper()
    anom  = str(row["anomaly"]).upper()
    return setup in ANOMALIES_BY_SETUP and anom in [a.upper() for a in ANOMALIES_BY_SETUP[setup]]

sumdf = sumdf[sumdf.apply(_keep_defined, axis=1)].copy()

# ---------------------------------------------------------------------------
# A) BEST pct per (setup, anomaly, win, kfold, method)
# ---------------------------------------------------------------------------
best_pct_per_method = (sumdf.sort_values(
                            ["setup","anomaly","win","kfold","method",
                             "auc_pr_median_filled","roc_auc_median_filled","pct"],
                            ascending=[True,True,True,True,True, False,False, True]
                       )
                       .groupby(["setup","anomaly","win","kfold","method"], as_index=False)
                       .head(1)
                       .rename(columns={"pct":"best_pct_by_median"}))

BEST_PCT_PER_METHOD_CSV = RES_DIR / "BEST_pct_per_win_kfold_PER_METHOD.csv"
best_pct_per_method.to_csv(BEST_PCT_PER_METHOD_CSV, index=False)
print(f"[OK] Saved best pct per (setup,anomaly,win,kfold,method) → {BEST_PCT_PER_METHOD_CSV}")

# ---------------------------------------------------------------------------
# B) For each (setup, anomaly, win, kfold): pick BEST method (using its BEST pct)
# ---------------------------------------------------------------------------
best_pct_per_method["_mrank"] = best_pct_per_method["method"].map(method_rank).fillna(999).astype(int)

best_method_per_case = (best_pct_per_method.sort_values(
                            ["setup","anomaly","win","kfold",
                             "auc_pr_median_filled","roc_auc_median_filled","_mrank","best_pct_by_median"],
                            ascending=[True,True,True,True, False,False, True, True]
                        )
                        .groupby(["setup","anomaly","win","kfold"], as_index=False)
                        .head(1))

# ---------------------------------------------------------------------------
# C) PER PLATFORM: choose ONE (win,kfold) by mean AUCPR across anomalies
#    (each anomaly contributes its best method@best pct for that win/kfold)
# ---------------------------------------------------------------------------
platform_rows = []
detail_rows = []

print("\n[SCAN] PER-PLATFORM best (WIN,KF) by mean AUCPR across platform anomalies")

for setup in ["DDR4","DDR5"]:
    sub = best_method_per_case[best_method_per_case["setup"].astype(str).str.upper() == setup].copy()
    if sub.empty:
        print(f"  [WARN] No summarized rows for {setup}")
        continue

    # aggregate across anomalies
    agg = (sub.groupby(["setup","win","kfold"], as_index=False)
             .agg(mean_auc_pr=("auc_pr_median_filled","mean"),
                  mean_roc_auc=("roc_auc_median_filled","mean"),
                  n_anoms=("anomaly","nunique")))

    # prefer combos covering more anomalies, then mean AUCPR, mean ROC, then smaller win/kfold
    agg = agg.sort_values(
        ["n_anoms","mean_auc_pr","mean_roc_auc","win","kfold"],
        ascending=[False, False, False, True, True]
    )
    top = agg.iloc[0]
    win_best = int(top["win"])
    kf_best  = int(top["kfold"])

    platform_rows.append({
        "setup": setup,
        "win": win_best,
        "kfold": kf_best,
        "mean_auc_pr_across_anomalies": float(top["mean_auc_pr"]),
        "mean_roc_auc_across_anomalies": float(top["mean_roc_auc"]),
        "num_anomalies_covered": int(top["n_anoms"]),
        "selection_note": "mean across anomalies; each anomaly uses its best (method,pct) for this (win,kfold)",
    })

    chosen = sub[(sub["win"]==win_best) & (sub["kfold"]==kf_best)].copy()
    chosen = chosen.sort_values(
        ["auc_pr_median_filled","roc_auc_median_filled","best_pct_by_median"],
        ascending=[False, False, True]
    )

    print(f"  [PLATFORM BEST • {setup}] WIN={win_best} K={kf_best} "
          f"mean AUCPR={float(top['mean_auc_pr']):.3f} mean ROC={float(top['mean_roc_auc']):.3f} "
          f"(anoms covered={int(top['n_anoms'])})")

    for _, r in chosen.iterrows():
        detail_rows.append({
            "setup": setup,
            "anomaly": str(r["anomaly"]),
            "win": win_best,
            "kfold": kf_best,
            "best_pct_by_median": int(r["best_pct_by_median"]),
            "method": str(r["method"]),
            "auc_pr_median": float(r["auc_pr_median"]),
            "auc_pr_iqr": float(r["auc_pr_iqr"]),
            "auc_pr_min": float(r["auc_pr_min"]),
            "auc_pr_max": float(r["auc_pr_max"]),
            "roc_auc_median": float(r["roc_auc_median"]),
            "roc_auc_iqr": float(r["roc_auc_iqr"]),
            "roc_auc_min": float(r["roc_auc_min"]),
            "roc_auc_max": float(r["roc_auc_max"]),
            "n_runs": int(r["n_runs"]),
        })
        print(f"      {setup}/{r['anomaly']}: method={r['method']} best%={int(r['best_pct_by_median'])} "
              f"AUCPR_med={float(r['auc_pr_median']):.3f} ROC_med={float(r['roc_auc_median']):.3f}")

platform_best = pd.DataFrame(platform_rows).sort_values(["setup"]).reset_index(drop=True)
platform_details = pd.DataFrame(detail_rows).sort_values(["setup","anomaly"]).reset_index(drop=True)

OUT_PLAT = RES_DIR / "BEST_in_DesignSpace_Post_per_platform.csv"
OUT_DET  = RES_DIR / "BEST_in_DesignSpace_Post_per_platform_details.csv"

platform_best.to_csv(OUT_PLAT, index=False)
platform_details.to_csv(OUT_DET, index=False)

print(f"\n[OK] Saved platform winners → {OUT_PLAT}")
print(f"[OK] Saved platform details → {OUT_DET}")

[OK] Saved best pct per (setup,anomaly,win,kfold,method) → /Users/hsiaopingni/octaneX_v7_4functions/Results/BEST_pct_per_win_kfold_PER_METHOD.csv

[SCAN] PER-PLATFORM best (WIN,KF) by mean AUCPR across platform anomalies
  [PLATFORM BEST • DDR4] WIN=512 K=3 mean AUCPR=1.000 mean ROC=1.000 (anoms covered=2)
      DDR4/DROOP: method=dC_aJ best%=10 AUCPR_med=1.000 ROC_med=1.000
      DDR4/RH: method=dC_aJ best%=10 AUCPR_med=1.000 ROC_med=1.000
  [PLATFORM BEST • DDR5] WIN=1024 K=10 mean AUCPR=1.000 mean ROC=1.000 (anoms covered=2)
      DDR5/DROOP: method=dC_aJ best%=10 AUCPR_med=1.000 ROC_med=1.000
      DDR5/SPECTRE: method=dC_aJ best%=10 AUCPR_med=1.000 ROC_med=1.000

[OK] Saved platform winners → /Users/hsiaopingni/octaneX_v7_4functions/Results/BEST_in_DesignSpace_Post_per_platform.csv
[OK] Saved platform details → /Users/hsiaopingni/octaneX_v7_4functions/Results/BEST_in_DesignSpace_Post_per_platform_details.csv


In [69]:
import re
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# -------------------- locate rank dirs robustly --------------------
ROOT = Path(globals().get("ROOT", "/Users/hsiaopingni/octaneX_v7"))
RES_DIR_RAW = Path(globals().get("RES_DIR", ROOT / "Results"))
RES_DIR = RES_DIR_RAW.parent if RES_DIR_RAW.name == "Explainability_SHAP_BestCases" else RES_DIR_RAW

# You can override these in globals() if you want:
#   RANK_HYB_DIR = Path(".../FeatureRankOUT_HYBRID")
#   RANK_CPM_DIR = Path(".../FeatureRankOUT")
RANK_HYB_DIR = globals().get("RANK_HYB_DIR", None)
RANK_CPM_DIR = globals().get("RANK_CPM_DIR", None)

# Search lists
RANK_HYB_DIRS = []
RANK_CPM_DIRS = []

if RANK_HYB_DIR is not None:
    RANK_HYB_DIRS.append(Path(RANK_HYB_DIR))
else:
    RANK_HYB_DIRS += [
        ROOT / "FeatureRankOUT_HYBRID",
        RES_DIR / "FeatureRankOUT_HYBRID",
        Path("/Volumes/Untitled") / "FeatureRankOUT_HYBRID",
        Path("/Volumes/Untitled") / "octaneX" / "FeatureRankOUT_HYBRID",
        Path.home() / "Desktop" / "octaneX" / "FeatureRankOUT_HYBRID",
    ]

if RANK_CPM_DIR is not None:
    RANK_CPM_DIRS.append(Path(RANK_CPM_DIR))
else:
    RANK_CPM_DIRS += [
        ROOT / "FeatureRankOUT",
        RES_DIR / "FeatureRankOUT",
        Path("/Volumes/Untitled") / "FeatureRankOUT",
        Path("/Volumes/Untitled") / "octaneX" / "FeatureRankOUT",
        Path.home() / "Desktop" / "octaneX" / "FeatureRankOUT",
    ]

# OUT directory must exist
OUT = Path(globals().get("OUT", RES_DIR / "paper_exports"))
OUT.mkdir(parents=True, exist_ok=True)

def _find_rank_file(dirs, fname: str) -> Path | None:
    for d in dirs:
        d = Path(d)
        if d.exists():
            p = d / fname
            if p.exists():
                return p
    # fallback slow search (only if needed)
    for d in dirs:
        d = Path(d)
        if d.exists():
            hits = list(d.rglob(fname))
            if hits:
                return hits[0]
    return None

# -------------------- your stability logic --------------------
def _norm_name(s: str) -> str:
    s = re.sub(r"\s+","_",str(s)).lower().replace("%","pct")
    s = re.sub(r"[^a-z0-9_]+","_", s)
    return re.sub(r"_+","_", s).strip("_")

def _read_rank_csv(p: Path | None, prefer_col=None):
    if p is None or (not Path(p).exists()):
        return []
    df = pd.read_csv(p)
    if df.empty:
        return []
    col = prefer_col if (prefer_col and prefer_col in df.columns) else df.columns[0]
    return [_norm_name(x) for x in df[col].astype(str).tolist()]

def _top_set(lst, pct):
    if not lst:
        return set()
    k = max(1, int(np.ceil(len(lst) * (pct/100.0))))
    return set(lst[:k])

def _jacc(A, B):
    if not A and not B:
        return np.nan
    den = len(A | B)
    return (len(A & B) / den) if den else np.nan

rows = []
cases_for_ranks = best[["setup","win","kfold"]].drop_duplicates()

PCTS = list(range(10,101,10))
for _, rr in cases_for_ranks.iterrows():
    setup, win, kf = str(rr["setup"]), int(rr["win"]), int(rr["kfold"])
    fname = f"{setup}_{win}_{kf}_0_memory.csv"

    pH = _find_rank_file(RANK_HYB_DIRS, fname)
    pC = _find_rank_file(RANK_CPM_DIRS, fname)

    H = _read_rank_csv(pH, prefer_col="feature_display")  # hybrid file often has feature_display
    C = _read_rank_csv(pC, prefer_col="feature")          # cpmi file often has feature

    for mode, L in [("Hybrid", H), ("CP-MI", C)]:
        if not L:
            continue
        tops = {p: _top_set(L, p) for p in PCTS}
        consec = [_jacc(tops[a], tops[b]) for a, b in zip(PCTS[:-1], PCTS[1:])]
        if consec:
            rows.append({"mode": mode, "val": float(np.nanmean(consec))})

stab_df = pd.DataFrame(rows)

if not stab_df.empty:
    plt.figure(figsize=(5.2,3.8), dpi=160)
    data, labels = [], []

    if (stab_df["mode"]=="Hybrid").any():
        data.append(stab_df.loc[stab_df["mode"]=="Hybrid","val"].values)
        labels.append("Hybrid")
    if (stab_df["mode"]=="CP-MI").any():
        data.append(stab_df.loc[stab_df["mode"]=="CP-MI","val"].values)
        labels.append("CP-MI")

    if not data:
        print("[SKIP] stability_box_consecutive_memory (no values after filtering)")
    else:
        plt.boxplot(data, labels=labels)
        plt.ylim(0,1)
        plt.ylabel("Consecutive Jaccard (Memory) ↑")
        plt.title("Within-method rank stability across Top-% steps")
        plt.tight_layout()
        out_path = OUT / "stability_box_consecutive_memory.png"
        plt.savefig(out_path)
        plt.close()
        print("[WROTE]", out_path)
else:
    print("[SKIP] stability_box_consecutive_memory (no memory rank files found)")



[WROTE] /Users/hsiaopingni/octaneX_v7/Results/Explainability_Comparisons/stability_box_consecutive_memory.png


In [65]:
# pip install scipy numpy statsmodels (if needed)
import numpy as np
from scipy.stats import wilcoxon, ttest_rel
from statsmodels.stats.multitest import multipletests

def paired_tests(baseline, hybrid, label, test='wilcoxon'):
    """
    baseline, hybrid: 1D arrays of per-fold scores (e.g., AUC-PR across folds)
    label: short name to print (e.g., 'DDR5-DROOP AUC-PR')
    test: 'wilcoxon' (default) or 'ttest'
    """
    baseline = np.asarray(baseline, float)
    hybrid   = np.asarray(hybrid, float)
    assert baseline.shape == hybrid.shape, "Use paired arrays with same folds."

    diff = hybrid - baseline
    mean_gain = diff.mean()
    std_gain  = diff.std(ddof=1)

    if test == 'ttest':
        t_stat, p = ttest_rel(hybrid, baseline, alternative='greater')  # test: hybrid > baseline
        # Cohen's dz for paired samples
        dz = mean_gain / (diff.std(ddof=1) + 1e-12)
        eff = ('Cohen_dz', dz)
        stat = ('t', t_stat)
    else:
        # Wilcoxon signed-rank (zero_method='wilcox' ignores zero diffs)
        stat_w, p = wilcoxon(hybrid, baseline, alternative='greater', zero_method='wilcox')
        # Cliff's delta (paired) approximation
        # For small n, dz also OK; Cliff’s delta more robust:
        # Compute ordinal effect size:
        gt = sum((d1 > d2) for d1 in hybrid for d2 in baseline)
        lt = sum((d1 < d2) for d1 in hybrid for d2 in baseline)
        cliff_delta = (gt - lt) / (len(hybrid)*len(baseline))
        eff = ('Cliffs_delta', cliff_delta)
        stat = ('W', stat_w)

    result = {
        'label': label,
        'n_folds': len(diff),
        'baseline_mean': float(baseline.mean()),
        'hybrid_mean': float(hybrid.mean()),
        'mean_gain': float(mean_gain),
        'std_gain': float(std_gain),
        'stat_name': stat[0],
        'stat_value': float(stat[1]),
        'p_value': float(p),
        'effect_name': eff[0],
        'effect_value': float(eff[1]),
    }
    return result

# ==== EXAMPLE USAGE ====
# Replace the arrays below with your per-fold results (same folds, same order).
# Example: DDR5–DROOP AUC-PR (5 or 10 values each)
octane_aucpr = np.array([0.95, 0.96, 0.99, 0.97, 0.98])   # baseline per-fold
xoct_aucpr   = np.array([0.99, 1.00, 1.00, 0.99, 1.00])   # hybrid per-fold

octane_roc   = np.array([0.99, 1.00, 1.00, 1.00, 1.00])
xoct_roc     = np.array([1.00, 1.00, 1.00, 1.00, 1.00])

results = []
results.append(paired_tests(octane_aucpr, xoct_aucpr, 'DDR5-DROOP AUC-PR', test='wilcoxon'))
results.append(paired_tests(octane_roc,   xoct_roc,   'DDR5-DROOP ROC-AUC', test='wilcoxon'))

# If you have many workloads/metrics, do FDR across all p-values:
pvals = [r['p_value'] for r in results]
rej, pvals_fdr, *_ = multipletests(pvals, alpha=0.05, method='fdr_bh')
for r, pfdr, rj in zip(results, pvals_fdr, rej):
    r['p_value_fdr'] = float(pfdr)
    r['significant_fdr_0.05'] = bool(rj)

# Pretty print
for r in results:
    print(
        f"{r['label']}: mean gain={r['mean_gain']:.4f} ± {r['std_gain']:.4f}, "
        f"{r['stat_name']}={r['stat_value']:.3f}, p={r['p_value']:.4g}, "
        f"FDR-p={r.get('p_value_fdr', r['p_value']):.4g}, "
        f"{r['effect_name']}={r['effect_value']:.3f}"
    )


DDR5-DROOP AUC-PR: mean gain=0.0260 ± 0.0134, W=15.000, p=0.03125, FDR-p=0.0625, Cliffs_delta=0.920
DDR5-DROOP ROC-AUC: mean gain=0.0020 ± 0.0045, W=1.000, p=0.5, FDR-p=0.5, Cliffs_delta=0.200


In [66]:
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import wilcoxon
from pathlib import Path

# === Auto-detect ROOT directory ===
EXT_DRIVE = Path("/Volumes/Untitled")
LOCAL_ROOT = Path("/Users/hsiaopingni/octaneX_v7")
ROOT = EXT_DRIVE / "octaneX" if EXT_DRIVE.exists() else LOCAL_ROOT

OUT_DIR = ROOT / "Results" / "Figures"
OUT_DIR.mkdir(parents=True, exist_ok=True)
print(f"Saving outputs to: {OUT_DIR.resolve()}")

# === Example per-fold data (replace with actual results) ===
octane_aucpr = np.array([0.95, 0.96, 0.99, 0.97, 0.98])
xoct_aucpr   = np.array([0.99, 1.00, 1.00, 0.99, 1.00])

octane_roc   = np.array([0.99, 1.00, 1.00, 1.00, 1.00])
xoct_roc     = np.array([1.00, 1.00, 1.00, 1.00, 1.00])

# === Compute statistics ===
def stat_summary(baseline, hybrid, label):
    diff = hybrid - baseline
    mean_gain = diff.mean()
    stat, p = wilcoxon(hybrid, baseline, alternative='greater')
    return {'label': label, 'gain': mean_gain, 'stat': stat, 'p': p, 'diff': diff}

res_aucpr = stat_summary(octane_aucpr, xoct_aucpr, 'AUC-PR')
res_roc   = stat_summary(octane_roc, xoct_roc, 'ROC-AUC')

# === Plot ===
fig, ax = plt.subplots(1, 2, figsize=(9, 4))

for i, (metric, base, hybrid, res) in enumerate([
    ('AUC-PR', octane_aucpr, xoct_aucpr, res_aucpr),
    ('ROC-AUC', octane_roc, xoct_roc, res_roc)
]):
    x = np.arange(len(base)) + 1
    ax[i].plot(x, base, 'o--', label='OCTANE (Baseline)', color='gray')
    ax[i].plot(x, hybrid, 's-', label='X-OCTANE (Hybrid)', color='tab:blue')
    ax[i].axhline(y=np.mean(base), color='gray', linestyle=':', alpha=0.6)
    ax[i].axhline(y=np.mean(hybrid), color='tab:blue', linestyle='--', alpha=0.7)
    ax[i].set_xlabel('Cross-Validation Fold')
    ax[i].set_ylabel(metric)
    ax[i].set_ylim(0.9, 1.02)
    ax[i].grid(True, alpha=0.3)
    ax[i].set_title(f"{metric} — Δ={res['gain']:.3f}, p={res['p']:.3g}")

ax[0].legend(loc='lower right')
plt.tight_layout()

# === Save figures ===
png_path = OUT_DIR / "XOCTANE_vs_OCTANE_stats.png"
pdf_path = OUT_DIR / "XOCTANE_vs_OCTANE_stats.pdf"
plt.savefig(png_path, dpi=300, bbox_inches="tight")
plt.savefig(pdf_path, bbox_inches="tight")
plt.close()

print(f"Saved: {png_path.name} and {pdf_path.name}")

# === Text summary ===
for res in [res_aucpr, res_roc]:
    print(f"{res['label']}: mean gain={res['gain']:.4f}, Wilcoxon W={res['stat']:.3f}, p={res['p']:.4g}")


Saving outputs to: /Users/hsiaopingni/octaneX_v7/Results/Figures
Saved: XOCTANE_vs_OCTANE_stats.png and XOCTANE_vs_OCTANE_stats.pdf
AUC-PR: mean gain=0.0260, Wilcoxon W=15.000, p=0.03125
ROC-AUC: mean gain=0.0020, Wilcoxon W=1.000, p=0.5


In [42]:
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import wilcoxon, ttest_rel
from pathlib import Path

# === Auto-detect ROOT directory ===
EXT_DRIVE = Path("/Volumes/Untitled")
LOCAL_ROOT = Path("/Users/hsiaopingni/octaneX_v7_4functions")
ROOT = EXT_DRIVE / "octaneX" if EXT_DRIVE.exists() else LOCAL_ROOT

OUT_DIR = ROOT / "Results" / "Figures"
OUT_DIR.mkdir(parents=True, exist_ok=True)
print(f"Saving outputs to: {OUT_DIR.resolve()}")

# === Example per-fold data (replace with real results) ===
octane_aucpr = np.array([0.93, 0.96, 0.97, 0.94, 0.98])
xoct_aucpr   = np.array([0.98, 0.99, 1.00, 0.99, 1.00])

octane_roc   = np.array([0.90, 0.93, 0.95, 0.92, 0.94])
xoct_roc     = np.array([0.96, 0.97, 0.99, 0.98, 0.99])

# === Function for paired comparison summary ===
def paired_stats(baseline, hybrid, label):
    diff = hybrid - baseline
    mean_gain = diff.mean()
    w_stat, w_p = wilcoxon(hybrid, baseline, alternative='greater')
    t_stat, t_p = ttest_rel(hybrid, baseline)
    return {
        'label': label,
        'gain': mean_gain,
        'wilcoxon_p': w_p,
        't_p': t_p,
        'diff': diff
    }

res_aucpr = paired_stats(octane_aucpr, xoct_aucpr, 'AUC-PR')
res_roc   = paired_stats(octane_roc, xoct_roc, 'ROC-AUC')

# === Plot ===
fig, ax = plt.subplots(1, 2, figsize=(9, 4))

for i, (metric, base, hybrid, res) in enumerate([
    ('AUC-PR', octane_aucpr, xoct_aucpr, res_aucpr),
    ('ROC-AUC', octane_roc, xoct_roc, res_roc)
]):
    x = np.arange(1, len(base) + 1)
    ax[i].plot(x, base, 'o--', label='OCTANE (Baseline)', color='gray')
    ax[i].plot(x, hybrid, 's-', label='X-OCTANE (Hybrid)', color='tab:blue')
    ax[i].axhline(np.mean(base), color='gray', linestyle=':', alpha=0.6)
    ax[i].axhline(np.mean(hybrid), color='tab:blue', linestyle='--', alpha=0.7)
    ax[i].set_xlabel('Cross-Validation Fold')
    ax[i].set_ylabel(metric)
    ax[i].set_ylim(0.85, 1.02)
    ax[i].grid(alpha=0.3)
    ax[i].set_title(
        f"{metric}\nΔ={res['gain']:.3f} | "
        f"p₍W₎={res['wilcoxon_p']:.3f}, p₍t₎={res['t_p']:.3f}"
    )

ax[0].legend(loc='lower right', fontsize=9)
plt.tight_layout()

# === Save outputs ===
png_path = OUT_DIR / "XOCTANE_vs_OCTANE_ttest_wilcoxon.png"
pdf_path = OUT_DIR / "XOCTANE_vs_OCTANE_ttest_wilcoxon.pdf"
plt.savefig(png_path, dpi=300, bbox_inches="tight")
plt.savefig(pdf_path, bbox_inches="tight")
plt.close()

# === Summary printout ===
print("\n=== Statistical Summary ===")
for res in [res_aucpr, res_roc]:
    print(f"{res['label']}:")
    print(f"  Mean gain: {res['gain']:.4f}")
    print(f"  Wilcoxon p-value: {res['wilcoxon_p']:.4f}")
    print(f"  Paired t-test p-value: {res['t_p']:.4f}")
    print()
print(f"Figures saved to:\n  {png_path}\n  {pdf_path}")


Saving outputs to: /Users/hsiaopingni/octaneX_v7_4functions/Results/Figures

=== Statistical Summary ===
AUC-PR:
  Mean gain: 0.0360
  Wilcoxon p-value: 0.0312
  Paired t-test p-value: 0.0039

ROC-AUC:
  Mean gain: 0.0500
  Wilcoxon p-value: 0.0312
  Paired t-test p-value: 0.0004

Figures saved to:
  /Users/hsiaopingni/octaneX_v7_4functions/Results/Figures/XOCTANE_vs_OCTANE_ttest_wilcoxon.png
  /Users/hsiaopingni/octaneX_v7_4functions/Results/Figures/XOCTANE_vs_OCTANE_ttest_wilcoxon.pdf


# **Jaccard agreement J@𝑝

In [51]:
# === Fix "[SKIP] No rows ..." by auto-resolving the correct (WIN,KF) per platform ===
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt

# --- Paths ---
ROOT = Path(globals().get("ROOT", "/Users/hsiaopingni/octaneX_v7_4functions"))
RES_DIR = Path(globals().get("RES_DIR", ROOT / "Results"))
if RES_DIR.name in ("Explainability_SHAP_BestCases", "Explainability_SHAP_BestPlatforms"):
    RES_DIR = RES_DIR.parent

OUT_DIR = RES_DIR / "Explainability_SHAP_BestPlatforms"
OUT_DIR.mkdir(parents=True, exist_ok=True)

CSV  = OUT_DIR / "SHAP_vs_CPMI_summary_PLATFORM.csv"
PLAT = RES_DIR / "BEST_in_DesignSpace_Post_per_platform.csv"

assert CSV.exists(), f"Missing {CSV}. Run the per-platform SHAP-vs-CPMI summary cell first."
assert PLAT.exists(), f"Missing {PLAT}. Generate per-platform winners first."

# --- Load ---
df = pd.read_csv(CSV).copy()
pb = pd.read_csv(PLAT).copy()

df.columns = [c.lower() for c in df.columns]
pb.columns = [c.lower() for c in pb.columns]

need = {"setup","win","kfold","subspace","k","jaccard"}
missing = need - set(df.columns)
assert not missing, f"Missing columns in summary CSV: {sorted(missing)}. Have: {list(df.columns)}"

# Force numeric typing for robust comparisons
df["win"] = pd.to_numeric(df["win"], errors="coerce")
df["kfold"] = pd.to_numeric(df["kfold"], errors="coerce")
df["jaccard"] = pd.to_numeric(df["jaccard"], errors="coerce")

# --- Plot styling to match your screenshot ---
COLORS  = {"compute":"tab:orange", "memory":"tab:blue", "sensors":"tab:green"}
MARKERS = {"compute":"o",          "memory":"s",        "sensors":"^"}
LABELS  = {"compute":"Compute",    "memory":"Memory",  "sensors":"Sensors"}

def _k_to_pct(kstr):
    try:
        return int(str(kstr).replace("top","").replace("%",""))
    except Exception:
        return np.nan

def _available_pairs_for_setup(setup_label: str):
    sub = df[df["setup"].astype(str).str.upper() == setup_label.upper()].copy()
    sub = sub.dropna(subset=["win","kfold"])
    pairs = sorted(set((int(w), int(k)) for w,k in zip(sub["win"], sub["kfold"])))
    return pairs

def _resolve_platform_pair(setup_label: str):
    """
    Try to use (win,kfold) from BEST_in_DesignSpace_Post_per_platform.csv.
    If that pair isn't present in the summary CSV, fallback to the first available pair in the summary.
    """
    pairs_avail = _available_pairs_for_setup(setup_label)
    if not pairs_avail:
        return None

    row = pb[pb["setup"].astype(str).str.upper() == setup_label.upper()].copy()
    if not row.empty:
        win0 = int(pd.to_numeric(row["win"].iloc[0], errors="coerce"))
        kf0  = int(pd.to_numeric(row["kfold"].iloc[0], errors="coerce"))
        if (win0, kf0) in pairs_avail:
            return (win0, kf0)

    # fallback: pick the pair with most rows (most complete)
    sub = df[df["setup"].astype(str).str.upper() == setup_label.upper()].copy()
    sub = sub.dropna(subset=["win","kfold"])
    counts = (sub.groupby(["win","kfold"]).size().reset_index(name="n")
                .sort_values("n", ascending=False))
    w = int(counts.iloc[0]["win"])
    k = int(counts.iloc[0]["kfold"])
    return (w, k)

def plot_jaccard_platform(setup_label: str, win: int, kfold: int, out_png: Path):
    sub = df[
        (df["setup"].astype(str).str.upper() == setup_label.upper()) &
        (df["win"].round(0) == int(win)) &
        (df["kfold"].round(0) == int(kfold))
    ].copy()

    if sub.empty:
        print(f"[SKIP] No rows for {setup_label} WIN={win} K={kfold}")
        print(f"       Available (WIN,K) for {setup_label}: {_available_pairs_for_setup(setup_label)}")
        return

    sub["pct"] = sub["k"].map(_k_to_pct)
    sub = sub[np.isfinite(sub["pct"])].copy()
    sub["pct"] = sub["pct"].astype(int)

    agg = (sub.groupby(["subspace","pct"], as_index=False)["jaccard"]
             .median()
             .sort_values(["subspace","pct"]))

    plt.figure(figsize=(7.6, 4.3), dpi=160)
    ax = plt.gca()

    for ssp in ["compute","memory","sensors"]:
        cur = agg[agg["subspace"].astype(str).str.lower() == ssp]
        if cur.empty:
            continue
        x = cur["pct"].to_numpy()
        y = np.clip(cur["jaccard"].to_numpy(dtype=float), 0.0, 1.0)
        ax.plot(x, y, color=COLORS[ssp], marker=MARKERS[ssp],
                linewidth=2.2, markersize=6, label=LABELS[ssp])

    ax.set_xlim(8, 102)
    ax.set_ylim(0.0, 1.02)
    ax.set_xticks(list(range(10, 101, 10)))
    ax.set_xlabel("Top-% of features", fontsize=16)
    ax.set_ylabel("Jaccard index (CP-MI vs SHAP)", fontsize=16)
    ax.grid(True, which="major", linestyle="--", linewidth=0.8, alpha=0.35)
    ax.legend(loc="lower right", frameon=True, fontsize=12)
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)
    ax.tick_params(labelsize=12)

    plt.tight_layout()
    out_png.parent.mkdir(parents=True, exist_ok=True)
    plt.savefig(out_png, dpi=220, bbox_inches="tight", pad_inches=0.05)
    plt.close()
    print("[WROTE]", out_png)

# ---- Quick diagnostic: show what exists for DDR5 ----
print("[INFO] Available (WIN,K) in summary CSV:")
for setup in ["DDR4","DDR5"]:
    print(f"  {setup}: {_available_pairs_for_setup(setup)}")

# ---- Generate plots using resolved platform pairs ----
for setup in ["DDR4","DDR5"]:
    pair = _resolve_platform_pair(setup)
    if pair is None:
        print(f"[WARN] No rows at all for {setup} in {CSV}")
        continue
    win, kf = pair
    out_png = OUT_DIR / f"Jaccard_PLATFORM_{setup}_WIN{win}_K{kf}.png"
    plot_jaccard_platform(setup, win=win, kfold=kf, out_png=out_png)

[INFO] Available (WIN,K) in summary CSV:
  DDR4: [(512, 3)]
  DDR5: [(1024, 5)]
[WROTE] /Users/hsiaopingni/octaneX_v7_4functions/Results/Explainability_SHAP_BestPlatforms/Jaccard_PLATFORM_DDR4_WIN512_K3.png
[WROTE] /Users/hsiaopingni/octaneX_v7_4functions/Results/Explainability_SHAP_BestPlatforms/Jaccard_PLATFORM_DDR5_WIN1024_K5.png


# **Platform Agreement

In [50]:
# === Table 6 (PER PLATFORM, paper-faithful): Agreement between CP–MI and SHAP ===
# You requested: DDR5 (Platform) WIN=1024 K=5  ✅ (forced override)
#
# Revision (fix for low J@k with high rho):
#   - Compute J@p on the SHARED feature universe per subspace (intersection-normalized).
#   - Compute rho on the SHARED universe as well (percentile-rank Spearman).
#
# Inputs:
#   - Results/BEST_in_DesignSpace_Post_per_platform_details.csv   (best pct+method per anomaly; used if matches WIN/K)
#   - FeatureRankOUT/<setup>_<win>_<kfold>_0_{compute|memory|sensors}.csv
#   - Results/Explainability_SHAP_BestPlatforms/SHAP_BESTPLAT_full_<setup>_<anomaly>_WIN<w>_KF<k>_PCT<p>_M<method>.csv
#
# Output:
#   - Results/Explainability_SHAP_BestPlatforms/Table6_platform_agreement.csv
#
# NOTE:
#   This script uses a MANUAL platform config override (DDR5 WIN=1024 K=5).
#   If your per-platform details CSV does not contain rows for that (WIN,K),
#   it will fall back to discovering SHAP files on disk for that (WIN,K) per anomaly.
# --------------------------------------------------------------------------------------------

import re
import numpy as np
import pandas as pd
from pathlib import Path
from typing import Optional, Tuple
from scipy.stats import spearmanr

# -------------------- Paths (robust) --------------------
ROOT = Path(globals().get("ROOT", "/Users/hsiaopingni/octaneX_v7_4functions"))
RES_DIR_RAW = Path(globals().get("RES_DIR", ROOT / "Results"))
RES_DIR = RES_DIR_RAW.parent if RES_DIR_RAW.name in ("Explainability_SHAP_BestCases", "Explainability_SHAP_BestPlatforms") else RES_DIR_RAW

OUT_DIR = RES_DIR / "Explainability_SHAP_BestPlatforms"
OUT_DIR.mkdir(parents=True, exist_ok=True)

PLAT_DETAIL = RES_DIR / "BEST_in_DesignSpace_Post_per_platform_details.csv"
assert PLAT_DETAIL.exists(), f"Missing: {PLAT_DETAIL}"

# Rank dirs (CP–MI)
RANK_DIRS = list(globals().get("RANK_DIRS", [
    ROOT / "FeatureRankOUT",
    Path("/Volumes/Untitled") / "FeatureRankOUT",
    Path("/Volumes/Untitled") / "octaneX" / "FeatureRankOUT",
    Path.home() / "Desktop" / "octaneX" / "FeatureRankOUT",
]))

# SHAP dir (per platform)
SHAP_DIR = RES_DIR / "Explainability_SHAP_BestPlatforms"
assert SHAP_DIR.exists(), f"Missing SHAP dir: {SHAP_DIR}"

# -------------------- Table settings --------------------
PCT_FOR_TABLE = int(globals().get("PCT_FOR_TABLE", 80))  # default 80 for convergence region
SUBSPACES = ("compute", "memory", "sensors")

# -------------------- REQUIRED OVERRIDE (your request) --------------------
# Force the (WIN,K) used for Table 6 per platform:
PLATFORM_CFG = {
    "DDR4": {"win": 512,  "kfold": 3, "label": "DDR4 (Platform)"},
    "DDR5": {"win": 1024, "kfold": 5, "label": "DDR5 (Platform)"},
}

# -------------------- Normalization helpers --------------------
def _norm_name(s: str) -> str:
    s = re.sub(r"\s+", "_", str(s)).lower().replace("%", "pct")
    s = re.sub(r"[^a-z0-9_]+", "_", s)
    return re.sub(r"_+", "_", s).strip("_")

def _top_k_from_pct(n: int, pct: int) -> int:
    return max(1, int(np.ceil(n * (pct / 100.0)))) if n > 0 and pct > 0 else 0

def _jacc(A: set, B: set) -> float:
    if not A and not B:
        return np.nan
    den = len(A | B)
    return (len(A & B) / den) if den else np.nan

def _restrict_to_common(order: list[str], common: set[str]) -> list[str]:
    return [f for f in order if f in common]

def _percentile_rank_from_order(features_in_order: list[str]) -> dict[str, float]:
    n = len(features_in_order)
    if n <= 1:
        return {f: 1.0 for f in features_in_order} if n == 1 else {}
    # best=1, worst=0
    return {f: (1.0 - (i / (n - 1))) for i, f in enumerate(features_in_order)}

def _jacc_top_pct_on_common(cp_list: list[str], sh_list: list[str], pct: int) -> float:
    common = set(cp_list) & set(sh_list)
    if len(common) == 0:
        return np.nan
    cp_c = _restrict_to_common(cp_list, common)
    sh_c = _restrict_to_common(sh_list, common)
    k = _top_k_from_pct(len(common), pct)  # top pct of COMMON size
    return _jacc(set(cp_c[:k]), set(sh_c[:k]))

def _spearman_on_common(cp_order: list[str], sh_order: list[str]) -> float:
    common = set(cp_order) & set(sh_order)
    if len(common) < 3:
        return np.nan
    cp_c = _restrict_to_common(cp_order, common)
    sh_c = _restrict_to_common(sh_order, common)
    cp_pr = _percentile_rank_from_order(cp_c)
    sh_pr = _percentile_rank_from_order(sh_c)
    uni = sorted(common)
    x = np.array([cp_pr.get(f, 0.0) for f in uni], dtype=float)
    y = np.array([sh_pr.get(f, 0.0) for f in uni], dtype=float)
    rho, _ = spearmanr(x, y)
    return float(rho)

# -------------------- CP-MI ranks --------------------
def _find_rank_file(setup: str, win: int, kfold: int, sub: str) -> Optional[Path]:
    fname = f"{setup}_{win}_{kfold}_0_{sub}.csv"
    for d in RANK_DIRS:
        p = Path(d) / fname
        if p.exists():
            return p
    for d in RANK_DIRS:
        d = Path(d)
        if d.exists():
            hits = list(d.rglob(fname))
            if hits:
                return hits[0]
    return None

def _read_cpmi_rank_list(setup: str, win: int, kfold: int, sub: str) -> list[str]:
    p = _find_rank_file(setup, win, kfold, sub)
    if p is None:
        return []
    df = pd.read_csv(p)
    if df.empty:
        return []
    col = "feature" if "feature" in df.columns else df.columns[0]
    return [_norm_name(x) for x in df[col].astype(str).tolist()]

# -------------------- SHAP loading --------------------
def _load_details() -> pd.DataFrame:
    d = pd.read_csv(PLAT_DETAIL).copy()
    d.columns = [c.lower() for c in d.columns]
    if "best_method" in d.columns and "method" not in d.columns:
        d = d.rename(columns={"best_method":"method"})
    if "best_pct_by_median" not in d.columns and "pct" in d.columns:
        d = d.rename(columns={"pct":"best_pct_by_median"})
    return d

DETAILS = _load_details()
need_pd = {"setup","anomaly","win","kfold","best_pct_by_median","method"}
miss2 = need_pd - set(DETAILS.columns)
if miss2:
    raise KeyError(f"{PLAT_DETAIL} missing columns: {sorted(miss2)}. Have: {list(DETAILS.columns)}")

def _find_best_pct_method_from_details(setup: str, anomaly: str, win: int, kfold: int) -> Optional[Tuple[int,str]]:
    sub = DETAILS[
        (DETAILS["setup"].astype(str).str.upper() == setup.upper()) &
        (DETAILS["anomaly"].astype(str).str.upper() == anomaly.upper()) &
        (pd.to_numeric(DETAILS["win"], errors="coerce") == int(win)) &
        (pd.to_numeric(DETAILS["kfold"], errors="coerce") == int(kfold))
    ].copy()
    if sub.empty:
        return None
    r = sub.iloc[0]
    pct = int(pd.to_numeric(r["best_pct_by_median"], errors="coerce"))
    method = str(r["method"]).strip()
    return pct, method

def _find_shap_file(setup: str, anomaly: str, win: int, kfold: int, pct: int, method: str) -> Optional[Path]:
    exact = SHAP_DIR / f"SHAP_BESTPLAT_full_{setup}_{anomaly}_WIN{win}_KF{kfold}_PCT{pct}_M{method}.csv"
    if exact.exists():
        return exact
    # fallback: any pct/method match for that (setup, anomaly, win, kfold)
    hits = sorted(SHAP_DIR.glob(f"SHAP_BESTPLAT_full_{setup}_{anomaly}_WIN{win}_KF{kfold}_PCT*_M*.csv"))
    return hits[0] if hits else None

def _load_shap_full_from_file(p: Path) -> pd.DataFrame:
    df = pd.read_csv(p)
    if df.empty:
        return df
    df.columns = [c.lower() for c in df.columns]
    if "shap_mean_abs" not in df.columns and "importance" in df.columns:
        df = df.rename(columns={"importance": "shap_mean_abs"})
    need = {"feature","subspace","shap_mean_abs"}
    if not need.issubset(set(df.columns)):
        return pd.DataFrame()
    df["feature"] = df["feature"].astype(str).map(_norm_name)
    df["subspace"] = df["subspace"].astype(str).str.lower()
    df["shap_mean_abs"] = pd.to_numeric(df["shap_mean_abs"], errors="coerce").fillna(0.0)
    return df

def _load_shap_full(setup: str, anomaly: str, win: int, kfold: int) -> pd.DataFrame:
    """
    Prefer details-specified (pct,method). If not available, fall back to any SHAP file for that win/kfold.
    """
    pm = _find_best_pct_method_from_details(setup, anomaly, win, kfold)
    if pm is not None:
        pct, method = pm
        p = _find_shap_file(setup, anomaly, win, kfold, pct, method)
        if p is not None:
            return _load_shap_full_from_file(p)

    # fallback: any file for this (setup, anomaly, win, kfold)
    hits = sorted(SHAP_DIR.glob(f"SHAP_BESTPLAT_full_{setup}_{anomaly}_WIN{win}_KF{kfold}_PCT*_M*.csv"))
    if not hits:
        return pd.DataFrame()
    return _load_shap_full_from_file(hits[0])

def _available_anomalies_for(setup: str) -> list[str]:
    # anomalies from details file if possible, otherwise from shap filenames
    sub = DETAILS[DETAILS["setup"].astype(str).str.upper() == setup.upper()]
    anoms = sorted(sub["anomaly"].astype(str).str.upper().unique().tolist())
    if anoms:
        return anoms
    # fallback: parse filenames
    anoms2 = set()
    for p in SHAP_DIR.glob(f"SHAP_BESTPLAT_full_{setup}_*_WIN*_KF*_PCT*_M*.csv"):
        parts = p.name.split("_")
        if len(parts) >= 4:
            anoms2.add(parts[3].upper())  # SHAP_BESTPLAT_full_<setup>_<anomaly>_...
    return sorted(anoms2)

# -------------------- Compute platform agreement --------------------
def platform_metrics(setup: str, win: int, kfold: int) -> tuple[float, float, int]:
    """
    Returns:
      J_platform: median across anomalies of mean-subspace J@p (shared-universe)
      rho_platform: median across anomalies of mean-subspace rho (shared-universe)
      n_used: number of anomalies used
    """
    anomalies = _available_anomalies_for(setup)
    case_J, case_rho = [], []
    used = 0

    for anom in anomalies:
        shap_full = _load_shap_full(setup, anom, win, kfold)
        if shap_full.empty:
            continue

        Js, Rhos = [], []
        for subspace in SUBSPACES:
            cp_list = _read_cpmi_rank_list(setup, win, kfold, subspace)
            if not cp_list:
                continue

            sh_sub = shap_full[shap_full["subspace"] == subspace][["feature","shap_mean_abs"]].copy()
            if sh_sub.empty:
                continue

            sh_list = sh_sub.sort_values("shap_mean_abs", ascending=False)["feature"].tolist()

            Js.append(_jacc_top_pct_on_common(cp_list, sh_list, PCT_FOR_TABLE))
            Rhos.append(_spearman_on_common(cp_list, sh_list))

        if Js:
            case_J.append(float(np.nanmean(Js)))
        if Rhos:
            case_rho.append(float(np.nanmean(Rhos)))

        used += 1

    J_platform = float(np.nanmedian(case_J)) if case_J else np.nan
    rho_platform = float(np.nanmedian(case_rho)) if case_rho else np.nan
    return J_platform, rho_platform, used

# -------------------- Build Table 6 --------------------
rows = []
for setup, cfg in PLATFORM_CFG.items():
    win = int(cfg["win"])
    kf  = int(cfg["kfold"])
    label = str(cfg["label"])

    Jp, rho, n_used = platform_metrics(setup, win, kf)

    rows.append({
        "Platform": label,
        "Window Size": win,
        "K": kf,
        "Cases used": n_used,
        f"J@{PCT_FOR_TABLE}%": Jp,
        "rho": rho,
        f"J@{PCT_FOR_TABLE}% (approx)": (f"≈ {Jp:.2f}" if np.isfinite(Jp) else "NA"),
        "rho (approx)": (f"≈ {rho:.2f}" if np.isfinite(rho) else "NA"),
    })

tab6 = pd.DataFrame(rows)
tab6_out = tab6[["Platform","Window Size","K","Cases used", f"J@{PCT_FOR_TABLE}% (approx)", "rho (approx)"]].copy()

OUT_CSV = OUT_DIR / "Table6_platform_agreement.csv"
tab6_out.to_csv(OUT_CSV, index=False)

print(tab6_out.to_string(index=False))
print("\n[OK] Wrote:", OUT_CSV)
print(f"[INFO] J and rho computed on the SHARED feature universe at top-{PCT_FOR_TABLE}% per subspace.")


       Platform  Window Size  K  Cases used J@80% (approx) rho (approx)
DDR4 (Platform)          512  3           2         ≈ 0.94       ≈ 0.81
DDR5 (Platform)         1024  5           2         ≈ 0.86       ≈ 0.73

[OK] Wrote: /Users/hsiaopingni/octaneX_v7_4functions/Results/Explainability_SHAP_BestPlatforms/Table6_platform_agreement.csv
[INFO] J and rho computed on the SHARED feature universe at top-80% per subspace.


# **Percentile Concordance

In [53]:
# === Fig. 9 style (PER PLATFORM): Percentile-rank concordance (CP–MI vs SHAP) ===
# You requested: DDR5 (Platform) WIN=1024 and K=5 ✅
#
# CONSISTENT with revised Table 6:
#   - same _norm_name()
#   - restrict to SHARED feature universe per subspace (intersection)
#   - percentile ranks computed within the shared universe
#
# UPDATED (visual quality):
#   - legend moved OUTSIDE (never blocks points)
#   - smaller points + semi-transparency + white edges (better overlap readability)
#   - diagonal behind points
#   - Spearman ρ shown as in-axes annotation (cleaner than a large title)
#   - optional TOPK per subspace to reduce clutter (default None = keep all)
#
# Inputs:
#   - Results/BEST_in_DesignSpace_Post_per_platform_details.csv
#   - FeatureRankOUT/<setup>_<win>_<kfold>_0_{compute|memory|sensors}.csv
#   - Results/Explainability_SHAP_BestPlatforms/SHAP_BESTPLAT_full_<setup>_<anomaly>_WIN<w>_KF<k>_PCT<p>_M<method>.csv
#
# Outputs:
#   - Results/Explainability_SHAP_BestPlatforms/Fig9_percentile_concordance_DDR4_WIN512_K3.png
#   - Results/Explainability_SHAP_BestPlatforms/Fig9_percentile_concordance_DDR5_WIN1024_K5.png
# --------------------------------------------------------------------------------------------

import re
import numpy as np
import pandas as pd
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
from pathlib import Path
from typing import Optional, Tuple
from scipy.stats import spearmanr

# -------------------- Paths (robust) --------------------
ROOT = Path(globals().get("ROOT", "/Users/hsiaopingni/octaneX_v7_4functions"))
RES_DIR_RAW = Path(globals().get("RES_DIR", ROOT / "Results"))
RES_DIR = RES_DIR_RAW.parent if RES_DIR_RAW.name in ("Explainability_SHAP_BestCases", "Explainability_SHAP_BestPlatforms") else RES_DIR_RAW

OUT_DIR = RES_DIR / "Explainability_SHAP_BestPlatforms"
OUT_DIR.mkdir(parents=True, exist_ok=True)

DETAILS_CSV = RES_DIR / "BEST_in_DesignSpace_Post_per_platform_details.csv"
assert DETAILS_CSV.exists(), f"Missing: {DETAILS_CSV}"

# Rank dirs (CP–MI)
RANK_DIRS = list(globals().get("RANK_DIRS", [
    ROOT / "FeatureRankOUT",
    Path("/Volumes/Untitled") / "FeatureRankOUT",
    Path("/Volumes/Untitled") / "octaneX" / "FeatureRankOUT",
    Path.home() / "Desktop" / "octaneX" / "FeatureRankOUT",
]))

# SHAP dir (same as output dir by your naming)
SHAP_DIR = RES_DIR / "Explainability_SHAP_BestPlatforms"
assert SHAP_DIR.exists(), f"Missing SHAP dir: {SHAP_DIR}"

SUBSPACES = ("compute", "memory", "sensors")

# -------------------- Case configs (forced to your request) --------------------
SETUP_A = dict(setup="DDR4", anomaly="DROOP",   win=512,  kfold=3, tag="DDR4_WIN512_K3")
SETUP_B = dict(setup="DDR5", anomaly="DROOP",   win=1024, kfold=5, tag="DDR5_WIN1024_K5")
# If Fig.9 DDR5 should be SPECTRE instead of DROOP:
# SETUP_B["anomaly"] = "SPECTRE"

# -------------------- Plot controls --------------------
# If DDR5 looks too cluttered, set TOPK_PER_SUBSPACE to 25 (or 20/30).
# None = keep all common features.
TOPK_PER_SUBSPACE: Optional[int] = 25  # e.g., 25

# -------------------- Shared helpers (match Table 6) --------------------
def _norm_name(s: str) -> str:
    s = re.sub(r"\s+", "_", str(s)).lower().replace("%", "pct")
    s = re.sub(r"[^a-z0-9_]+", "_", s)
    return re.sub(r"_+", "_", s).strip("_")

def _find_rank_file(setup: str, win: int, kfold: int, sub: str) -> Optional[Path]:
    fname = f"{setup}_{win}_{kfold}_0_{sub}.csv"
    for d in RANK_DIRS:
        p = Path(d) / fname
        if p.exists():
            return p
    # fallback: recursive search
    for d in RANK_DIRS:
        d = Path(d)
        if d.exists():
            hits = list(d.rglob(fname))
            if hits:
                return hits[0]
    return None

def _read_cpmi_rank_list(setup: str, win: int, kfold: int, sub: str) -> list[str]:
    p = _find_rank_file(setup, win, kfold, sub)
    if p is None:
        return []
    df = pd.read_csv(p)
    if df.empty:
        return []
    col = "feature" if "feature" in df.columns else df.columns[0]
    return [_norm_name(x) for x in df[col].astype(str).tolist()]

# ---- details: best pct + method (if present for this win/kfold) ----
DETAILS = pd.read_csv(DETAILS_CSV).copy()
DETAILS.columns = [c.lower() for c in DETAILS.columns]
if "best_method" in DETAILS.columns and "method" not in DETAILS.columns:
    DETAILS = DETAILS.rename(columns={"best_method": "method"})
if "best_pct_by_median" not in DETAILS.columns and "pct" in DETAILS.columns:
    DETAILS = DETAILS.rename(columns={"pct": "best_pct_by_median"})

def _best_pct_method_from_details(setup: str, anomaly: str, win: int, kfold: int) -> Optional[Tuple[int, str]]:
    sub = DETAILS[
        (DETAILS["setup"].astype(str).str.upper() == setup.upper()) &
        (DETAILS["anomaly"].astype(str).str.upper() == anomaly.upper()) &
        (pd.to_numeric(DETAILS["win"], errors="coerce") == int(win)) &
        (pd.to_numeric(DETAILS["kfold"], errors="coerce") == int(kfold))
    ].copy()
    if sub.empty:
        return None
    r = sub.iloc[0]
    pct = int(pd.to_numeric(r["best_pct_by_median"], errors="coerce"))
    method = str(r["method"]).strip()
    return pct, method

def _find_shap_file(setup: str, anomaly: str, win: int, kfold: int,
                    pct: Optional[int], method: Optional[str]) -> Optional[Path]:
    # 1) exact (best if pct+method known)
    if pct is not None and method is not None:
        exact = SHAP_DIR / f"SHAP_BESTPLAT_full_{setup}_{anomaly}_WIN{win}_KF{kfold}_PCT{pct}_M{method}.csv"
        if exact.exists():
            return exact
        hits = sorted(SHAP_DIR.glob(
            f"SHAP_BESTPLAT_full_{setup}_{anomaly}_WIN{win}_KF{kfold}_PCT*_M{method}.csv"
        ))
        if hits:
            return hits[0]
    # 2) fallback: any method/pct for that (setup, anomaly, win, kfold)
    hits2 = sorted(SHAP_DIR.glob(
        f"SHAP_BESTPLAT_full_{setup}_{anomaly}_WIN{win}_KF{kfold}_PCT*_M*.csv"
    ))
    return hits2[0] if hits2 else None

def _load_shap_full(setup: str, anomaly: str, win: int, kfold: int) -> Tuple[pd.DataFrame, Optional[int], Optional[str], Optional[Path]]:
    pm = _best_pct_method_from_details(setup, anomaly, win, kfold)
    pct, method = (pm if pm is not None else (None, None))
    p = _find_shap_file(setup, anomaly, win, kfold, pct, method)
    if p is None or not p.exists():
        return pd.DataFrame(), pct, method, None

    df = pd.read_csv(p)
    if df.empty:
        return df, pct, method, p

    df.columns = [c.lower() for c in df.columns]
    if "shap_mean_abs" not in df.columns and "importance" in df.columns:
        df = df.rename(columns={"importance": "shap_mean_abs"})

    need = {"feature", "subspace", "shap_mean_abs"}
    if not need.issubset(set(df.columns)):
        return pd.DataFrame(), pct, method, p

    df["feature"] = df["feature"].astype(str).map(_norm_name)
    df["subspace"] = df["subspace"].astype(str).str.lower()
    df["shap_mean_abs"] = pd.to_numeric(df["shap_mean_abs"], errors="coerce").fillna(0.0)
    return df, pct, method, p

def _restrict_to_common(order: list[str], common: set[str]) -> list[str]:
    return [f for f in order if f in common]

def _percentile_rank_from_order(features_in_order: list[str]) -> dict[str, float]:
    n = len(features_in_order)
    if n <= 1:
        return {features_in_order[0]: 1.0} if n == 1 else {}
    # best=1, worst=0
    return {f: (1.0 - (i / (n - 1))) for i, f in enumerate(features_in_order)}

# -------------------- Plot function --------------------
def plot_percentile_concordance_one(setup: str, anomaly: str, win: int, kfold: int, out_png: Path):
    shap, pct_best, method_best, shap_path = _load_shap_full(setup, anomaly, win, kfold)
    if shap.empty:
        print(f"[SKIP] Missing SHAP full for {setup}/{anomaly} WIN={win} KF={kfold}")
        return

    pts = []
    for sub in SUBSPACES:
        cp_order = _read_cpmi_rank_list(setup, win, kfold, sub)
        sh_sub = shap[shap["subspace"] == sub][["feature", "shap_mean_abs"]].copy()
        if (not cp_order) or sh_sub.empty:
            continue

        # SHAP order (desc)
        sh_order = sh_sub.sort_values("shap_mean_abs", ascending=False)["feature"].tolist()

        # shared universe
        common = set(cp_order) & set(sh_order)
        if len(common) < 3:
            continue

        # preserve each method's order, restricted to common
        cp_c = _restrict_to_common(cp_order, common)
        sh_c = _restrict_to_common(sh_order, common)

        # OPTIONAL: reduce clutter by plotting only top-K from each list, then re-intersect
        if TOPK_PER_SUBSPACE is not None:
            cp_c = cp_c[:int(TOPK_PER_SUBSPACE)]
            sh_c = sh_c[:int(TOPK_PER_SUBSPACE)]
            common = set(cp_c) & set(sh_c)
            if len(common) < 3:
                continue
            cp_c = _restrict_to_common(cp_c, common)
            sh_c = _restrict_to_common(sh_c, common)

        # percentile ranks within shared universe
        cp_pr = _percentile_rank_from_order(cp_c)
        sh_pr = _percentile_rank_from_order(sh_c)

        for f in common:
            pts.append({"subspace": sub, "x": float(cp_pr.get(f, 0.0)), "y": float(sh_pr.get(f, 0.0))})

    if not pts:
        print(f"[SKIP] No aligned features for {setup}/{anomaly} WIN={win} KF={kfold}")
        return

    P = pd.DataFrame(pts)
    rho, _ = spearmanr(P["x"].to_numpy(dtype=float), P["y"].to_numpy(dtype=float))
    rho = float(rho) if np.isfinite(rho) else np.nan

    # --- Style (paper-ready) ---
    fig, ax = plt.subplots(figsize=(7.6, 4.3), dpi=160)

    colors  = {"compute": "tab:orange", "memory": "tab:blue", "sensors": "tab:green"}
    markers = {"compute": "o",          "memory": "s",        "sensors": "^"}
    labels  = {"compute": "Compute",    "memory": "Memory",  "sensors": "Sensors"}

    # diagonal behind points
    ax.plot([0, 1], [0, 1], "--", color="gray", linewidth=1.4, label="y = x", zorder=1)

    # points
    for sub in ["compute", "memory", "sensors"]:
        Q = P[P["subspace"] == sub]
        if Q.empty:
            continue
        ax.scatter(
            Q["x"], Q["y"],
            s=26, alpha=0.70,
            marker=markers[sub],
            color=colors[sub],
            label=labels[sub],
            edgecolors="white", linewidths=0.4,
            zorder=3
        )

    ax.set_xlim(0.0, 1.0)
    ax.set_ylim(0.0, 1.0)

    ax.set_xlabel("CP-MI percentile rank", fontsize=16)
    ax.set_ylabel("SHAP percentile rank", fontsize=16)
    ax.grid(True, linestyle="--", linewidth=0.8, alpha=0.30)

    ax.tick_params(labelsize=11)
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)

    # Spearman text (cleaner than big title)
    ax.text(
        0.02, 0.98, f"Spearman ρ = {rho:.2f}",
        transform=ax.transAxes, ha="left", va="top", fontsize=14
    )

    # Legend OUTSIDE (never blocks data)
    ax.legend(
        loc="center left",
        bbox_to_anchor=(1.02, 0.5),
        frameon=False,
        fontsize=12,
        borderaxespad=0.0
    )

    extra = []
    if pct_best is not None: extra.append(f"best%={pct_best}")
    if method_best is not None: extra.append(f"M={method_best}")
    if shap_path is not None: extra.append(shap_path.name)
    extra_txt = " | ".join(extra)

    # leave room for legend on right
    fig.tight_layout(rect=[0, 0, 0.82, 1])

    out_png.parent.mkdir(parents=True, exist_ok=True)
    fig.savefig(out_png, dpi=220, bbox_inches="tight", pad_inches=0.05)
    plt.close(fig)

    print("[WROTE]", out_png, f"({extra_txt})")

# -------------------- Generate Setup A and Setup B --------------------
outA = OUT_DIR / f"Fig9_percentile_concordance_{SETUP_A['tag']}.png"
outB = OUT_DIR / f"Fig9_percentile_concordance_{SETUP_B['tag']}.png"

plot_percentile_concordance_one(SETUP_A["setup"], SETUP_A["anomaly"], SETUP_A["win"], SETUP_A["kfold"], outA)
plot_percentile_concordance_one(SETUP_B["setup"], SETUP_B["anomaly"], SETUP_B["win"], SETUP_B["kfold"], outB)

[WROTE] /Users/hsiaopingni/octaneX_v7_4functions/Results/Explainability_SHAP_BestPlatforms/Fig9_percentile_concordance_DDR4_WIN512_K3.png (best%=10 | M=dC_aJ | SHAP_BESTPLAT_full_DDR4_DROOP_WIN512_KF3_PCT10_MdC_aJ.csv)
[WROTE] /Users/hsiaopingni/octaneX_v7_4functions/Results/Explainability_SHAP_BestPlatforms/Fig9_percentile_concordance_DDR5_WIN1024_K5.png (SHAP_BESTPLAT_full_DDR5_DROOP_WIN1024_KF5_PCT20_MdC_aJ.csv)


In [59]:
# === Fig. 9 style (PER PLATFORM): Percentile-rank concordance (CP–MI vs SHAP) ===
# You requested: DDR5 (Platform) WIN=1024 and K=5 ✅
#
# CONSISTENT with revised Table 6:
#   - same _norm_name()
#   - restrict to SHARED feature universe per subspace (intersection)
#   - percentile ranks computed within the shared universe
#
# UPDATED (visual quality):
#   - legend moved OUTSIDE (never blocks points)
#   - smaller points + semi-transparency + white edges (better overlap readability)
#   - diagonal behind points
#   - (REMOVED) Spearman ρ text annotation in-axes (per your request)
#   - optional TOPK per subspace to reduce clutter (default 25; set None to keep all)
#
# Inputs:
#   - Results/BEST_in_DesignSpace_Post_per_platform_details.csv
#   - FeatureRankOUT/<setup>_<win>_<kfold>_0_{compute|memory|sensors}.csv
#   - Results/Explainability_SHAP_BestPlatforms/SHAP_BESTPLAT_full_<setup>_<anomaly>_WIN<w>_KF<k>_PCT<p>_M<method>.csv
#
# Outputs:
#   - Results/Explainability_SHAP_BestPlatforms/Fig9_percentile_concordance_DDR4_WIN512_K3.png
#   - Results/Explainability_SHAP_BestPlatforms/Fig9_percentile_concordance_DDR5_WIN1024_K5.png
# --------------------------------------------------------------------------------------------

import re
import numpy as np
import pandas as pd
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
from pathlib import Path
from typing import Optional, Tuple
from scipy.stats import spearmanr

# -------------------- Paths (robust) --------------------
ROOT = Path(globals().get("ROOT", "/Users/hsiaopingni/octaneX_v7_4functions"))
RES_DIR_RAW = Path(globals().get("RES_DIR", ROOT / "Results"))
RES_DIR = RES_DIR_RAW.parent if RES_DIR_RAW.name in ("Explainability_SHAP_BestCases", "Explainability_SHAP_BestPlatforms") else RES_DIR_RAW

OUT_DIR = RES_DIR / "Explainability_SHAP_BestPlatforms"
OUT_DIR.mkdir(parents=True, exist_ok=True)

DETAILS_CSV = RES_DIR / "BEST_in_DesignSpace_Post_per_platform_details.csv"
assert DETAILS_CSV.exists(), f"Missing: {DETAILS_CSV}"

# Rank dirs (CP–MI)
RANK_DIRS = list(globals().get("RANK_DIRS", [
    ROOT / "FeatureRankOUT",
    Path("/Volumes/Untitled") / "FeatureRankOUT",
    Path("/Volumes/Untitled") / "octaneX" / "FeatureRankOUT",
    Path.home() / "Desktop" / "octaneX" / "FeatureRankOUT",
]))

# SHAP dir (same as output dir by your naming)
SHAP_DIR = RES_DIR / "Explainability_SHAP_BestPlatforms"
assert SHAP_DIR.exists(), f"Missing SHAP dir: {SHAP_DIR}"

SUBSPACES = ("compute", "memory", "sensors")

# -------------------- Case configs (forced to your request) --------------------
SETUP_A = dict(setup="DDR4", anomaly="DROOP",   win=512,  kfold=3, tag="DDR4_WIN512_K3")
SETUP_B = dict(setup="DDR5", anomaly="DROOP",   win=1024, kfold=5, tag="DDR5_WIN1024_K5")
# If Fig.9 DDR5 should be SPECTRE instead of DROOP:
# SETUP_B["anomaly"] = "SPECTRE"

# -------------------- Plot controls --------------------
# If DDR5 looks too cluttered, set TOPK_PER_SUBSPACE to 25 (or 20/30).
# None = keep all common features.
TOPK_PER_SUBSPACE: Optional[int] = 25  # e.g., 25

# -------------------- Shared helpers (match Table 6) --------------------
def _norm_name(s: str) -> str:
    s = re.sub(r"\s+", "_", str(s)).lower().replace("%", "pct")
    s = re.sub(r"[^a-z0-9_]+", "_", s)
    return re.sub(r"_+", "_", s).strip("_")

def _find_rank_file(setup: str, win: int, kfold: int, sub: str) -> Optional[Path]:
    fname = f"{setup}_{win}_{kfold}_0_{sub}.csv"
    for d in RANK_DIRS:
        p = Path(d) / fname
        if p.exists():
            return p
    # fallback: recursive search
    for d in RANK_DIRS:
        d = Path(d)
        if d.exists():
            hits = list(d.rglob(fname))
            if hits:
                return hits[0]
    return None

def _read_cpmi_rank_list(setup: str, win: int, kfold: int, sub: str) -> list[str]:
    p = _find_rank_file(setup, win, kfold, sub)
    if p is None:
        return []
    df = pd.read_csv(p)
    if df.empty:
        return []
    col = "feature" if "feature" in df.columns else df.columns[0]
    return [_norm_name(x) for x in df[col].astype(str).tolist()]

# ---- details: best pct + method (if present for this win/kfold) ----
DETAILS = pd.read_csv(DETAILS_CSV).copy()
DETAILS.columns = [c.lower() for c in DETAILS.columns]
if "best_method" in DETAILS.columns and "method" not in DETAILS.columns:
    DETAILS = DETAILS.rename(columns={"best_method": "method"})
if "best_pct_by_median" not in DETAILS.columns and "pct" in DETAILS.columns:
    DETAILS = DETAILS.rename(columns={"pct": "best_pct_by_median"})

def _best_pct_method_from_details(setup: str, anomaly: str, win: int, kfold: int) -> Optional[Tuple[int, str]]:
    sub = DETAILS[
        (DETAILS["setup"].astype(str).str.upper() == setup.upper()) &
        (DETAILS["anomaly"].astype(str).str.upper() == anomaly.upper()) &
        (pd.to_numeric(DETAILS["win"], errors="coerce") == int(win)) &
        (pd.to_numeric(DETAILS["kfold"], errors="coerce") == int(kfold))
    ].copy()
    if sub.empty:
        return None
    r = sub.iloc[0]
    pct = int(pd.to_numeric(r["best_pct_by_median"], errors="coerce"))
    method = str(r["method"]).strip()
    return pct, method

def _find_shap_file(setup: str, anomaly: str, win: int, kfold: int,
                    pct: Optional[int], method: Optional[str]) -> Optional[Path]:
    # 1) exact (best if pct+method known)
    if pct is not None and method is not None:
        exact = SHAP_DIR / f"SHAP_BESTPLAT_full_{setup}_{anomaly}_WIN{win}_KF{kfold}_PCT{pct}_M{method}.csv"
        if exact.exists():
            return exact
        hits = sorted(SHAP_DIR.glob(
            f"SHAP_BESTPLAT_full_{setup}_{anomaly}_WIN{win}_KF{kfold}_PCT*_M{method}.csv"
        ))
        if hits:
            return hits[0]
    # 2) fallback: any method/pct for that (setup, anomaly, win, kfold)
    hits2 = sorted(SHAP_DIR.glob(
        f"SHAP_BESTPLAT_full_{setup}_{anomaly}_WIN{win}_KF{kfold}_PCT*_M*.csv"
    ))
    return hits2[0] if hits2 else None

def _load_shap_full(setup: str, anomaly: str, win: int, kfold: int) -> Tuple[pd.DataFrame, Optional[int], Optional[str], Optional[Path]]:
    pm = _best_pct_method_from_details(setup, anomaly, win, kfold)
    pct, method = (pm if pm is not None else (None, None))
    p = _find_shap_file(setup, anomaly, win, kfold, pct, method)
    if p is None or not p.exists():
        return pd.DataFrame(), pct, method, None

    df = pd.read_csv(p)
    if df.empty:
        return df, pct, method, p

    df.columns = [c.lower() for c in df.columns]
    if "shap_mean_abs" not in df.columns and "importance" in df.columns:
        df = df.rename(columns={"importance": "shap_mean_abs"})

    need = {"feature", "subspace", "shap_mean_abs"}
    if not need.issubset(set(df.columns)):
        return pd.DataFrame(), pct, method, p

    df["feature"] = df["feature"].astype(str).map(_norm_name)
    df["subspace"] = df["subspace"].astype(str).str.lower()
    df["shap_mean_abs"] = pd.to_numeric(df["shap_mean_abs"], errors="coerce").fillna(0.0)
    return df, pct, method, p

def _restrict_to_common(order: list[str], common: set[str]) -> list[str]:
    return [f for f in order if f in common]

def _percentile_rank_from_order(features_in_order: list[str]) -> dict[str, float]:
    n = len(features_in_order)
    if n <= 1:
        return {features_in_order[0]: 1.0} if n == 1 else {}
    # best=1, worst=0
    return {f: (1.0 - (i / (n - 1))) for i, f in enumerate(features_in_order)}

# -------------------- Plot function --------------------
def plot_percentile_concordance_one(setup: str, anomaly: str, win: int, kfold: int, out_png: Path):
    shap, pct_best, method_best, shap_path = _load_shap_full(setup, anomaly, win, kfold)
    if shap.empty:
        print(f"[SKIP] Missing SHAP full for {setup}/{anomaly} WIN={win} KF={kfold}")
        return

    pts = []
    for sub in SUBSPACES:
        cp_order = _read_cpmi_rank_list(setup, win, kfold, sub)
        sh_sub = shap[shap["subspace"] == sub][["feature", "shap_mean_abs"]].copy()
        if (not cp_order) or sh_sub.empty:
            continue

        # SHAP order (desc)
        sh_order = sh_sub.sort_values("shap_mean_abs", ascending=False)["feature"].tolist()

        # shared universe
        common = set(cp_order) & set(sh_order)
        if len(common) < 3:
            continue

        # preserve each method's order, restricted to common
        cp_c = _restrict_to_common(cp_order, common)
        sh_c = _restrict_to_common(sh_order, common)

        # OPTIONAL: reduce clutter by plotting only top-K from each list, then re-intersect
        if TOPK_PER_SUBSPACE is not None:
            cp_c = cp_c[:int(TOPK_PER_SUBSPACE)]
            sh_c = sh_c[:int(TOPK_PER_SUBSPACE)]
            common = set(cp_c) & set(sh_c)
            if len(common) < 3:
                continue
            cp_c = _restrict_to_common(cp_c, common)
            sh_c = _restrict_to_common(sh_c, common)

        # percentile ranks within shared universe
        cp_pr = _percentile_rank_from_order(cp_c)
        sh_pr = _percentile_rank_from_order(sh_c)

        for f in common:
            pts.append({"subspace": sub, "x": float(cp_pr.get(f, 0.0)), "y": float(sh_pr.get(f, 0.0))})

    if not pts:
        print(f"[SKIP] No aligned features for {setup}/{anomaly} WIN={win} KF={kfold}")
        return

    P = pd.DataFrame(pts)
    # (still computed if you want it later, but NOT displayed on plot)
    rho, _ = spearmanr(P["x"].to_numpy(dtype=float), P["y"].to_numpy(dtype=float))
    rho = float(rho) if np.isfinite(rho) else np.nan

    # --- Style (paper-ready) ---
    fig, ax = plt.subplots(figsize=(7.6, 4.3), dpi=160)

    colors  = {"compute": "tab:orange", "memory": "tab:blue", "sensors": "tab:green"}
    markers = {"compute": "o",          "memory": "s",        "sensors": "^"}
    labels  = {"compute": "Compute",    "memory": "Memory",  "sensors": "Sensors"}

    # diagonal behind points
    ax.plot([0, 1], [0, 1], "--", color="gray", linewidth=1.4, label="y = x", zorder=1)

    # points
    for sub in ["compute", "memory", "sensors"]:
        Q = P[P["subspace"] == sub]
        if Q.empty:
            continue
        ax.scatter(
            Q["x"], Q["y"],
            s=26, alpha=0.70,
            marker=markers[sub],
            color=colors[sub],
            label=labels[sub],
            edgecolors="white", linewidths=0.4,
            zorder=3
        )

    ax.set_xlim(0.0, 1.0)
    ax.set_ylim(0.0, 1.0)

    ax.set_xlabel("CP-MI percentile rank", fontsize=16)
    ax.set_ylabel("SHAP percentile rank", fontsize=16)
    ax.grid(True, linestyle="--", linewidth=0.8, alpha=0.30)

    ax.tick_params(labelsize=11)
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)

    # (REMOVED) Spearman text annotation per your request
    # ax.text(0.02, 0.98, f"Spearman ρ = {rho:.2f}", transform=ax.transAxes, ha="left", va="top", fontsize=14)

    # Legend OUTSIDE (never blocks data)
    ax.legend(
        loc="center left",
        bbox_to_anchor=(1.02, 0.5),
        frameon=False,
        fontsize=12,
        borderaxespad=0.0
    )

    extra = []
    if pct_best is not None: extra.append(f"best%={pct_best}")
    if method_best is not None: extra.append(f"M={method_best}")
    if shap_path is not None: extra.append(shap_path.name)
    extra_txt = " | ".join(extra)

    # leave room for legend on right
    fig.tight_layout(rect=[0, 0, 0.82, 1])

    out_png.parent.mkdir(parents=True, exist_ok=True)
    fig.savefig(out_png, dpi=220, bbox_inches="tight", pad_inches=0.05)
    plt.close(fig)

    print("[WROTE]", out_png, f"({extra_txt})")

# -------------------- Generate Setup A and Setup B --------------------
outA = OUT_DIR / f"Fig9_percentile_concordance_{SETUP_A['tag']}.png"
outB = OUT_DIR / f"Fig9_percentile_concordance_{SETUP_B['tag']}.png"

plot_percentile_concordance_one(SETUP_A["setup"], SETUP_A["anomaly"], SETUP_A["win"], SETUP_A["kfold"], outA)
plot_percentile_concordance_one(SETUP_B["setup"], SETUP_B["anomaly"], SETUP_B["win"], SETUP_B["kfold"], outB)

[WROTE] /Users/hsiaopingni/octaneX_v7_4functions/Results/Explainability_SHAP_BestPlatforms/Fig9_percentile_concordance_DDR4_WIN512_K3.png (best%=10 | M=dC_aJ | SHAP_BESTPLAT_full_DDR4_DROOP_WIN512_KF3_PCT10_MdC_aJ.csv)
[WROTE] /Users/hsiaopingni/octaneX_v7_4functions/Results/Explainability_SHAP_BestPlatforms/Fig9_percentile_concordance_DDR5_WIN1024_K5.png (SHAP_BESTPLAT_full_DDR5_DROOP_WIN1024_KF5_PCT20_MdC_aJ.csv)


In [69]:
# === Build LaTeX Table: Median AUC-PR vs top-p (10/30/70/100) for specific (WIN,K) ===
# Uses per-run CSV (source of truth):
#   per_run_metrics_all_PIPELINE.csv
#
# Rule for each cell (setup, anomaly, win, kfold, pct):
#   - compute auc_pr_median across run_id
#   - if multiple methods exist, take the BEST (max) auc_pr_median across methods
#
# Targets (your request):
#   - Setup A (DDR4): WIN=512,  K=3
#   - Setup B (DDR5): WIN=1024, K=5
#
# Output:
#   - prints LaTeX table ready to paste into Overleaf

from pathlib import Path
import pandas as pd
import numpy as np

# ----------------------------
# CONFIG
# ----------------------------
TARGETS = {
    "A": {"setup":"DDR4", "win":512,  "kfold":3, "anomalies":["RH","DROOP"]},       # RH -> TRRespass
    "B": {"setup":"DDR5", "win":1024, "kfold":5, "anomalies":["SPECTRE","DROOP"]},
}
PCTS = [10, 30, 70, 100]

def paper_anomaly_name(anom: str) -> str:
    a = str(anom).strip().upper()
    if a == "RH": return "TRRespass"
    if a == "DROOP": return "Droop"
    if a == "SPECTRE": return "Spectre"
    return a.title()

# ----------------------------
# LOAD per-run CSV (uploaded path first)
# ----------------------------
CANDS = [
    Path("/Users/hsiaopingni/octaneX_v7_4functions/Results/per_run_metrics_all_PIPELINE.csv"),
    Path("/Users/hsiaopingni/octaneX_v7_4functions/Results/per_run_metrics_all_PIPELINE.csv"),
    Path("/Volumes/Untitled/octaneX_results/per_run_metrics_all_PIPELINE.csv"),
]
PER_RUN = next((p for p in CANDS if p.exists()), None)
if PER_RUN is None:
    raise FileNotFoundError("Missing per_run_metrics_all_PIPELINE.csv. Looked for:\n  - " + "\n  - ".join(map(str, CANDS)))

df = pd.read_csv(PER_RUN).copy()
df.columns = [c.lower() for c in df.columns]

need = {"setup","anomaly","win","kfold","pct","run_id","auc_pr","method"}
missing = need - set(df.columns)
if missing:
    raise KeyError(f"per_run_metrics_all_PIPELINE.csv missing columns: {sorted(missing)}. Have: {list(df.columns)}")

# Normalize types
for c in ["win","kfold","pct","auc_pr"]:
    df[c] = pd.to_numeric(df[c], errors="coerce")
df["setup"] = df["setup"].astype(str)
df["anomaly"] = df["anomaly"].astype(str)
df["method"] = df["method"].astype(str)
df["run_id"] = df["run_id"].astype(str)

df = df.dropna(subset=["setup","anomaly","win","kfold","pct","auc_pr","method"]).copy()
df["auc_pr"] = df["auc_pr"].clip(0.0, 1.0)

# ----------------------------
# Compute per-(setup, anomaly, win, kfold, pct, method) median over run_id
# then take max across method for each (setup, anomaly, win, kfold, pct)
# ----------------------------
med_by_method = (
    df.groupby(["setup","anomaly","win","kfold","pct","method"], as_index=False)
      .agg(auc_pr_median=("auc_pr","median"))
)

best_over_method = (
    med_by_method.groupby(["setup","anomaly","win","kfold","pct"], as_index=False)
                .agg(auc_pr_median=("auc_pr_median","max"))
)

# Helper: get cell value
def get_aucpr_median(setup, anomaly, win, kfold, pct):
    sub = best_over_method[
        (best_over_method["setup"].str.upper() == str(setup).upper()) &
        (best_over_method["anomaly"].str.upper() == str(anomaly).upper()) &
        (best_over_method["win"] == int(win)) &
        (best_over_method["kfold"] == int(kfold)) &
        (best_over_method["pct"] == int(pct))
    ]
    if sub.empty:
        return None
    return float(sub["auc_pr_median"].iloc[0])

# ----------------------------
# Build table rows
# ----------------------------
rows = []
for setup_label, cfg in TARGETS.items():
    setup = cfg["setup"]
    win = cfg["win"]
    kfold = cfg["kfold"]
    for anom in cfg["anomalies"]:
        row = {"Setup": setup_label, "Anomaly": paper_anomaly_name(anom)}
        for p in PCTS:
            v = get_aucpr_median(setup, anom, win, kfold, p)
            row[f"{p}%"] = f"{v:.3f}" if v is not None and np.isfinite(v) else "NA"
        rows.append(row)

tab = pd.DataFrame(rows)

# Optional: order anomalies as your paper example
order = ["TRRespass","Droop","Spectre"]
tab["__ord"] = tab["Anomaly"].map(lambda x: order.index(x) if x in order else 999)
tab = tab.sort_values(["Setup","__ord","Anomaly"]).drop(columns=["__ord"]).reset_index(drop=True)

# ----------------------------
# Emit LaTeX (paste into Overleaf)
# ----------------------------
latex_lines = []
latex_lines.append(r"\begin{table}[bp]")
latex_lines.append(r"    \centering")
latex_lines.append(r"    \caption{Median AUC--PR summary vs top $p$ features retained across setups.}")
latex_lines.append(r"    \label{tab:aucpr_summary}")
latex_lines.append(r"    \setlength{\tabcolsep}{6pt}")
latex_lines.append(r"    \renewcommand{\arraystretch}{1.15}")
latex_lines.append(r"    \footnotesize")
latex_lines.append(r"    \begin{tabular}{||c|c|c|c|c|c||}")
latex_lines.append(r"        \hline")
latex_lines.append(r"        \multirow{2}{*}{\textbf{Setup}} &")
latex_lines.append(r"        \multirow{2}{*}{\textbf{Anomaly}} &")
latex_lines.append(r"        \multicolumn{4}{c||}{$p$} \\")
latex_lines.append(r"        \cline{3-6}")
latex_lines.append(r"        & & \textbf{10\%} & \textbf{30\%} & \textbf{70\%} & \textbf{100\%} \\")
latex_lines.append(r"        \hline")

for _, r in tab.iterrows():
    latex_lines.append(
        f"        {r['Setup']} & {r['Anomaly']} & {r['10%']} & {r['30%']} & {r['70%']} & {r['100%']} \\\\"
    )

latex_lines.append(r"        \hline")
latex_lines.append(r"    \end{tabular}")
latex_lines.append(r"\end{table}")

latex = "\n".join(latex_lines)
print(latex)

# Also save to a file in the current session (optional)
out_tex = Path("/Users/hsiaopingni/octaneX_v7_4functions/Results/aucpr_summary_table_from_perrun.tex")
out_tex.write_text(latex)
print(f"\n[OK] Saved LaTeX to: {out_tex}")

\begin{table}[bp]
    \centering
    \caption{Median AUC--PR summary vs top $p$ features retained across setups.}
    \label{tab:aucpr_summary}
    \setlength{\tabcolsep}{6pt}
    \renewcommand{\arraystretch}{1.15}
    \footnotesize
    \begin{tabular}{||c|c|c|c|c|c||}
        \hline
        \multirow{2}{*}{\textbf{Setup}} &
        \multirow{2}{*}{\textbf{Anomaly}} &
        \multicolumn{4}{c||}{$p$} \\
        \cline{3-6}
        & & \textbf{10\%} & \textbf{30\%} & \textbf{70\%} & \textbf{100\%} \\
        \hline
        A & TRRespass & 1.000 & 1.000 & 1.000 & 1.000 \\
        A & Droop & 1.000 & 1.000 & 0.996 & 1.000 \\
        B & Droop & 0.978 & 0.959 & 0.975 & 0.973 \\
        B & Spectre & 1.000 & 1.000 & 1.000 & 1.000 \\
        \hline
    \end{tabular}
\end{table}

[OK] Saved LaTeX to: /Users/hsiaopingni/octaneX_v7_4functions/Results/aucpr_summary_table_from_perrun.tex


In [77]:
import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import wilcoxon, ttest_rel
from pathlib import Path

# =========================
# Paths
# =========================
EXT_DRIVE = Path("/Volumes/Untitled")
LOCAL_ROOT = Path("/Users/hsiaopingni/octaneX_v7_4functions")
ROOT = (EXT_DRIVE / "octaneX") if (EXT_DRIVE.exists() and (EXT_DRIVE / "octaneX").exists()) else LOCAL_ROOT

OUT_DIR = ROOT / "Results" / "Figures"
OUT_DIR.mkdir(parents=True, exist_ok=True)
print(f"[OK] Saving outputs to: {OUT_DIR.resolve()}")

# Prefer uploaded file first (this session), then local paths
CSV_CANDS = [
    Path("/mnt/data/per_run_metrics_all_PIPELINE.csv"),
    ROOT / "Results" / "per_run_metrics_all_PIPELINE.csv",
    Path("/Volumes/Untitled/octaneX_results/per_run_metrics_all_PIPELINE.csv"),
]
PER_RUN = next((p for p in CSV_CANDS if p.exists()), None)
if PER_RUN is None:
    raise FileNotFoundError("Cannot find per_run_metrics_all_PIPELINE.csv. Tried:\n  - " + "\n  - ".join(map(str, CSV_CANDS)))
print(f"[OK] Using per-run CSV: {PER_RUN}")

# =========================
# Baseline OCTANE arrays (unchanged)
# =========================
octane_aucpr = np.array([0.93, 0.96, 0.97, 0.94, 0.98])
octane_roc   = np.array([0.90, 0.93, 0.95, 0.92, 0.94])

# =========================
# X-OCTANE selection (your exact request)
# =========================
XOCT_METHOD   = "dE_aM"   # your "X-OCTANE" proxy inside per_run_metrics_all_PIPELINE.csv
TARGET_SETUP  = "DDR5"
TARGET_ANOM   = "DROOP"
TARGET_WIN    = 1024
TARGET_KFOLD  = 5
TARGET_PCT    = 80

# workload order you use across the project
WORKLOAD_ORDER = ["DFT","DJ","DP","GL","GS","HA","JA","MM","NI","OE","PI","SH","TR"]
WL_RANK = {w:i for i,w in enumerate(WORKLOAD_ORDER)}

# load only needed columns
usecols = ["setup","anomaly","win","kfold","pct","method","workload","auc_pr","roc_auc"]
df = pd.read_csv(PER_RUN, usecols=usecols).copy()
df.columns = [c.lower() for c in df.columns]

for c in ["win","kfold","pct","auc_pr","roc_auc"]:
    df[c] = pd.to_numeric(df[c], errors="coerce")

df["setup"]    = df["setup"].astype(str).str.upper()
df["anomaly"]  = df["anomaly"].astype(str).str.upper()
df["method"]   = df["method"].astype(str)
df["workload"] = df["workload"].astype(str).str.upper()

mask = (
    (df["setup"] == TARGET_SETUP) &
    (df["anomaly"] == TARGET_ANOM) &
    (df["win"] == TARGET_WIN) &
    (df["kfold"] == TARGET_KFOLD) &
    (df["pct"] == TARGET_PCT) &
    (df["method"] == XOCT_METHOD)
)

dx = df[mask].dropna(subset=["auc_pr","roc_auc"]).copy()
if dx.empty:
    raise RuntimeError(
        f"No rows found for: {TARGET_SETUP} {TARGET_ANOM} WIN{TARGET_WIN} K{TARGET_KFOLD} PCT{TARGET_PCT} method={XOCT_METHOD}."
    )

# sort by workload order then take first 5
dx["wl_ord"] = dx["workload"].map(WL_RANK).fillna(999).astype(int)
dx = dx.sort_values(["wl_ord", "workload"]).reset_index(drop=True)

dx5 = dx.head(5).copy()
xoct_aucpr = np.clip(dx5["auc_pr"].to_numpy(float), 0.0, 1.0)
xoct_roc   = np.clip(dx5["roc_auc"].to_numpy(float), 0.0, 1.0)
xoct_wl    = dx5["workload"].tolist()

print("\n[X-OCTANE (workload-ordered) selection]")
for wl, ap, rc in zip(xoct_wl, xoct_aucpr, xoct_roc):
    print(f"  {wl}: AUC-PR={ap:.6f}, ROC-AUC={rc:.6f}")

# Honest check vs baseline (cannot force these to be all above)
print("\n[CHECK vs baseline OCTANE arrays]")
for i, wl in enumerate(xoct_wl):
    print(f"  Fold{i+1} ({wl}): "
          f"AUC-PR {'OK' if xoct_aucpr[i] >= octane_aucpr[i] else 'LOW'} "
          f"({xoct_aucpr[i]:.3f} vs {octane_aucpr[i]:.3f}), "
          f"ROC {'OK' if xoct_roc[i] >= octane_roc[i] else 'LOW'} "
          f"({xoct_roc[i]:.3f} vs {octane_roc[i]:.3f})")

# =========================
# Stats + plot (unchanged)
# =========================
def paired_stats(baseline, hybrid, label):
    diff = hybrid - baseline
    mean_gain = diff.mean()
    # Wilcoxon can fail if all diffs are 0; handle safely
    try:
        w_stat, w_p = wilcoxon(hybrid, baseline, alternative='greater')
    except Exception:
        w_p = np.nan
    try:
        t_stat, t_p = ttest_rel(hybrid, baseline)
    except Exception:
        t_p = np.nan
    return {'label': label, 'gain': mean_gain, 'wilcoxon_p': w_p, 't_p': t_p, 'diff': diff}

res_aucpr = paired_stats(octane_aucpr, xoct_aucpr, 'AUC-PR')
res_roc   = paired_stats(octane_roc, xoct_roc, 'ROC-AUC')

fig, ax = plt.subplots(1, 2, figsize=(9, 4))
for i, (metric, base, hybrid, res) in enumerate([
    ('AUC-PR', octane_aucpr, xoct_aucpr, res_aucpr),
    ('ROC-AUC', octane_roc, xoct_roc, res_roc)
]):
    x = np.arange(1, len(base) + 1)
    ax[i].plot(x, base, 'o--', label='OCTANE', color='gray')
    ax[i].plot(x, hybrid, 's-', label='X-OCTANE', color='tab:blue')
    ax[i].axhline(np.mean(base), color='gray', linestyle=':', alpha=0.6)
    ax[i].axhline(np.mean(hybrid), color='tab:blue', linestyle='--', alpha=0.7)
    ax[i].set_xlabel('Cross-Validation Fold')
    ax[i].set_ylabel(metric)
    ax[i].set_ylim(0.85, 1.02)
    ax[i].grid(alpha=0.3)
    ax[i].set_title(
        f"{metric}\nΔ={res['gain']:.3f} | "
        f"p₍W₎={res['wilcoxon_p']:.3f}, p₍t₎={res['t_p']:.3f}"
    )
ax[0].legend(loc='lower right', fontsize=9)
plt.tight_layout()

png_path = OUT_DIR / "XOCTANE_vs_OCTANE_ttest_wilcoxon.png"
pdf_path = OUT_DIR / "XOCTANE_vs_OCTANE_ttest_wilcoxon.pdf"
plt.savefig(png_path, dpi=300, bbox_inches="tight")
plt.savefig(pdf_path, bbox_inches="tight")
plt.close()

print("\n=== Statistical Summary ===")
for res in [res_aucpr, res_roc]:
    print(f"{res['label']}:")
    print(f"  Mean gain: {res['gain']:.4f}")
    print(f"  Wilcoxon p-value: {res['wilcoxon_p']:.4f}")
    print(f"  Paired t-test p-value: {res['t_p']:.4f}")
    print()
print(f"Figures saved to:\n  {png_path}\n  {pdf_path}")

# =============================================================================
# LaTeX improvement table (computed from THIS per_run CSV)
# IMPORTANT:
# per_run_metrics_all_PIPELINE.csv does NOT contain a separate CP-MI-only vs CP-MI+SHAP model,
# so CP-MI+SHAP over CP-MI deltas cannot be derived reliably from it unless both variants exist.
# If you still want a table from this file, you MUST specify baseline/hybrid method labels.
# Below uses BASE_METHOD vs HYBRID_METHOD (method-level proxy).
# =============================================================================
BASE_METHOD   = "dC_aM"
HYBRID_METHOD = "dE_aM"

def compute_mean_delta(setup, anomaly, win, kfold, pct):
    sub = df[(df["setup"]==setup)&(df["anomaly"]==anomaly)&(df["win"]==win)&(df["kfold"]==kfold)&(df["pct"]==pct)].copy()
    sub = sub[sub["method"].isin([BASE_METHOD, HYBRID_METHOD])].copy()
    if sub.empty:
        return None

    keys = ["setup","anomaly","win","kfold","pct","workload"]
    b = sub[sub["method"]==BASE_METHOD][keys+["auc_pr","roc_auc"]].rename(columns={"auc_pr":"auc_b","roc_auc":"roc_b"})
    h = sub[sub["method"]==HYBRID_METHOD][keys+["auc_pr","roc_auc"]].rename(columns={"auc_pr":"auc_h","roc_auc":"roc_h"})
    m = b.merge(h, on=keys, how="inner")
    if m.empty:
        return None
    return float((m["auc_h"]-m["auc_b"]).mean()), float((m["roc_h"]-m["roc_b"]).mean())

# Using your requested configs in the conversation:
# Setup A: DDR4 WIN512 K3, anomalies DROOP + RH (RH is "TRRespass" in paper)
# Setup B: DDR5 WIN1024 K5, anomalies DROOP + SPECTRE
table_rows = []
targets = [
    ("A","DDR4","DROOP",512,3,80,"Droop"),
    ("A","DDR4","RH",512,3,80,"TRRespass"),
    ("B","DDR5","DROOP",1024,5,80,"Droop"),
    ("B","DDR5","SPECTRE",1024,5,80,"Spectre"),
]
for setup_lbl, setup, anom, win, kf, pct, paper_anom in targets:
    d = compute_mean_delta(setup, anom, win, kf, pct)
    if d is None:
        d_pr, d_roc = np.nan, np.nan
    else:
        d_pr, d_roc = d
    table_rows.append((setup_lbl, paper_anom, d_pr, d_roc))

avg_pr  = np.nanmean([r[2] for r in table_rows])
avg_roc = np.nanmean([r[3] for r in table_rows])


[OK] Saving outputs to: /Users/hsiaopingni/octaneX_v7_4functions/Results/Figures
[OK] Using per-run CSV: /Users/hsiaopingni/octaneX_v7_4functions/Results/per_run_metrics_all_PIPELINE.csv

[X-OCTANE (workload-ordered) selection]
  DFT: AUC-PR=0.971168, ROC-AUC=0.966049
  DJ: AUC-PR=0.958250, ROC-AUC=0.959877
  DP: AUC-PR=1.000000, ROC-AUC=1.000000
  GL: AUC-PR=0.907735, ROC-AUC=0.904321
  GS: AUC-PR=1.000000, ROC-AUC=1.000000

[CHECK vs baseline OCTANE arrays]
  Fold1 (DFT): AUC-PR OK (0.971 vs 0.930), ROC OK (0.966 vs 0.900)
  Fold2 (DJ): AUC-PR LOW (0.958 vs 0.960), ROC OK (0.960 vs 0.930)
  Fold3 (DP): AUC-PR OK (1.000 vs 0.970), ROC OK (1.000 vs 0.950)
  Fold4 (GL): AUC-PR LOW (0.908 vs 0.940), ROC LOW (0.904 vs 0.920)
  Fold5 (GS): AUC-PR OK (1.000 vs 0.980), ROC OK (1.000 vs 0.940)

=== Statistical Summary ===
AUC-PR:
  Mean gain: 0.0114
  Wilcoxon p-value: 0.3125
  Paired t-test p-value: 0.4292

ROC-AUC:
  Mean gain: 0.0380
  Wilcoxon p-value: 0.0625
  Paired t-test p-value: 0.06